A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
function val = MultiDomainIntegration(f,a,b,c,N,eps)
% Description: Implements a method to evaluate the integral
%
%   int_a^c f(x) dx = int_a^b f(x) dx + int_b^c f(x) dx := A_0 + A_1
%
% where a < b <= c and that by first evaluating the integral A_0 using 
% Matlab's own 'integral' function directly and then evaluating the 
% remaining integral A_1 using the multi-domain integration method of 
% (Zhu, 2010) (still with Matlab's 'integral' function).
%
% Parameters:
%   f:   [function] Function to integrate.
%   a:   [1x1 real] Left endpoint of integration interval.
%   b:   [1x1 real] The interval midpoint from where we use the method of
%        (Zhu, 2010).
%   c:   [1x1 real] Right endpoint of integration interval.
%   N:   [1x1 real] Number of sub-intervals to divide [b,c] into.
%   eps: [1x1 real] We stop the integration of [b,c] when the absolute 
%         value of a sub-integral is smaller than eps.
%
% Output: 
%   val: [1x1 real] Value of integral.
%
% References:
%   - Jianwei Zhu, Applications of Fourier Transform to Smile Modelling,
%   Springer, 2010.
%
    
    if b < a || b > c
        error('MultiDomainIntegration: Invalid integration points.');
    end
    
    dz = (c - b) / N;
    val = integral(f,a,b);
    if c > b
        for i=1:N
            valNew = integral(f,b + (i-1)*dz,b + i*dz);
            val = val + valNew;
            if abs(valNew) < eps
                return;
            end
        end
    end
    
end


In [ ]:
% MultiAssetModelLGM2FLGM2FOld20181031.m

classdef MultiAssetModelLGM2FLGM2F < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    % 20181031
    
    properties
        modelNameMap
        stateNumberMap
        refModel
        refModelName
        stochasticModelNames
        correlationMatrix
        numOfFactor
        foreignModelName
        payCurrency
        useBrigoCorrYN
    end
    
    methods
        function multiAsset = MultiAssetModelLGM2FLGM2F(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                
                refModelName = modelParams('refModelName');
                multiAsset.refModelName = refModelName;
                multiAsset.refModel = multiAsset.modelNameMap(refModelName);
                
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                if isKey(modelParams,'useBrigoCorrYN')
                    multiAsset.useBrigoCorrYN = modelParams('correlationMatrix');
                else
                    multiAsset.useBrigoCorrYN = 'YES';
                end
                % LGM2F + LGM2F model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                % beginning index of multiassetModel map
                multiAsset.stateNumberMap = containers.Map;
                
                cumNumOfFactors = 0;
                startIdx = 1;
                for i=1: length(stochasticModelNames)
                    multiAsset.stateNumberMap(stochasticModelNames{i}) = startIdx;
                    numOfFactor = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor;
                    cumNumOfFactors = cumNumOfFactors + numOfFactor;
                    startIdx = startIdx + numOfFactor;
                end
                multiAsset.numOfFactor = cumNumOfFactors;
                
                stochasticModelNames = multiAsset.stochasticModelNames;
                
                % find foreignModel(USD) 
                % we assume that there is only one foreign model
                % need to be updated later
                % multiple foreign Model
                % we only need to care about domestic RefModel and each
                % Foreign Models
                
                for i=1:length(stochasticModelNames)
                    if ~strcmp(stochasticModelNames{i}, refModelName)
                        multiAsset.foreignModelName = stochasticModelNames{i};
                        break;
                    end
                end
                
                quantoLGM2F_LGM2FmodelParams.fxVolCalibrated = false;
                
                % construct USDQuantoLGM2F_LGM2F 
                % from KRWLGM2FRisky and USDLGM2F
                
                quantoLGM2F_LGM2FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                quantoLGM2F_LGM2FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
                
                % market correlation
                %domestic & foreign
                quantoLGM2F_LGM2FmodelParams.corr_df = multiAsset.correlationMatrix(1,2);
                %domestic & fx
                quantoLGM2F_LGM2FmodelParams.corr_dx = multiAsset.correlationMatrix(1,3);
                %foreign & fx
                quantoLGM2F_LGM2FmodelParams.corr_fx = multiAsset.correlationMatrix(2,3);

               
                
                quantoLGM2F_LGM2FmodelParams.corr_drds = quantoLGM2F_LGM2FmodelParams.domesticModel.correl;
                quantoLGM2F_LGM2FmodelParams.corr_frfs = quantoLGM2F_LGM2FmodelParams.foreignModel.correl;
                
                % model correlation setting to be updated !!
                % Brigo , MarketModel & proxy ..
%                 useBrigoCorrYNYN = 'true';
                
                if strcmp(modelParams('useBrigoCorrYN'),'original') || strcmp(modelParams('useBrigoCorrYN'),'YES')
                    gamma_df = multiAsset.computeBrigo_2F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_df);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_drfs = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs = gamma_df;
                    
                    
                    gamma_dx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_dx);
                    quantoLGM2F_LGM2FmodelParams.corr_drx = gamma_dx;
                    quantoLGM2F_LGM2FmodelParams.corr_dsx = gamma_dx;
                    
                    gamma_fx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_fx);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_frx = gamma_fx;
                    quantoLGM2F_LGM2FmodelParams.corr_fsx = gamma_fx;
                    
                elseif strcmp(modelParams('useBrigoCorrYN'),'adjust')
                    gamma_df = multiAsset.computeBrigo_2F2F_CorrelationAdj(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_df);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.tenor = ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs = gamma_df;
                    
                    % Leave 1Fx2F Correlation invariant!
                    gamma_dx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_dx);
                    quantoLGM2F_LGM2FmodelParams.corr_drx = gamma_dx;
                    quantoLGM2F_LGM2FmodelParams.corr_dsx = gamma_dx;
                    
                    gamma_fx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_fx);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_frx = gamma_fx;
                    quantoLGM2F_LGM2FmodelParams.corr_fsx = gamma_fx;    
                elseif strcmp(modelParams('useBrigoCorrYN'),'simple_adjust')
                
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_drx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_frx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                
                elseif strcmp(modelParams('useBrigoCorrYN'),'calibrateToHist')
                    
                    calibOut = multiAsset.CalibrateToCrossCorrelation(modelParams('crossCorrelation'));
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfr = calibOut.corr_drfr;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr = calibOut.corr_dsfr;
                    quantoLGM2F_LGM2FmodelParams.corr_drfs = calibOut.corr_drfs;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs = calibOut.corr_dsfs;
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_frx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                    
                else
                
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_drx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_frx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                end
                %fx Black Vol Setting
                
                % BSCK1FLGM2F calibration needed !
                fxModelName = strcat(multiAsset.foreignModelName,refModelName);
                fxModelName2 = strcat(refModelName,multiAsset.foreignModelName);
                
                if isKey(multiAsset.mktData,fxModelName)
                    fxMktData = multiAsset.mktData(fxModelName);
                elseif isKey(multiAsset.mktData,fxModelName2)
                    fxMktData = multiAsset.mktData(fxModelName2);
                else
                    error('wrong fx asset name!')
                end
                
                fxImpVolRawData = fxMktData('impliedTermVol').params('rawData');

                fxImpVol.tenor = fxImpVolRawData(:,1);
                fxImpVol.quote = fxImpVolRawData(:,2);

                quantoLGM2F_LGM2FmodelParams.fxBlackVol = fxImpVol;
                usdquantoLGM2F_LGM2F = QuantoLGM2F_LGM2F(quantoLGM2F_LGM2FmodelParams);
                multiAsset.modelNameMap(multiAsset.foreignModelName) = usdquantoLGM2F_LGM2F;
                
                
            end
        end
        
        function out = computeBrigo_2F2F_CorrelationAdj(multiAsset,params,corrIn)
            % adjusted version so that 
            % correlation matrix is positive definite
            
                
            %domestic           
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;
            
            %foreign
            
            foreign_dH_r_Tenor = params.foreignModel.dH1.tenor;
            foreign_dH_r_Quote = params.foreignModel.dH1.quote;

            foreign_Alpha_r_Tenor = params.foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = params.foreignModel.Alpha1.quote;

            foreign_dH_s_Tenor = params.foreignModel.dH2.tenor;
            foreign_dH_s_Quote = params.foreignModel.dH2.quote;

            foreign_Alpha_s_Tenor = params.foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = params.foreignModel.Alpha2.quote;
            
            corr_frfs_Tenor = params.foreignModel.correl.tenor;
            corr_frfs_Quote = params.foreignModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);...
                         corr_frfs_Tenor(:);...
                         foreign_Alpha_r_Tenor(:);foreign_Alpha_s_Tenor(:); ...
                         foreign_dH_r_Tenor(:)],foreign_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_dr = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_ds = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                corr_drds = corr_drds_Values(k);
                
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                foreign_dH_s_Values(k) = H_interpolation(foreign_dH_s_Tenor, foreign_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_fr = foreign_dH_r_Values(k)*foreign_Alpha_r_Values(k);
                sigma_fs = foreign_dH_s_Values(k)*foreign_Alpha_s_Values(k);
                corr_frfs = corr_frfs_Values(k);
                
                
                numerator = sigma_dr*sigma_dr + sigma_ds*sigma_ds + 2*corr_drds*sigma_dr*sigma_ds;
                numerator = numerator * (sigma_fr*sigma_fr + sigma_fs*sigma_fs + 2*corr_frfs*sigma_fr*sigma_fs);
                numerator = sqrt(numerator);
                
%                 denominator = sigma_dr +sigma_ds;
%                 denominator = denominator*(sigma_fr +sigma_fs);
                % adjusted version so that correlation matrix is positive
                % definite
                denominator = sigma_dr * sigma_fr +sigma_ds * sigma_fs;
                
                gammaValues(k) = numerator/denominator*corrIn;
                % adjust to be in range -0.99 ~ 0.99
                gammaValues(k) = min(max(gammaValues(k),-0.99),0.99);
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = computeBrigo_2F2F_Correlation(multiAsset,params,corrIn)
            %domestic           
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;
            
            %foreign
            
            foreign_dH_r_Tenor = params.foreignModel.dH1.tenor;
            foreign_dH_r_Quote = params.foreignModel.dH1.quote;

            foreign_Alpha_r_Tenor = params.foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = params.foreignModel.Alpha1.quote;

            foreign_dH_s_Tenor = params.foreignModel.dH2.tenor;
            foreign_dH_s_Quote = params.foreignModel.dH2.quote;

            foreign_Alpha_s_Tenor = params.foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = params.foreignModel.Alpha2.quote;
            
            corr_frfs_Tenor = params.foreignModel.correl.tenor;
            corr_frfs_Quote = params.foreignModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);...
                         corr_frfs_Tenor(:);...
                         foreign_Alpha_r_Tenor(:);foreign_Alpha_s_Tenor(:); ...
                         foreign_dH_r_Tenor(:)],foreign_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_dr = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_ds = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                corr_drds = corr_drds_Values(k);
                
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                foreign_dH_s_Values(k) = H_interpolation(foreign_dH_s_Tenor, foreign_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_fr = foreign_dH_r_Values(k)*foreign_Alpha_r_Values(k);
                sigma_fs = foreign_dH_s_Values(k)*foreign_Alpha_s_Values(k);
                corr_frfs = corr_frfs_Values(k);
                
                
                numerator = sigma_dr*sigma_dr + sigma_ds*sigma_ds + 2*corr_drds*sigma_dr*sigma_ds;
                numerator = numerator * (sigma_fr*sigma_fr + sigma_fs*sigma_fs + 2*corr_frfs*sigma_fr*sigma_fs);
                numerator = sqrt(numerator);
                
                denominator = sigma_dr +sigma_ds;
                denominator = denominator*(sigma_fr +sigma_fs);
                
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = computeBrigo_1F2F_CorrelationAdj(multiAsset,params,corrIn)
            % adjusted version so that 
            % correlation matrix is positive definite           
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
%                 denominator = sigma_r +sigma_s;
                denominator = sigma_r;
                
                gammaValues(k) = numerator/denominator*corrIn;
                % adjust to be in range -0.99 ~ 0.99
                gammaValues(k) = min(max(gammaValues(k),-0.99),0.99);
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = computeBrigo_1F2F_Correlation(multiAsset,params,corrIn)
                       
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
                denominator = sigma_r +sigma_s;
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(multiAsset,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(multiAsset,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(multiAsset,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            out1 = SimpleSRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper);
            out2 = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralowerS,raupperS);
%             out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
            outVec = out1*out2;
            out = outVec;
        end
        
        function out = SimpleDRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
%                     outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                    outVec(i) = multiAsset.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = DFRisky(multiAsset,t)
            out = multiAsset.refModel.DFRisky(t);
        end
        
        function out = stateLocalDrift(multiAsset,fromTime, toTime)
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            out = foreignModel.LocalDrift(fromTime/365.0,toTime/365.0);
        end
        
        function out = stateLocalVariance(multiAsset,fromTime, toTime)
            % we need to generalize this hard coding !!
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            % we use quantoLGM1F_LGM2F to calculate localcovariance term
            % is there any better solution ?  :(
            
            out = foreignModel.LocalCovariance(fromTime/365.0,toTime/365.0);
        end
        
        function out =  Rate_Index(multiAsset,rateType,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT,PSAOption)
            if strcmp(PSAOption,'PSAAll')
                out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            elseif strcmp(PSAOption,'PSA')
                if strcmp(rateType,'CMS')
                    out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
            else
                if strcmp(rateType,'CMS')
                    out = CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
                
            end
        end
        
        function out =  Libor_SimpleDate(multiAsset,curveName,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3:4,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3:4,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3:4,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        
        function out = discountPayoffRisky(multiAsset,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            % for multiAsset discounting is done by refModel
            dfNumMat = multiAsset.refModel.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
            referenceModelStatesT = modelStatesT(1:2,:);
            
            discountT = multiAsset.refModel.discountFactorRisky(eventTime, numMatTime,numMatTime,referenceModelStatesT);
            
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountFactorRisky(multiAsset, startTime, endTime,numMatTime,modelStatesT)
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
           referenceModelStatesT = modelStatesT(1:2,:);
           out = multiAsset.refModel.discountFactorRisky(startTime, endTime,numMatTime,referenceModelStatesT);
           
        end
        
        function out = CMSCMTSpreadDigitalArgDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2)
           % probability that CMS tenor1 - CMT tenor 2  < strike
           %CMS 1
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
%            tau1 = 1.0/(domesticModel.freq);
           % hard coding for LGM2F for the time-being
           tau1 = 0.25;
           
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau1);
           
           cms_fwd   = cmsDigitalArg.fwdCMS;
           cms_mu    = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_var   = cms_sigma * cms_sigma;
           cms_c_nN  = cmsDigitalArg.c_nN;
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           cmtDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor2,tau2);
           
           cmt_fwd   = cmtDigitalArg.fwdCMS;
           cmt_mu    = cmtDigitalArg.mu;
           cmt_sigma = cmtDigitalArg.sigma;
           cmt_var   = cmt_sigma * cmt_sigma;
           cmt_c_nN  = cmtDigitalArg.c_nN;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (4,4) matrix (KRW dr, KRW ds, USD fr USD fs) as (1,2,3,4)
           coVar = foreignModel.LocalCovariance(0,to);
           
           %cms: KRW
           %cmt :USD
           cmscmt_covar =                cms_c_nN(1) * cmt_c_nN(1) * coVar(1,3);
           cmscmt_covar = cmscmt_covar + cms_c_nN(2) * cmt_c_nN(1) * coVar(2,3);
           
           cmscmt_covar = cmscmt_covar + cms_c_nN(1) * cmt_c_nN(2) * coVar(1,4);
           cmscmt_covar = cmscmt_covar + cms_c_nN(2) * cmt_c_nN(2) * coVar(2,4);
           
           spread_var = cms_var + cmt_var - 2.0 * cmscmt_covar;
           
           out.fwdSpread = cms_fwd - cmt_fwd; 
           out.mu = cms_mu - cmt_mu;
           out.sigma = sqrt(spread_var);
        end
        
        function out = CMSCMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,strike)
           % probability that CMS tenor1 - CMT tenor 2 < strike
           %CMS 1
           cmsCmtSpreadDigitalArg = multiAsset.CMSCMTSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2);
           mu = cmsCmtSpreadDigitalArg.mu;
           sigma = cmsCmtSpreadDigitalArg.sigma;
           z1= (strike - mu)/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSCMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,lower,upper)
            
            lower_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,lower);
            upper_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,upper);
%             out = lower_value - upper_value;
            out = upper_value - lower_value;
        end
        
        function out = CMSCMSSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor > strike
           %CMS
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau1 = 1.0/(foreignModel.freq);
           cmsDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau1);
           
           cms_mu = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_c_nN = cmsDigitalArg.c_nN;
           
           
           %CMS Spread
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           cmsSpreadDigitalArg = domesticModel.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           cmsSpread_c_nN1 = cmsSpreadDigitalArg.c_nN1;
           cmsSpread_c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           cmsSpread_mu = cmsSpreadDigitalArg.mu;
           cmsSpread_sigma = cmsSpreadDigitalArg.sigma;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3,4)
           varCovar = foreignModel.LocalCovariance(0,to);
           
           % covariance between USD CMS Rate & KRW CMS Spread
           %cms: USD
           %cmtSpread :KRW CMS 1 - KRW CMS 2
           cms_cmsSpread_covar = 0.0;
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,3);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,3);
           
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(2) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,4);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(2) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,4);
           
           corr = cms_cmsSpread_covar/(cms_sigma*cmsSpread_sigma);
           
           z1 = (cms_mu       - strike)/cms_sigma;
           z2 = (cmsSpread_mu - strikeS)/cmsSpread_sigma;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function impCorrel = ImpCorrel(multiAsset,tx,tnr1,tnr2)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = foreignModel.LocalCovariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j+2,k+2);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k+2);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function variances = LocalCovariance(multiAsset,from,to,params)

            sig(1) = params.domestic_Alpha_r;
            sig(2) = params.domestic_Alpha_s;
            
            sig(3) = params.foreign_Alpha_r;
            sig(4) = params.foreign_Alpha_s;
            
            corr_drds = params.corr_drds;
            corr_drfr = params.corr_drfr;
            corr_dsfr = params.corr_dsfr;
            
            corr_frfs = params.corr_frfs;
            corr_drfs = params.corr_drfs;
            corr_dsfs = params.corr_dsfs;
            
            correl = zeros(4,4);
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
  
           lastU  = from;
           
           variances = zeros(4,4);
           correl =ones(4,4);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end
                if (i< alphaSize) vol(4) = sig(4).quote(i);else vol(4) = sig(4).quote(alphaSize);end
                
                if (i< alphaSize) correl(1,2) = corr_drds.quote(i);else correl(1,2) = corr_drds.quote(alphaSize);end
                if (i< alphaSize) correl(1,3) = corr_drfr.quote(i);else correl(1,3) = corr_drfr.quote(alphaSize);end
                if (i< alphaSize) correl(1,4) = corr_drfs.quote(i);else correl(1,4) = corr_drfs.quote(alphaSize);end
                
                if (i< alphaSize) correl(2,3) = corr_dsfr.quote(i);else correl(2,3) = corr_dsfr.quote(alphaSize);end
                if (i< alphaSize) correl(2,4) = corr_dsfs.quote(i);else correl(2,4) = corr_dsfs.quote(alphaSize);end
                
                if (i< alphaSize) correl(3,4) = corr_frfs.quote(i);else correl(3,4) = corr_frfs.quote(alphaSize);end
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(4,1) = correl(1,4);
                
                correl(3,2) = correl(2,3);
                correl(4,2) = correl(2,4);
                
                correl(4,3) = correl(3,4);
                
                for j=1:4
                    for k=1:4

                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function impCorrel = ComputeImpliedCorrelFwdSwap(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            alpha_1 = zeros(2,1);
            alpha_1(1) = H_interpolation(domesticModel.Alpha1.tenor, domesticModel.Alpha1.quote,tx,0);
            alpha_1(2) = H_interpolation(domesticModel.Alpha2.tenor, domesticModel.Alpha2.quote,tx,0);
            
            alpha_2 = zeros(2,1);
            alpha_2(1) = H_interpolation(foreignModel.Alpha1.tenor, foreignModel.Alpha1.quote,tx,0);
            alpha_2(2) = H_interpolation(foreignModel.Alpha2.tenor, foreignModel.Alpha2.quote,tx,0);
            
            corr = zeros(4,4);
            for i=1:4
                corr(i,i) = 1.0;
            end
            corr(1,3) = H_interpolation(params.corr_drfr.tenor, params.corr_drfr.quote,tx,0);
            corr(1,4) = H_interpolation(params.corr_drfs.tenor, params.corr_drfs.quote,tx,0);
            corr(3,1) = corr(1,3);
            corr(4,1) = corr(1,4);
            
            corr(2,3) = H_interpolation(params.corr_dsfr.tenor, params.corr_dsfr.quote,tx,0);
            corr(2,4) = H_interpolation(params.corr_dsfs.tenor, params.corr_dsfs.quote,tx,0);
            corr(3,2) = corr(2,3);
            corr(4,2) = corr(2,4);
            
%             variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*alpha_1(j)*corr(j,k)*c_nN_1(k)*alpha_1(k);
                    var2 = var2 + c_nN_2(j)*alpha_2(j)*corr(j+2,k+2)*c_nN_2(k)*alpha_2(k);
                    coVar = coVar + c_nN_1(j)*alpha_1(j)*corr(j,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function impCorrel = ComputeImpliedCorrel(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j+2,k+2);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k+2);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function out = ComputeImpliedCorrelMatrix(multiAsset,targetExpiry,tenor,params)
            out = zeros(length(tenor),length(tenor));
            for i=1:length(tenor)
                for j=1:length(tenor)
%                     out(i,j) = multiAsset.ComputeImpliedCorrel(targetExpiry,tenor(i),tenor(j),params);
                    out(i,j) = multiAsset.ComputeImpliedCorrelFwdSwap(targetExpiry,tenor(i),tenor(j),params);
                end
            end
            
        end
        
        function out = ComputeImpliedCorrelCube(multiAsset,targetExpiry,tenor,corr_drfr,corr_dsfr,corr_drfs,corr_dsfs)
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            % synchronize model parameters time-steps start
            corr_drds_Tenor = domesticModel.correl.tenor;
            corr_drds_Quote = domesticModel.correl.quote;
            
            corr_frfs_Tenor = foreignModel.correl.tenor;
            corr_frfs_Quote = foreignModel.correl.quote;
            
            corr_drfr_Tenor = corr_drfr.tenor;
            corr_drfr_Quote = corr_drfr.quote;
            
            corr_dsfr_Tenor = corr_dsfr.tenor;
            corr_dsfr_Quote = corr_dsfr.quote;
            
            corr_drfs_Tenor = corr_drfs.tenor;
            corr_drfs_Quote = corr_drfs.quote;
            
            corr_dsfs_Tenor = corr_dsfs.tenor;
            corr_dsfs_Quote = corr_dsfs.quote;

            domestic_Alpha_r_Tenor = domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = domesticModel.Alpha1.quote;

            domestic_Alpha_s_Tenor = domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = domesticModel.Alpha2.quote;

            foreign_Alpha_r_Tenor = foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = foreignModel.Alpha1.quote;
            
            foreign_Alpha_s_Tenor = foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = foreignModel.Alpha2.quote;
            
            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_frfs_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drfs_Tenor(:);corr_dsfs_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_Alpha_r_Tenor(:)],foreign_Alpha_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            
            corr_drfs_Values = zeros(length(sigmaTimes),1);
            corr_dsfs_Values = zeros(length(sigmaTimes),1);

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);

            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                
                corr_drfs_Values(k) = H_interpolation(corr_drfs_Tenor, corr_drfs_Quote,sigmaTimes(k),0);
                corr_dsfs_Values(k) = H_interpolation(corr_dsfs_Tenor, corr_dsfs_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);

            end
            
            params.corr_drds.tenor = sigmaTimes;
            params.corr_frfs.tenor = sigmaTimes;
            
            params.corr_drfr.tenor = sigmaTimes;
            params.corr_dsfr.tenor = sigmaTimes;
            params.corr_drfs.tenor = sigmaTimes;
            params.corr_dsfs.tenor = sigmaTimes;
            params.domestic_Alpha_r.tenor = sigmaTimes;
            params.domestic_Alpha_s.tenor = sigmaTimes;
            params.foreign_Alpha_r.tenor = sigmaTimes;
            params.foreign_Alpha_s.tenor = sigmaTimes;
            
            params.corr_drds.quote = corr_drds_Values;
            params.corr_frfs.quote = corr_frfs_Values;
            
            params.corr_drfr.quote = corr_drfr_Values;
            params.corr_dsfr.quote = corr_dsfr_Values;
            params.corr_drfs.quote = corr_drfs_Values;
            params.corr_dsfs.quote = corr_dsfs_Values;
            params.domestic_Alpha_r.quote = domestic_Alpha_r_Values;
            params.domestic_Alpha_s.quote = domestic_Alpha_s_Values;
            params.foreign_Alpha_r.quote = foreign_Alpha_r_Values;
            params.foreign_Alpha_s.quote = foreign_Alpha_s_Values;
            
            
            out = zeros(length(targetExpiry),length(tenor),length(tenor));
            
            for i=1:length(targetExpiry)
                correlMatrix = multiAsset.ComputeImpliedCorrelMatrix(targetExpiry(i),tenor,params);
                for j=1:length(tenor)
                    for k=1:length(tenor)
                        out(i,j,k) = correlMatrix(j,k);
                    end
                end
            end
        end
        
        function out = CalibrateToCrossCorrelation(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
            targetExpiryOrig = 1:1:15;
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(targetExpiryOrig),length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                optParams.modelParamsSize = length(targetExpiry)*4;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr, drfs, dsfr, dsfs : 4
                tvar = zeros(optParams.modelParamsSize,1);
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

                options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];

                % observation ,crossCorrelationMatrix
                x = zeros(optParams.numOfEquation,1);
                tic
    %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);

                toc

                tic
                targetfun = @(tvar) TargetFunctionLGM2FLGM2FCrossCorrelationGlobal( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
                out.corr_drfs.tenor = targetExpiry;
                out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = popt(j +   length(targetExpiry));
                    out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
                    out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
                    
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
                temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
                temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                out.corrDiff = TargetFunctionLGM2FLGM2FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                temporalCorr_diff(idx,:,:) = out.corrDiff(1,:,:);
                
            end
            
        end
        
    end
    
end



In [ ]:
classdef MultiAssetModelLGM2FLGM2F < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    % 20181031
    
    properties
        modelNameMap
        stateNumberMap
        correlMap
        
        refModel
        refModelName
        stochasticModelNames
        correlationMatrix
        numOfFactor
        foreignModelName
        payCurrency
        useBrigoCorrYN
    end
    
    methods
        function multiAsset = MultiAssetModelLGM2FLGM2F(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                
                refModelName = modelParams('refModelName');
                multiAsset.refModelName = refModelName;
                multiAsset.refModel = multiAsset.modelNameMap(refModelName);
                
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                multiAsset.correlMap = mktData('correlMap');
                
%                 if isKey(modelParams,'useBrigoCorrYN')
%                     multiAsset.useBrigoCorrYN = modelParams('correlationMatrix');
%                 else
%                     multiAsset.useBrigoCorrYN = 'YES';
%                 end
                multiAsset.useBrigoCorrYN = modelParams('useBrigoCorrYN');
                % LGM2F + LGM2F model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                % beginning index of multiassetModel map
                multiAsset.stateNumberMap = containers.Map;
                
                cumNumOfFactors = 0;
                startIdx = 1;
                for i=1: length(stochasticModelNames)
                    multiAsset.stateNumberMap(stochasticModelNames{i}) = startIdx;
                    numOfFactor = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor;
                    cumNumOfFactors = cumNumOfFactors + numOfFactor;
                    startIdx = startIdx + numOfFactor;
                end
                multiAsset.numOfFactor = cumNumOfFactors;
                
                stochasticModelNames = multiAsset.stochasticModelNames;
                
                % find foreignModel(USD) 
                % we assume that there is only one foreign model
                % need to be updated later
                % multiple foreign Model
                % we only need to care about domestic RefModel and each
                % Foreign Models
                
                for i=1:length(stochasticModelNames)
                    if ~strcmp(stochasticModelNames{i}, refModelName)
                        multiAsset.foreignModelName = stochasticModelNames{i};
                        break;
                    end
                end
                
                quantoLGM2F_LGM2FmodelParams.fxVolCalibrated = false;
                
                % construct USDQuantoLGM2F_LGM2F 
                % from KRWLGM2FRisky and USDLGM2F
                
                quantoLGM2F_LGM2FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                quantoLGM2F_LGM2FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
                
              %% market correlation
                % domestic & foreign
                domestic_foreign_correlMapKey = convertStringsToChars(strcat(refModelName,'_',multiAsset.foreignModelName));
                domestic_foreign_correlMapKey2 = convertStringsToChars(strcat(multiAsset.foreignModelName,'_',refModelName));
                
                if isKey(multiAsset.correlMap,domestic_foreign_correlMapKey)
                    quantoLGM2F_LGM2FmodelParams.corr_df = multiAsset.correlMap(domestic_foreign_correlMapKey);
                elseif isKey(multiAsset.correlMap,domestic_foreign_correlMapKey2)
                    quantoLGM2F_LGM2FmodelParams.corr_df = multiAsset.correlMap(domestic_foreign_correlMapKey);
                else
                    error('no correl in correlMap');
                end
                
                % domestic & fx
                fxAssetName = convertStringsToChars(strcat(refModelName,multiAsset.foreignModelName));
                domestic_fx_correlMapKey = convertStringsToChars(strcat(refModelName,'_',fxAssetName));
                domestic_fx_correlMapKey2 = convertStringsToChars(strcat(fxAssetName,'_',refModelName));
                
                if isKey(multiAsset.correlMap,domestic_fx_correlMapKey)
                    quantoLGM2F_LGM2FmodelParams.corr_dx = multiAsset.correlMap(domestic_fx_correlMapKey);
                elseif isKey(multiAsset.correlMap,domestic_fx_correlMapKey2)
                    quantoLGM2F_LGM2FmodelParams.corr_dx = multiAsset.correlMap(domestic_fx_correlMapKey2);
                else
                    error('no correl in correlMap');
                end
                
                % foreign & fx
                fxAssetName = convertStringsToChars(strcat(refModelName,multiAsset.foreignModelName));
                foreign_fx_correlMapKey = convertStringsToChars(strcat(multiAsset.foreignModelName,'_',fxAssetName));
                foreign_fx_correlMapKey2 = convertStringsToChars(strcat(fxAssetName,'_',multiAsset.foreignModelName));
                
                if isKey(multiAsset.correlMap,foreign_fx_correlMapKey)
                    quantoLGM2F_LGM2FmodelParams.corr_fx = multiAsset.correlMap(foreign_fx_correlMapKey);
                elseif isKey(multiAsset.correlMap,foreign_fx_correlMapKey2)
                    quantoLGM2F_LGM2FmodelParams.corr_fx = multiAsset.correlMap(foreign_fx_correlMapKey2);
                else
                    error('no correl in correlMap');
                end
%                 
%                 %domestic & foreign
%                 quantoLGM2F_LGM2FmodelParams.corr_df = multiAsset.correlationMatrix(1,2);
%                 %domestic & fx
%                 quantoLGM2F_LGM2FmodelParams.corr_dx = multiAsset.correlationMatrix(1,3);
%                 %foreign & fx
%                 quantoLGM2F_LGM2FmodelParams.corr_fx = multiAsset.correlationMatrix(2,3);

               
                
                quantoLGM2F_LGM2FmodelParams.corr_drds = quantoLGM2F_LGM2FmodelParams.domesticModel.correl;
                quantoLGM2F_LGM2FmodelParams.corr_frfs = quantoLGM2F_LGM2FmodelParams.foreignModel.correl;
                
                % model correlation setting to be updated !!
                % Brigo , MarketModel & proxy ..
%                 useBrigoCorrYNYN = 'true';
                
                if strcmp(modelParams('useBrigoCorrYN'),'original') || strcmp(modelParams('useBrigoCorrYN'),'YES')
                    gamma_df = multiAsset.computeBrigo_2F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_df);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_drfs = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs = gamma_df;
                    
                    
                    gamma_dx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_dx);
                    quantoLGM2F_LGM2FmodelParams.corr_drx = gamma_dx;
                    quantoLGM2F_LGM2FmodelParams.corr_dsx = gamma_dx;
                    
                    gamma_fx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_fx);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_frx = gamma_fx;
                    quantoLGM2F_LGM2FmodelParams.corr_fsx = gamma_fx;
                    
                elseif strcmp(modelParams('useBrigoCorrYN'),'adjust')
                    gamma_df = multiAsset.computeBrigo_2F2F_CorrelationAdj(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_df);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr = gamma_df;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.tenor = ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs = gamma_df;
                    
                    % Leave 1Fx2F Correlation invariant!
                    gamma_dx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_dx);
                    quantoLGM2F_LGM2FmodelParams.corr_drx = gamma_dx;
                    quantoLGM2F_LGM2FmodelParams.corr_dsx = gamma_dx;
                    
                    gamma_fx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM2F_LGM2FmodelParams,quantoLGM2F_LGM2FmodelParams.corr_fx);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_frx = gamma_fx;
                    quantoLGM2F_LGM2FmodelParams.corr_fsx = gamma_fx;    
                elseif strcmp(modelParams('useBrigoCorrYN'),'simple_adjust')
                
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_drx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_frx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                    
                    targetCorrel = modelParams('crossCorrelation');
                    quantoLGM2F_LGM2FmodelParams.targetCorrel = targetCorrel(2:size(targetCorrel,2),2:size(targetCorrel,2));
                    quantoLGM2F_LGM2FmodelParams.tenor = targetCorrel(1,2:size(targetCorrel,2))';
                    quantoLGM2F_LGM2FmodelParams.targetExpiry = [5];
                    
                    tvar = zeros(4,1);
                    tvar(1) = quantoLGM2F_LGM2FmodelParams.corr_df;
                    tvar(2) = 0.0;
                    tvar(3) = 0.0;
                    tvar(4) = quantoLGM2F_LGM2FmodelParams.corr_df;
                    
                    corrDiff = TargetFunctionLGM2FLGM2FCrossCorrelationGlobalMatrix(tvar,multiAsset,quantoLGM2F_LGM2FmodelParams);
                    aaa = 1.0;
                
                elseif strcmp(modelParams('useBrigoCorrYN'),'calibrateToHist')
                    
%                     calibOut = multiAsset.CalibrateToCrossCorrelation(modelParams('crossCorrelation'));
                    calibOut = multiAsset.CalibrateToCrossCorrelation2T(modelParams('crossCorrelation'));
%                     calibOut = multiAsset.CalibrateToCrossCorrelationBrigoTrick(modelParams('crossCorrelation'));
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfr = calibOut.corr_drfr;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr = calibOut.corr_dsfr;
                    quantoLGM2F_LGM2FmodelParams.corr_drfs = calibOut.corr_drfs;
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs = calibOut.corr_dsfs;
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_frx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.quote = 0.0*quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                    
                else
                
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.quote = quantoLGM2F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsfs.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_drx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_dsx.quote = quantoLGM2F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM2F_LGM2FmodelParams.corr_frx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.quote = quantoLGM2F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM2F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                end
                %fx Black Vol Setting
                
                % BSCK1FLGM2F calibration needed !
                fxModelName = strcat(multiAsset.foreignModelName,refModelName);
                fxModelName2 = strcat(refModelName,multiAsset.foreignModelName);
                
                if isKey(multiAsset.mktData,fxModelName)
                    fxMktData = multiAsset.mktData(fxModelName);
                elseif isKey(multiAsset.mktData,fxModelName2)
                    fxMktData = multiAsset.mktData(fxModelName2);
                else
                    error('wrong fx asset name!')
                end
                
                fxImpVolRawData = fxMktData('impliedTermVol').params('rawData');

                fxImpVol.tenor = fxImpVolRawData(:,1);
                fxImpVol.quote = fxImpVolRawData(:,2);

                if strcmp(multiAsset.foreignModelName,'KTB')
                    quantoLGM2F_LGM2FmodelParams.fxVolCalibrated = true;
                    fxImpVol.quote = zeros(length(fxImpVol.tenor),1);
                end

                quantoLGM2F_LGM2FmodelParams.fxBlackVol = fxImpVol;
                usdquantoLGM2F_LGM2F = QuantoLGM2F_LGM2F(quantoLGM2F_LGM2FmodelParams);
                multiAsset.modelNameMap(multiAsset.foreignModelName) = usdquantoLGM2F_LGM2F;
                
                
            end
        end
        
        function out = computeBrigo_2F2F_CorrelationAdj(multiAsset,params,corrIn)
            % adjusted version so that 
            % correlation matrix is positive definite
            
                
            %domestic           
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;
            
            %foreign
            
            foreign_dH_r_Tenor = params.foreignModel.dH1.tenor;
            foreign_dH_r_Quote = params.foreignModel.dH1.quote;

            foreign_Alpha_r_Tenor = params.foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = params.foreignModel.Alpha1.quote;

            foreign_dH_s_Tenor = params.foreignModel.dH2.tenor;
            foreign_dH_s_Quote = params.foreignModel.dH2.quote;

            foreign_Alpha_s_Tenor = params.foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = params.foreignModel.Alpha2.quote;
            
            corr_frfs_Tenor = params.foreignModel.correl.tenor;
            corr_frfs_Quote = params.foreignModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);...
                         corr_frfs_Tenor(:);...
                         foreign_Alpha_r_Tenor(:);foreign_Alpha_s_Tenor(:); ...
                         foreign_dH_r_Tenor(:)],foreign_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_dr = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_ds = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                corr_drds = corr_drds_Values(k);
                
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                foreign_dH_s_Values(k) = H_interpolation(foreign_dH_s_Tenor, foreign_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_fr = foreign_dH_r_Values(k)*foreign_Alpha_r_Values(k);
                sigma_fs = foreign_dH_s_Values(k)*foreign_Alpha_s_Values(k);
                corr_frfs = corr_frfs_Values(k);
                
                
                numerator = sigma_dr*sigma_dr + sigma_ds*sigma_ds + 2*corr_drds*sigma_dr*sigma_ds;
                numerator = numerator * (sigma_fr*sigma_fr + sigma_fs*sigma_fs + 2*corr_frfs*sigma_fr*sigma_fs);
                numerator = sqrt(numerator);
                
%                 denominator = sigma_dr +sigma_ds;
%                 denominator = denominator*(sigma_fr +sigma_fs);
                % adjusted version so that correlation matrix is positive
                % definite
                denominator = sigma_dr * sigma_fr +sigma_ds * sigma_fs;
                
                gammaValues(k) = numerator/denominator*corrIn;
                % adjust to be in range -0.99 ~ 0.99
                gammaValues(k) = min(max(gammaValues(k),-0.99),0.99);
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = computeBrigo_2F2F_Correlation(multiAsset,params,corrIn)
            %domestic           
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;
            
            %foreign
            
            foreign_dH_r_Tenor = params.foreignModel.dH1.tenor;
            foreign_dH_r_Quote = params.foreignModel.dH1.quote;

            foreign_Alpha_r_Tenor = params.foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = params.foreignModel.Alpha1.quote;

            foreign_dH_s_Tenor = params.foreignModel.dH2.tenor;
            foreign_dH_s_Quote = params.foreignModel.dH2.quote;

            foreign_Alpha_s_Tenor = params.foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = params.foreignModel.Alpha2.quote;
            
            corr_frfs_Tenor = params.foreignModel.correl.tenor;
            corr_frfs_Quote = params.foreignModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:);domestic_dH_s_Tenor(:);...
                         corr_frfs_Tenor(:);...
                         foreign_Alpha_r_Tenor(:);foreign_Alpha_s_Tenor(:); ...
                         foreign_dH_r_Tenor(:)],foreign_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
            foreign_dH_r_Values = zeros(length(sigmaTimes),1);
            foreign_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_dr = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_ds = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                corr_drds = corr_drds_Values(k);
                
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_dH_r_Values(k) = H_interpolation(foreign_dH_r_Tenor, foreign_dH_r_Quote,sigmaTimes(k),0);
                foreign_dH_s_Values(k) = H_interpolation(foreign_dH_s_Tenor, foreign_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_fr = foreign_dH_r_Values(k)*foreign_Alpha_r_Values(k);
                sigma_fs = foreign_dH_s_Values(k)*foreign_Alpha_s_Values(k);
                corr_frfs = corr_frfs_Values(k);
                
                
                numerator = sigma_dr*sigma_dr + sigma_ds*sigma_ds + 2*corr_drds*sigma_dr*sigma_ds;
                numerator = numerator * (sigma_fr*sigma_fr + sigma_fs*sigma_fs + 2*corr_frfs*sigma_fr*sigma_fs);
                numerator = sqrt(numerator);
                
                denominator = sigma_dr +sigma_ds;
                denominator = denominator*(sigma_fr +sigma_fs);
                
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = computeBrigo_1F2F_CorrelationAdj(multiAsset,params,corrIn)
            % adjusted version so that 
            % correlation matrix is positive definite           
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
%                 denominator = sigma_r +sigma_s;
                denominator = sigma_r;
                
                gammaValues(k) = numerator/denominator*corrIn;
                % adjust to be in range -0.99 ~ 0.99
                gammaValues(k) = min(max(gammaValues(k),-0.99),0.99);
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = computeBrigo_1F2F_Correlation(multiAsset,params,corrIn)
                       
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
                denominator = sigma_r +sigma_s;
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(multiAsset,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(multiAsset,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(multiAsset,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            out1 = SimpleSRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper);
            out2 = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralowerS,raupperS);
%             out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
            outVec = out1*out2;
            out = outVec;
        end
        
        function out = SimpleDRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
%                     outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                    outVec(i) = multiAsset.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = DFRisky(multiAsset,t)
            out = multiAsset.refModel.DFRisky(t);
        end
        
        function out = stateLocalDrift(multiAsset,fromTime, toTime)
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            out = foreignModel.LocalDrift(fromTime/365.0,toTime/365.0);
        end
        
        function out = stateLocalVariance(multiAsset,fromTime, toTime)
            % we need to generalize this hard coding !!
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            % we use quantoLGM1F_LGM2F to calculate localcovariance term
            % is there any better solution ?  :(
            
            out = foreignModel.LocalCovariance(fromTime/365.0,toTime/365.0);
        end
        
        function out =  Rate_Index(multiAsset,rateType,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT,PSAOption)
            if strcmp(PSAOption,'PSAAll')
                out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            elseif strcmp(PSAOption,'PSA')
                if strcmp(rateType,'CMS')
                    out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
            else
                if strcmp(rateType,'CMS')
                    out = CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
                
            end
        end
        
        function out =  Libor_SimpleDate(multiAsset,curveName,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3:4,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3:4,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3:4,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        
        function out = discountPayoffRisky(multiAsset,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            % for multiAsset discounting is done by refModel
            dfNumMat = multiAsset.refModel.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
            referenceModelStatesT = modelStatesT(1:2,:);
            
            discountT = multiAsset.refModel.discountFactorRisky(eventTime, numMatTime,numMatTime,referenceModelStatesT);
            
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountFactorRisky(multiAsset, startTime, endTime,numMatTime,modelStatesT)
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
           referenceModelStatesT = modelStatesT(1:2,:);
           out = multiAsset.refModel.discountFactorRisky(startTime, endTime,numMatTime,referenceModelStatesT);
           
        end
        
        function out = CMSCMTSpreadDigitalArgDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2)
           % probability that CMS tenor1 - CMT tenor 2  < strike
           %CMS 1
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
%            tau1 = 1.0/(domesticModel.freq);
           % hard coding for LGM2F for the time-being
           tau1 = 0.25;
           
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau1);
           
           cms_fwd   = cmsDigitalArg.fwdCMS;
           cms_mu    = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_var   = cms_sigma * cms_sigma;
           cms_c_nN  = cmsDigitalArg.c_nN;
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           cmtDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor2,tau2);
           
           cmt_fwd   = cmtDigitalArg.fwdCMS;
           cmt_mu    = cmtDigitalArg.mu;
           cmt_sigma = cmtDigitalArg.sigma;
           cmt_var   = cmt_sigma * cmt_sigma;
           cmt_c_nN  = cmtDigitalArg.c_nN;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (4,4) matrix (KRW dr, KRW ds, USD fr USD fs) as (1,2,3,4)
           coVar = foreignModel.LocalCovariance(0,to);
           
           %cms: KRW
           %cmt :USD
           cmscmt_covar =                cms_c_nN(1) * cmt_c_nN(1) * coVar(1,3);
           cmscmt_covar = cmscmt_covar + cms_c_nN(2) * cmt_c_nN(1) * coVar(2,3);
           
           cmscmt_covar = cmscmt_covar + cms_c_nN(1) * cmt_c_nN(2) * coVar(1,4);
           cmscmt_covar = cmscmt_covar + cms_c_nN(2) * cmt_c_nN(2) * coVar(2,4);
           
           spread_var = cms_var + cmt_var - 2.0 * cmscmt_covar;
           
           out.fwdSpread = cms_fwd - cmt_fwd; 
           out.mu = cms_mu - cmt_mu;
           out.sigma = sqrt(spread_var);
        end
        
        function out = CMSCMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,strike)
           % probability that CMS tenor1 - CMT tenor 2 < strike
           %CMS 1
           cmsCmtSpreadDigitalArg = multiAsset.CMSCMTSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2);
           mu = cmsCmtSpreadDigitalArg.mu;
           sigma = cmsCmtSpreadDigitalArg.sigma;
           z1= (strike - mu)/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSCMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,lower,upper)
            
            lower_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,lower);
            upper_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,upper);
%             out = lower_value - upper_value;
            out = upper_value - lower_value;
        end
        
        function out = CMSCMSSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor > strike
           %CMS
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau1 = 1.0/(foreignModel.freq);
           cmsDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau1);
           
           cms_mu = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_c_nN = cmsDigitalArg.c_nN;
           
           
           %CMS Spread
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           cmsSpreadDigitalArg = domesticModel.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           cmsSpread_c_nN1 = cmsSpreadDigitalArg.c_nN1;
           cmsSpread_c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           cmsSpread_mu = cmsSpreadDigitalArg.mu;
           cmsSpread_sigma = cmsSpreadDigitalArg.sigma;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3,4)
           varCovar = foreignModel.LocalCovariance(0,to);
           
           % covariance between USD CMS Rate & KRW CMS Spread
           %cms: USD
           %cmtSpread :KRW CMS 1 - KRW CMS 2
           cms_cmsSpread_covar = 0.0;
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,3);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,3);
           
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(2) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,4);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(2) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,4);
           
           corr = cms_cmsSpread_covar/(cms_sigma*cmsSpread_sigma);
           
           z1 = (cms_mu       - strike)/cms_sigma;
           z2 = (cmsSpread_mu - strikeS)/cmsSpread_sigma;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end

        function out = CMSRACMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor > strike
           % domestic CMS
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           % hard coding for LGM2F for the time-being
           tau1 = 0.25;
           tau1 = tau;
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau1);
           
           cms_mu = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_c_nN = cmsDigitalArg.c_nN;
           
           
           % foreign CMS Spread
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           
           cmsSpreadDigitalArg = foreignModel.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau2);
           
           cmsSpread_c_nN1 = cmsSpreadDigitalArg.c_nN1;
           cmsSpread_c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           cmsSpread_mu = cmsSpreadDigitalArg.mu;
           cmsSpread_sigma = cmsSpreadDigitalArg.sigma;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, KTB fr KTB fs) as (1,2,3,4)
           varCovar = foreignModel.LocalCovariance(0,to);
           
           % covariance between KRW CMS Rate & KTB CMS Spread
           %cms: KRW
           %cmtSpread :KTB CMS 1 - KTB CMS 2
           cms_cmsSpread_covar = 0.0;
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,3);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,3);
           
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(2) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,4);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(2) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,4);
           
           corr = cms_cmsSpread_covar/(cms_sigma*cmsSpread_sigma);
           
           z1 = (cms_mu       - strike)/cms_sigma;
           z2 = (cmsSpread_mu - strikeS)/cmsSpread_sigma;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end

        function out = CMSRACMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = multiAsset.CMSRACMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = multiAsset.CMSRACMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = multiAsset.CMSRACMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = multiAsset.CMSRACMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function impCorrel = ImpCorrel(multiAsset,tx,tnr1,tnr2)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = foreignModel.LocalCovariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j+2,k+2);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k+2);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function variances = LocalCovariance(multiAsset,from,to,params)

            sig(1) = params.domestic_Alpha_r;
            sig(2) = params.domestic_Alpha_s;
            
            sig(3) = params.foreign_Alpha_r;
            sig(4) = params.foreign_Alpha_s;
            
            corr_drds = params.corr_drds;
            corr_drfr = params.corr_drfr;
            corr_dsfr = params.corr_dsfr;
            
            corr_frfs = params.corr_frfs;
            corr_drfs = params.corr_drfs;
            corr_dsfs = params.corr_dsfs;
            
            correl = zeros(4,4);
            
            alphaSize = length(sig(1).quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
  
           lastU  = from;
           
           variances = zeros(4,4);
           correl =ones(4,4);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end
                if (i< alphaSize) vol(4) = sig(4).quote(i);else vol(4) = sig(4).quote(alphaSize);end
                
                if (i< alphaSize) correl(1,2) = corr_drds.quote(i);else correl(1,2) = corr_drds.quote(alphaSize);end
                if (i< alphaSize) correl(1,3) = corr_drfr.quote(i);else correl(1,3) = corr_drfr.quote(alphaSize);end
                if (i< alphaSize) correl(1,4) = corr_drfs.quote(i);else correl(1,4) = corr_drfs.quote(alphaSize);end
                
                if (i< alphaSize) correl(2,3) = corr_dsfr.quote(i);else correl(2,3) = corr_dsfr.quote(alphaSize);end
                if (i< alphaSize) correl(2,4) = corr_dsfs.quote(i);else correl(2,4) = corr_dsfs.quote(alphaSize);end
                
                if (i< alphaSize) correl(3,4) = corr_frfs.quote(i);else correl(3,4) = corr_frfs.quote(alphaSize);end
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(4,1) = correl(1,4);
                
                correl(3,2) = correl(2,3);
                correl(4,2) = correl(2,4);
                
                correl(4,3) = correl(3,4);
                
                for j=1:4
                    for k=1:4

                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function impCorrel = ComputeImpliedCorrelFwdSwap(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            alpha_1 = zeros(2,1);
            alpha_1(1) = H_interpolation(domesticModel.Alpha1.tenor, domesticModel.Alpha1.quote,tx,0);
            alpha_1(2) = H_interpolation(domesticModel.Alpha2.tenor, domesticModel.Alpha2.quote,tx,0);
            
            alpha_2 = zeros(2,1);
            alpha_2(1) = H_interpolation(foreignModel.Alpha1.tenor, foreignModel.Alpha1.quote,tx,0);
            alpha_2(2) = H_interpolation(foreignModel.Alpha2.tenor, foreignModel.Alpha2.quote,tx,0);
            
            corr = zeros(4,4);
            for i=1:4
                corr(i,i) = 1.0;
            end
            corr(1,3) = H_interpolation(params.corr_drfr.tenor, params.corr_drfr.quote,tx,0);
            corr(1,4) = H_interpolation(params.corr_drfs.tenor, params.corr_drfs.quote,tx,0);
            corr(3,1) = corr(1,3);
            corr(4,1) = corr(1,4);
            
            corr(2,3) = H_interpolation(params.corr_dsfr.tenor, params.corr_dsfr.quote,tx,0);
            corr(2,4) = H_interpolation(params.corr_dsfs.tenor, params.corr_dsfs.quote,tx,0);
            corr(3,2) = corr(2,3);
            corr(4,2) = corr(2,4);
            
%             variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*alpha_1(j)*corr(j,k)*c_nN_1(k)*alpha_1(k);
                    var2 = var2 + c_nN_2(j)*alpha_2(j)*corr(j+2,k+2)*c_nN_2(k)*alpha_2(k);
                    coVar = coVar + c_nN_1(j)*alpha_1(j)*corr(j,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function impCorrel = ComputeImpliedCorrel(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j+2,k+2);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k+2);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function out = ComputeImpliedCorrelMatrix(multiAsset,targetExpiry,tenor,params)
            out = zeros(length(tenor),length(tenor));
            for i=1:length(tenor)
                for j=1:length(tenor)
%                     out(i,j) = multiAsset.ComputeImpliedCorrel(targetExpiry,tenor(i),tenor(j),params);
                    out(i,j) = multiAsset.ComputeImpliedCorrelFwdSwap(targetExpiry,tenor(i),tenor(j),params);
                end
            end
            
        end
        
        function out = ComputeImpliedCorrelCube(multiAsset,targetExpiry,tenor,corr_drfr,corr_dsfr,corr_drfs,corr_dsfs)
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            % synchronize model parameters time-steps start
            corr_drds_Tenor = domesticModel.correl.tenor;
            corr_drds_Quote = domesticModel.correl.quote;
            
            corr_frfs_Tenor = foreignModel.correl.tenor;
            corr_frfs_Quote = foreignModel.correl.quote;
            
            corr_drfr_Tenor = corr_drfr.tenor;
            corr_drfr_Quote = corr_drfr.quote;
            
            corr_dsfr_Tenor = corr_dsfr.tenor;
            corr_dsfr_Quote = corr_dsfr.quote;
            
            corr_drfs_Tenor = corr_drfs.tenor;
            corr_drfs_Quote = corr_drfs.quote;
            
            corr_dsfs_Tenor = corr_dsfs.tenor;
            corr_dsfs_Quote = corr_dsfs.quote;

            domestic_Alpha_r_Tenor = domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = domesticModel.Alpha1.quote;

            domestic_Alpha_s_Tenor = domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = domesticModel.Alpha2.quote;

            foreign_Alpha_r_Tenor = foreignModel.Alpha1.tenor;
            foreign_Alpha_r_Quote = foreignModel.Alpha1.quote;
            
            foreign_Alpha_s_Tenor = foreignModel.Alpha2.tenor;
            foreign_Alpha_s_Quote = foreignModel.Alpha2.quote;
            
            sigmaTimes = unique(union([corr_drds_Tenor(:);corr_frfs_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:);...
                         corr_drfs_Tenor(:);corr_dsfs_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:);foreign_Alpha_r_Tenor(:)],foreign_Alpha_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            corr_frfs_Values = zeros(length(sigmaTimes),1);
            
            corr_drfr_Values = zeros(length(sigmaTimes),1);
            corr_dsfr_Values = zeros(length(sigmaTimes),1);
            
            corr_drfs_Values = zeros(length(sigmaTimes),1);
            corr_dsfs_Values = zeros(length(sigmaTimes),1);

            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            
            foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
            foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);

            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
                
                corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
                corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
                
                corr_drfs_Values(k) = H_interpolation(corr_drfs_Tenor, corr_drfs_Quote,sigmaTimes(k),0);
                corr_dsfs_Values(k) = H_interpolation(corr_dsfs_Tenor, corr_dsfs_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
                foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);

            end
            
            params.corr_drds.tenor = sigmaTimes;
            params.corr_frfs.tenor = sigmaTimes;
            
            params.corr_drfr.tenor = sigmaTimes;
            params.corr_dsfr.tenor = sigmaTimes;
            params.corr_drfs.tenor = sigmaTimes;
            params.corr_dsfs.tenor = sigmaTimes;
            params.domestic_Alpha_r.tenor = sigmaTimes;
            params.domestic_Alpha_s.tenor = sigmaTimes;
            params.foreign_Alpha_r.tenor = sigmaTimes;
            params.foreign_Alpha_s.tenor = sigmaTimes;
            
            params.corr_drds.quote = corr_drds_Values;
            params.corr_frfs.quote = corr_frfs_Values;
            
            params.corr_drfr.quote = corr_drfr_Values;
            params.corr_dsfr.quote = corr_dsfr_Values;
            params.corr_drfs.quote = corr_drfs_Values;
            params.corr_dsfs.quote = corr_dsfs_Values;
            params.domestic_Alpha_r.quote = domestic_Alpha_r_Values;
            params.domestic_Alpha_s.quote = domestic_Alpha_s_Values;
            params.foreign_Alpha_r.quote = foreign_Alpha_r_Values;
            params.foreign_Alpha_s.quote = foreign_Alpha_s_Values;
            
            
            out = zeros(length(targetExpiry),length(tenor),length(tenor));
            
            for i=1:length(targetExpiry)
                correlMatrix = multiAsset.ComputeImpliedCorrelMatrix(targetExpiry(i),tenor,params);
                for j=1:length(tenor)
                    for k=1:length(tenor)
                        out(i,j,k) = correlMatrix(j,k);
                    end
                end
            end
        end
        
        function out = CalibrateToCrossCorrelationBrigoTrick(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
            targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                optParams.modelParamsSize = length(targetExpiry)*1;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr, drfs, dsfr, dsfs : 4
                tvar = zeros(optParams.modelParamsSize,1);
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

                options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];

                % observation ,crossCorrelationMatrix
                x = zeros(optParams.numOfEquation,1);
                tic
    %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);

                toc

                tic
%                 targetfun = @(tvar) TargetFunctionLGM2FLGM2FCrossCorrelationGlobal( tvar,multiAsset,optParams);
                targetfun = @(tvar) TargetFunctionLGM2FLGM2FCrossCorrelationGlobalBrigoTrick( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
                out.corr_drfs.tenor = targetExpiry;
                out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = 0.0;
                    out.corr_drfs.quote(j) = 0.0;
                    out.corr_dsfs.quote(j) = popt(j);
                    
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
                temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
                temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                tvar2 = zeros(4,1);
                
                tvar2(1) = popt(1);
                tvar2(2) = 0.0;
                tvar2(3) = 0.0;
                tvar2(4) = popt(1);
                
                out.corrDiff = TargetFunctionLGM2FLGM2FCrossCorrelationGlobalMatrix(tvar2,multiAsset,optParams);
                temporalCorr_diff = out.corrDiff;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelationOrig(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
            targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                optParams.modelParamsSize = length(targetExpiry)*4;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr, drfs, dsfr, dsfs : 4
                tvar = zeros(optParams.modelParamsSize,1);
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

                options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];

                % observation ,crossCorrelationMatrix
                x = zeros(optParams.numOfEquation,1);
                tic
    %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);

                toc

                tic
                targetfun = @(tvar) TargetFunctionLGM2FLGM2FCrossCorrelationGlobal( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
                out.corr_drfs.tenor = targetExpiry;
                out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = popt(j +   length(targetExpiry));
                    out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
                    out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
                    
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
                temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
                temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                out.corrDiff = TargetFunctionLGM2FLGM2FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                temporalCorr_diff = out.corrDiff;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelation(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
            targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                optParams.modelParamsSize = length(targetExpiry)*4;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr, drfs, dsfr, dsfs : 4
                tvar = zeros(optParams.modelParamsSize,1);
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

                options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];

                % observation ,crossCorrelationMatrix
                x = zeros(optParams.numOfEquation,1);
                tic
    %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);

                toc

                tic
                targetfun = @(tvar) TargetFunctionLGM2FLGM2FCrossCorrelationGlobal( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
                out.corr_drfs.tenor = targetExpiry;
                out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = popt(j +   length(targetExpiry));
                    out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
                    out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
                    
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
                temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
                temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                out.corrDiff = TargetFunctionLGM2FLGM2FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                temporalCorr_diff = out.corrDiff;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelation2T(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
            targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [5;10];
            
            temporalCorr_drfr = zeros(1);
            temporalCorr_drfs = zeros(1);
            temporalCorr_dsfr = zeros(1);
            temporalCorr_dsfs = zeros(1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            
            targetExpiry = targetExpiryOrig;
            
            optParams.tenor = tenor;
            optParams.targetCorrel = targetCorrel;
            optParams.targetExpiry = targetExpiry;
%                 optParams.modelParamsSize = length(targetExpiry)*4;
            optParams.modelParamsSize = 4;
            optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
            % drfr, drfs, dsfr, dsfs : 4
            tvar = zeros(optParams.modelParamsSize,1);
            for i=1:length(tvar)
                tvar(i) = 0.8;
            end

            options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];

            % observation ,crossCorrelationMatrix
            x = zeros(optParams.numOfEquation,1);
            tic
%             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);

            toc

            tic
            targetfun = @(tvar) TargetFunctionLGM2FLGM2FCrossCorrelationGlobalSimple( tvar,multiAsset,optParams);
            options=optimset('Algorithm','interior-point');
            lb = -0.99*ones(optParams.modelParamsSize,1);
            ub = +0.99*ones(optParams.modelParamsSize,1);

            [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
            toc

            out.corr_drfr.tenor = targetExpiry(1);
            out.corr_dsfr.tenor = targetExpiry(1);
            out.corr_drfs.tenor = targetExpiry(1);
            out.corr_dsfs.tenor = targetExpiry(1);

            out.corr_drfr.quote(1) = popt(1);
            out.corr_dsfr.quote(1) = popt(2);
            out.corr_drfs.quote(1) = popt(3);
            out.corr_dsfs.quote(1) = popt(4);

%                 for j=1:length(targetExpiry)
%                     out.corr_drfr.quote(j) = popt(j);
%                     out.corr_dsfr.quote(j) = popt(j +   length(targetExpiry));
%                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
%                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
%                     
%                 end

            temporalCorr_drfr(1) = out.corr_drfr.quote(1);
            temporalCorr_dsfr(1) = out.corr_dsfr.quote(1);
            temporalCorr_drfs(1) = out.corr_drfs.quote(1);
            temporalCorr_dsfs(1) = out.corr_dsfs.quote(1);

            out.corrDiff = TargetFunctionLGM2FLGM2FCrossCorrelationGlobalMatrix2T(popt,multiAsset,optParams);
            temporalCorr_diff = out.corrDiff;
                
        end
        
    end
    
end



In [ ]:
classdef MultiAssetModelLGM2FLGM1F < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        stateNumberMap
        refModel
        refModelName
        stochasticModelNames
        correlationMatrix
        numOfFactor
        foreignModelName
        payCurrency
    end
    
    methods
        function multiAsset = MultiAssetModelLGM2FLGM1F(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                
                refModelName = modelParams('refModelName');
                multiAsset.refModelName = refModelName;
                multiAsset.refModel = multiAsset.modelNameMap(refModelName);
                
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                % LGM2F + LGM1F model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                % beginning index of multiassetModel map
                multiAsset.stateNumberMap = containers.Map;
                
                cumNumOfFactors = 0;
                startIdx = 1;
                for i=1: length(stochasticModelNames)
                    multiAsset.stateNumberMap(stochasticModelNames{i}) = startIdx;
                    numOfFactor = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor;
                    cumNumOfFactors = cumNumOfFactors + numOfFactor;
                    startIdx = startIdx + numOfFactor;
                end
                multiAsset.numOfFactor = cumNumOfFactors;
                
                stochasticModelNames = multiAsset.stochasticModelNames;
                
                % find foreignModel(USD) 
                % we assume that there is only one foreign model
                % need to be updated later
                % multiple foreign Model
                % we only need to care about domestic RefModel and each
                % Foreign Models
                
                for i=1:length(stochasticModelNames)
                    if ~strcmp(stochasticModelNames{i}, refModelName)
                        multiAsset.foreignModelName = stochasticModelNames{i};
                        break;
                    end
                end
                
                quantoLGM1F_LGM2FmodelParams.fxVolCalibrated = false;
                
                % construct USDQuantoLGM1F_LGM2F 
                % from KRWLGM2FRisky and USDLGM1F
                
                quantoLGM1F_LGM2FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                quantoLGM1F_LGM2FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
                
                % market correlation
                %domestic & foreign
                quantoLGM1F_LGM2FmodelParams.corr_df = multiAsset.correlationMatrix(1,2);
                %domestic & fx
                quantoLGM1F_LGM2FmodelParams.corr_dx = multiAsset.correlationMatrix(1,3);
                %foreign & fx
                quantoLGM1F_LGM2FmodelParams.corr_fx = multiAsset.correlationMatrix(2,3);

               
                
                quantoLGM1F_LGM2FmodelParams.corr_drds = quantoLGM1F_LGM2FmodelParams.domesticModel.correl;
                
                % model correlation setting to be updated !!
                % Brigo , MarketModel & proxy ..
                useBrigoCorrYN = 'YES';
                
                if strcmp(useBrigoCorrYN,'YES')
                    gamma_df = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM1F_LGM2FmodelParams,quantoLGM1F_LGM2FmodelParams.corr_df);
                    quantoLGM1F_LGM2FmodelParams.corr_drfr = gamma_df;
                    quantoLGM1F_LGM2FmodelParams.corr_dsfr = gamma_df;
                    
                    gamma_dx = multiAsset.computeBrigo_1F2F_Correlation(quantoLGM1F_LGM2FmodelParams,quantoLGM1F_LGM2FmodelParams.corr_dx);
                    quantoLGM1F_LGM2FmodelParams.corr_drx = gamma_dx;
                    quantoLGM1F_LGM2FmodelParams.corr_dsx = gamma_dx;
                    
                    quantoLGM1F_LGM2FmodelParams.corr_frx.quote = quantoLGM1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                else
                
                    quantoLGM1F_LGM2FmodelParams.corr_drfr.quote = quantoLGM1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM1F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoLGM1F_LGM2FmodelParams.corr_dsfr.quote = quantoLGM1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoLGM1F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);

                    quantoLGM1F_LGM2FmodelParams.corr_drx.quote = quantoLGM1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM1F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoLGM1F_LGM2FmodelParams.corr_dsx.quote = quantoLGM1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoLGM1F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoLGM1F_LGM2FmodelParams.corr_frx.quote = quantoLGM1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoLGM1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                end
                %fx Black Vol Setting
                
                % BSCK1FLGM2F calibration needed !
                fxModelName = strcat(multiAsset.foreignModelName,refModelName);
                fxMktData = multiAsset.mktData(fxModelName);
                fxImpVolRawData = fxMktData('impliedTermVol').params('rawData');

                fxImpVol.tenor = fxImpVolRawData(:,1);
                fxImpVol.quote = fxImpVolRawData(:,2);

                quantoLGM1F_LGM2FmodelParams.fxBlackVol = fxImpVol;
                usdquantoLGM1F_LGM2F = QuantoLGM1F_LGM2F(quantoLGM1F_LGM2FmodelParams);
                multiAsset.modelNameMap(multiAsset.foreignModelName) = usdquantoLGM1F_LGM2F;
                
                
            end
        end
        
        function out = computeBrigo_1F2F_Correlation(multiAsset,params,corrIn)
                       
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
                denominator = sigma_r +sigma_s;
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(multiAsset,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(multiAsset,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(multiAsset,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            out1 = SimpleSRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper);
            out2 = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralowerS,raupperS);
%             out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
            outVec = out1*out2;
            out = outVec;
        end
        
        function out = SimpleDRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
%                     outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                    outVec(i) = multiAsset.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = DFRisky(multiAsset,t)
            out = multiAsset.refModel.DFRisky(t);
        end
        
        function out = stateLocalDrift(multiAsset,fromTime, toTime)
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            out = foreignModel.LocalDrift(fromTime/365.0,toTime/365.0);
        end
        
        function out = stateLocalVariance(multiAsset,fromTime, toTime)
            % we need to generalize this hard coding !!
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            % we use quantoLGM1F_LGM2F to calculate localcovariance term
            % is there any better solution ?  :(
            
            out = foreignModel.LocalCovariance(fromTime/365.0,toTime/365.0);
        end
        
        function out =  Libor_SimpleDate(multiAsset,curveName,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        
        function out = discountPayoffRisky(multiAsset,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            % for multiAsset discounting is done by refModel
            dfNumMat = multiAsset.refModel.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
            referenceModelStatesT = modelStatesT(1:2,:);
            
            discountT = multiAsset.refModel.discountFactorRisky(eventTime, numMatTime,numMatTime,referenceModelStatesT);
            
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountFactorRisky(multiAsset, startTime, endTime,numMatTime,modelStatesT)
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
           referenceModelStatesT = modelStatesT(1:2,:);
           out = multiAsset.refModel.discountFactorRisky(startTime, endTime,numMatTime,referenceModelStatesT);
           
        end
        
        function out = CMSCMTSpreadDigitalArgDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2)
           % probability that CMS tenor1 - CMT tenor 2  < strike
           %CMS 1
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
%            tau1 = 1.0/(domesticModel.freq);
           % hard coding for LGM2F for the time-being
           tau1 = 0.25;
           
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau1);
           
           cms_fwd   = cmsDigitalArg.fwdCMS;
           cms_mu    = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_var   = cms_sigma * cms_sigma;
           cms_c_nN  = cmsDigitalArg.c_nN;
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           cmtDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor2,tau2);
           
           cmt_fwd   = cmtDigitalArg.fwdCMS;
           cmt_mu    = cmtDigitalArg.mu;
           cmt_sigma = cmtDigitalArg.sigma;
           cmt_var   = cmt_sigma * cmt_sigma;
           cmt_c_nN  = cmtDigitalArg.c_nN;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3)
           coVar = foreignModel.LocalCovariance(0,to);
           
           %cms: KRW
           %cmt :USD
           cmscmt_covar =                cms_c_nN(1) * cmt_c_nN(1) * coVar(1,3);
           cmscmt_covar = cmscmt_covar + cms_c_nN(2) * cmt_c_nN(1) * coVar(2,3); 
           spread_var = cms_var + cmt_var - 2.0 * cmscmt_covar;
           
           out.fwdSpread = cms_fwd - cmt_fwd; 
           out.mu = cms_mu - cmt_mu;
           out.sigma = sqrt(spread_var);
        end
        
        function out = CMSCMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,strike)
           % probability that CMS tenor1 - CMT tenor 2 < strike
           %CMS 1
           cmsCmtSpreadDigitalArg = multiAsset.CMSCMTSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2);
           mu = cmsCmtSpreadDigitalArg.mu;
           sigma = cmsCmtSpreadDigitalArg.sigma;
           z1= (strike - mu)/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSCMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,lower,upper)
            
            lower_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,lower);
            upper_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,upper);
%             out = lower_value - upper_value;
            out = upper_value - lower_value;
        end
        
        function out = CMSCMSSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor > strike
           %CMS
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau1 = 1.0/(foreignModel.freq);
           cmsDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau1);
           
           cms_mu = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_c_nN = cmsDigitalArg.c_nN;
           
           
           %CMS Spread
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           cmsSpreadDigitalArg = domesticModel.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           cmsSpread_c_nN1 = cmsSpreadDigitalArg.c_nN1;
           cmsSpread_c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           cmsSpread_mu = cmsSpreadDigitalArg.mu;
           cmsSpread_sigma = cmsSpreadDigitalArg.sigma;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3)
           varCovar = foreignModel.LocalCovariance(0,to);
           
           % covariance between USD CMS Rate & KRW CMS Spread
           %cms: USD
           %cmtSpread :KRW CMS 1 - KRW CMS 2
           cms_cmsSpread_covar = 0.0;
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,3);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,3);
           
           corr = cms_cmsSpread_covar/(cms_sigma*cmsSpread_sigma);
           
           z1 = (cms_mu       - strike)/cms_sigma;
           z2 = (cmsSpread_mu - strikeS)/cmsSpread_sigma;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
    end
    
end



In [ ]:
classdef MultiAssetModelLGM2FEQDupire < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        stateNumberMap
        refModel
        refModelName
        stochasticModelNames
        correlationMatrix
        numOfFactor
%         foreignModelName
        eqModelName
        
        payCurrency
        useBrigoCorrYN
    end
    
    methods
        function multiAsset = MultiAssetModelLGM2FEQDupire(mktData, modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                multiAsset.modelNameMap = modelParams('modelNameMap');
                
                refModelName = modelParams('refModelName');
                multiAsset.refModelName = refModelName;
                multiAsset.refModel = multiAsset.modelNameMap(refModelName);
                
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                multiAsset.useBrigoCorrYN = 'NO';
                if isKey(modelParams,'useBrigoCorrYN')
                    multiAsset.useBrigoCorrYN = modelParams('useBrigoCorrYN');
                else
                    multiAsset.useBrigoCorrYN = 'NO';
                end
                
                
                % CK1F + EQDupire model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                % beginning index of multiassetModel map
                multiAsset.stateNumberMap = containers.Map;
                
                cumNumOfFactors = 0;
                startIdx = 1;
                for i=1: length(stochasticModelNames)
                    multiAsset.stateNumberMap(stochasticModelNames{i}) = startIdx;
                    childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                    
                    numOfFactors = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactors;
                    
%                     if isfield(childModel,'numOfFactor')
%                         numOfFactors = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor;
%                     else
%                         numOfFactors = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactors;
%                     end
                    
                    cumNumOfFactors = cumNumOfFactors + numOfFactors;
                    startIdx = startIdx + numOfFactors;
                end
                multiAsset.numOfFactor = cumNumOfFactors;
                
                stochasticModelNames = multiAsset.stochasticModelNames;
                
                % find eqModel(USD) 
                % we assume that there is only one foreign model
                % need to be updated later
                % multiple foreign Model
                % we only need to care about domestic RefModel and each
                % Foreign Models
                
                for i=1:length(stochasticModelNames)
                    if ~strcmp(stochasticModelNames{i}, refModelName)
                        multiAsset.eqModelName = stochasticModelNames{i};
                        break;
                    end
                end
                
                if isKey(modelParams,'eqVolCalibrated')
                    eqDuire_CK1FmodelParams.eqVolCalibrated = modelParams('eqVolCalibrated');
                else
                    eqDuire_CK1FmodelParams.eqVolCalibrated = false;
                end
                
                % construct USDQuantoCK1F_CK1F
                % from KRWCK1FRisky and USDCK1F
                
                eqDuire_CK1FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                eqDuire_CK1FmodelParams.eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
                
                % market correlation
                %domestic & foreign
                eqDuire_CK1FmodelParams.corr_de = multiAsset.correlationMatrix(1,2);
                
                
                
                %% added at 20201030
%                 eqDuire_CK1FmodelParams.domestic_meanR_r = eqDuire_CK1FmodelParams.domesticModel.meanR_r; 
%                 eqDuire_CK1FmodelParams.domestic_vol_r = eqDuire_CK1FmodelParams.domesticModel.vol_r;
                
                
                % model correlation setting to be updated !!
                % Brigo , MarketModel & proxy ..
                % useBrigoCorrYN = 'NO';
                useBrigoCorrYN = multiAsset.useBrigoCorrYN;
                
                if strcmp(modelParams('useBrigoCorrYN'),'original') || strcmp(modelParams('useBrigoCorrYN'),'YES')
                    gamma_de = multiAsset.computeBrigo_1F2F_Correlation(eqDuire_CK1FmodelParams,eqDuire_CK1FmodelParams.corr_de);
                    eqDuire_CK1FmodelParams.corr_dre = gamma_de;
                    eqDuire_CK1FmodelParams.corr_dse = gamma_de;

                elseif strcmp(useBrigoCorrYN,'simple_adjust')
                    eqDuire_CK1FmodelParams.corr_dre.quote = eqDuire_CK1FmodelParams.corr_de * ones(1,1);
                    eqDuire_CK1FmodelParams.corr_dre.tenor = ones(1,1);
                    
                    eqDuire_CK1FmodelParams.corr_dse.quote = eqDuire_CK1FmodelParams.corr_de * ones(1,1);
                    eqDuire_CK1FmodelParams.corr_dse.tenor = ones(1,1);
                        
                else                
                    eqDuire_CK1FmodelParams.corr_dre.quote = eqDuire_CK1FmodelParams.corr_de * ones(1,1);
                    eqDuire_CK1FmodelParams.corr_dre.tenor = ones(1,1);
                    
                    eqDuire_CK1FmodelParams.corr_dse.quote = eqDuire_CK1FmodelParams.corr_de * ones(1,1);
                    eqDuire_CK1FmodelParams.corr_dse.tenor = ones(1,1);
                end
                
                
                %eq Black Vol Setting
                % BSCK1F calibration needed !
%                 fxModelName = strcat(multiAsset.foreignModelName, refModelName);
%                 fxModelName2 = strcat(refModelName,multiAsset. foreignModelName);
%                 
                if isKey(multiAsset.mktData, multiAsset.eqModelName)
                    eqMktData = multiAsset.mktData(multiAsset.eqModelName);
                else
                    error('wrong eq asset name!')
                end
                
%                 eqImpVolRawData = eqMktData('impliedVolSurface').params('rawData');
% 
% %                 eqImpVol.tenor = eqImpVolRawData(2:end,1);
%                 eqImpVol.tenor = eqImpVolRawData(2:end,1)/365.0;
%                 eqImpVol.quote = eqImpVolRawData(2:end,2);
%                 eqDuire_CK1FmodelParams.eqBlackVol = eqImpVol;
                
                
                eqLocalVolSurface = eqDuire_CK1FmodelParams.eqModel.localVolSurface;
                eqLocalVolSurface.tenor = eqLocalVolSurface.expiry/365.0; 
                eqLocalVolSurface.eqLocalVolValuesMat = eqLocalVolSurface.localVol;
                
                eqDuire_CK1FmodelParams.eqLocalVolSurface = eqLocalVolSurface;
                
                hybridLocalVolSurface.localVol = eqLocalVolSurface.localVol;
                hybridLocalVolSurface.timeStep = eqLocalVolSurface.expiry;
                hybridLocalVolSurface.strike = eqLocalVolSurface.Ks;
                
                eqDuire_CK1FmodelParams.hybridLocalVolSurface = hybridLocalVolSurface;
                
                % eventTimeStep info from securityInfo
                securityInfo = modelParams('securityInfo');
                eventDates = securityInfo.params('eventDates');
                valueDateH = securityInfo.params('valueDateH');
                
                eventDays = zeros(length(eventDates),1);
                for idx=1:length(eventDates)
                    eventDays(idx) = DateDiff(eventDates{idx},valueDateH);
                end
                
                eqDuire_CK1FmodelParams.eventDays = eventDays;
%                 eqDupire_CK1F = EQDupire_CK1F(eqDuire_CK1FmodelParams);
                tic
                eqDupire_CK1F = EQDupire_LGM2F(eqDuire_CK1FmodelParams);
                toc
                disp('hybridVol Caculation Finished')
                
                multiAsset.modelNameMap(multiAsset.eqModelName) = eqDupire_CK1F;
                
            end
            
        end
        
        function out = computeBrigo_1F2F_Correlation(multiAsset,params,corrIn)
                       
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
                denominator = sigma_r +sigma_s;
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralower,raupper,hedgeBuffer)
            buffer = hedgeBuffer;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(multiAsset,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(multiAsset,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(multiAsset,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,hedgeBuffer)
            outVec = 0.0;
            out1 = SimpleSRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper,hedgeBuffer);
            out2 = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralowerS,raupperS,hedgeBuffer);
%             out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
            outVec = out1*out2;
            out = outVec;
        end
        
        function out = SimpleDRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,hedgeBuffer,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
%                     outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                    outVec(i) = multiAsset.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS,hedgeBuffer);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = DFRisky(multiAsset,t)
            out = multiAsset.refModel.DFRisky(t);
        end
        
        function out = stateLocalDrift(multiAsset, fromTime, toTime)
            eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
            out = eqModel.LocalDrift(fromTime/365.0,toTime/365.0);
        end
        
%         function out = stateLocalVariance(multiAsset, fromTime, toTime,numMatTime,lastX,dZ)
%             % we need to generalize this hard coding !!
%             eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
%             % we use eqDupire_ck1f(hybrid model) to calculate localcovariance term
%             % is there any better solution ?  :(
%             
%             out = eqModel.LocalCovariance(fromTime/365.0,toTime/365.0,numMatTime/365.0,lastX,dZ);
%         end
        
       function out = computeNextMCModelState(multiAsset, fromTime, toTime,numMatTime,lastX,dZ)
            % we need to generalize this hard coding !!
            eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
            % we use eqDupire_ck1f(hybrid model) to calculate localcovariance term
            % is there any better solution ?  :(
            
            out = eqModel.computeNextMCModelState(fromTime/365.0,toTime/365.0,numMatTime/365.0,lastX,dZ);
       end
        
%        function out = computeForwardStartingVanillaMC(multiAsset,valueDate,vanillaParams)
%             % model schedule generation
%             expiryDate = H_Date(vanillaParams.params('maturityDate'));
%             startDate  = H_Date(vanillaParams.params('startDate'));
%             
%             scheduleSize = 2;
%             % resetDate
%             % expiryDate
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             daysToReset = DateDiff(startDate,valueDate);
%             
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
% %             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%             eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
%             
%             % we use hybridLocalVolSurface's timeStep as pricing timeStep
%             
%             timeStep = eqModel.hybridLocalVolSurface.timeStep;
%             timeStep = sort(union(timeStep,0),'ascend');
%             
% %             expiry = round(365.0*eqModel.eqBlackVol.tenor);
% %             expiredChance = 0;
% %             aliveChance = 0;
% % %             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
% %             volExpiry = expiry(find(expiry <= maturity));
% %             % we include valueDate in the scheduel
% %             volExpiry =[volExpiry;0];
% %             
% %             timeStep = volExpiry;
% %             
% %             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
% %                 dailyTimeSteps = [maturity:-1:0]';
% %                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
% %             else
% %                 timeStepP = multiAsset.modelParams('mcOneTimeStep');
% %                 dailyTimeSteps = [maturity:-timeStepP:0]';
% %                 dailyTimeSteps = [dailyTimeSteps;0];
% %                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
% %             end
% %             
% %             timeStep = sort(union(timeStep,daysToReset),'ascend');
%             
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             lastAliveExerciseIdx = 1;
%             
%             eventTimeIdx(1) = find(timeStep == daysToReset);
%             eventTimeIdx(2) = find(timeStep == maturity);
%             
%              % numeraire info
%             numMatTime = timeStep(end);
%             dfNumMat = multiAsset.DFRisky(numMatTime/365.0);
% 
%             numeraire.numMatTime = numMatTime;
%             numeraire.dfNumMat = dfNumMat;
%                 
% %             for i= scheduleSize:-1:lastAliveExerciseIdx
% %                 idx = find(timeStep == maturity);
% %                 eventTimeIdx(i) = min(idx);
% %             end
%             
%             %% MC init 
%             rng('default');
%             rng(0);
%              
%             numOfFactors = multiAsset.numOfFactor;
% 
%             NMC = eqModel.modelParams('NMC'); 
%             MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,numOfFactors*MCTimeStep);
%             Z=[Z;-Z];
%             Z=Z';
% 
%             %TimeInversion
%             sizeZ = size(Z,1);
%             A = zeros(sizeZ,sizeZ);
%             for i=1:sizeZ
%                 A(i,sizeZ + 1 -i) = 1;
%             end
%             Z = A*Z;
% 
% %             U = Z';
%             U = Z;
% 
%            
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            % IR 1 Factor & EQ 1 Factor & 1 Auxillary Factor
%            %(drift as in cheyette model)
%            %  initX = zeros(2,NMC);
%            initX = zeros(3,NMC);
%            
%            
%            nextX = initX;
%            %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            relativeStrike =  vanillaParams.params('relativeStrike');
%            
%            detFwdReset    = eqModel.FwdFactor(daysToReset);
%            detFwdMaturity = eqModel.FwdFactor(maturity);
%            
%            fwdRatio = detFwdMaturity/detFwdReset;
%            
% %            relativeStrike =  [0.5:0.1:1.5];
%            
%            reSetS = zeros(1,NMC);      
% %            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            modelStatesSize = NMC;
%            
%            %europeanValue
%            europeanValue = PricerInfo(1, modelStatesSize);
%            europeanValueUnd = PricerInfo(1, modelStatesSize);
%            nMFuture      = PricerInfo(1, modelStatesSize);
%            
%            
%            
% %            europeanValue.npv = 0;
% %            europeanValue.payoff = zeros(1,modelStatesSize);
% %            europeanValue.payoffStates.cashflow = zeros(1,modelStatesSize);
% %            europeanValue.payoffStates.cashflow_npv = zeros(1,modelStatesSize);
%            
%            %americanValue
% %            americanValue.npv = 0;
% %            americanValue.payoff = zeros(1,modelStatesSize);
% %            americanValue.payoffStates.cashflow = zeros(1,modelStatesSize);
% %            americanValue.payoffStates.cashflow_npv = zeros(1,modelStatesSize);
%            
%            
%            % dummy nearest month future price
% %            nMFuture.npv = 0;
% %            nMFuture.payoff = zeros(1,modelStatesSize);
% %            nMFuture.payoffStates.cashflow = zeros(1,modelStatesSize);
% %            nMFuture.payoffStates.cashflow_npv = zeros(1,modelStatesSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(1,modelStatesSize);
%            payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
%            
%            payoffStateB.cashflow = zeros(1,modelStatesSize);
%            payoffStateB.cashflow_npv = zeros(1,modelStatesSize);
%            
%            payoffStateC.cashflow = zeros(1,modelStatesSize);
%            payoffStateC.cashflow_npv = zeros(1,modelStatesSize);
%            
% %            currentTime = timeStep(totalTimeStepSize);
% %            currentTimeIdx = totalTimeStepSize;
% %            currentNodeIdx = scheduleSize;
%            
%             for i=lastAliveExerciseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if i== 1
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 else
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 end
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = multiAsset.inductMCForwardNew(timeStep(idx),timeStep(idx+1),numMatTime,nextX,U(numOfFactors*(idx-1)+1:numOfFactors*(idx-1)+numOfFactors,:));
%                     
%                     nextX = mcmcOut.nextX;
%                     % one step forward induction end
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                 end
%                 
%                 % at startDate, we reset strike
%                 if i==1
% %                     comFwd = multiAsset.EQFwdMC(currentTime,nextX);
%                     comFwd = multiAsset.EQSpotMC(multiAsset.eqModelName,valueDate,currentTime, numMatTime, nextX);
%                     
%                     reSetS = comFwd;
%                     
%                 end
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%            
% %            comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
%            comFwd = multiAsset.EQSpotMC(multiAsset.eqModelName,valueDate,currentTime, numMatTime, nextX);
%            df_payTime = multiAsset.DFRisky(currentTime/365.0);
%            if strcmp(vanillaParams.params('callPutFlag'),'C')
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
%                     payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
%                end
%            else
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
%                     payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            %process payoff at maturity
%            europeanValue.ProcessPayoff(multiAsset,currentTime,numeraire,nextX,payoffStateA.cashflow,currentNodeIdx);
%            europeanValueUnd.ProcessPayoff(multiAsset,currentTime,numeraire,nextX,payoffStateB.cashflow,currentNodeIdx);
%            nMFuture.ProcessPayoff(multiAsset,currentTime,numeraire,nextX,payoffStateC.cashflow,currentNodeIdx);
%            
% % %            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
% %            df_ep = 1.0;
% %            
% %            % we don't discount forward price
% %            for i=1:modelStatesSize
% %                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
% %                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
% %                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
% %            end
% %            
% %            
% %            %process payoff at maturity
% %             europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
% %             europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
% %             europeanValue.payoff = payoffStateA.cashflow_npv;
% %             
% %             americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
% %             americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
% %             americanValue.payoff = payoffStateB.cashflow_npv;
% %             
% %             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
% %             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
% %             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            europeanValue.npv = mean(europeanValue.payoff);
%            europeanValueUnd.npv = mean(europeanValueUnd.payoff)/df_payTime;
%            nMFuture.npv = mean(nMFuture.payoff);
%             
%            out.europeanValue = europeanValue; 
%            out.europeanValueUnd = europeanValueUnd;
%            out.nMFuture = nMFuture;
%            out.fwdRatio = fwdRatio;
%            
%         end
        
        function out =  Rate_Index(multiAsset,rateType,curveName,valueDate,observeTime,numMatTime,tenor,modelStatesT,PSAOption)
            if strcmp(PSAOption,'PSAAll')
                out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            elseif strcmp(PSAOption,'PSA')
                if strcmp(rateType,'CMS')
                    out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
            else
                if strcmp(rateType,'CMS')
                    out = CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
                
            end
        end
        
        function out =  Libor_SimpleDate(multiAsset,curveName,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(1:2,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               error('aaa')
%                foreignModelStatesT = modelStatesT(2,:);
%                foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
%                out = foreignModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  EQSpotMC(multiAsset,curveName,valueDate,evalTime, numMatTime, modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.eqModelName)
               irModelStatesT = modelStatesT(1:2,:);
               eqModelStatesT = modelStatesT(3,:);
               auxillaryStateT = modelStatesT(4,:);
               
               eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
               out = eqModel.EQSpotMC(valueDate,evalTime,numMatTime,irModelStatesT,eqModelStatesT,auxillaryStateT);
           else
               
           end
            
        end
        
        function out =  CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        
        function out = discountPayoffRisky(multiAsset,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            % for multiAsset discounting is done by refModel
            dfNumMat = multiAsset.refModel.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
            referenceModelStatesT = modelStatesT(1:2,:);
            
            discountT = multiAsset.refModel.discountFactorRisky(eventTime, numMatTime,numMatTime,referenceModelStatesT);
            
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountFactorRisky(multiAsset, startTime, endTime,numMatTime,modelStatesT)
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
           referenceModelStatesT = modelStatesT(1:2,:);
           out = multiAsset.refModel.discountFactorRisky(startTime, endTime,numMatTime,referenceModelStatesT);
           
        end
        
        function out = CMSCMTSpreadDigitalArgDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2)
           % probability that CMS tenor1 - CMT tenor 2  < strike
           %CMS 1
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
%            tau1 = 1.0/(domesticModel.freq);
           % hard coding for LGM2F for the time-being
           tau1 = 0.25;
           
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau1);
           
           cms_fwd   = cmsDigitalArg.fwdCMS;
           cms_mu    = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_var   = cms_sigma * cms_sigma;
           cms_c_nN  = cmsDigitalArg.c_nN;
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           cmtDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor2,tau2);
           
           cmt_fwd   = cmtDigitalArg.fwdCMS;
           cmt_mu    = cmtDigitalArg.mu;
           cmt_sigma = cmtDigitalArg.sigma;
           cmt_var   = cmt_sigma * cmt_sigma;
           cmt_c_nN  = cmtDigitalArg.c_nN;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (2,2) matrix (KRW dr, USD fr) as (1,2)
           coVar = foreignModel.LocalCovariance(0,to);
           
           %cms: KRW
           %cmt :USD
           cmscmt_covar = cms_c_nN(1) * cmt_c_nN(1) * coVar(1,2);
           spread_var = cms_var + cmt_var - 2.0 * cmscmt_covar;
           
           out.cms_fwd = cms_fwd;
           out.cmt_fwd = cmt_fwd;
           out.fwdSpread = cms_fwd - cmt_fwd; 
           out.mu = cms_mu - cmt_mu;
           out.sigma = sqrt(spread_var);
        end
        
        function out = CMSCMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,strike)
           % probability that CMS tenor1 - CMT tenor 2 < strike
           %CMS 1
           cmsCmtSpreadDigitalArg = multiAsset.CMSCMTSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2);
           mu = cmsCmtSpreadDigitalArg.mu;
           sigma = cmsCmtSpreadDigitalArg.sigma;
           
           out.cms_fwd = cmsCmtSpreadDigitalArg.cms_fwd;
           out.cmt_fwd = cmsCmtSpreadDigitalArg.cmt_fwd;
           out.fwdSpread = cmsCmtSpreadDigitalArg.fwdSpread;
           out.mu = cmsCmtSpreadDigitalArg.mu;
           out.sigma = cmsCmtSpreadDigitalArg.sigma;
           
           z1= (strike - mu)/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out.value = 0.0;
           else
               out.value = H_ncdf(z1);
           end
           
        end
        
        function out = CMSCMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,lower,upper)
            
            lowerOut = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,lower);
            upperOut = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,upper);
%             out = lower_value - upper_value;
            out.value = upperOut.value - lowerOut.value;
            out.cms_fwd = lowerOut.cms_fwd;
            out.cmt_fwd = lowerOut.cmt_fwd;
            out.fwdSpread = lowerOut.fwdSpread;
            out.mu = lowerOut.mu;
            out.sigma = lowerOut.sigma;
        end
        
        function out = CMSCMSSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor > strike
           %CMS
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau1 = 1.0/(foreignModel.freq);
           cmsDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau1);
           
           cms_mu = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_c_nN = cmsDigitalArg.c_nN;
           
           
           %CMS Spread
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           cmsSpreadDigitalArg = domesticModel.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           cmsSpread_c_nN1 = cmsSpreadDigitalArg.c_nN1;
           cmsSpread_c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           cmsSpread_mu = cmsSpreadDigitalArg.mu;
           cmsSpread_sigma = cmsSpreadDigitalArg.sigma;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3)
           varCovar = foreignModel.LocalCovariance(0,to);
           
           % covariance between USD CMS Rate & KRW CMS Spread
           %cms: USD
           %cmtSpread :KRW CMS 1 - KRW CMS 2
           cms_cmsSpread_covar = 0.0;
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,3);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,3);
           
           corr = cms_cmsSpread_covar/(cms_sigma*cmsSpread_sigma);
           
           z1 = (cms_mu       - strike)/cms_sigma;
           z2 = (cmsSpread_mu - strikeS)/cmsSpread_sigma;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
           % to be checked later 
           if isnan(out)
               disp('bivariate cumulative normal pdf returns NAN!')
               out = 0.0;
           end
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = computeForwardStartingVanillaMC(multiAsset,valueDate,vanillaParams)
            % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            startDate  = H_Date(vanillaParams.params('startDate'));
            
            scheduleSize = 2;
            % resetDate
            % expiryDate
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            daysToReset = DateDiff(startDate,valueDate);
            
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
            
            % we use hybridLocalVolSurface's timeStep as pricing timeStep
            
            timeStep = eqModel.hybridLocalVolSurface.timeStep;
            timeStep = sort(union(timeStep,0),'ascend');
            
%             expiry = round(365.0*eqModel.eqBlackVol.tenor);
%             expiredChance = 0;
%             aliveChance = 0;
% %             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = multiAsset.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
%             
%             timeStep = sort(union(timeStep,daysToReset),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            
            eventTimeIdx(1) = find(timeStep == daysToReset);
            eventTimeIdx(2) = find(timeStep == maturity);
            
             % numeraire info
            numMatTime = timeStep(end);
            dfNumMat = multiAsset.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
                
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
            
            %% MC init 
            rng('default');
            rng(0);
             
            numOfFactors = multiAsset.numOfFactor;

            NMC = eqModel.modelParams('NMC'); 
            MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%             U = Z';
            U = Z;

           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           % IR 2 Factor & EQ 1 Factor & 1 Auxillary Factor
           %(drift as in cheyette model)
           %  initX = zeros(2,NMC);
%            initX = zeros(3,NMC);
           initX = zeros(4,NMC);
           
           
           nextX = initX;
           %% grid generation based on basePrice
            
           
          %% payoff at maturity
           relativeStrike =  vanillaParams.params('relativeStrike');
           
           detFwdReset    = eqModel.FwdFactor(daysToReset);
           detFwdMaturity = eqModel.FwdFactor(maturity);
           
           fwdRatio = detFwdMaturity/detFwdReset;
           
%            relativeStrike =  [0.5:0.1:1.5];
           
           reSetS = zeros(1,NMC);      
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
           modelStatesSize = NMC;
           
           %europeanValue
           europeanValue = PricerInfo(1, modelStatesSize);
           europeanValueUnd = PricerInfo(1, modelStatesSize);
           nMFuture      = PricerInfo(1, modelStatesSize);
           
           
           
%            europeanValue.npv = 0;
%            europeanValue.payoff = zeros(1,modelStatesSize);
%            europeanValue.payoffStates.cashflow = zeros(1,modelStatesSize);
%            europeanValue.payoffStates.cashflow_npv = zeros(1,modelStatesSize);
           
           %americanValue
%            americanValue.npv = 0;
%            americanValue.payoff = zeros(1,modelStatesSize);
%            americanValue.payoffStates.cashflow = zeros(1,modelStatesSize);
%            americanValue.payoffStates.cashflow_npv = zeros(1,modelStatesSize);
           
           
           % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(1,modelStatesSize);
%            nMFuture.payoffStates.cashflow = zeros(1,modelStatesSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(1,modelStatesSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateB.cashflow = zeros(1,modelStatesSize);
           payoffStateB.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateC.cashflow = zeros(1,modelStatesSize);
           payoffStateC.cashflow_npv = zeros(1,modelStatesSize);
           
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
           
            for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                if i== 1
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                else
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                end
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = multiAsset.inductMCForwardNew(timeStep(idx),timeStep(idx+1),numMatTime,nextX,U(numOfFactors*(idx-1)+1:numOfFactors*(idx-1)+numOfFactors,:));
                    
                    nextX = mcmcOut.nextX;
                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                end
                
                % at startDate, we reset strike
                if i==1
%                     comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    comFwd = multiAsset.EQSpotMC(multiAsset.eqModelName,valueDate,currentTime, numMatTime, nextX);
                    
                    reSetS = comFwd;
                    
                end
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
%            comFwd = eqCOMDupireSpotGF.EQFwdMC(currentTime,nextX);
           comFwd = multiAsset.EQSpotMC(multiAsset.eqModelName,valueDate,currentTime, numMatTime, nextX);
           df_payTime = multiAsset.DFRisky(currentTime/365.0);
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(comFwd(i)/reSetS(i) - relativeStrike*fwdRatio,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i)/reSetS(i) + relativeStrike*fwdRatio,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           %process payoff at maturity
           europeanValue.ProcessPayoff(multiAsset,currentTime,numeraire,nextX,payoffStateA.cashflow,currentNodeIdx);
           europeanValueUnd.ProcessPayoff(multiAsset,currentTime,numeraire,nextX,payoffStateB.cashflow,currentNodeIdx);
           nMFuture.ProcessPayoff(multiAsset,currentTime,numeraire,nextX,payoffStateC.cashflow,currentNodeIdx);
           
% %            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            df_ep = 1.0;
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             europeanValue.payoff = payoffStateA.cashflow_npv;
%             
%             americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             americanValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValueUnd.npv = mean(europeanValueUnd.payoff)/df_payTime;
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.europeanValueUnd = europeanValueUnd;
           out.nMFuture = nMFuture;
           out.fwdRatio = fwdRatio;
        end
        
        function out = computeStepdown1SMCEQ(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
            
            % we use hybridLocalVolSurface's timeStep as pricing timeStep
            
            timeStep = eqModel.hybridLocalVolSurface.timeStep;
            timeStep = sort(union(timeStep,0),'ascend');
            
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
%             timeStep = sort(union(timeStep,maturity),'ascend');
            
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
            
             % numeraire info
            numMatTime = timeStep(end);
            dfNumMat = multiAsset.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
           
          %% MCMC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
%             useQuasiRandomYN = 'true';
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
%                 NMC = 2^16;
                NMC = multiAsset.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = multiAsset.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                numOfFactors = multiAsset.numOfFactor;
                NMC = eqModel.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,numOfFactors*MCTimeStep);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                U = Z;

            end
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
%            % initial modelState
%            initX = ones(NMC,1);
           
            % initial modelState
           % IR 1 Factor & EQ 1 Factor & 1 Auxillary Factor
           %(drift as in cheyette model)
           %  initX = zeros(2,NMC);
%            initX = zeros(3,NMC);
            % IR 2Factor & EQ 1 Factor & 1 Auxillary Factor
           initX = zeros(4,NMC);
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           
           %PricerInfo init
           
           hitValue   = PricerInfo(1, modelStatesSize);
           unhitValue = PricerInfo(1, modelStatesSize);
           nMFuture   = PricerInfo(1, modelStatesSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateB.cashflow = zeros(1,modelStatesSize);
           payoffStateB.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateC.cashflow = zeros(1,modelStatesSize);
           payoffStateC.cashflow_npv = zeros(1,modelStatesSize);
           
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(scheduleSize,modelStatesSize);
           worstP = zeros(scheduleSize,modelStatesSize);
           
           if KIYN == true
               isKI = ones(1,modelStatesSize);
           else
               isKI = zeros(1,modelStatesSize);
           end
           
           % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
           comFwd = multiAsset.EQSpotMC(multiAsset.eqModelName,valueDate,currentTime, numMatTime, nextX);
                    
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
                    mcmcOut   = multiAsset.inductMCForwardNew(timeStep(idx),timeStep(idx+1),numMatTime,nextX,U(numOfFactors*(idx-1)+1:numOfFactors*(idx-1)+numOfFactors,:));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    
%                     comFwd = eqCOMDupireSpotGF.EQSpotMC(currentTime,nextX);
                    comFwd = multiAsset.EQSpotMC(multiAsset.eqModelName,valueDate,currentTime, numMatTime, nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
%                 dummyisKI = ones(modelStatesSize,1);
%                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
%                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
                comFwdWorst = comFwd/payoffInfo.basePrice; 
%                 columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,numeraire,nextX,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.cashflow(currentNodeIdx,:) = comFwd;
                nMFuture.cashflow_npv(currentNodeIdx,:) = comFwd ;
                nMFuture.payoff = comFwd;
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.cashflow_npv(currentNodeIdx,:);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(1,modelStatesSize);
                    contiValue = zeros(1,modelStatesSize);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoff = ones(1,modelStatesSize);
                    onePayoffD = multiAsset.discountPayoffRisky(currentTime,numMatTime ...
                    ,nextX,onePayoff);
                
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(i-1,idx) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD(idx);
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
%                     decisionVar1 = worstP(:,i-1);
                    decisionVar1 = worstP(i-1,:);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(i-1,idx) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(i-1,idx) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(i-1,idx) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(i-1,idx) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(i-1,idx) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD(idx);
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function columnOut = MCPayoffWorst(multiAsset,idx,comFwdWorst,currentTime,numeraire,nextX, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(scheduleSize,modelStatesSize);
            columnOut.worstP = zeros(scheduleSize,modelStatesSize);
%             
%             columnOut.isStop = zeros(modelStatesSize,scheduleSize);
%             columnOut.worstP = zeros(modelStatesSize,scheduleSize);
%             
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn = PricerInfo(1, modelStatesSize);
            
%             columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
%             columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%             columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
            payoffA = zeros(1,modelStatesSize);
            
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            onePayoff = ones(1,modelStatesSize);
            df_ep     = multiAsset.discountPayoffRisky(currentTime,numeraire.numMatTime ...
                    ,nextX,onePayoff);
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx,idx11) = 1;
                           columnOut.worstP(idx,idx11) = comFwdWorst(idx11);
%                            payoffA(idx11) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.cashflow(idx,idx11) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.cashflow_npv(idx,idx11) = nominal*(1.0 + coupon)*df_ep(idx11);
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.cashflow_npv(idx,idx11);
                        end
                        
%                         columnOut.worstP(idx,idx11) = comFwdWorst(idx11);
                    
                end
                
%                 columnOut.payoffColumn.ProcessAssignPayoff(multiAsset,currentTime,numeraire,nextX,payoffA,idx);
           
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx-1,idx11) == 1 % terminated path
                        columnOut.isStop(idx,idx11) = 1;
                        columnOut.worstP(idx,idx11) = columnOut.worstP(idx-1,idx11);
                        columnOut.payoffColumn.cashflow(idx,idx11) = columnOut.payoffColumn.cashflow(idx-1,idx11); %cached cashflow
                        columnOut.payoffColumn.cashflow_npv(idx,idx11) = columnOut.payoffColumn.cashflow_npv(idx-1,idx11); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx,idx11) = 1;
                            columnOut.worstP(idx,idx11) = comFwdWorst(idx11);
%                             payoffA(idx11) = nominal*(1.0 + coupon);
                            
                            columnOut.payoffColumn.cashflow(idx,idx11) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.cashflow_npv(idx,idx11) = nominal*(1.0 + coupon)*df_ep(idx11);
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.cashflow_npv(idx,idx11);
                        end
                        
%                         columnOut.worstP(idx,idx11) = comFwdWorst(idx11);
                        
                    end
                    
                end
                
%                 columnOut.payoffColumn.ProcessAssignPayoff(multiAsset,currentTime,numeraire,nextX,payoffA,idx);
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx-1,idx11) == 1 % terminated path
                        columnOut.isStop(idx,idx11) = 1;
                        columnOut.worstP(idx,idx11) = columnOut.worstP(idx-1,idx11);
                        columnOut.payoffColumn.cashflow(idx,idx11) = columnOut.payoffColumn.cashflow(idx-1,idx11); %cached cashflow
                        columnOut.payoffColumn.cashflow_npv(idx,idx11) = columnOut.payoffColumn.cashflow_npv(idx-1,idx11); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx,idx11) = 1;
                        columnOut.worstP(idx,idx11) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1
%                                 payoffA(idx11) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
%                                 payoffA(idx11) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1;
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
%                                 payoffA(idx11) = nominal * (1.0 + coupon);
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1
%                                 payoffA(idx11) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                                
                            elseif comFwdWorst(idx11) < strike
%                                 payoffA(idx11) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
%                                 payoffA(idx11) = nominal * (1.0 + coupon);
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
%                                 payoffA(idx11) = nominal * (1.0 + coupon);
                                columnOut.payoffColumn.cashflow(idx,idx11) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
%                            columnOut.payoffColumn.ProcessAssignPayoff(multiAsset,currentTime,numeraire,nextX,payoffA,idx);
                        columnOut.payoffColumn.cashflow_npv(idx,idx11) = columnOut.payoffColumn.cashflow(idx,idx11)*df_ep(idx11);
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.cashflow_npv(idx,idx11);
                    end    
                end
            end    
            
            
        end
        
        function out = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            modelStatesSize = length(comFwd);
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
                    out(idx) = 1.0;
                end
            end
        end
        
        function mcOut = inductMCForwardNew(multiAsset,fromTime,toTime,numMatTime,lastX,dZ)
            
              nextX = multiAsset.computeNextMCModelState(fromTime,toTime,numMatTime,lastX,dZ);
              mcOut.nextX = nextX;  
            
%             stateLocalVariance = multiAsset.stateLocalVariance(fromTime,toTime,numMatTime,lastX,dZ);
%             stateLocalDrift = multiAsset.stateLocalDrift(fromTime,toTime);
%             stateAbsoluteDrift = multiAsset.stateAbsoluteDrift(fromTime,toTime);
%             L = chol(stateLocalVariance,'lower');
%             prevModelStates = lastX;
%             modelStates = zeros(size(lastX));
%             modelStates = stateLocalDrift * prevModelStates + L*dZ; 
%             mcOut.nextX = modelStates;   

        end
        
        function out = ImpliedBlackVol(multiAsset,price,fwd,strike,expiry,vanillayOptionType)
            eqModel = multiAsset.modelNameMap(multiAsset.eqModelName);
            out = eqModel.ImpliedBlackVol(price,fwd,strike,expiry,vanillayOptionType);
        
        end
%         
%         function mcmcOut = inductMCMCForwardNew(multiAsset,fromTime,toTime,nextX,nextXIdx,dZ)
%             
%             predictor = nextX;
%             predictorIdx = nextXIdx;
%             numOfFactors = multiAsset.numOfFactors;
%             stochasticModelNames = multiAsset.stochasticModelNames;
%             
%             for i=1:length(stochasticModelNames)
%                 startIdx = multiAsset.stateNumberMap(stochasticModelNames{i});
%                 endIdx = startIdx + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor - 1;
%                 
%                 mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCMCForwardNew(fromTime,toTime,lastX(:,startIdx:endIdx),dZ(:,startIdx:endIdx));
%                 predictor(:,startIdx:endIdx) = mcOut1.nextX;
%                 predictorIdx(:,startIdx:endIdx) = mcOut1.nextXIdx;
%             end
%             
%             mcmcOut.nextX = predictor;
%             mcmcOut.nextXIdx = predictorIdx;
%             
%         end
        
    end
    
end



In [ ]:
classdef MultiAssetModelLGM2FCK1F < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        stateNumberMap
        refModel
        refModelName
        stochasticModelNames
        correlationMatrix
        numOfFactors
        foreignModelName
        payCurrency
        useBrigoCorrYN
    end
    
    methods
        function multiAsset = MultiAssetModelLGM2FCK1F(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                
                refModelName = modelParams('refModelName');
                multiAsset.refModelName = refModelName;
                multiAsset.refModel = multiAsset.modelNameMap(refModelName);
                
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                if isKey(modelParams,'useBrigoCorrYN')
                    multiAsset.useBrigoCorrYN = modelParams('useBrigoCorrYN');
                else
                    multiAsset.useBrigoCorrYN = 'NO';
                end
                % LGM2F + CK1F model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                % beginning index of multiassetModel map
                multiAsset.stateNumberMap = containers.Map;
                
                cumNumOfFactors = 0;
                startIdx = 1;
                for i=1: length(stochasticModelNames)
                    multiAsset.stateNumberMap(stochasticModelNames{i}) = startIdx;
                    numOfFactors = multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactors;
                    cumNumOfFactors = cumNumOfFactors + numOfFactors;
                    startIdx = startIdx + numOfFactors;
                end
                multiAsset.numOfFactors = cumNumOfFactors;
                
                stochasticModelNames = multiAsset.stochasticModelNames;
                
                % find foreignModel(USD) 
                % we assume that there is only one foreign model
                % need to be updated later
                % multiple foreign Model
                % we only need to care about domestic RefModel and each
                % Foreign Models
                
                for i=1:length(stochasticModelNames)
                    if ~strcmp(stochasticModelNames{i}, refModelName)
                        multiAsset.foreignModelName = stochasticModelNames{i};
                        break;
                    end
                end
                
                quantoCK1F_LGM2FmodelParams.fxVolCalibrated = false;
                
                % construct USDQuantoCK1F_LGM2F 
                % from KRWLGM2FRisky and USDCK1F
                
                quantoCK1F_LGM2FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                quantoCK1F_LGM2FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
                
                % market correlation
                %domestic & foreign
                quantoCK1F_LGM2FmodelParams.corr_df = multiAsset.correlationMatrix(1,2);
                %domestic & fx
                quantoCK1F_LGM2FmodelParams.corr_dx = multiAsset.correlationMatrix(1,3);
                %foreign & fx
                quantoCK1F_LGM2FmodelParams.corr_fx = multiAsset.correlationMatrix(2,3);

               
                
                quantoCK1F_LGM2FmodelParams.corr_drds = quantoCK1F_LGM2FmodelParams.domesticModel.correl;
                
                % model correlation setting to be updated !!
                % Brigo , MarketModel & proxy ..
%                 useBrigoCorrYN = 'YES';
%                 useBrigoCorrYN = 'NO';
                useBrigoCorrYN = multiAsset.useBrigoCorrYN;
                
                if strcmp(useBrigoCorrYN,'YES')
                    gamma_df = multiAsset.computeBrigo_1F2F_Correlation(quantoCK1F_LGM2FmodelParams,quantoCK1F_LGM2FmodelParams.corr_df);
                    quantoCK1F_LGM2FmodelParams.corr_drfr = gamma_df;
                    quantoCK1F_LGM2FmodelParams.corr_dsfr = gamma_df;
                    
                    gamma_dx = multiAsset.computeBrigo_1F2F_Correlation(quantoCK1F_LGM2FmodelParams,quantoCK1F_LGM2FmodelParams.corr_dx);
                    quantoCK1F_LGM2FmodelParams.corr_drx = gamma_dx;
                    quantoCK1F_LGM2FmodelParams.corr_dsx = gamma_dx;
                    
                    quantoCK1F_LGM2FmodelParams.corr_frx.quote = quantoCK1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                elseif strcmp(useBrigoCorrYN,'simple_adjust')
                    quantoCK1F_LGM2FmodelParams.corr_drfr.quote = quantoCK1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsfr.quote = 0.0*quantoCK1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_drx.quote = quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_frx.quote = quantoCK1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    tempExpiry = 5.0;
                    tempTenor = [1;2;3;4;5;7;10;15];
                    
                    outSwapRateImpCorrel = multiAsset.ComputeImpliedCorrelMatrix(tempExpiry,tempTenor,quantoCK1F_LGM2FmodelParams);
                    aaa = 1.0;
                elseif strcmp(useBrigoCorrYN,'simple_adjust2')
                    quantoCK1F_LGM2FmodelParams.corr_drfr.quote = 0.0*quantoCK1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsfr.quote = 1.0*quantoCK1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_drx.quote = quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_frx.quote = quantoCK1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    tempExpiry = 5.0;
                    tempTenor = [1;2;3;4;5;7;10;15];
                    
                    outSwapRateImpCorrel = multiAsset.ComputeImpliedCorrelMatrix(tempExpiry,tempTenor,quantoCK1F_LGM2FmodelParams);
                    aaa = 1.0;
                elseif strcmp(modelParams('useBrigoCorrYN'),'calibrateToHist')
                    

%                     calibOut = multiAsset.CalibrateToCrossCorrelationLevmar(modelParams('crossCorrelation'));
%                     calibOut = multiAsset.CalibrateToCrossCorrelationWithConstraint(modelParams('crossCorrelation'));
                    calibOut = multiAsset.CalibrateToCrossCorrelation2T(modelParams('crossCorrelation'));
%                     calibOut = multiAsset.CalibrateToCrossCorrelation(modelParams('crossCorrelation'));
                    
%                     calibOut = multiAsset.CalibrateToCrossCorrelation_drfr(modelParams('crossCorrelation'));
%                     calibOut = multiAsset.CalibrateToCrossCorrelation_dsfr(modelParams('crossCorrelation'));
                    
                    quantoCK1F_LGM2FmodelParams.corr_drfr = calibOut.corr_drfr;
                    quantoCK1F_LGM2FmodelParams.corr_dsfr = calibOut.corr_dsfr;
%                     quantoLGM2F_LGM2FmodelParams.corr_drfs = calibOut.corr_drfs;
%                     quantoLGM2F_LGM2FmodelParams.corr_dsfs = calibOut.corr_dsfs;
                    tempExpiry = 5.0;
                    tempTenor = [1;2;3;4;5;7;10];
                    
                    outSwapRateImpCorrel = multiAsset.ComputeImpliedCorrelMatrix(tempExpiry,tempTenor,quantoCK1F_LGM2FmodelParams);
                    
                    quantoCK1F_LGM2FmodelParams.corr_drx.quote = quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsx.quote = 0.0*quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_frx.quote = quantoCK1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                    
                    quantoCK1F_LGM2FmodelParams.corr_fsx.quote = 0.0*quantoCK1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_fsx.tenor = ones(1,1);
                else
                
                    quantoCK1F_LGM2FmodelParams.corr_drfr.quote = quantoCK1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drfr.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsfr.quote = quantoCK1F_LGM2FmodelParams.corr_df*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsfr.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_drx.quote = quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_drx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_dsx.quote = quantoCK1F_LGM2FmodelParams.corr_dx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_dsx.tenor = ones(1,1);

                    quantoCK1F_LGM2FmodelParams.corr_frx.quote = quantoCK1F_LGM2FmodelParams.corr_fx*ones(1,1);
                    quantoCK1F_LGM2FmodelParams.corr_frx.tenor = ones(1,1);
                end
                %fx Black Vol Setting
                
                % BSCK1FLGM2F calibration needed !
                fxModelName = strcat(multiAsset.foreignModelName,refModelName);
                fxModelName2 = strcat(refModelName,multiAsset.foreignModelName);
                
                if isKey(multiAsset.mktData,fxModelName)
                    fxMktData = multiAsset.mktData(fxModelName);
                elseif isKey(multiAsset.mktData,fxModelName2)
                    fxMktData = multiAsset.mktData(fxModelName2);
                else
                    error('wrong fx asset name!')
                end
                
                fxImpVolRawData = fxMktData('impliedTermVol').params('rawData');

                fxImpVol.tenor = fxImpVolRawData(:,1);
                fxImpVol.quote = fxImpVolRawData(:,2);

                quantoCK1F_LGM2FmodelParams.fxBlackVol = fxImpVol;
                usdQuantoCK1F_LGM2F = QuantoCK1F_LGM2F(quantoCK1F_LGM2FmodelParams);
                multiAsset.modelNameMap(multiAsset.foreignModelName) = usdQuantoCK1F_LGM2F;
                
                
            end
        end
        
        function out = CalibrateToCrossCorrelationLevmar(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [1;2;3;5;7;10;15];
            targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                % drfr, dsfr correl : 2
                % length(targetExpiry):1
                optParams.modelParamsSize = length(targetExpiry)*2;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr,  dsfr,  : 2
                tvar = zeros(optParams.modelParamsSize,1);
                % initial
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

                options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];

                % observation ,crossCorrelationMatrix
                x = zeros(optParams.numOfEquation,1);
                tic
                [ret, popt, info, covar]=levmar('TargetFunctionLGM2FCK1FCrossCorrelationGlobalLevmar',tvar,x, 1000, options,'unc',multiAsset,optParams);

                toc

%                 tic
%                 targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal( tvar,multiAsset,optParams);
%                 constraintfun  = @(x) ConstraintFunctionLGM2F_CK1F(x);
%                 
%                 options=optimset('Algorithm','interior-point');
%                 lb = -0.99*ones(optParams.modelParamsSize,1);
%                 ub = +0.99*ones(optParams.modelParamsSize,1);
% 
%                 [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,constraintfun,options);
% %                 [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
%                 
%                 toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
%                 out.corr_drfs.tenor = targetExpiry;
%                 out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = popt(j +   1);
%                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
%                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
%                     
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
%                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
%                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                out.calibOut = calibOut;
%                 temporalCorr_diff = out.corrDiff;
                aaa = 1.0;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelationWithConstraint(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [1;2;3;5;7;10;15];
            targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                % drfr, dsfr correl : 2
                % length(targetExpiry):1
                optParams.modelParamsSize = length(targetExpiry)*2;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr,  dsfr,  : 2
                tvar = zeros(optParams.modelParamsSize,1);
                % initial
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

%                 options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
% 
%                 % observation ,crossCorrelationMatrix
%                 x = zeros(optParams.numOfEquation,1);
%                 tic
%     %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);
% 
%                 toc

                tic
                targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal( tvar,multiAsset,optParams);
                constraintfun  = @(x) ConstraintFunctionLGM2F_CK1F(x);
                
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,constraintfun,options);
%                 [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                
                toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
%                 out.corr_drfs.tenor = targetExpiry;
%                 out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = popt(j +   1);
%                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
%                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
%                     
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
%                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
%                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                out.calibOut = calibOut;
%                 temporalCorr_diff = out.corrDiff;
                aaa = 1.0;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelation2T(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [1;2;3;5;7;10;15];
            targetExpiryOrig = 1:1:15;
            
            temporalCorr_drfr = zeros(1);
%             temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(1);
%             temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            
            targetExpiry = targetExpiryOrig;
            
            optParams.tenor = tenor;
            optParams.targetCorrel = targetCorrel;
            optParams.targetExpiry = targetExpiry;
            % drfr, dsfr correl : 2
            % length(targetExpiry):1
            optParams.modelParamsSize = 2;
            optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
            % drfr,  dsfr,  : 2
            tvar = zeros(optParams.modelParamsSize,1);
            % initial
            for i=1:length(tvar)
                tvar(i) = 0.8;
            end

%                 options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
% 
%                 % observation ,crossCorrelationMatrix
%                 x = zeros(optParams.numOfEquation,1);
%                 tic
%     %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);
% 
%                 toc

            tic
            targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobalSimple( tvar,multiAsset,optParams);
            constraintfun  = @(x) ConstraintFunctionLGM2F_CK1F(x);
            options=optimset('Algorithm','interior-point');
            lb = -0.99*ones(optParams.modelParamsSize,1);
            ub = +0.99*ones(optParams.modelParamsSize,1);

            [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,constraintfun,options);
            toc

            out.corr_drfr.tenor = targetExpiry(1);
            out.corr_dsfr.tenor = targetExpiry(1);
%                 out.corr_drfs.tenor = targetExpiry;
%                 out.corr_dsfs.tenor = targetExpiry;

            out.corr_drfr.quote(1) = popt(1);
            out.corr_dsfr.quote(1) = popt(2);
                
%             for j=1:length(targetExpiry)
%                 out.corr_drfr.quote(j) = popt(j);
%                 out.corr_dsfr.quote(j) = popt(j +   1);
% %                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
% %                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
% %                     
%             end

            temporalCorr_drfr(1) = out.corr_drfr.quote(1);
            temporalCorr_dsfr(1) = out.corr_dsfr.quote(1);
%                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
%                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);

            calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
            out.calibOut = calibOut;
%                 temporalCorr_diff = out.corrDiff;
            aaa = 1.0;
                
%             for idx =1:length(targetExpiryOrig)
%                 targetExpiry = targetExpiryOrig(idx);
%             
%                 optParams.tenor = tenor;
%                 optParams.targetCorrel = targetCorrel;
%                 optParams.targetExpiry = targetExpiry;
%                 % drfr, dsfr correl : 2
%                 % length(targetExpiry):1
%                 optParams.modelParamsSize = length(targetExpiry)*2;
%                 optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
%                 % drfr,  dsfr,  : 2
%                 tvar = zeros(optParams.modelParamsSize,1);
%                 % initial
%                 for i=1:length(tvar)
%                     tvar(i) = 0.8;
%                 end
% 
% %                 options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
% % 
% %                 % observation ,crossCorrelationMatrix
% %                 x = zeros(optParams.numOfEquation,1);
% %                 tic
% %     %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);
% % 
% %                 toc
% 
%                 tic
%                 targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal( tvar,multiAsset,optParams);
%                 options=optimset('Algorithm','interior-point');
%                 lb = -0.99*ones(optParams.modelParamsSize,1);
%                 ub = +0.99*ones(optParams.modelParamsSize,1);
% 
%                 [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
%                 toc
% 
%                 out.corr_drfr.tenor = targetExpiry;
%                 out.corr_dsfr.tenor = targetExpiry;
% %                 out.corr_drfs.tenor = targetExpiry;
% %                 out.corr_dsfs.tenor = targetExpiry;
% 
%                 for j=1:length(targetExpiry)
%                     out.corr_drfr.quote(j) = popt(j);
%                     out.corr_dsfr.quote(j) = popt(j +   1);
% %                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
% %                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
% %                     
%                 end
% 
%                 temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
%                 temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
% %                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
% %                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
%                 
%                 calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
%                 out.calibOut = calibOut;
% %                 temporalCorr_diff = out.corrDiff;
%                 aaa = 1.0;
%                 
%             end
            
        end
        
        function out = CalibrateToCrossCorrelation(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [1;2;3;5;7;10;15];
            targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                % drfr, dsfr correl : 2
                % length(targetExpiry):1
                optParams.modelParamsSize = length(targetExpiry)*2;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr,  dsfr,  : 2
                tvar = zeros(optParams.modelParamsSize,1);
                % initial
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

%                 options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
% 
%                 % observation ,crossCorrelationMatrix
%                 x = zeros(optParams.numOfEquation,1);
%                 tic
%     %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);
% 
%                 toc

                tic
                targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

                out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
%                 out.corr_drfs.tenor = targetExpiry;
%                 out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
                    out.corr_dsfr.quote(j) = popt(j +   1);
%                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
%                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
%                     
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
%                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
%                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                out.calibOut = calibOut;
%                 temporalCorr_diff = out.corrDiff;
                aaa = 1.0;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelation_drfr(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [1;2;3;5;7;10;15];
            targetExpiryOrig = [5];
            
            temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                % drfr correl : 1
                % length(targetExpiry):1
%                 optParams.modelParamsSize = length(targetExpiry)*2;
                optParams.modelParamsSize = length(targetExpiry)*1;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr : 1
                tvar = zeros(optParams.modelParamsSize,1);
                % initial
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

%                 options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
% 
%                 % observation ,crossCorrelationMatrix
%                 x = zeros(optParams.numOfEquation,1);
%                 tic
%     %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);
% 
%                 toc

                tic
                targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal_drfr( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

                out.corr_drfr.tenor = targetExpiry;
%                 out.corr_dsfr.tenor = targetExpiry;
%                 out.corr_drfs.tenor = targetExpiry;
%                 out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_drfr.quote(j) = popt(j);
%                     out.corr_dsfr.quote(j) = popt(j +   1);
%                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
%                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
%                     
                end

                temporalCorr_drfr(idx) = out.corr_drfr.quote(1);
%                 temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
%                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
%                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix_drfr(popt,multiAsset,optParams);
%                 calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                out.calibOut = calibOut;
%                 temporalCorr_diff = out.corrDiff;
                aaa = 1.0;
                
            end
            
        end
        
        function out = CalibrateToCrossCorrelation_dsfr(multiAsset,crossCorrelation)
            tenor  = crossCorrelation(1,2:size(crossCorrelation,2))';
            targetCorrel= crossCorrelation(2:size(crossCorrelation,1),2:size(crossCorrelation,2));
            
%             targetExpiry = [1;2;3;5;10];
%             targetExpiryOrig = 1:1:15;
%             targetExpiryOrig = [1;2;3;5;7;10;15];
            targetExpiryOrig = [5];
            
%             temporalCorr_drfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_drfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_dsfr = zeros(length(targetExpiryOrig),1);
%             temporalCorr_dsfs = zeros(length(targetExpiryOrig),1);
            temporalCorr_diff = zeros(length(tenor),length(tenor));
            for idx =1:length(targetExpiryOrig)
                targetExpiry = targetExpiryOrig(idx);
            
                optParams.tenor = tenor;
                optParams.targetCorrel = targetCorrel;
                optParams.targetExpiry = targetExpiry;
                % drfr correl : 1
                % length(targetExpiry):1
%                 optParams.modelParamsSize = length(targetExpiry)*2;
                optParams.modelParamsSize = length(targetExpiry)*1;
                optParams.numOfEquation = length(tenor)*length(tenor)*length(targetExpiry);
                % drfr : 1
                tvar = zeros(optParams.modelParamsSize,1);
                % initial
                for i=1:length(tvar)
                    tvar(i) = 0.8;
                end

%                 options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
% 
%                 % observation ,crossCorrelationMatrix
%                 x = zeros(optParams.numOfEquation,1);
%                 tic
%     %             [ret, popt, info, covar]=levmar('TargetFunctionLGM2FLGM2FCrossCorrelationGlobal',tvar,x, 1000, options,'unc',multiAsset,optParams);
% 
%                 toc

                tic
%                 targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal_drfr( tvar,multiAsset,optParams);
                targetfun = @(tvar) TargetFunctionLGM2FCK1FCrossCorrelationGlobal_dsfr( tvar,multiAsset,optParams);
                options=optimset('Algorithm','interior-point');
                lb = -0.99*ones(optParams.modelParamsSize,1);
                ub = +0.99*ones(optParams.modelParamsSize,1);

                [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                toc

%                 out.corr_drfr.tenor = targetExpiry;
                out.corr_dsfr.tenor = targetExpiry;
%                 out.corr_drfs.tenor = targetExpiry;
%                 out.corr_dsfs.tenor = targetExpiry;

                for j=1:length(targetExpiry)
                    out.corr_dsfr.quote(j) = popt(j);
%                     out.corr_dsfr.quote(j) = popt(j +   1);
%                     out.corr_drfs.quote(j) = popt(j + 2*length(targetExpiry));
%                     out.corr_dsfs.quote(j) = popt(j + 3*length(targetExpiry));
%                     
                end

                temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
%                 temporalCorr_dsfr(idx) = out.corr_dsfr.quote(1);
%                 temporalCorr_drfs(idx) = out.corr_drfs.quote(1);
%                 temporalCorr_dsfs(idx) = out.corr_dsfs.quote(1);
                
                calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix_dsfr(popt,multiAsset,optParams);
%                 calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix_drfr(popt,multiAsset,optParams);
%                 calibOut = TargetFunctionLGM2FCK1FCrossCorrelationGlobalMatrix(popt,multiAsset,optParams);
                out.calibOut = calibOut;
%                 temporalCorr_diff = out.corrDiff;
                aaa = 1.0;
                
            end
            
        end
        
        function out = computeBrigo_1F2F_Correlation(multiAsset,params,corrIn)
                       
            domestic_dH_r_Tenor = params.domesticModel.dH1.tenor;
            domestic_dH_r_Quote = params.domesticModel.dH1.quote;

            domestic_Alpha_r_Tenor = params.domesticModel.Alpha1.tenor;
            domestic_Alpha_r_Quote = params.domesticModel.Alpha1.quote;

            domestic_dH_s_Tenor = params.domesticModel.dH2.tenor;
            domestic_dH_s_Quote = params.domesticModel.dH2.quote;

            domestic_Alpha_s_Tenor = params.domesticModel.Alpha2.tenor;
            domestic_Alpha_s_Quote = params.domesticModel.Alpha2.quote;
            
            corr_drds_Tenor = params.domesticModel.correl.tenor;
            corr_drds_Quote = params.domesticModel.correl.quote;

            sigmaTimes = unique(union([corr_drds_Tenor(:);...
                         domestic_Alpha_r_Tenor(:);domestic_Alpha_s_Tenor(:); ...
                         domestic_dH_r_Tenor(:)],domestic_dH_s_Tenor));

            sigmaTimes = sort(sigmaTimes,'ascend');
            
            corr_drds_Values = zeros(length(sigmaTimes),1);
            
            domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
            domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
            domestic_dH_r_Values = zeros(length(sigmaTimes),1);
            domestic_dH_s_Values = zeros(length(sigmaTimes),1);
            
            gammaValues = zeros(length(sigmaTimes),1);


            for k=1:length(sigmaTimes)
                corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
                
                domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
                domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
                
                domestic_dH_r_Values(k) = H_interpolation(domestic_dH_r_Tenor, domestic_dH_r_Quote,sigmaTimes(k),0);
                domestic_dH_s_Values(k) = H_interpolation(domestic_dH_s_Tenor, domestic_dH_s_Quote,sigmaTimes(k),0);
                
                sigma_r = domestic_dH_r_Values(k)*domestic_Alpha_r_Values(k);
                sigma_s = domestic_dH_s_Values(k)*domestic_Alpha_s_Values(k);
                
                corr_rs = corr_drds_Values(k);
                
                numerator = sigma_r*sigma_r + sigma_s*sigma_s + 2*corr_rs*sigma_r*sigma_s; 
                numerator = sqrt(numerator);
                denominator = sigma_r +sigma_s;
                gammaValues(k) = numerator/denominator*corrIn;
            end
            
            gamma.tenor = sigmaTimes;
            gamma.quote = gammaValues;
            
            out = gamma;
            
        end
        
        function out = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(multiAsset,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(multiAsset,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(multiAsset,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            out1 = SimpleSRPrOverHedge(multiAsset,cmsC,cmsN,ralower,raupper);
            out2 = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralowerS,raupperS);
%             out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
            outVec = out1*out2;
            out = outVec;
        end
        
        function out = SimpleDRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(multiAsset,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
%                     outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                    outVec(i) = multiAsset.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        function impCorrel = ComputeImpliedCorrelFwdSwap(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            alpha_1 = zeros(2,1);
            alpha_1(1) = H_interpolation(domesticModel.Alpha1.tenor, domesticModel.Alpha1.quote,tx,0);
            alpha_1(2) = H_interpolation(domesticModel.Alpha2.tenor, domesticModel.Alpha2.quote,tx,0);
            
            alpha_2 = zeros(1,1);
            alpha_2(1) = H_interpolation(foreignModel.vol_r.tenor, foreignModel.vol_r.quote,tx,0);
%             alpha_2(2) = H_interpolation(foreignModel.Alpha2.tenor, foreignModel.Alpha2.quote,tx,0);
            
            corr = zeros(3,3);
            for i=1:3
                corr(i,i) = 1.0;
            end
            
            
            corr_drds = domesticModel.correl;
            
            corr(1,2) = H_interpolation(corr_drds.tenor, corr_drds.quote,tx,0);
            
            corr(1,3) = H_interpolation(params.corr_drfr.tenor, params.corr_drfr.quote,tx,0);
%             corr(1,4) = H_interpolation(params.corr_drfs.tenor, params.corr_drfs.quote,tx,0);
            corr(3,1) = corr(1,3);
%             corr(4,1) = corr(1,4);
            
            corr(2,3) = H_interpolation(params.corr_dsfr.tenor, params.corr_dsfr.quote,tx,0);
%             corr(2,4) = H_interpolation(params.corr_dsfs.tenor, params.corr_dsfs.quote,tx,0);
            corr(3,2) = corr(2,3);
%             corr(4,2) = corr(2,4);
            
%             variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*alpha_1(j)*corr(j,k)*c_nN_1(k)*alpha_1(k);
                end
            end
            
            for j=1:1
                for k=1:1
                    var2 = var2 + c_nN_2(j)*alpha_2(j)*corr(j+2,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end

            for j=1:2
                for k=1:1
                    coVar = coVar + c_nN_1(j)*alpha_1(j)*corr(j,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end

            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function impCorrel = ComputeImpliedCorrelFwdSwap_drfr(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            alpha_1 = zeros(2,1);
            alpha_1(1) = H_interpolation(domesticModel.Alpha1.tenor, domesticModel.Alpha1.quote,tx,0);
            alpha_1(2) = H_interpolation(domesticModel.Alpha2.tenor, domesticModel.Alpha2.quote,tx,0);
            
            alpha_2 = zeros(1,1);
            alpha_2(1) = H_interpolation(foreignModel.vol_r.tenor, foreignModel.vol_r.quote,tx,0);
%             alpha_2(2) = H_interpolation(foreignModel.Alpha2.tenor, foreignModel.Alpha2.quote,tx,0);
            
            corr = zeros(3,3);
            for i=1:3
                corr(i,i) = 1.0;
            end
            
            
            corr_drds = domesticModel.correl;
            
            corr(1,2) = H_interpolation(corr_drds.tenor, corr_drds.quote,tx,0);
            
            corr(1,3) = H_interpolation(params.corr_drfr.tenor, params.corr_drfr.quote,tx,0);
%             corr(1,4) = H_interpolation(params.corr_drfs.tenor, params.corr_drfs.quote,tx,0);
            corr(3,1) = corr(1,3);
%             corr(4,1) = corr(1,4);
            
            corr(2,3) = 0.0;
%             corr(2,3) = H_interpolation(params.corr_dsfr.tenor, params.corr_dsfr.quote,tx,0);
%             corr(2,4) = H_interpolation(params.corr_dsfs.tenor, params.corr_dsfs.quote,tx,0);
            corr(3,2) = corr(2,3);
%             corr(4,2) = corr(2,4);
            
%             variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*alpha_1(j)*corr(j,k)*c_nN_1(k)*alpha_1(k);
                end
            end
            
            for j=1:1
                for k=1:1
                    var2 = var2 + c_nN_2(j)*alpha_2(j)*corr(j+2,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end

            for j=1:2
                for k=1:1
                    coVar = coVar + c_nN_1(j)*alpha_1(j)*corr(j,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end

            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function impCorrel = ComputeImpliedCorrelFwdSwap_dsfr(multiAsset,tx,tnr1,tnr2,params)
            tau = 0.25;
            domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            
            pv01_1 = domesticModel.PV01(tx,tx+tnr1,tau);
            fsr_1  = domesticModel.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = domesticModel.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = foreignModel.PV01(tx,tx+tnr2,tau);
            fsr_2  = foreignModel.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = foreignModel.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            alpha_1 = zeros(2,1);
            alpha_1(1) = H_interpolation(domesticModel.Alpha1.tenor, domesticModel.Alpha1.quote,tx,0);
            alpha_1(2) = H_interpolation(domesticModel.Alpha2.tenor, domesticModel.Alpha2.quote,tx,0);
            
            alpha_2 = zeros(1,1);
            alpha_2(1) = H_interpolation(foreignModel.vol_r.tenor, foreignModel.vol_r.quote,tx,0);
%             alpha_2(2) = H_interpolation(foreignModel.Alpha2.tenor, foreignModel.Alpha2.quote,tx,0);
            
            corr = zeros(3,3);
            for i=1:3
                corr(i,i) = 1.0;
            end
            
            
            corr_drds = domesticModel.correl;
            
            corr(1,2) = H_interpolation(corr_drds.tenor, corr_drds.quote,tx,0);
            
            corr(1,3) = 0.0;
%             corr(1,3) = H_interpolation(params.corr_drfr.tenor, params.corr_drfr.quote,tx,0);
%             corr(1,4) = H_interpolation(params.corr_drfs.tenor, params.corr_drfs.quote,tx,0);
            corr(3,1) = corr(1,3);
%             corr(4,1) = corr(1,4);
            
%             corr(2,3) = 0.0;
            corr(2,3) = H_interpolation(params.corr_dsfr.tenor, params.corr_dsfr.quote,tx,0);
%             corr(2,4) = H_interpolation(params.corr_dsfs.tenor, params.corr_dsfs.quote,tx,0);
            corr(3,2) = corr(2,3);
%             corr(4,2) = corr(2,4);
            
%             variances = multiAsset.LocalCovariance(0,tx,params);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*alpha_1(j)*corr(j,k)*c_nN_1(k)*alpha_1(k);
                end
            end
            
            for j=1:1
                for k=1:1
                    var2 = var2 + c_nN_2(j)*alpha_2(j)*corr(j+2,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end

            for j=1:2
                for k=1:1
                    coVar = coVar + c_nN_1(j)*alpha_1(j)*corr(j,k+2)*c_nN_2(k)*alpha_2(k);
                end
            end

            impCorrel = coVar/sqrt(var1)/sqrt(var2);
            
        end
        
        function out = ComputeImpliedCorrelMatrix(multiAsset,targetExpiry,tenor,params)
            
%             domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
%             foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
%             
%             % synchronize model parameters time-steps start
%             corr_drds_Tenor = domesticModel.correl.tenor;
%             corr_drds_Quote = domesticModel.correl.quote;
%             
% %             corr_frfs_Tenor = foreignModel.correl.tenor;
% %             corr_frfs_Quote = foreignModel.correl.quote;
%             
%             corr_drfr_Tenor = corr_drfr.tenor;
%             corr_drfr_Quote = corr_drfr.quote;
%             
%             corr_dsfr_Tenor = corr_dsfr.tenor;
%             corr_dsfr_Quote = corr_dsfr.quote;
%             
% %             corr_drfs_Tenor = corr_drfs.tenor;
% %             corr_drfs_Quote = corr_drfs.quote;
% %             
% %             corr_dsfs_Tenor = corr_dsfs.tenor;
% %             corr_dsfs_Quote = corr_dsfs.quote;
% 
% %             domestic_Alpha_r_Tenor = domesticModel.Alpha1.tenor;
% %             domestic_Alpha_r_Quote = domesticModel.Alpha1.quote;
% % 
% %             domestic_Alpha_s_Tenor = domesticModel.Alpha2.tenor;
% %             domestic_Alpha_s_Quote = domesticModel.Alpha2.quote;
% % 
% %             foreign_Alpha_r_Tenor = foreignModel.Alpha1.tenor;
% %             foreign_Alpha_r_Quote = foreignModel.Alpha1.quote;
% %             
% %             foreign_Alpha_s_Tenor = foreignModel.Alpha2.tenor;
% %             foreign_Alpha_s_Quote = foreignModel.Alpha2.quote;
%             
%             sigmaTimes = unique(union([corr_drds_Tenor(:);corr_drfr_Tenor(:);corr_dsfr_Tenor(:)]);
% 
%             sigmaTimes = sort(sigmaTimes,'ascend');
%             
%             corr_drds_Values = zeros(length(sigmaTimes),1);
% %             corr_frfs_Values = zeros(length(sigmaTimes),1);
%             
%             corr_drfr_Values = zeros(length(sigmaTimes),1);
%             corr_dsfr_Values = zeros(length(sigmaTimes),1);
%             
% %             corr_drfs_Values = zeros(length(sigmaTimes),1);
% %             corr_dsfs_Values = zeros(length(sigmaTimes),1);
% % 
% %             domestic_Alpha_r_Values = zeros(length(sigmaTimes),1);
% %             domestic_Alpha_s_Values = zeros(length(sigmaTimes),1);
% %             
% %             foreign_Alpha_r_Values = zeros(length(sigmaTimes),1);
% %             foreign_Alpha_s_Values = zeros(length(sigmaTimes),1);
% 
%             for k=1:length(sigmaTimes)
%                 corr_drds_Values(k) = H_interpolation(corr_drds_Tenor, corr_drds_Quote,sigmaTimes(k),0);
% %                 corr_frfs_Values(k) = H_interpolation(corr_frfs_Tenor, corr_frfs_Quote,sigmaTimes(k),0);
%                 
%                 corr_drfr_Values(k) = H_interpolation(corr_drfr_Tenor, corr_drfr_Quote,sigmaTimes(k),0);
%                 corr_dsfr_Values(k) = H_interpolation(corr_dsfr_Tenor, corr_dsfr_Quote,sigmaTimes(k),0);
%                 
% %                 corr_drfs_Values(k) = H_interpolation(corr_drfs_Tenor, corr_drfs_Quote,sigmaTimes(k),0);
% %                 corr_dsfs_Values(k) = H_interpolation(corr_dsfs_Tenor, corr_dsfs_Quote,sigmaTimes(k),0);
% %                 
% %                 domestic_Alpha_r_Values(k) = H_interpolation(domestic_Alpha_r_Tenor, domestic_Alpha_r_Quote,sigmaTimes(k),0);
% %                 domestic_Alpha_s_Values(k) = H_interpolation(domestic_Alpha_s_Tenor, domestic_Alpha_s_Quote,sigmaTimes(k),0);
% %                 
% %                 foreign_Alpha_r_Values(k) = H_interpolation(foreign_Alpha_r_Tenor, foreign_Alpha_r_Quote,sigmaTimes(k),0);
% %                 foreign_Alpha_s_Values(k) = H_interpolation(foreign_Alpha_s_Tenor, foreign_Alpha_s_Quote,sigmaTimes(k),0);
% 
%             end
%             
%             params.corr_drds.tenor = sigmaTimes;
% %             params.corr_frfs.tenor = sigmaTimes;
%             
%             params.corr_drfr.tenor = sigmaTimes;
%             params.corr_dsfr.tenor = sigmaTimes;
% %             params.corr_drfs.tenor = sigmaTimes;
% %             params.corr_dsfs.tenor = sigmaTimes;
% %             params.domestic_Alpha_r.tenor = sigmaTimes;
% %             params.domestic_Alpha_s.tenor = sigmaTimes;
% %             params.foreign_Alpha_r.tenor = sigmaTimes;
% %             params.foreign_Alpha_s.tenor = sigmaTimes;
%             
%             params.corr_drds.quote = corr_drds_Values;
% %             params.corr_frfs.quote = corr_frfs_Values;
%             
%             params.corr_drfr.quote = corr_drfr_Values;
%             params.corr_dsfr.quote = corr_dsfr_Values;
% %             params.corr_drfs.quote = corr_drfs_Values;
% %             params.corr_dsfs.quote = corr_dsfs_Values;
% %             params.domestic_Alpha_r.quote = domestic_Alpha_r_Values;
% %             params.domestic_Alpha_s.quote = domestic_Alpha_s_Values;
% %             params.foreign_Alpha_r.quote = foreign_Alpha_r_Values;
% %             params.foreign_Alpha_s.quote = foreign_Alpha_s_Values;
            
            out = zeros(length(tenor),length(tenor));
            for i=1:length(tenor)
                for j=1:length(tenor)
%                     out(i,j) = multiAsset.ComputeImpliedCorrel(targetExpiry,tenor(i),tenor(j),params);
                    out(i,j) = multiAsset.ComputeImpliedCorrelFwdSwap(targetExpiry,tenor(i),tenor(j),params);
                end
            end
            
        end
        
        function out = ComputeImpliedCorrelMatrix_drfr(multiAsset,targetExpiry,tenor,params)
 
            
            out = zeros(length(tenor),length(tenor));
            for i=1:length(tenor)
                for j=1:length(tenor)
%                     out(i,j) = multiAsset.ComputeImpliedCorrel(targetExpiry,tenor(i),tenor(j),params);
                    out(i,j) = multiAsset.ComputeImpliedCorrelFwdSwap_drfr(targetExpiry,tenor(i),tenor(j),params);
                end
            end
            
        end
        
        function out = ComputeImpliedCorrelMatrix_dsfr(multiAsset,targetExpiry,tenor,params)
 
            
            out = zeros(length(tenor),length(tenor));
            for i=1:length(tenor)
                for j=1:length(tenor)
%                     out(i,j) = multiAsset.ComputeImpliedCorrel(targetExpiry,tenor(i),tenor(j),params);
%                     out(i,j) = multiAsset.ComputeImpliedCorrelFwdSwap_drfr(targetExpiry,tenor(i),tenor(j),params);
                    out(i,j) = multiAsset.ComputeImpliedCorrelFwdSwap_dsfr(targetExpiry,tenor(i),tenor(j),params);
                end
            end
            
        end
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = DFRisky(multiAsset,t)
            out = multiAsset.refModel.DFRisky(t);
        end
        
        function out = stateLocalDrift(multiAsset,fromTime, toTime)
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            out = foreignModel.LocalDrift(fromTime/365.0,toTime/365.0);
        end
        
        function out = stateLocalVariance(multiAsset,fromTime, toTime)
            % we need to generalize this hard coding !!
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            % we use quantock1f_lgm2f to calculate localcovariance term
            % is there any better solution ?  :(
            
            out = foreignModel.LocalCovariance(fromTime/365.0,toTime/365.0);
        end
        
        function out =  Rate_Index(multiAsset,rateType,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT,PSAOption)
            if strcmp(PSAOption,'PSAAll')
                out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            elseif strcmp(PSAOption,'PSA')
                if strcmp(rateType,'CMS')
                    out = CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
            else
                if strcmp(rateType,'CMS')
                    out = CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT);
                elseif strcmp(rateType,'Libor')
                    % Libor startTime is set to be observeTime 
                    out = Libor_SimpleDate(multiAsset,curveName,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT); 
                else
                    error('unknown rateType !')
                end
                
            end
        end
        
        function out =  Libor_SimpleDate(multiAsset,curveName,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1:2,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(3,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        
        function out = discountPayoffRisky(multiAsset,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            % for multiAsset discounting is done by refModel
            dfNumMat = multiAsset.refModel.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
            referenceModelStatesT = modelStatesT(1:2,:);
            
            discountT = multiAsset.refModel.discountFactorRisky(eventTime, numMatTime,numMatTime,referenceModelStatesT);
            
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountFactorRisky(multiAsset, startTime, endTime,numMatTime,modelStatesT)
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
           referenceModelStatesT = modelStatesT(1:2,:);
           out = multiAsset.refModel.discountFactorRisky(startTime, endTime,numMatTime,referenceModelStatesT);
           
        end
        
        function out = CMSCMTSpreadDigitalArgDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2)
           % probability that CMS tenor1 - CMT tenor 2  < strike
           %CMS 1
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
%            tau1 = 1.0/(domesticModel.freq);
           % hard coding for LGM2F for the time-being
           tau1 = 0.25;
           
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau1);
           
           cms_fwd   = cmsDigitalArg.fwdCMS;
           cms_mu    = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_var   = cms_sigma * cms_sigma;
           cms_c_nN  = cmsDigitalArg.c_nN;
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           cmtDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor2,tau2);
           
           cmt_fwd   = cmtDigitalArg.fwdCMS;
           cmt_mu    = cmtDigitalArg.mu;
           cmt_sigma = cmtDigitalArg.sigma;
           cmt_var   = cmt_sigma * cmt_sigma;
           cmt_c_nN  = cmtDigitalArg.c_nN;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3)
           coVar = foreignModel.LocalCovariance(0,to);
           
           %cms: KRW
           %cmt :USD
           cmscmt_covar =                cms_c_nN(1) * cmt_c_nN(1) * coVar(1,3);
           cmscmt_covar = cmscmt_covar + cms_c_nN(2) * cmt_c_nN(1) * coVar(2,3); 
           spread_var = cms_var + cmt_var - 2.0 * cmscmt_covar;
           
           out.fwdSpread = cms_fwd - cmt_fwd; 
           out.mu = cms_mu - cmt_mu;
           out.sigma = sqrt(spread_var);
        end
        
        function out = CMSCMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,strike)
           % probability that CMS tenor1 - CMT tenor 2 < strike
           %CMS 1
           cmsCmtSpreadDigitalArg = multiAsset.CMSCMTSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2);
           mu = cmsCmtSpreadDigitalArg.mu;
           sigma = cmsCmtSpreadDigitalArg.sigma;
           z1= (strike - mu)/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSCMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,lower,upper)
            
            lower_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,lower);
            upper_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,upper);
%             out = lower_value - upper_value;
            out = upper_value - lower_value;
        end
        
        function out = CMSCMSSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor > strike
           %CMS
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau1 = 1.0/(foreignModel.freq);
           cmsDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau1);
           
           cms_mu = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_c_nN = cmsDigitalArg.c_nN;
           
           
           %CMS Spread
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           cmsSpreadDigitalArg = domesticModel.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           cmsSpread_c_nN1 = cmsSpreadDigitalArg.c_nN1;
           cmsSpread_c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           cmsSpread_mu = cmsSpreadDigitalArg.mu;
           cmsSpread_sigma = cmsSpreadDigitalArg.sigma;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (3,3) matrix (KRW dr, KRW ds, USD fr) as (1,2,3)
           varCovar = foreignModel.LocalCovariance(0,to);
           
           % covariance between USD CMS Rate & KRW CMS Spread
           %cms: USD
           %cmtSpread :KRW CMS 1 - KRW CMS 2
           cms_cmsSpread_covar = 0.0;
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(1) - cmsSpread_c_nN2(1)) * varCovar(1,3);
           cms_cmsSpread_covar = cms_cmsSpread_covar + cms_c_nN(1) * (cmsSpread_c_nN1(2) - cmsSpread_c_nN2(2)) * varCovar(2,3);
           
           corr = cms_cmsSpread_covar/(cms_sigma*cmsSpread_sigma);
           
           z1 = (cms_mu       - strike)/cms_sigma;
           z2 = (cmsSpread_mu - strikeS)/cmsSpread_sigma;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
           % to be checked later 
           if isnan(out)
               disp('bivariate cumulative normal pdf returns NAN!')
               out = 0.0;
           end
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = multiAsset.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
%         function mcOut = inductMCForwardNew(multiAsset,fromTime,toTime,lastX,dZ)
%             
%             predictor = lastX;
% %             numOfFactors = multiAsset.numOfFactors;
%             stochasticModelNames = multiAsset.stochasticModelNames;
%             
%             for i=1:length(stochasticModelNames)
%                 startIdx = multiAsset.stateNumberMap(stochasticModelNames{i});
%                 endIdx = startIdx + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor - 1;
%                 
%                 mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardNew(fromTime,toTime,lastX(:,startIdx:endIdx),dZ(:,startIdx:endIdx));
%                 predictor(:,startIdx:endIdx) = mcOut1.nextX;
%             end
%             
%             mcOut.nextX = predictor;
%             
%         end
%         
%         function mcmcOut = inductMCMCForwardNew(multiAsset,fromTime,toTime,nextX,nextXIdx,dZ)
%             
%             predictor = nextX;
%             predictorIdx = nextXIdx;
%             numOfFactors = multiAsset.numOfFactors;
%             stochasticModelNames = multiAsset.stochasticModelNames;
%             
%             for i=1:length(stochasticModelNames)
%                 startIdx = multiAsset.stateNumberMap(stochasticModelNames{i});
%                 endIdx = startIdx + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor - 1;
%                 
%                 mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCMCForwardNew(fromTime,toTime,lastX(:,startIdx:endIdx),dZ(:,startIdx:endIdx));
%                 predictor(:,startIdx:endIdx) = mcOut1.nextX;
%                 predictorIdx(:,startIdx:endIdx) = mcOut1.nextXIdx;
%             end
%             
%             mcmcOut.nextX = predictor;
%             mcmcOut.nextXIdx = predictorIdx;
%             
%         end
        
    end
    
end



In [ ]:
classdef MultiAssetModelLGM1FLGM1F < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        refModel
        refModelName
        stochasticModelNames
        correlationMatrix
        numOfFactor
        foreignModelName
        payCurrency
    end
    
    methods
        function multiAsset = MultiAssetModelLGM1FLGM1F(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                
                refModelName = modelParams('refModelName');
                multiAsset.refModelName = refModelName;
                multiAsset.refModel = multiAsset.modelNameMap(refModelName);
                
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                % 1 factor LGM model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                numOfFactor = 0;
                for i=1: length(stochasticModelNames)
                    numOfFactor = numOfFactor + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactor;
                end
                multiAsset.numOfFactor = numOfFactor;
                
%                 KTBLGM1F = multiAsset.modelNameMap('KTB');
                % Check whether KTB is in the modelNames
                stochasticModelNames = multiAsset.stochasticModelNames;
                
                % we have two factor model
                % find foreignModel
                for i=1:length(stochasticModelNames)
                    if ~strcmp(stochasticModelNames{i}, refModelName)
                        multiAsset.foreignModelName = stochasticModelNames{i};
                        break;
                    end
                end
                
                ktbExist = false;
                if multiAsset.foreignModelName == 'KTB'
                    ktbExist = true;
                end
                
                if ktbExist
                    
%                     multiAsset.foreignModelName = 'KTB';
                    
                    quantoLGM1FmodelParams.fxVolCalibrated = true;
                    % construct KTBQuantoLGM1F from KRWLGM1FRisky and KTB
                    quantoLGM1FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                    quantoLGM1FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
                    quantoLGM1FmodelParams.corr_df = multiAsset.correlationMatrix(1,2);
                    quantoLGM1FmodelParams.corr_dx = multiAsset.correlationMatrix(1,3);
%                     quantoLGM1FmodelParams.corr_xf = multiAsset.correlationMatrix(2,3);
                    quantoLGM1FmodelParams.corr_fx = multiAsset.correlationMatrix(2,3);
                    
                    
                    % dummy fxBlackVol for KTB 
                    quantoLGM1FmodelParams.fxBlackVol.tenor = quantoLGM1FmodelParams.domesticModel.Alpha1.tenor;
                    quantoLGM1FmodelParams.fxBlackVol.quote = zeros(length(quantoLGM1FmodelParams.fxBlackVol.tenor),1);
                    
                    
                    KTBQuantoLGM1F = QuantoLGM1F(quantoLGM1FmodelParams); 
                    multiAsset.modelNameMap(multiAsset.foreignModelName) = KTBQuantoLGM1F;
                    
                else
                    
%                     multiAsset.foreignModelName = 'USD';
                    
                    quantoLGM1FmodelParams.fxVolCalibrated = false;
                    % construct KTBQuantoLGM1F from KRWLGM1FRisky and KTB
                    quantoLGM1FmodelParams.domesticModel = multiAsset.modelNameMap(refModelName);
                    quantoLGM1FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
                    quantoLGM1FmodelParams.corr_df = multiAsset.correlationMatrix(1,2);
                    %  fxBlackVol for USD
                    %domestic & fx
                    quantoLGM1FmodelParams.corr_dx = multiAsset.correlationMatrix(1,3);
                    %foreign & fx
                    quantoLGM1FmodelParams.corr_fx = multiAsset.correlationMatrix(2,3);
                    
                    % BSLGMLGM calibration needed !
                    fxModelName = strcat(multiAsset.foreignModelName,refModelName);
                    fxMktData = multiAsset.mktData(fxModelName);
                    fxImpVolRawData = fxMktData('impliedTermVol').params('rawData');
                    
                    fxImpVol.tenor = fxImpVolRawData(:,1);
                    fxImpVol.quote = fxImpVolRawData(:,2);
                    
                    quantoLGM1FmodelParams.fxBlackVol = fxImpVol;
                    usdQuantoLGM1F = QuantoLGM1F(quantoLGM1FmodelParams);
                    multiAsset.modelNameMap(multiAsset.foreignModelName) = usdQuantoLGM1F;
                    
%                     % initialize fxBlackVol with implied Vol
%                     paramsIn.fxImpVol = fxImpVol;
%                     
%                     paramsIn.fxBlackVol.tenor = usdkrwImpVolRawData(:,1);
%                     paramsIn.fxBlackVol.quote = usdkrwImpVolRawData(:,2);
%                     
%                     paramsIn.domestic_dH =  quantoLGM1FmodelParams.domesticModel.dH1;
%                     paramsIn.domestic_Alpha = quantoLGM1FmodelParams.domesticModel.Alpha1;
%                     
%                     paramsIn.foreign_dH =  quantoLGM1FmodelParams.foreignModel.dH1;
%                     paramsIn.foreign_Alpha = quantoLGM1FmodelParams.foreignModel.Alpha1;
%                     paramsIn.corr_df = quantoLGM1FmodelParams.corr_df;
%                     paramsIn.corr_dx = quantoLGM1FmodelParams.corr_dx;
%                     paramsIn.corr_fx = quantoLGM1FmodelParams.corr_fx;
%                     
%                     paramsIn.domesticModel = quantoLGM1FmodelParams.domesticModel;
%                     
%                     
%                     % without synchronization just initialize QuantoLGM1F
%                     % for BSLGMLGM Calibration
%                     
%                     
%                     multiAsset.modelNameMap(multiAsset.foreignModelName) = usdQuantoLGM1F;
%                     quantoLGM1FmodelParams.foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
%                     
%                     paramsIn.foreignModel = quantoLGM1FmodelParams.foreignModel;
%                     
%                     % BSLGMLGM Calibration Start
%                     out = multiAsset.CalibrateBSLGMLGMVol(paramsIn);
%                     re =  out.re;
%                     rmseTotal = out.rmseTotal;
%                     
%                     % BSLGMLGM Calibration End
%                     
%                     quantoLGM1FmodelParams.fxBlackVol = out.fxBlackVol;
%                     quantoLGM1FmodelParams.fxBlackVolCalibratedYN = true;
%                     usdQuantoLGM1F = QuantoLGM1F(quantoLGM1FmodelParams); 
                    
%                     multiAsset.modelNameMap(multiAsset.foreignModelName) = usdQuantoLGM1F;
                    
                end
                
                
                
            end
        end
        
        function out = SimpleSRPrOverHedge(multiAsset,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(multiAsset,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(multiAsset,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(multiAsset,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = multiAsset.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = multiAsset.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = DFRisky(multiAsset,t)
            out = multiAsset.refModel.DFRisky(t);
        end
        
        function out = stateLocalDrift(multiAsset,fromTime, toTime)
            out = eye(2,2);
        end
        
        function out = stateLocalVariance(multiAsset,fromTime, toTime)
            % we need to generalize this hard coding !!
            foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
            % we use quantolgm1f to calculate localcovariance term
            % is there any better solution ?  :(
            
            out = foreignModel.LocalCovariance(fromTime/365.0,toTime/365.0);
        end
        
        function out =  Libor_SimpleDate(multiAsset,curveName,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           % domestic Libor
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,domesticModelStatesT);
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(2,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.Libor_SimpleDate(valueDate,evalTime, fwdStartTime, numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        function out =  CMS_PSA_SimpleDate(multiAsset,curveName,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           
           if strcmp(curveName,multiAsset.refModelName)
               domesticModelStatesT = modelStatesT(1,:);
               domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
               out = domesticModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,domesticModelStatesT);
           
           elseif strcmp(curveName,multiAsset.foreignModelName)
               foreignModelStatesT = modelStatesT(2,:);
               foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
               out = foreignModel.CMS_PSA_SimpleDate(valueDate,observeTime,numMatTime, tenor,foreignModelStatesT);
               
           end
            
        end
        
        
        function out = discountPayoffRisky(multiAsset,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            % for multiAsset discounting is done by refModel
            dfNumMat = multiAsset.refModel.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
            referenceModelStatesT = modelStatesT(1,:);
            
            discountT = multiAsset.refModel.discountFactorRisky(eventTime, numMatTime,numMatTime,referenceModelStatesT);
            
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountFactorRisky(multiAsset, startTime, endTime,numMatTime,modelStatesT)
            %KRW is the first modelStates
            % to do list : need to define GetModelsStates in the futures !!!
           referenceModelStatesT = modelStatesT(1,:);
           out = multiAsset.refModel.discountFactorRisky(startTime, endTime,numMatTime,referenceModelStatesT);
           
        end
        
        function out = CMSCMTSpreadDigitalArgDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2)
           % probability that CMS tenor1 - CMT tenor 2  < strike
           %CMS 1
           domesticModel = multiAsset.modelNameMap(multiAsset.refModelName);
           tau1 = 1.0/(domesticModel.freq);
           cmsDigitalArg = domesticModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau1);
           
           cms_fwd   = cmsDigitalArg.fwdCMS;
           cms_mu    = cmsDigitalArg.mu;
           cms_sigma = cmsDigitalArg.sigma;
           cms_var   = cms_sigma * cms_sigma;
           cms_c_nN  = cmsDigitalArg.c_nN;
           
           foreignModel = multiAsset.modelNameMap(multiAsset.foreignModelName);
           tau2 = 1.0/(foreignModel.freq);
           cmtDigitalArg = foreignModel.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor2,tau2);
           
           cmt_fwd   = cmtDigitalArg.fwdCMS;
           cmt_mu    = cmtDigitalArg.mu;
           cmt_sigma = cmtDigitalArg.sigma;
           cmt_var   = cmt_sigma * cmt_sigma;
           cmt_c_nN  = cmtDigitalArg.c_nN;
           
           %covariance Term
           to      = toDate.DateDiff(valueDate)/365.0;
           
           % coVar is (2,2) matrix
           coVar = foreignModel.LocalCovariance(0,to);
           
           
           cmscmt_covar = cms_c_nN(1) * cmt_c_nN(1) * coVar(1,2);
           spread_var = cms_var + cmt_var - 2.0 * cmscmt_covar;
           
           out.fwdSpread = cms_fwd - cmt_fwd; 
           out.mu = cms_mu - cmt_mu;
           out.sigma = sqrt(spread_var);
        end
        
        function out = CMSCMTSpreadDigitalDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,strike)
           % probability that CMS tenor1 - CMT tenor 2 < strike
           %CMS 1
           cmsCmtSpreadDigitalArg = multiAsset.CMSCMTSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2);
           mu = cmsCmtSpreadDigitalArg.mu;
           sigma = cmsCmtSpreadDigitalArg.sigma;
           z1= (strike - mu)/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSCMTSpreadDigitalRangeDate(multiAsset,valueDate,toDate,tpDate,tenor1,tenor2,lower,upper)
            
            lower_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,lower);
            upper_value = multiAsset.CMSCMTSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,upper);
%             out = lower_value - upper_value;
            out = upper_value - lower_value;
        end
        
        function mcmcOut = inductMCForwardNew(multiAsset,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardNew(fromTime,toTime,lastX(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCMCForwardNew(multiAsset,fromTime,toTime,nextX,nextXIdx,dZ)
            
            predictor = nextX;
            predictorIdx = nextXIdx;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCMCForwardNew(fromTime,toTime,nextX(:,i),nextXIdx(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
                predictorIdx(:,i) = mcOut1.nextXIdx;
            end
            
            mcmcOut.nextX = predictor;
            mcmcOut.nextXIdx = predictorIdx;
            
        end
        
%         function out =computeBSLGMLGMVariance(multiasset,a,b,T,quantolgm1f,corr_df,corr_dx,corr_fx, ...
%                                   fxBlackVol, domestic_dH, domestic_Alpha, foreign_dH, foreign_Alpha)
%                  
%           % synchronize model parameters time-steps start
% 
%             fxBlackVolTenor = fxBlackVol.tenor;
%             fxBlackVolQuote = fxBlackVol.quote;
% 
%             domestic_dHTenor = domestic_dH.tenor;
%             domestic_dHQuote = domestic_dH.quote;
% 
%             foreign_dHTenor = foreign_dH.tenor;
%             foreign_dHQuote = foreign_dH.quote;
% 
%             domestic_AlphaTenor = domestic_Alpha.tenor;
%             domestic_AlphaQuote = domestic_Alpha.quote;
% 
%             foreign_AlphaTenor = foreign_Alpha.tenor;
%             foreign_AlphaQuote = foreign_Alpha.quote;
% 
% 
% 
%             sigmaTimes = unique(union([domestic_AlphaTenor(:);foreign_AlphaTenor(:); ...
%                          domestic_dHTenor(:);foreign_dHTenor(:)],fxBlackVolTenor));
% 
% 
%             sigmaTimes = sort(sigmaTimes,'ascend');
% 
% 
% 
%             domestic_AlphaValues = zeros(length(sigmaTimes),1);
%             foreign_AlphaValues = zeros(length(sigmaTimes),1);
%             domestic_dHValues = zeros(length(sigmaTimes),1);
%             foreign_dHValues = zeros(length(sigmaTimes),1);
%             fxBlackVolValues = zeros(length(sigmaTimes),1);
% 
% 
%             for k=1:length(sigmaTimes)
%                 domestic_AlphaValues(k) = H_interpolation(domestic_AlphaTenor, domestic_AlphaQuote,sigmaTimes(k),0);
%                 foreign_AlphaValues(k) = H_interpolation(foreign_AlphaTenor, foreign_AlphaQuote,sigmaTimes(k),0);
%                 domestic_dHValues(k) = H_interpolation(domestic_dHTenor, domestic_dHQuote,sigmaTimes(k),0);
%                 foreign_dHValues(k) = H_interpolation(foreign_dHTenor, foreign_dHQuote,sigmaTimes(k),0);
%                 fxBlackVolValues(k) = H_interpolation(fxBlackVolTenor, fxBlackVolQuote,sigmaTimes(k),0);
%             end
%             
%             % synchronize model parameters time-steps end
%             
%             var_x_x = quantolgm1f.LocalVariance_Alpha2(a,b,sigmaTimes,fxBlackVolValues,fxBlackVolValues);
%             
%             var_f_f = quantolgm1f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,foreign_dHValues,foreign_dHValues, ...
%                                                          foreign_AlphaValues,foreign_AlphaValues);
%                                                      
%             var_d_d = quantolgm1f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,domestic_dHValues,domestic_dHValues, ...
%                                                          domestic_AlphaValues,domestic_AlphaValues);
%                                                      
%             covar_f_x =  1.0*corr_fx*-1.0*quantolgm1f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
%                                             foreign_dHValues,foreign_AlphaValues,fxBlackVolValues);
%                                         
%             covar_d_x = -1.0*corr_dx*-1.0*quantolgm1f.GeneralVariance_BSLGM(a,b,T,sigmaTimes, ...
%                                             domestic_dHValues,domestic_AlphaValues,fxBlackVolValues);
%                                         
%             covar_d_f =  1.0*corr_df* 1.0*quantolgm1f.GeneralVariance_LGMLGM(a,b,T,T,sigmaTimes,...
%                                             domestic_dHValues,foreign_dHValues,domestic_AlphaValues,foreign_AlphaValues);
%                                                          
%             value = var_x_x + var_f_f + var_d_d -2.0*(covar_f_x + covar_d_x + covar_d_f);                                         
%             
%             out = value;                                         
%                                                      
%             
%         end
        
%         function out = FindFXBlackVol(multiasset,index,params)
%             
%             expiry = params.fxImpVol.tenor(index);
%             impVol = params.fxImpVol.quote(index);
%             blackVariance = impVol*impVol*expiry;
%             optParams.blackVariance = blackVariance;
%             
%             optParams.params = params;
%             optParams.index = index;
%             modelParamSize = 1;
%             tvar = zeros(modelParamSize,1);
%             lb = zeros(modelParamSize,1);
%             ub = zeros(modelParamSize,1);
%             
%             tvar(1) = params.fxImpVol.quote(index);
%             lb(1)= 0.0001;
%             ub(1)= 10;
%             
%             %options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
%             options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
%             numOfEquation =1;
%             x=  zeros(numOfEquation,1);
%             [ret, popt, info, covar]=levmar('TargetFunctionBSLGMLGMVariance',tvar,x, 200, options,'unc',multiasset,optParams);
%             
%             out.fxBlackVol_index = popt(1);
%             out.re = TargetFunctionBSLGMLGMVariance(popt,multiasset,optParams);
%             params.fxBlackVol.quote(index) = popt(1);
% 
%         end
        
%         function out = CalibrateBSLGMLGMVol(multiasset, paramsIn)
%  
%                 expirySize = length(paramsIn.fxImpVol.tenor);
%                 out.fxBlackVol.quote = zeros(expirySize,1);
%                 out.fxBlackVol.tenor = paramsIn.fxImpVol.tenor;
%                 out.re =  zeros(expirySize,1);
%                 out.rmseTotal = 0;
%                 for i=1:expirySize
%                     index = i;
%                     BootstrapOut = multiasset.FindFXBlackVol(index,paramsIn);
%                     out.fxBlackVol.quote(i) = BootstrapOut.fxBlackVol_index;
%                     out.re(i) = BootstrapOut.re;
%                     out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
%                 end
%                 out.rmseTotal = out.rmseTotal/expirySize;
%                 
%                 out.rmseTotal = sqrt(out.rmseTotal);
%                  
%                 
%         end
        
    end
    
end



In [ ]:
classdef MultiAssetModelLC < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        refModel
        stochasticModelNames
        correlationMatrix
        numOfFactors
        
        localCorrInfo
        
        %% parametrized local correlation info
        lc_alpha
        lc_alpahBarrier
        lc_lambdaCorr
    end
    
    methods
        function multiAsset = MultiAssetModel(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                multiAsset.refModel = modelParams('refModel');
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                % 1 factor local vol model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                numOfFactors = 0;
                for i=1: length(stochasticModelNames)
                    numOfFactors = numOfFactors + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactors;
                end
                multiAsset.numOfFactors = numOfFactors;
                
                multiAsset.localCorrInfo = {};
                
                multiAsset.lc_alpha = 0.0;
                multiAsset.lc_alpahBarrier = 0.0;
                multiAsset.lc_lambdaCorr = 1.0;
            end
        end
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = CommoFwdNMMC(multiAsset,observeTime,modelStates)
            
%             modelNames = multiAsset.modelNameMap.keys;
            numOfFactors = multiAsset.numOfFactors;
            stateSize = size(modelStates,1);
            stochasticFwd = zeros(stateSize,numOfFactors);
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                stochasticFwd(:,i) = multiAsset.modelNameMap(stochasticModelNames{i}).CommoFwdNMMC(observeTime,modelStates(:,i));
            end
             
             out = stochasticFwd;
             
        end
        
        function out = EQFwdMC(multiAsset,observeTime,modelStates)
            
            numOfFactors = multiAsset.numOfFactors;
            stateSize = size(modelStates,1);
            stochasticFwd = zeros(stateSize,numOfFactors);
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                stochasticFwd(:,i) = multiAsset.modelNameMap(stochasticModelNames{i}).EQFwdMC(observeTime,modelStates(:,i));
            end
             
             out = stochasticFwd;
             
        end
        
        function out = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
%             modelStatesSize = length(comFwd);
            modelStatesSize = size(comFwd,1);
            
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if isKI(idx) > 0
                    out(idx) = 1.0;
                else
                    for ii=1:length(basePrice)
                        if comFwd(idx,ii) <= basePrice(ii)*lowerBarrier
                            out(idx) = 1.0;
                            break;
                        end
                    end
                end
            end
        end
        
        function mcmcOut = inductMCForwardNew(multiAsset,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardNew(fromTime,toTime,lastX(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
            end
            
            mcmcOut.nextX = predictor;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                deltaX = log(predictor./lastX);
                termDeltaX = log(predictor./1.0); 
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.termCorrX1X2 = corr(termDeltaX(:,1),termDeltaX(:,2));
%                 corrInfo.corrX1X3 = corr(deltaX(:,1),deltaX(:,3));
%                 corrInfo.corrX2X3 = corr(deltaX(:,2),deltaX(:,3));

%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
                
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
        end
        
        function mcmcOut = inductMCForwardQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoS1S2 = multiAsset.correlationMatrix(1,2);
 
            
            Lsmall = eye(2,2);
            
            dummyR = rhoS1S2*rhoS1S2/(1-rhoSV(1)^2)/(1-rhoSV(2)^2);
            Lsmall(2,1) = sqrt(dummyR);
            Lsmall(2,2) = sqrt(1.0-dummyR);
            
            L = eye(4,4);
            L(3:4,3:4) = Lsmall;
            % W is (numOfFactors,NMC) matrix
            W = dZ';  
            WCorr = L*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoSiSj = multiAsset.correlationMatrix;
 
            rhoSmall = eye(numOfFactors/2,numOfFactors/2);
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i==j
                        rhoSmall(i,j) = 1.0;
                    else
                        rhoSmall(i,j) = rhoSiSj(i,j)/sqrt(1-rhoSV(i)^2)/sqrt(1-rhoSV(j)^2);
                    end
                end    
            end
            
%             rhoSmall = 0.5*(rhoSmall + rhoSmall');
            
            LSmall = chol(rhoSmall,'lower');
            
            L = eye(numOfFactors,numOfFactors);
            L(numOfFactors/2+1:numOfFactors,numOfFactors/2+1:numOfFactors) = LSmall;
            % W is (numOfFactors,NMC) matrix
            W = dZ';  
            WCorr = L*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaX(:,2));
                corrInfo.covarX1X2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaV(:,1),deltaV(:,2));
                corrInfo.covarV1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaV(:,2));
                corrInfo.covarX1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,2),deltaV(:,1));
                corrInfo.covarX2V1 = dummyCovarX1X2(1,2);
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardPerfectCorrMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoSiSj = multiAsset.correlationMatrix;
 
            rhoSmall = eye(numOfFactors/2,numOfFactors/2);
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i==j
                        rhoSmall(i,j) = 1.0;
                    else
                        sigma_i = sqrt(1-rhoSV(i)^2);
                        sigma_j = sqrt(1-rhoSV(j)^2);
                        rhoSmall(i,j) = (rhoSiSj(i,j)-rhoSV(i)*rhoSV(j)) ...
                                        /(sigma_i*sigma_j);
                    end
                end    
            end
            
%             rhoSmall = 0.5*(rhoSmall + rhoSmall');
            
%             LSmall = chol(rhoSmall,'lower');
            
            rhoBig = eye(numOfFactors/2 +1,numOfFactors/2 +1);
            
            rhoBig(2:numOfFactors/2+1,2:numOfFactors/2+1) = rhoSmall;
            L = chol(rhoBig,'lower');
            % W is (numOfFactors,NMC) matrix
            W = dZ';
            
            % make dW_v_i as dW_v_1 so that variaces are perfectly
            % correlated
            WDummy = zeros(numOfFactors/2 +1,size(W,2));
            
            WDummy(1,:) = W(1,:);
            for i=1:length(stochasticModelNames)
                WDummy(1+i,:) = W(numOfFactors/2+i,:);
            end
            
            WCorrDummy = L*WDummy;
            WCorr = zeros(size(W));
            
            for i=1:length(stochasticModelNames)
                WCorr(i,:) = WCorrDummy(1,:);
                WCorr(numOfFactors/2+i,:) = WCorrDummy(1+i,:);
            end
            
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaX(:,2));
                corrInfo.covarX1X2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaV(:,1),deltaV(:,2));
                corrInfo.covarV1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaV(:,2));
                corrInfo.covarX1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,2),deltaV(:,1));
                corrInfo.covarX2V1 = dummyCovarX1X2(1,2);
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        % input asset Correlation -> output L with parsimonious correlation
        % assumption
        function out = CholeskyDecomposeParsimonious(multiAsset,corrIn)
          %% Cholesky Decomposition Start
          
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
            rhoSiSj = corrIn;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
            
            try
                RPars = chol(corrTotal,'lower');
            catch
                aaa = corrTotal;
                bbb = rhoSiSj(1,2);
            end
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            out = dummyL2;
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
          %% Cholesky Decomposition Start
          
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                
                termDeltaX = log(predictorX./1.0);
                corrInfo.termCorrX1X2 = corr(termDeltaX(:,1),termDeltaX(:,2));
                
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                
%                 corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
%                 corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
%                 corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
%                 corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
%                 
%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
%                 
%                 corrInfo.varV1 = var(deltaV(:,1));
%                 corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQEND(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
          %% Cholesky Decomposition Start
          
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                
                % 1&2
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                % 1&3
                corrInfo.corrX1X3 = corr(deltaX(:,1),deltaX(:,3));
                corrInfo.corrV1V3 = corr(deltaV(:,1),deltaV(:,3));
                
                corrInfo.corrX3V3 = corr(deltaX(:,3),deltaV(:,3));
                
                corrInfo.corrX1V3 = corr(deltaX(:,1),deltaV(:,3));
                corrInfo.corrX3V1 = corr(deltaX(:,3),deltaV(:,1));
                
                % 2&3
                corrInfo.corrX2X3 = corr(deltaX(:,2),deltaX(:,3));
                corrInfo.corrV2V3 = corr(deltaV(:,2),deltaV(:,3));
                
                corrInfo.corrX2V3 = corr(deltaX(:,2),deltaV(:,3));
                corrInfo.corrX3V2 = corr(deltaX(:,3),deltaV(:,2));
                
                
%                 corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
%                 corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
%                 corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
%                 corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
%                 
%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
%                 
%                 corrInfo.varV1 = var(deltaV(:,1));
%                 corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQE_EffectiveLocalCorr(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                
                corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
                corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
                corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
                corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQETestVV0(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
%                         
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = 0.0;
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = 0.0;
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = 0.0;
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQETestVV1(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
%                         
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = 0.999;
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function out = GetLocalCorrSLV(multiAsset,fromTime,x,y)
            
            timeStep = multiAsset.localCorrInfo.timeStepLVF;
            
            idx = find(timeStep > fromTime);
            if isempty(idx)
                idxNow = length(timeStep);
            else
                idxNow = min(idx);
            end
            
            localCorrXY = multiAsset.localCorrInfo.localCorrXY{idxNow};
            strikeX = multiAsset.localCorrInfo.strikeX{idxNow};
            strikeY = multiAsset.localCorrInfo.strikeY{idxNow};
            
            strikeXVector = strikeX(:);
            strikeYVector = strikeY(:);
            localCorrXYVector = localCorrXY(:);
            
            F = scatteredInterpolant(strikeXVector,strikeYVector,localCorrXYVector,'linear','nearest');
            out = F(x,y);
            
            out = max(-0.99,min(out,0.99));
            
%             out = griddata(strikeXVector,strikeYVector,localCorrXYVector,x,y);
%             out = interp2FlatExtrap(strikeX,strikeY,localCorrXY',x,y);
            
        end
            
        
        function out = GenerateEffectiveLocalCorr(multiAsset,valueDateH,effectiveLocalCorrParams)
            tic

            stochasticModelNames = multiAsset.stochasticModelNames;
            
            numOfStochasticAssets = length(stochasticModelNames);
            
            childModels = cell(numOfStochasticAssets,1);
            
            for i=1:numOfStochasticAssets
                childModels{i} = multiAsset.modelNameMap(stochasticModelNames{i});
            end
            
            levFuncInfo1 = childModels{1}.levFuncInfo;
            timeStepLVF = levFuncInfo1.timeStepLVF;
            
            timeStep = [0;timeStepLVF];
            timeStep = sort(timeStep,'ascend');
            % timeStepLVF do not have 0 therefore we count 0 additionaly
            totalTimeStepSize = length(timeStepLVF) + 1;
            
            numOfBins = effectiveLocalCorrParams.numOfBins;
            numOfPath = effectiveLocalCorrParams.numOfPath;
            numOfPathPerBins = effectiveLocalCorrParams.numOfPathPerBins;
            numOfPathPerBins1D = numOfPath/numOfBins;
            
          %% initialize EffectiveLocalCorr Info
           localCorrXY = cell(totalTimeStepSize-1,1);
           strikeX = cell(totalTimeStepSize-1,1);
           strikeY = cell(totalTimeStepSize-1,1);
           expectedVarianceX = cell(totalTimeStepSize-1,1);
           

           expectedVarianceY = cell(totalTimeStepSize-1,1);
           
           for idxhh=1:totalTimeStepSize-1
                localCorrXY{idxhh} = zeros(numOfBins,numOfBins);
                strikeX{idxhh} = zeros(numOfBins,numOfBins);
                strikeY{idxhh} = zeros(numOfBins,numOfBins);
                expectedVarianceX{idxhh} = zeros(numOfBins,numOfBins);
                expectedVarianceY{idxhh} = zeros(numOfBins,numOfBins);
           end
                      
           multiAsset.localCorrInfo.localCorrXY = localCorrXY;
           multiAsset.localCorrInfo.timeStepLVF = timeStepLVF;
           multiAsset.localCorrInfo.strikeX = strikeX;
           multiAsset.localCorrInfo.strikeY = strikeY;
           
           multiAsset.localCorrInfo.expectedVarianceX = expectedVarianceX;
           multiAsset.localCorrInfo.expectedVarianceY = expectedVarianceY;
            
          %% local correlation from the local vol model
           rhoSiSj = multiAsset.correlationMatrix;
           corr0 = rhoSiSj(1,2);

           v10 = childModels{1}.v0;
           v20 = childModels{2}.v0;
           smallTime = 0.01;
           lv10 = childModels{1}.GetLocalVol(smallTime,1.0);
           lv20 = childModels{2}.GetLocalVol(smallTime,1.0);
           
          %% localCorrXY initialize
            
            % initial strikes
            dt = timeStep(2) - timeStep(1);
            dx1 = 1.0 * 5.0*lv10*sqrt(dt/365)/numOfBins*2.0;
            dx2 = 1.0 * 5.0*lv20*sqrt(dt/365)/numOfBins*2.0;
            
            for i=1:numOfBins
                for j=1:numOfBins
                    multiAsset.localCorrInfo.strikeX{1}(i,j) = 1.0 + (i-numOfBins/2.0)*dx1;
                    multiAsset.localCorrInfo.strikeY{1}(i,j) = 1.0 + (j-numOfBins/2.0)*dx2;

                    multiAsset.localCorrInfo.expectedVarianceX{1}(i,j) =  childModels{1}.v0;
                    multiAsset.localCorrInfo.expectedVarianceY{1}(i,j) =  childModels{2}.v0;

                    % if we use local Vol & local Corr
                    % this part should be updated to replaced with
                    % local look up part
                    
                    % rho_slv = rho_lv/(sqrt(v1v2)/sqrt(v1*v2));
                    multiAsset.localCorrInfo.localCorrXY{1}(i,j) = 1.0* corr0;
                end
            end
            
           %% MC init 
            useQuasiRandomYN = 'false';
            numOfFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = numOfPath; 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = numOfPath; 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
            end
            
            currentTime = 0; %NowDate
            currentTimeIdx = 1; % 1st Time Steps
            currentNodeIdx = 1; % no eventDate for nowDate
            lastTimeIdx = currentTimeIdx;
            
            % initial modelState
            %  X  = S(t)/F(0,t)  : spot process
            %  V  = instantaneous variance proces
           
           initX = cell(numOfStochasticAssets,1);
           initV = cell(numOfStochasticAssets,1);
           
           for i=1:numOfStochasticAssets
               initX{i} = ones(NMC,1);
               initV{i} = ones(NMC,1)*childModels{i}.v0;
           end
           
%            initX = ones(NMC,1);
%            initV = ones(NMC,1)*eqHeston.v0;
%            
           nextX = initX;
           nextV = initV;
           
         %% Path Generation Forward
          startTimeIdx = 1;
          endTimeIdx = length(timeStep)-1;
         %% induction Forward start
         %% below for the timeStep(idx) to timeStep(idx+1) 
         %% generated leverageFunction applied to timeStep(idx+1) to timeStep(idx+2)
         %% therefore we only need to generate from startTimeIdx to endTimeIdx-1
          toc

            
          for idx= startTimeIdx:1:endTimeIdx-1
               if idx == endTimeIdx
                 abcd = 1.0;
               end
              
               % Get Effective Local Correlation SLV
               effectivCorrSLV = GetLocalCorrSLV(multiAsset,timeStep(idx) + 0.01,nextX{1},nextX{2});
               Ut = U(:,numOfFactors*(idx-1)+1:numOfFactors*(idx-1)+numOfFactors);
               
               tic
 
               for idxNMC=1:NMC
                    corrIn = eye(2,2);
                    corrIn(1,2) = effectivCorrSLV(idxNMC);
                    corrIn(2,1) = corrIn(1,2);

                    outL = CholeskyDecomposeParsimonious(multiAsset,corrIn);
                    U0 = Ut(idxNMC,1:numOfFactors);
                    dZ = U0';
                    dZCorr = outL*dZ;
                    U0 = dZCorr';
                    Ut(idxNMC,:) = U0;
               end

               toc 
                
                if strcmp(childModels{1}.modelParams('MCScheme'), 'QE')
                    for i=1:numOfStochasticAssets
                        mcmcOut   = childModels{i}.inductMCForwardQE(timeStep(idx),timeStep(idx+1),nextX{i},nextV{i},U(:,2*(i-1)+1),U(:,2*(i-1)+2));
                    
                        nextX{i} = mcmcOut.nextX;
                        nextV{i} = mcmcOut.nextV;

                    end
                    
                    
                end
                
             %% we sort simul path, asceding w.r.t nextX
               sortedX0 = cell(numOfStochasticAssets,1);
               sortIdx0 = cell(numOfStochasticAssets,1);
               
             
               sortedX = cell(numOfStochasticAssets,1);
               sortIdx = cell(numOfStochasticAssets,1);
               sortedV = cell(numOfStochasticAssets,1);
               
               sortIdx2D = cell(numOfStochasticAssets,1);
               interMediatedX = cell(numOfStochasticAssets,1);
               interMediatedV = cell(numOfStochasticAssets,1);
               
               for idxZZ=1:numOfStochasticAssets
                    interMediatedX{idxZZ} = zeros(numOfPathPerBins1D,1);
                    interMediatedV{idxZZ} = zeros(numOfPathPerBins1D,1);
               end
               
               [sortedX0{1}, sortIdx0{1}] = sort(nextX{1},'ascend');
               
               for idxhh1 = 1:numOfBins
                   startIdx = numOfPathPerBins1D*(idxhh1-1)+1;
                   endXIdx = startIdx + numOfPathPerBins1D;
                   interMediatedX{1} = nextX{1}(sortIdx0{1}(startIdx:endXIdx-1));
                   interMediatedX{2} = nextX{2}(sortIdx0{1}(startIdx:endXIdx-1));
                   interMediatedV{1} = nextV{1}(sortIdx0{1}(startIdx:endXIdx-1));
                   interMediatedV{2} = nextV{2}(sortIdx0{1}(startIdx:endXIdx-1));
                   
                   [dummySortedX, dummySortIdx] = sort(interMediatedX{2},'ascend');
                   
                   for idxhh2=1:numOfBins
                       startIdx2 = numOfPathPerBins*(idxhh2-1)+1;
                       endXIdx2 = startIdx2 + numOfPathPerBins;
                       startIdxNew = startIdx + startIdx2 - 1;
                       endXIdxNew  = startIdx  + endXIdx2  - 1; 
                       sortedX{1}(startIdxNew:endXIdxNew-1,1) = interMediatedX{1}(dummySortIdx(startIdx2:endXIdx2-1));
                       sortedX{2}(startIdxNew:endXIdxNew-1,1) = interMediatedX{2}(dummySortIdx(startIdx2:endXIdx2-1));
                       
                       sortedV{1}(startIdxNew:endXIdxNew-1,1) = interMediatedV{1}(dummySortIdx(startIdx2:endXIdx2-1));
                       sortedV{2}(startIdxNew:endXIdxNew-1,1) = interMediatedV{2}(dummySortIdx(startIdx2:endXIdx2-1));
                       
                   end
                   
               end
               
               
%                sortedV{i} = nextV{i}(sortIdx{i});
%                    
%                for i=1:numOfStochasticAssets
%                    [sortedX{i}, sortIdx{i}] = sort(nextX{i},'ascend');
%                    sortedV{i} = nextV{i}(sortIdx{i});
%                end
               
               % X1, X2
               
               startX1Idx = 1;
               endX1Idx = 0;
               
               startX2Idx = 1;
               endX2Idx = 0;
               
%                expectedVariance = cell(numOfStochasticAssets,1);
%                for i=1:numOfStochasticAssets
%                    expectedVariance{i} = zeros(numOfBins,1);
%                end
%                
%                strikesVariance = cell(numOfStochasticAssets,1);
%                for i=1:numOfStochasticAssets
%                    strikesVariance{i} = zeros(numOfBins,1);
%                end
               
               % test version for marginal prob dist
               % Sigma11(S1,S2),Sigma22(S1,S2)
               % Sigma12(S1,S2)
               

               
%                for i=1:numOfStochasticAssets
%                    for idxS1=1:numOfBins
%                        inc = numOfPathPerBins1D;
%                        startX1Idx = 1 + numOfPathPerBins1D*(idxS1-1);
%                        endX1Idx   = startX1Idx + numOfPathPerBins1D;
%                        sum = 0.0;
%                        
%                        for idxW=startX1Idx:endX1Idx-1
%                            sum = sum + sortedV{i}(idxW);
%                        end
% 
%                        %average of variance
%                        sum = sum/inc;
%                        strikesVariance{i}(idxS1) = 0.5*(sortedX{i}(startX1Idx)+sortedX{i}(endX1Idx-1));
%                        expectedVariance{i}(idxS1) = sum;
%                     end
%                end
               
               startX1Idx = 1;
               endX1Idx = 0;
               
               
             %% Effective Joint Marginal Distribution of S1,S2 
             
               for idxS1=1:numOfBins
                   startX1Idx = 1 + numOfBins*numOfPathPerBins*(idxS1-1);
                   endX1Idx   = startX1Idx + numOfPathPerBins;
                   
                   startX2Idx = 1;
                   endX2Idx = 0;
                       
                   for idxS2=1:numOfBins
                       startX2Idx = startX1Idx + numOfPathPerBins*(idxS2-1);
                       endX2Idx = startX2Idx + numOfPathPerBins;
                       
                       sum = 0.0;
                       sumx = 0.0;
                       sumy = 0.0;
                       for idxW=startX2Idx:endX2Idx-1
                           sum  = sum  + sqrt(sortedV{1}(idxW)*sortedV{2}(idxW));
                           sumx = sumx + sqrt(sortedV{1}(idxW)*sortedV{1}(idxW));
                           sumy = sumy + sqrt(sortedV{2}(idxW)*sortedV{2}(idxW));
                       end

                       %average of variance
                       sum = sum/numOfPathPerBins;
                       sumx = sumx/numOfPathPerBins;
                       sumy = sumy/numOfPathPerBins;
                       
                       multiAsset.localCorrInfo.strikeX{idx+1}(idxS1,idxS2) = 0.5*(sortedX{1}(startX2Idx)+sortedX{1}(endX2Idx-1));
                       multiAsset.localCorrInfo.strikeY{idx+1}(idxS1,idxS2) = 0.5*(sortedX{2}(startX2Idx)+sortedX{2}(endX2Idx-1));
 
                       multiAsset.localCorrInfo.expectedVarianceX{idx+1}(idxS1,idxS2) = sumx;
                       multiAsset.localCorrInfo.expectedVarianceY{idx+1}(idxS1,idxS2) = sumy;
                       
                    %% local correlation from the local vol model
                       rhoSiSj_LV = multiAsset.correlationMatrix;
                       corr0_LV = rhoSiSj_LV(1,2); 
                       % rho_slv = rho_lv/(sqrt(v1v2)/sqrt(v1*v2));
                       dummyRal = sum/sqrt(sumx*sumy);
                       
%                        dummyRal2 = min(dummyRal,1.0);
                       corrDummyRal = corr0_LV/dummyRal;
                       
                       corrDummyRal2 = min(corrDummyRal,0.99);
                       
                       if corrDummyRal2 > 1
                           dfr = 1.0;
                       end
                       multiAsset.localCorrInfo.localCorrXY{idx+1}(idxS1,idxS2) = corrDummyRal2; 
                       
                   end
               end
                
               aaa = 1.0;

          end
            
          toc
 
          out = multiAsset.localCorrInfo;
        end
        
        function out = GetEffectiveLocalCorr(multiAsset,fromTime,x,y)
            
            timeStep = multiAsset.localCorrInfo.timeStepLVF;
            
            idx = find(timeStep > fromTime);
            if isempty(idx)
                idxNow = length(timeStep);
            else
                idxNow = min(idx);
            end
            
%             localCorrXY = eqHeston.localCorrInfo.localCorrMap(modelXName,modelYName);
            localCorrXYt = eqHeston.localCorrInfo.localCorrXY{idxNow};
            strikeX = eqHeston.localCorrInfo.StrikeX;
            strikeY = eqHeston.localCorrInfo.StrikeY;
            
            out = interp2(strikeX,strikeY,localCorrXYt,x,y);

        end
        
        function mcmcOut = inductMCMCForwardNew(multiAsset,fromTime,toTime,nextX,nextXIdx,dZ)
            
            predictor = nextX;
            predictorIdx = nextXIdx;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCMCForwardNew(fromTime,toTime,nextX(:,i),nextXIdx(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
                predictorIdx(:,i) = mcOut1.nextXIdx;
            end
            
            mcmcOut.nextX = predictor;
            mcmcOut.nextXIdx = predictorIdx;
            
        end
        
        function columnOut = MCPayoffWorst(multiAsset,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            df_ep = multiAsset.DF(currentTime/365.0);
            
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
        end    
            
        
        function out = computeStepdown2SMC(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            WTIDupire = multiAsset.modelNameMap('WTI');
            expiry = WTIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = 150000;
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCEQLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIDupire = multiAsset.modelNameMap('HSCEI');
            expiry = HSCEIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = HSCEIDupire.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIDupire.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIDupire.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
         
           % corr , var1, var2 
           realizedCorr = zeros(MCTimeStep,3);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.termCorrX1X2;
%                         realizedCorr(idx,2) = mcmcOut.corrInfo.varX1;
%                         realizedCorr(idx,3) = mcmcOut.corrInfo.varX2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                out.realizedCorr = realizedCorr;
                out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown3SMCEQLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIDupire = multiAsset.modelNameMap('HSCEI');
            expiry = HSCEIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = HSCEIDupire.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIDupire.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIDupire.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
         
           % corr , var1, var2 
           realizedCorr = zeros(MCTimeStep,3);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
%                         realizedCorr(idx,2) = mcmcOut.corrInfo.corrX1X3;
%                         realizedCorr(idx,3) = mcmcOut.corrInfo.corrX2X3;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                out.realizedCorr = realizedCorr;
                out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCEQHeston(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIHeston = multiAsset.modelNameMap('HSCEI');
            expiry = HSCEIHeston.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = HSCEIHeston.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
           
            % corr X1X2,V1V2,X1V2,X2V1,X1V1,X2V2,
            % var X1,X2,V1,V2
            % COVAR x1x2,v1v2,x1v2,x2v1
            
           realizedCorr = zeros(MCTimeStep,6 + 4 + 4);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           HSCEIHeston = multiAsset.modelNameMap('HSCEI');
           SX5EHeston  = multiAsset.modelNameMap('SX5E');
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*HSCEIHeston.v0;
           
           initX2 = ones(NMC,1);
           initV2 = ones(NMC,1)*SX5EHeston.v0;

           initX = [initX1,initX2];
           initV = [initV1,initV2];
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE')
                        mcmcOut   = inductMCForwardMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE')
                        mcmcOut   = inductMCForwardPerfectCorrMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV0')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV0(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV1')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV1(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
%                         realizedCorr(idx,7) = mcmcOut.corrInfo.varX1;
%                         realizedCorr(idx,8) = mcmcOut.corrInfo.varX2;
%                         
%                         
%                         realizedCorr(idx,9) = mcmcOut.corrInfo.varV1;
%                         realizedCorr(idx,10) = mcmcOut.corrInfo.varV2;
                        
                        realizedCorr(idx,7) = mcmcOut.corrInfo.termCorrX1X2;
                        
                        if strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE') || ...
                           strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE') 
                            realizedCorr(idx,11) = mcmcOut.corrInfo.covarX1X2;
                            realizedCorr(idx,12) = mcmcOut.corrInfo.covarV1V2;
                            realizedCorr(idx,13) = mcmcOut.corrInfo.covarX1V2;
                            realizedCorr(idx,14) = mcmcOut.corrInfo.covarX2V1;
                            
                        end
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown3SMCEQHeston(multiAsset,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            % we use stochasticModelNames{1} for generic purpose
            DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
            
            expiry = DummyHeston.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = DummyHeston.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = DummyHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = DummyHeston.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = DummyHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
           
            % corr X1X2,V1V2,X1V2,X2V1,X1V1,X2V2,
            % var X1,X2,V1,V2
            % COVAR x1x2,v1v2,x1v2,x2v1
            
%            realizedCorr = zeros(MCTimeStep,6 + 4 + 4);
           realizedCorr = zeros(MCTimeStep,100);
           
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           numOfAsset = length(multiAsset.stochasticModelNames);
           
           DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*DummyHeston.v0;
           
           initX = initX1;
           initV = initV1;
           
           for idxHong=2:numOfAsset
               DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxHong});
               initXDummy = ones(NMC,1);
               initVDummy = ones(NMC,1)*DummyHeston.v0;
               
               initX = [initX,initXDummy];
               initV = [initV,initVDummy];
               
           end
           
           nextX = initX;
           nextV = initV;
           
%            HSCEIHeston = multiAsset.modelNameMap('HSCEI');
%            SX5EHeston  = multiAsset.modelNameMap('SX5E');
%            
%            initX1 = ones(NMC,1);
%            initV1 = ones(NMC,1)*HSCEIHeston.v0;
%            
%            initX2 = ones(NMC,1);
%            initV2 = ones(NMC,1)*SX5EHeston.v0;
% 
%            initX = [initX1,initX2];
%            initV = [initV1,initV2];
%            
%            nextX = initX;
%            nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQEND(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        % 1& 2
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
                        % 1& 3
                        realizedCorr(idx,7) = mcmcOut.corrInfo.corrX1X3;
                        realizedCorr(idx,8) = mcmcOut.corrInfo.corrV1V3;
                        realizedCorr(idx,9) = mcmcOut.corrInfo.corrX1V3;
                        realizedCorr(idx,10) = mcmcOut.corrInfo.corrX3V1;
                        
                        realizedCorr(idx,11) = mcmcOut.corrInfo.corrX3V3;
                        
                        % 2&3
                        realizedCorr(idx,12) = mcmcOut.corrInfo.corrX2X3;
                        realizedCorr(idx,13) = mcmcOut.corrInfo.corrV2V3;
                        realizedCorr(idx,14) = mcmcOut.corrInfo.corrX2V3;
                        realizedCorr(idx,15) = mcmcOut.corrInfo.corrX3V2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCEQHestonSLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIHeston = multiAsset.modelNameMap('HSCEI');
%             expiry = HSCEIHeston.localVolSurface.expiry;
            expiry = HSCEIHeston.levFuncInfo.timeStepLVF;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = HSCEIHeston.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
            
           realizedCorr = zeros(MCTimeStep,6+4);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           HSCEIHeston = multiAsset.modelNameMap('HSCEI');
           SX5EHeston  = multiAsset.modelNameMap('SX5E');
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*HSCEIHeston.v0;
           
           initX2 = ones(NMC,1);
           initV2 = ones(NMC,1)*SX5EHeston.v0;

           initX = [initX1,initX2];
           initV = [initV1,initV2];
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE')
                        mcmcOut   = inductMCForwardMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                     elseif strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE')
                        mcmcOut   = inductMCForwardPerfectCorrMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE_EffectiveLocalCorr')
                        mcmcOut   = inductMCForwardParsimoniousMultiQE_EffectiveLocalCorr(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV0')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV0(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV1')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV1(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
%                         realizedCorr(idx,7) = mcmcOut.corrInfo.varX1;
%                         realizedCorr(idx,8) = mcmcOut.corrInfo.varX2;
%                         
%                         
%                         realizedCorr(idx,9) = mcmcOut.corrInfo.varV1;
%                         realizedCorr(idx,10) = mcmcOut.corrInfo.varV2;
                        
                        realizedCorr(idx,7) = mcmcOut.corrInfo.termCorrX1X2;
                        
                        if strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE') || ...
                           strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE') 
                            realizedCorr(idx,11) = mcmcOut.corrInfo.covarX1X2;
                            realizedCorr(idx,12) = mcmcOut.corrInfo.covarV1V2;
                            realizedCorr(idx,13) = mcmcOut.corrInfo.covarX1V2;
                            realizedCorr(idx,14) = mcmcOut.corrInfo.covarX2V1;
                        
                        end
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 

           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown3SMCEQHestonSLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
%             expiry = HSCEIHeston.localVolSurface.expiry;
            expiry = DummyHeston.levFuncInfo.timeStepLVF;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = HSCEIHeston.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = DummyHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = DummyHeston.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = DummyHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
            
           realizedCorr = zeros(MCTimeStep,100);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           numOfAsset = length(multiAsset.stochasticModelNames);
           
           DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*DummyHeston.v0;
           
           initX = initX1;
           initV = initV1;
           
           for idxHong=2:numOfAsset
               DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxHong});
               initXDummy = ones(NMC,1);
               initVDummy = ones(NMC,1)*DummyHeston.v0;
               
               initX = [initX,initXDummy];
               initV = [initV,initVDummy];
               
           end
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQEND(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        % 1& 2
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
                        % 1& 3
                        realizedCorr(idx,7) = mcmcOut.corrInfo.corrX1X3;
                        realizedCorr(idx,8) = mcmcOut.corrInfo.corrV1V3;
                        realizedCorr(idx,9) = mcmcOut.corrInfo.corrX1V3;
                        realizedCorr(idx,10) = mcmcOut.corrInfo.corrX3V1;
                        
                        realizedCorr(idx,11) = mcmcOut.corrInfo.corrX3V3;
                        
                        % 2&3
                        realizedCorr(idx,12) = mcmcOut.corrInfo.corrX2X3;
                        realizedCorr(idx,13) = mcmcOut.corrInfo.corrV2V3;
                        realizedCorr(idx,14) = mcmcOut.corrInfo.corrX2V3;
                        realizedCorr(idx,15) = mcmcOut.corrInfo.corrX3V2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 

           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCMC(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            WTIDupire = multiAsset.modelNameMap('WTI');
            expiry = WTIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
%             useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            useQuasiRandomYN = 'false';
            useQuasiRandomYN = 'true';
            
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
                NMC = 2^18;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
                
                % we use gaussian copula to generated joint distribution
                % correlated brownian motions
                Z = dZ0';
                UU = zeros(MCTimeStep*numFactors,NMC);
                U = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     W = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*W;
                end
                
                U = H_ncdf(UU);
                
                U = U'; 
                
%                % apply brownian bridge
%                dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
%                U(:,1) = dZ(:,1);
%                for i=2:size(dZ,2)
%                    U(:,i) = dZ(:,i) - dZ(:,i-1); 
%                end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 5000;
%                 NMC = 50000;
                NMC = 150000;
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                UUU = zeros(MCTimeStep*numFactors,NMC);
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                
                UUU = H_ncdf(UU);
                U = UUU';
                
%                 abcd = zeros(MCTimeStep*numFactors,NMC);
%                 for i=1:MCTimeStep*numFactors
%                     for j=1:NMC
%                         abcd(i,j) = H_ncdf(UU(i,j));
%                     end
%                 end

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
           initXIdx = zeros(NMC,numFactors);
           stochasticModelNames = multiAsset.stochasticModelNames;
           for i=1:length(stochasticModelNames)
               Pricingidx = multiAsset.modelNameMap(stochasticModelNames{i}).localVolSurface.Pricingidx;
               initXIdx(:,i) = ones(NMC,1)*Pricingidx;
           end
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);
                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        
%         function out= expmReghai(multiAsset,A,t)
%             % Scale A by power of 2 so that its norm is < 1/2 .
%             A = A*t;
%             [f,e] = log2(norm(A,'inf'));
%             s = max(0,e+1);
%             A = A/2^s;
%             Aexp = eye(size(A,1)) + A;
%             for i=1:s
%                 Aexp = Aexp*Aexp;
%             end
%             out = Aexp;
%         end
%         
%         function path= bbgenerator(eqCOMDupireSpotGF,rn)
%             %generate brownian bridge 
%             
%             [Npaths,Nsteps]=size(rn);
%             path=zeros(Npaths,Nsteps);
%             map= zeros(1,Nsteps);
%             bridgeindex = zeros(1,Nsteps);
%             leftindex = zeros(1,Nsteps);
%             rightindex = zeros(1,Nsteps);
%             sigma = zeros(1,Nsteps);
%             leftweight = zeros(1,Nsteps);
%             rightweight = zeros(1,Nsteps);
%             
%             map(Nsteps)=1;
%             bridgeindex(1)=Nsteps;
%             sigma(1)=sqrt(Nsteps);
%             leftweight(1)=1; 
%             rightweight(1)=1; 
%             j=1;
%             for i=2:Nsteps    
%                 while map(j) 
%                     j=j+1;
%                 end
%                 k=j;
%                 while ~map(k) 
%                     k=k+1;
%                 end
%                 l=j+fix((k-1-j)/2);
%                 map(l)=i;
%                 bridgeindex(i)=l;
%                 leftindex(i)=j;
%                 rightindex(i)=k;
%                 leftweight(i)  = (k-l)/(k+1-j);
%                 rightweight(i) = (l+1-j)/(k+1-j);
%                 sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
%                 j=k+1;
%                 if j>=Nsteps+1
%                     j=1;
%                 end
%             end
% 
%             path(:,Nsteps)=sigma(1)*rn(:,1);
%             for i=2:Nsteps
%                 j=leftindex(i);
%                 k=rightindex(i);
%                 l=bridgeindex(i);
%                 if j~=1
%                     path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
%                 else
%                     path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
%                 end
%             end
%         end
%         
%         function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
%             size = tdma.size;
%             out = vector;
%             
%             for i=2:size
%                 out(i) = out(i) - tdma.ll(i)*out(i-1);
%             end
%             
%             out(size) = out(size)/tdma.dd(size);
%             
%             for i=size-1:-1:1
%                 out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
%             end
%             
%         end
%         
%         function out = LUDecompose(eqCOMDupireSpotGF,tdma)
%             out = tdma;
%             if out.bupdated == 1
%                 for i=1:out.size
%                     out.uu(i) = out.c(i);
%                 end
%                 
%                 out.dd(1) = out.b(1);
%                 for i=2:out.size
%                     out.ll(i) = out.a(i)/out.dd(i-1);
%                     out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
%                 end
%                 
%                 out.bupdated = 0;
%             end
%         
%         end
%         
%         function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
%     
%             x = zeros(n,1);
%             y = zeros(n,1);
%             d = zeros(n,1);
%             %fwd
% 
%             x(1) = 1.0;
%             x(2) = 1.0;
% 
%             for i=3:n
%                 x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
%             end
% 
%             %bwd
%             y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
%             y(n-1) = y(n);
% 
%             for i=n-2:-1:1
%                 y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
%             end
% 
%             %set d
% 
%             d(1) = x(1)*y(1)/b(1);
%             for i=2:n
%                 d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
%             end
% 
% 
%         end
% 
%         
%         function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)
% 
%             localVol = zeros(length(Ks),1);
%             volProxyKs = zeros(1,length(volProxy));
%             try
%                 fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
%             catch ME
%                 aaa = 0;
%             end
%             
%             if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
%                     aaa =1.0;
%                 else
%                     for i=1:length(volProxy)
%                         volProxyKs(i) = Ks(ipos(i));
%                     end
% 
%                     localVol = pchip(volProxyKs,volProxy,Ks);
%                     %flat extrapolation at the ends
%                     for i=1:length(Ks)
%                         if Ks(i) <= Ks(ipos(1))
%                             localVol(i) = localVol(ipos(1));
%                         elseif Ks(i) >= Ks(ipos(length(ipos)))
%                             localVol(i) = localVol(ipos(length(ipos)));
%                         end
%                     end
%                 end
%             elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
%                 for i=1:length(volProxy)
%                     volProxyKs(i) = fwdMoneynessPerExpiry(i);
%                 end
%                 
%                 for i=1:length(Ks)
%                     localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
%                 end   
%                 
%             else
%                 for i=1: length(volProxy)
%     %                 Extrapolation at the left and right
%                     if i == 1
%                         for j=1:ipos(i)-1
%                             localVol(j) = volProxy(i);
%                         end
%                     elseif i == length(volProxy)
%                         for j=ipos(i-1):ipos(i)-1
%                             localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
%                         end
%                         for j=ipos(i):length(Ks)
%                             localVol(j) = volProxy(i);
%                         end
%     %                  linear interpolation in the spot grid in-between    
%                     else
%                         for j=ipos(i-1):ipos(i)-1
%                             localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
%                         end
%                     end
%                 end
%             end
%             
%         end
%                         
%         
%         function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
%             
%             %filling proxy grid vol
%             proxy = zeros(length(lastC),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);
% 
%             %filling end
%             
%             meshC = lastC;
%             meshP = lastP;
%             
%             tdma.size = length(meshC);
%             tdma.bupdated = 0;
%             tdma.a = zeros(tdma.size,1);
%             tdma.b = zeros(tdma.size,1);
%             tdma.c = zeros(tdma.size,1);
%             tdma.uu = zeros(tdma.size,1);
%             tdma.ll = zeros(tdma.size,1);
%             tdma.dd = zeros(tdma.size,1);
%             
%             dK2 = dK*dK;
%             for i=2:tdma.size-1
%                 tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%             end
%             
%             tdma.a(1) = 0;
%             tdma.a(tdma.size) = 0;
%             tdma.c(1) = 0;
%             tdma.c(tdma.size) = 0;
%             tdma.b(1) = 1;
%             tdma.b(tdma.size) = 1;
%             tdma.bupdated = 1;
%             
%             tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%             pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
%             newC = pmeshC;
%             
%             pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
%             newP = pmeshP;
%         end
%         
%         function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
%             
%             %filling proxy grid vol
%             proxy = zeros(length(lastProb),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);
% 
%             %filling end
%             
%             meshProb = lastProb;
%             
%             size = length(meshProb);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             
%             condProb = zeros(size,size);
%             if optParams.params.GFType == -1
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             elseif optParams.params.GFType == 0
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expmdemo1(A_rg*NT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             
%             elseif optParams.params.GFType == 1
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg*NT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             elseif optParams.params.GFType == 2
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg);
%                 
%                 newProb = lastProb;
%                 condProb = eye(size,size);
%                 for t=1:NT
%                     newProb = newProb*A_exp1;
%                     condProb = A_exp1*A_exp1;
%                 end
%                 
%             elseif optParams.params.GFType == 3
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%             
%                 % Test for HugeDecomposition for inverse matrix
%                 [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
%                 A = gallery('tridiag',a1,b,c1);
%                 oneStep = OneStepGF(size,x,y,d,A);
%                 newProb = lastProb;
%                 newProb = newProb*oneStep;
%                 
%                 condProb = oneStep;
%                 
%             else
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 % rescale matrix element to smaller time step
%                 a= a*oneStepDt/dT;
%                 b = (b-1)*oneStepDt/dT + 1.0;
%                 c = c*oneStepDt/dT;
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%             
%                 % Test for HugeDecomposition for inverse matrix
%                 [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
%                 A = gallery('tridiag',a1,b,c1);
%                 oneStep = OneStepGF(size,x,y,d,A);
%                 newProb = lastProb;
%                 %numberOfTimeStep;
%                 %oneStepDt = 1.0/365;
%                 
%                 condProb = eye(size,size);
%                 for t=1:NT
%                     newProb = newProb*oneStep;
%                     condProb = oneStep*oneStep;
%                 end 
%                 
%             end
%             
%             out.newProb = newProb;
%             out.condProb = condProb;
%         end
%         
%         function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = params.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = params.Ks(2) - params.Ks(1);
%             dT = (fromTime - toTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             if params.backGFType == -1
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 newP = A_exp1*lastP;
%                 
%             elseif params.backGFType == 0
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expmdemo1(A_rg*NT);
%                 newP = A_exp1*lastP;
%             elseif params.backGFType == 1
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg*NT);
%                 newP = A_exp1*lastP;
%             elseif params.backGFType == 2
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg);
%                 
%                 newP = lastP;
%                 
%                 for t=1:NT
%                     newP = A_exp1*newP;
%                 end
%                 
%             elseif params.backGFType == -21
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
% %                 A_exp1 = expm(A_rg);
%                 
%                 newP = lastP;
%                 
%                 for t=1:NT
%                     newP = A_exp1*newP;
%                 end
%                 
%             elseif params.backGFType == 3
% 
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%                 
%                 tdma.size = length(lastP);
%                 tdma.a = a;
%                 tdma.b = b;
%                 tdma.c = c;
%                 tdma.uu = zeros(tdma.size,1);
%                 tdma.ll = zeros(tdma.size,1);
%                 tdma.dd = zeros(tdma.size,1);
% 
%                 tdma.bupdated = 1;
% 
%                 tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%                 newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);
% 
%                 
%             elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 % rescale matrix element to smaller time step
%                 a= a*oneStepDt/dT;
%                 b = (b-1)*oneStepDt/dT + 1.0;
%                 c = c*oneStepDt/dT;
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%                 
%                 tdma.size = length(lastP);
%                 tdma.a = a;
%                 tdma.b = b;
%                 tdma.c = c;
%                 tdma.uu = zeros(tdma.size,1);
%                 tdma.ll = zeros(tdma.size,1);
%                 tdma.dd = zeros(tdma.size,1);
% 
%                 tdma.bupdated = 1;
% 
%                 tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%                 
%                 newP = lastP;
%                 for t=1:NT
%                     newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
%                 end
% 
%             else 
%                 disp('unImplemented')
%             end
%         end
%         
%         function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = Ks(2) - Ks(1);
%             dT = (fromTime - toTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             newP = A_exp1*lastP;
%             
%         end
%         
%         function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(Ks),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(proxy);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = Ks(2) - Ks(1);
%             dT = (toTime - fromTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             out = A_exp1;
%             
%         end
%         
%         function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)
% 
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
%         end
%         
%         function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)
% 
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %             Ks = params.Ks;
%             localVol =  zeros(length(x),1);
%             fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             
%             % we use the same linear interpolation scheme as calibration
%             useVolProxyInterpolationYN = 'YES';
%             if strcmp(useVolProxyInterpolationYN,'YES')
%                 
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%             else
%                 volKnotPoints = zeros(length(ipos),1);
%                 for i=1: length(ipos)
%                     volKnotPoints(i) = Ks(ipos(i));
%                 end
% 
% 
%                 for i=1:length(x)
%                     xIdx = find(volKnotPoints <=x(i));
%                     if isempty(xIdx)
%                         localVol(i) = localVolLine(1);
%                     elseif max(xIdx) == ipos(length(ipos))
%                         localVol(i) = localVolLine(length(ipos));
%                     else
%                         localVolIdx = max(xIdx);
%                         weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
%                         localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
%                     end
%                 end
%             end
%             
%         end
%         
%         function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
%             
%             NMC = length(lastX);
%             % we generate conditional probability matrix first
%             condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
%             Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
%             
% %             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
%             uSample = U;
%             
%             nextX = zeros(NMC,1);
%             nextXIdx = zeros(NMC,1);
% %             Ks = params.Ks;
%             nPoints = length(Ks);
%             
%             for i=1:NMC
%                 IndexMc = lastXIdx(i);
%                 j= IndexMc;
%                 cumProb = Qa(IndexMc,:);
%                 idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
%                 nextX(i) = Ks(idxH);
%                 nextXIdx(i) = idxH;
% 
%             end
%             
%             out.nextX = nextX;
%             out.nextXIdx = nextXIdx;
% 
%             
%         end
%         
%         function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
%             
%             timeSchedule = timeScheduleInfo.timeSchedule;
%             dTSchedule365 = timeScheduleInfo.dTSchedule365;
%             volKnotIdx = timeScheduleInfo.volKnowIdx;
%             
%             predictor = lastX;
%             for j=startIdx:endIdx-1
%                 
% %                 localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j),params);
%                 localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
%                 Dt = dTSchedule365(j);
%                 sqrtDt = sqrt(dTSchedule365(j));
%                 for i=1:length(lastX)
%                     predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
% %                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                 end
%             end
%             newX = predictor;
%             
%         end
%         
%         function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
%             
%             predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
%             Dt = (toTime - fromTime)/365.0;
%             sqrtDt = sqrt(Dt);
%             for i=1:length(lastX)
% %                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
%                 predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
%                 
%             end
%             
%             mcmcOut.nextX = predictor;
%             
%         end
%         
%         
%         function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
%             % Calculate Object Function
%             % settings for bisection
%             a = 0.001;
%             b = 10;
%             Tol = 1e-10;
%             MaxIter = 1000;
% 
%             strikes = params.marketStrikes;
%             strikeSize = length(params.marketStrikes);
%             out.vols    = zeros(strikeSize,1);
%             out.interpOTMPrices = zeros(strikeSize,1);
%             out.interpBlackPrices = zeros(strikeSize,1);
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             cValue = meshProb*meshC;
%             pValue = meshProb*meshP;
%             
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 if K <= 1.0
%                       out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
%                      out.interpOTMPrices(i) = pValue(i);
%                      out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
%                      
%                 else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
%                     out.interpOTMPrices(i) = cValue(i);
%                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
%                 end
%             end
%             
%             
%         end
%         
%         function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
%             % Calculate Object Function
%             % settings for bisection
%             a = 0.001;
%             b = 10;
%             Tol = 1e-10;
%             MaxIter = 1000;
% 
%             strikes = params.marketStrikes;
%             strikeSize = length(params.marketStrikes);
%             out.vols    = zeros(strikeSize,1);
%             out.interpOTMPrices = zeros(strikeSize,1);
%             out.interpBlackPrices = zeros(strikeSize,1);
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             cValue = meshProb*meshC;
%             pValue = meshProb*meshP;
%             
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 if K <= 1.0
% %                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
%                      out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
%                      out.interpOTMPrices(i) = pValue(i);
%                      out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
%                      
%                 else
% %                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
%                     out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
%                     out.interpOTMPrices(i) = cValue(i);
%                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
%                 end
%             end
%             
%             
%         end
%         
%         function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
%             expiry = params.expiry;
%             
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             mesh = zeros(modelXSize,1);
%             
%             if strike <= 1.0
%                 for i=1:modelXSize
%                     mesh(i,1) = max(strike- modelX(i),0);
%                 end
% 
%                 volExpiry = expiry(find(expiry <= maturity));
%                 volExpiry = [volExpiry; 0];
%                 timeStep = sort(union(volExpiry,maturity),'ascend');
%                 for i=length(timeStep):-1:2
%                     mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
%                 end
%                 
%                 out = mesh(params.Pricingidx);
%                 
%             else
%                 for i=1:modelXSize
%                     mesh(i,1) = max(modelX(i)-strike,0);
%                 end
% 
%                 volExpiry = expiry(find(expiry <= maturity));
%                 volExpiry = [volExpiry; 0];
%                 timeStep = sort(union(volExpiry,maturity),'ascend');
%                 for i=length(timeStep):-1:2
%                     mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
%                 end
%                 
%                 out = mesh(params.Pricingidx);
%                 
%             end
%             
%         end
% %                              MCPayoff(eqCOMDupireSpotGF,i, scheduleSize, comFwd,isStop,dummyisKI,payoffInfo,hitValue);
%         function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
%             
%             modelStatesSize = length(comFwd);
%             scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
%             strike = payoffInfo.autoCallSchedule.strike(idx);
%             coupon = payoffInfo.autoCallSchedule.coupon(idx);
%             nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
%             lowerBarrier = payoffInfo.lowerBarrier;
%             callStrike1 = payoffInfo.callStrike1;
%             callValue1  = payoffInfo.callValue1;
%             
%             columnOut.isStop = zeros(modelStatesSize,scheduleSize);
%             columnOut.worstP = zeros(modelStatesSize,scheduleSize);
%             
%             columnOut.isStop = columnIn.isStop;
%             columnOut.worstP = columnIn.worstP;
%             
%             columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
%             columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%             columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%             
%             columnOut.payoffColumn = columnIn.payoffColumn;
%             
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%             % early redemption
%             % idx  : early redemption schedule iter
%             % idx11: simulation path iter
%             if idx == 1
%                 for idx11 =1:modelStatesSize % 1st early redemption
%                         % alive path
%                         if comFwd(idx11) >= basePrice*strike % early redemption conditon met
%                            columnOut.isStop(idx11,idx) = 1;
%                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
%                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
%                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                         end
%                         
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                     
%                 end
%                 
%             elseif idx > 1 && idx < scheduleSize
%                 for idx11 =1:modelStatesSize
%                     if columnOut.isStop(idx11,idx-1) == 1 % terminated path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
%                         columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
%                     else
%                         % alive path
%                         if comFwd(idx11) >= basePrice*strike % early redemption conditon met
%                             columnOut.isStop(idx11,idx) = 1;
%                             columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                             columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
%                             columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
%                             columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                         end
%                         
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                     end
%                 end
%                 
%             else % idx == scheduleSize
%                 for idx11 =1:modelStatesSize
%                     if columnOut.isStop(idx11,idx-1) == 1 % terminated path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
%                         columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
% 
%                     else
%                         %alive path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                         % barrier hit path
%                         if isKI(idx11) == 1
%                             if comFwd(idx11) < basePrice*callStrike1 
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
%                             elseif comFwd(idx11) < basePrice*strike
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
%                             else
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                         else % barrier unhit path
%                             if comFwd(idx11) < basePrice*callStrike1 
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
%                             elseif comFwd(idx11) < basePrice*strike
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
%                             else
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                             
%                             if comFwd(idx11) > basePrice*lowerBarrier
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                         end
%                         
%                         % payoff postprocess
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
%                         columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                     end    
%                 end
%             end    
%             
%             
%         end
%         
%         function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%             basePrice = payoffInfo.basePrice;
%             lowerBarrier = payoffInfo.lowerBarrier;
%             modelStatesSize = length(comFwd);
%             out = zeros(modelStatesSize,1);
%             for idx=1:modelStatesSize
%                 if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
%                     out(idx) = 1.0;
%                 end
%             end
%         end
%         
%         % NM : nearest month forward
%         function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%            % NM : nearest month forward
%         function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
%             if observeTime > maturityTime
%                 disp('observeTime is passed maturityTime!')
%                 out = 0;
%                 return;
%             end
%             
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(maturityTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%         function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
% %             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%         function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
% %             out =0;
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% MCMC init 
%             useQuasiRandomYN = 'true';
%             
%             if strcmp(useQuasiRandomYN,'true')
% %                 NMC = 2^14;
% %                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Ps = sobolset(MCTimeStep,'Skip',NMC);
%                 Q = qrandstream(Ps);
% 
%                 reset(Q);
%                 U0 = qrand(Q,NMC);
%                 U = zeros(NMC,MCTimeStep);
% 
%                 dZ0 = zeros(size(U,1),size(U,2));
%     %             dZ = norminv(U);
%                 % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
%                 % generate random number from the end
%                 for i=1:size(U,1)
%                     for j=1:size(U,2)
%                         dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
%                     end
%                 end
%             
%                % apply brownian bridge
%                dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
%                U(:,1) = dZ(:,1);
%                for i=2:size(dZ,2)
%                    U(:,i) = dZ(:,i) - dZ(:,i-1); 
%                end
% %                U = dZ;
%             else
%                 rng('default');
%                 rng(0);
%                 
%                 NMC = 40000;
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
% %                 Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
% 
%             end
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            initX = ones(NMC,1);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
%             
%           %% payoff init
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
%            
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
%            
%            
%            modelStatesSize = NMC;
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
%            
%            % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%            isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
% %           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%            %% Path Generation Forward
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     
%                     nextX = mcmcOut.nextX;
% %                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
% %                 dummyisKI = ones(modelStatesSize,1);
% %                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
% %                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
%                 nMFuture.payoff = comFwd;
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%             
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end
% 
% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
% %             out =0;
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% MCMC init 
%             useQuasiRandomYN = 'true';
%             
%             if strcmp(useQuasiRandomYN,'true')
% %                 NMC = 2^14;
% %                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Ps = sobolset(MCTimeStep,'Skip',NMC);
%                 Q = qrandstream(Ps);
% 
%                 reset(Q);
%                 U0 = qrand(Q,NMC);
%                 U = zeros(NMC,MCTimeStep);
% 
%                 % first random numbers goes to the enddate..
%                 for i=1:size(U,2)
%                     U(:,i) = U0(:,size(U,2)-(i-1));
%                 end
%             else
%                 rng('default');
%                 rng(0);
%                 
%                 NMC = 40000;
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
% %                 Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
% 
%             end
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
%            nextXIdx = initXIdx;
%             
%           %% payoff init
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
%            
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
%            
%            
%            modelStatesSize = NMC;
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
%            
%            % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%            isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
% %           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%            %% Path Generation Forward
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
%                     nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
% %                 dummyisKI = ones(modelStatesSize,1);
% %                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
% %                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
%                 nMFuture.payoff = comFwd;
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%             
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end
% 
% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%              
%         function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            
%            modelStatesSize = length(modelStates);
%            
%            %down barrier touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
%            
%            comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%            strikeMat = autoCallSchedule.strike(scheduleSize);
%            couponMat = autoCallSchedule.coupon(scheduleSize);
%            
% %            strikeOH = strikeMat + overHedgeShift;
%            % samsung overhedge payoff
%            if strcmp(SSpayoffYN,'YES')
%                for i=1:modelStatesSize
%                    if comFwd(i) < basePrice*callStrike1
%                         payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
%                    elseif comFwd(i) < basePrice*strikeMat 
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
%                    else
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
%                    end
%                end
% 
%                payoffStateB.cashflow = payoffStateA.cashflow;
% 
%                for i=1:modelStatesSize
%                     if comFwd(i) > basePrice*lowerBarrier
%                         payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
%                     end
%                end
%                
%            else
%                for i=1:modelStatesSize
%                    if comFwd(i) >= basePrice*strikeMat
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
%                    else
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
%                        payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
%                    end
%                end
% 
%                payoffStateB.cashflow = payoffStateA.cashflow;
% 
%                for i=1:modelStatesSize
%                     if comFwd(i) > basePrice*lowerBarrier
%                         payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
%                     end
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             hitValue.payoff = payoffStateA.cashflow_npv;
%             
%             unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             unhitValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
%                 
%                 endTimeIdx = endTimeIdx + 1;
%             %% induction start
%                 mesh = hitValue.payoff;
%                 meshB = unhitValue.payoff;
%                 for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx -1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%                     for j=1:modelStatesSize
%                         if comFwd(j) <= basePrice*lowerBarrier
%                            meshB(j) = mesh(j); 
%                         end
%                     end
%                 end
%                 
%                 hitValue.payoff = mesh;
%                 unhitValue.payoff = meshB;
%                 nMFuture.payoff = comFwd;
%               %% induction end
%                 
%                 % backward induction in the payoff script
%                 currentNodeIdx = currentNodeIdx -1;
%               %% AtDate operation on Early ExerciseDate
%                 
%                 if i > lastAliveExeriseIdx
%                     currentTime = timeStep(lastTimeIdx);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValue  = hitValue.payoff/onePayoffD;
%                     contiValueB = unhitValue.payoff/onePayoffD;
%                     
%                     %AutoCall applied
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                      % intermediate variables
%                     payoffStateA.cashflow = zeros(modelStatesSize,1);
%                     payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     payoffStateC.cashflow = zeros(modelStatesSize,1);
%                     payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     % samsung overhedge payoff
%                     if strcmp(SSpayoffYN,'YES')
%                         for j=1:modelStatesSize
%                             if comFwd(j)/basePrice >= strikeEarly
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
%                             elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                
%                                payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                             else
% %                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% %                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% % 
% %                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
% %                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
%                                 
%                                 payoffStateA.cashflow(j) = contiValue(j);
%                                 payoffStateB.cashflow(j) = contiValueB(j);
%                            end
%                         end
%                     else
%                         for j=1:modelStatesSize
%                             if comFwd(j) >= basePrice*strikeEarly
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
%                             else
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
%                            end
%                         end
%                         
%                     end
%                     % barrier
%                     for j=1:modelStatesSize
%                         if comFwd(j) <= basePrice*lowerBarrier
%                            payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
%                         end
%                     end
%                     
%                     payoffStateC.cashflow = comFwd;
%                     
%                     for j=1:modelStatesSize
%                         payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
%                         payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
%                         payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
%                     end
%                     
%                     %process payoff at early exercise date
%                     hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%                     hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%                     hitValue.payoff = payoffStateA.cashflow_npv;
%                     
%                     unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%                     unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%                     unhitValue.payoff = payoffStateB.cashflow_npv;
%                     
%                     nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%                     nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%                     nMFuture.payoff = payoffStateC.cashflow_npv;
%                     
%                 end
%                    
%            end
%            
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            
%            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = unhitValue.payoff(Pricingidx);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
%            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
%             % model schedule generation
%             expiryDate = H_Date(vanillaParams.params('maturityDate'));
%             scheduleSize = 1;
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             lastAliveExerciseIdx = 1;
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            modelStatesSize = length(modelStates);
%            
%            %europeanValue
%            europeanValue.npv = 0;
%            europeanValue.payoff = zeros(modelStatesSize,1);
%            europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
%            europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            %americanValue
%            americanValue.npv = 0;
%            americanValue.payoff = zeros(modelStatesSize,1);
%            americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
%            americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
%            
%            comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%            
%            if strcmp(vanillaParams.params('callPutFlag'),'C')
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
%                     payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
%                end
%            else
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
%                     payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             europeanValue.payoff = payoffStateA.cashflow_npv;
%             
%             americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             americanValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            
%            for i=scheduleSize:-1:lastAliveExerciseIdx
%                 % past schedule neglect
%                 startTimeIdx = eventTimeIdx(i);
%                 endTimeIdx = 1;  % valueDate
%                 
%                 endTimeIdx = endTimeIdx + 1;
%             %% induction start
%                 mesh = europeanValue.payoff;
%                 meshB = americanValue.payoff;
%                 for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     % one step backward induction end
%                     lastTimeIdx = lastTimeIdx -1;
%                     currentTime = timeStep(lastTimeIdx);
%                     
%                     %apply american option
%                     comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValueB  = meshB/onePayoffD;
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     if strcmp(vanillaParams.params('callPutFlag'),'C')
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     else
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     end
%                     payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
%                     meshB = payoffStateB.cashflow_npv;
%                     
%                 end
%                 
%                 europeanValue.payoff = mesh;
%                 americanValue.payoff = meshB;
% %                 nMFuture.payoff = comFwd;
%               %% induction end
%                 
%                 % backward induction in the payoff script
%                 currentNodeIdx = currentNodeIdx -1;
%            end
%            
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            
%            europeanValue.npv = europeanValue.payoff(Pricingidx);
%            americanValue.npv = americanValue.payoff(Pricingidx);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
%            out.europeanValue = europeanValue; 
%            out.americanValue = americanValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
%             expiry = params.expiry;
%             
%             initX = ones(NMC,1);
%             
%             payoff = zeros(NMC,length(strikes));
%             volExpiry = expiry(find(expiry <= maturity));
%             volExpiry = [volExpiry; 0];
%             timeStep = sort(union(volExpiry,maturity),'ascend');
%             nextX = initX;
%             %divide time interval into volExpiry
%             for i=2:length(timeStep)
%                 startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
%                 endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
%                 dZStep = dZ(:,startIdx+1:endIdx);
%                 nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
%             end
%             
%             
%             for j=1:length(strikes)
%                 if(strikes(j) <= 1)
%                     payoff(:,j) = max(strikes(j) - nextX,0);
%                 else
%                     payoff(:,j) = max(nextX - strikes(j),0);
%                 end
%             end
%             out = mean(payoff);
%             
%         end
%         
%         function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
%             expiry = params.expiry;
%             Ks = params.Ks;
%             initX = ones(NMC,1);
%             initXIdx = ones(NMC,1)*params.Pricingidx;
%             
%             payoff = zeros(NMC,length(strikes));
%             volExpiry = expiry(find(expiry <= maturity));
%             volExpiry = [volExpiry; 0];
%             timeStep = sort(union(volExpiry,maturity),'ascend');
%             nextX = initX;
%             nextXIdx = initXIdx;
%             for i=2:length(timeStep)
%                 mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
%                 nextX = mcmcOut.nextX;
%                 nextXIdx = mcmcOut.nextXIdx;
%             end
%             
%             
%             for j=1:length(strikes)
%                 if(strikes(j) <= 1)
%                     payoff(:,j) = max(strikes(j) - nextX,0);
%                 else
%                     payoff(:,j) = max(nextX - strikes(j),0);
%                 end
%             end
%             out = mean(payoff);
%             
%         end
%         
%         function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             pdeOTMPrices =  zeros(length(expiry),length(strikes));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
% %                 strikePerExpiry = params.fwdMoneyness(i,:);
%                 for j=1:length(strikePerExpiry)
%                     pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
%                 end
%             end
%             
%             out = pdeOTMPrices;
%             
%         end
%         
%         
%         % compute forward mc(euler scheme) for 8*21 vanilla otm products
%         % generate time steps from the enddate backwards 
%         
%         function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             % we generate the schedule from the enddate
%             timeStepSize = params.mcOneTimeStep;
%             MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
%             dummyTimeSchedule = zeros(MCTimeStep,1);
%             
%             dummyN = 0;
%             for i=1:MCTimeStep
%                 dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
%                 if dummyTimeSchedule(i) < 0
%                     break;
%                 end
%                 dummyN = dummyN +1;
%                 
%             end
%             
%             timeSchedule = zeros(dummyN,1);
%             for i=1:dummyN
%                 timeSchedule(i)= dummyTimeSchedule(i);
%             end
%             
%             volExpiry = [expiry; 0];
%             
%             newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
%             
%             dTSchedule = zeros(length(newTimeSchedule)-1,1);
%             dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
%             for i=length(newTimeSchedule)-1:-1:1
%                 dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
%                 dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
%             end
%             
%             % find the vol expiry points
%             
%             volKnotIdx = zeros(length(expiry),1);
%             for i=1:length(expiry)
%                 volKnotIdx(i) = find(newTimeSchedule==expiry(i));
%                 
%             end
%             
%             timeScheduleInfo.timeSchedule = newTimeSchedule;
%             timeScheduleInfo.dTSchedule = dTSchedule;
%             timeScheduleInfo.dTSchedule365 = dTSchedule365;
%             timeScheduleInfo.volKnowIdx = volKnotIdx;
%             
%             
%             % schedule create end
% %             NMC = params.NMC;
%             NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
% %             Ps = sobolset(length(dTSchedule));
%             
%             Ps = sobolset(length(dTSchedule),'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             Q.reset;
%             U = qrand(Q,NMC);
%             dZ0 = zeros(size(U,1),size(U,2));
% %             dZ = norminv(U);
%             % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
%             % generate random number from the end
%             for i=1:size(U,1)
%                 for j=1:size(U,2)
%                     dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
%                 end
%             end
%             
%            % apply brownian bridge
%            dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
%            
%            % first dZ is dummy , 0
%            dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
%            dZ(:,2) = dZ1(:,1);
%            for i=3:size(dZ1,2)+1
%                 dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
%            end
%            
%             mcOTMPrices =  zeros(length(expiry),size(strikes,2));
% %             mcOTMPrices =  zeros(length(expiry),length(strikes));
%             
%             mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
%                 mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
%                 for j=1:size(strikes,2)
%                     mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
%                 end
%             end
%             
%             out = mcOTMPrices;
%             
%         end
%         
%         
%         % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
%         % generate time steps using only events date 
%         
%         function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
%             mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
%                 mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
%                 for j=1:size(strikes,2)
%                     mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
%                 end
%             end
%             
%             out = mcOTMPrices;
%             
%         end
%         
%         function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
%             
%             expiry = params.expiry;
%             lastP = zeros(1,length(params.Ks));
%             lastP(params.Pricingidx) = 1.0;
%             newP = zeros(1,length(params.Ks));
%             
%             
%             idx = find(maturity <= expiry);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             
%             %relevent volProxy
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             
%             %relevent probMesh and dTDays
%             dTDays = 0;
%             if idxNow~=1
%                 probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
%                 lastP = probMesh;
%                 dTDays = (maturity - expiry(idxNow-1));
%             else % idxNow == 1
%                 dTDays = maturity;
%             end
%             
%             
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
% 
%             %filling end
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
% 
%             dK = Ks(2) - Ks(1);
%             dT = dTDays/365.0;
% 
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
% 
%             a(1) = 0;
%             c(size) = 0;
% 
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
% 
%             b(1) = 1;
%             b(size) = 1;
% 
%             % to create tridiagonal matrix
% 
%             a1 = a(2:size);
%             c1 = c(1:size-1);
% 
%             b_rg(1) = 0;
%             b_rg(size) = 0;
% 
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             newP = lastP*A_exp1;
%             
%             
%             impVol = zeros(1,length(strikes));
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             strikeSize = length(strikes);
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             impVolC = zeros(1,strikeSize);
%             impVolP = zeros(1,strikeSize);
%             impVol  = zeros(1,strikeSize);
%             cValue = newP*meshC;
%             pValue = newP*meshP;
%             
%             out.cValue = cValue;
%             out.pValue = pValue;
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
%                 impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
%                 if K <= 1.0
%                      impVol(i) = impVolP(i);
%                 else
%                      impVol(i) = impVolC(i);
%                 end
%             end
%             
%             out.impVolP = impVolP;
%             out.impVolC = impVolC;
%             out.impVol = impVol;
%             out.probMeshInt = newP;
%             
%         end
%         
%         function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
% %             expiry = params.expiry;
% %             strikes = params.marketStrikes;
% %             
% %             %mc random number initialize
% % %             NMC = 2^14;
% %             NMC = params.NMC;
% %             
% %             MCTimeStep = length(expiry); % -1 because 0
% % 
% %             Ps = sobolset(MCTimeStep,'Skip',NMC);
% % %             Ps = scramble(Ps,'MatousekAffineOwen');
% %             Q = qrandstream(Ps);
% %             
% %             reset(Q);
% %             U0 = qrand(Q,NMC);
% %             U = zeros(NMC,MCTimeStep);
% %             
% %             % first random numbers goes to the enddate..
% %             for i=1:size(U,2)
% %                 U(:,i) = U0(:,size(U,2)-(i-1));
% %             end            
% %             
%             targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
%             targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
%             cValue = zeros(1,size(targetStrikes,2));
%             pValue = zeros(1,size(targetStrikes,2));
%             probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
%             
%             for i=1:length(targetExpiry)
%                 strikePerExpiry = targetStrikes(i,:);
%                 outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
%                 targetImpVolPerExpiry = outDummy.impVol;
%                 cValue = outDummy.cValue;
%                 pValue = outDummy.pValue;
%                 probMeshIntPerExpiry = outDummy.probMeshInt;
%                 
%                 for j=1:size(targetStrikes,2)
%                     targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
%                     cValueSurface(i,j) = cValue(j);
%                     pValueSurface(i,j) = pValue(j);
%                     probMeshInt(i,j) = probMeshIntPerExpiry(j);
%                 end
%             end
%             
%             out.impVolSurface = targetImpVolSurface;
%             out.cValueSurface = cValueSurface;
%             out.pValueSurface = pValueSurface;
%             out.probMeshInt = probMeshInt;
%         end
%         
%         %generate marginal(cumulative distribution) at each event
%         %date(per expiry)
%         %using conditional transition Probability matrix
%         function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
% %             Ks = params.Ks;
% %             nPoints = length(params.Ks);
%             
%             nPoints = length(Ks);
%             condProb = condProbPerExpiry;
%             Qa = zeros(length(Ks),length(Ks));
%             cumM = 0;
%             cumM2 = 0;
%             %first row dealt separately
%             Qa(1,1) = condProb(1,1);
%             cumM2 = 0.0;
%             for j=2:nPoints
%                 cumM2 = cumM2 + condProb(1,j);
%                 Qa(1,j) = Qa(1,1) + cumM2;
%             end
%             
%             for i=2:nPoints
%                 cumM = 0.0;
%                 Qa(i,i) = condProb(i,i);
%                 for j=i-1:-1:1
%                     Qa(i,i) = Qa(i,i) + condProb(i,j);
%                 end
% 
%                 cumM = 0.0;
%                 for j=i-1:-1:1
%                     cumM = cumM + condProb(i,j+1);
%                     Qa(i,j) = Qa(i,i) - cumM;
%                 end
% 
%                 cumM2 = 0.0;
%                 for j=i+1:nPoints
%                     cumM2 = cumM2 + condProb(i,j);
%                     Qa(i,j) = Qa(i,i) + cumM2;
%                 end
%             end
%             
%             out = Qa;
%         end
%         
%         %generate marginal(cumulative distribution) for all eventdates
%         %date(per expiry)
%         %using conditional transition Probability matrix
%         
%         function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
%             Ks = params.Ks;
%             marginal = zeros(params.expirySize,length(Ks),length(Ks));
%             condProbPerExpiry = zeros(length(Ks),length(Ks));
%             marginalPerExpiry = zeros(length(Ks),length(Ks));
%             for i=1:params.expirySize
%                 condProbPerExpiry(:,:) = condProb(i,:,:);
% %                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
%                 marginal(i,:,:) = marginalPerExpiry(:,:);
%             end
%         end
%         
%         
%         % Target function for FixedPointIteration Optimizer
%         
%         function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
%             idxNow = optParams.idxNow;
%             if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
%             mkt_dT=mkt_dT/365.0;
%             
%             
%           %% Solve  Forward PDE(GF) One time
%             outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
%             
%             newProb = outGF.newProb;
%             optParams.meshProb = newProb;
%            %% PDE(GF) Solve End
%            % find the implied vol of dupire model
%            if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
%                target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
%            else
%                target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
%            end
%            marketImpVol = optParams.params.blackVol;
%            expiry = optParams.params.expiry;
%            strike = optParams.params.strike;
%            
%            out.newLocalVol = zeros(length(optParams.strikeSize),1);
%            out.localVolError = zeros(length(optParams.strikeSize),1);
%            out.impVolError =  zeros(length(optParams.strikeSize),1);
%            out.modelImpVol = zeros(length(optParams.strikeSize),1);
%            out.marketImpVol = zeros(length(optParams.strikeSize),1);
%            out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
%            out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
%            out.priceRe = zeros(length(optParams.strikeSize),1);
%            
%            for i=1:optParams.strikeSize
%                 out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
%                 out.localVolError(i) = out.newLocalVol(i) - tvar(i);
%                 out.modelImpVol(i) = target.vols(i);
%                 out.marketImpVol(i) =  marketImpVol(idxNow,i);
%                 out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
%                 out.interpOTMPrices(i) = target.interpOTMPrices(i);
%                 out.interpBlackPrices(i) = target.interpBlackPrices(i);
%                 out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
%            end
%            
%            % for shorter tenor we only fit for the restricted interval
%             if idxNow <= optParams.params.numOfCutoffTenor
%                 for i=1:optParams.lowerIdx-1
%                     out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
%                 end
% 
%                 for i=optParams.upperIdx+1:optParams.strikeSize
%                     out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
%                 end
%             end
%             
%             out.volErrorTotal = 0.0;
%             out.priceReTotal = 0.0;
%             %for one month we only fit betwee lower and upper
%             dummyN = 0;
%             if idxNow <= optParams.params.numOfCutoffTenor
%                 for i=optParams.lowerIdx:optParams.upperIdx
%                     out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
%                     out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
%                     dummyN = dummyN +1;
%                 end
%             else
%                 for i=1:length(out.impVolError)
%                     out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
%                     out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
%                     dummyN = dummyN +1;
%                 end
%             end
%             out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
%             out.priceReTotal = sqrt(out.priceReTotal/dummyN);
%             
%         end
%         
%          % Fixed Point Iteration Optimizer
%          % start from implied vol find the local vol using Reghai's fixed
%          % point iteration optimization
%          % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
%          
%         function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
%             
%             n= 0;
%             errorTry = 1000.0; % Initial ErrorBound make it big enough
%             tvarOld = tvar;
%             if optParams.params.targetType == 1
%                 while (errorTry > tol) && (n < maxIter)
%                     out = targetfunc(tvar);
%                     tvarOld = tvar;
%                     % reduce implied vol error
%                     errorTry = out.volErrorTotal;
%                     tvar = out.newLocalVol;
%                     n = n+1;
%                 end
%                 x = tvar;
%                 finalOut = targetfunc(tvarOld);
%                 fval = finalOut.volErrorTotal;
%                 marketImpVol = finalOut.marketImpVol;
%                 modelImpVol  = finalOut.modelImpVol;
%                 interpBlackPrices = finalOut.interpBlackPrices;
%                 interpOTMPrices = finalOut.interpOTMPrices;
%                 volError     = finalOut.impVolError;
%                 volErrorTotal = finalOut.volErrorTotal;
%                 priceRe      = finalOut.priceRe;
%                 priceReTotal = finalOut.priceReTotal;
%                 nIter = n;
%             else
%                 while (errorTry > tol) && (n < maxIter)
%                     out = targetfunc(tvar);
%                     tvarOld = tvar;
%                     % reduce price relative error
%                     errorTry = out.priceReTotal;
%                     tvar = out.newLocalVol;
%                     n = n+1;
%                 end
%                 x = tvar;
%                 finalOut = targetfunc(tvarOld);
%                 fval = finalOut.volErrorTotal;
%                 marketImpVol = finalOut.marketImpVol;
%                 modelImpVol  = finalOut.modelImpVol;
%                 interpBlackPrices = finalOut.interpBlackPrices;
%                 interpOTMPrices = finalOut.interpOTMPrices;
%                 volError     = finalOut.impVolError;
%                 volErrorTotal = finalOut.volErrorTotal;
%                 priceRe      = finalOut.priceRe;
%                 priceReTotal = finalOut.priceReTotal;
%                 nIter = n;
%             end
%             
%         end
%         
%         % Fixed Local Vol per expiry by bootstrapping
%         
%         function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
%             %initialize calibration parameters
%             
%             optParams.idxNow = idxNow;
%             optParams.dT = dT;
%             optParams.params = params;
%             optParams.ipos = zeros(params.strikeSize,1);
%             optParams.Ks = zeros(params.Ns,1);
%             optParams.Ks = params.Ks;
%             optParams.strikeSize = params.strikeSize;
%             optParams.meshSize = params.Ns;
%             
%             optParams.meshProb = zeros(1,params.Ns);
%             
%             optParams.proxy = zeros(params.Ns,1);
%             optParams.dK = params.Ks(2) - params.Ks(1);
%             
%             optParams.marketStrikes = zeros(params.strikeSize,1);
%             optParams.blackOTMPrices = params.blackOTMPrices;
%             optParams.callCurve = zeros(params.strikeSize,1);
%             optParams.putCurve = zeros(params.strikeSize,1);
%             
%             for i=1:params.strikeSize
%                 optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
%             end
%                 
% %             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
% %                 % market strike points in fwd PDE grid
% %                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
% %                 for i=1:params.strikeSize
% %                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
% %                 end
% %             else
% %                 for i=1:params.strikeSize
% %                     optParams.marketStrikes(i) = params.strike(i);
% %                 end
% %             end
%             
%             % positions to separate vol proxy regimes
%             for i=1:params.strikeSize
%                 optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
%             end
% %             optParams.ipos(params.strikeSize) = optParams.meshSize;
%             
%             optParams.lastProb = lastProb;
%             
%             % exception for one month maturity for restricted interval
%             
%             lowerCutoff = params.lowerCutoffVector(idxNow);
%             upperCutoff = params.upperCutoffVector(idxNow);
%             numOfCutoffTenor = params.numOfCutoffTenor; 
%             
%             lowerIdx = 1;
%             upperIdx = params.strikeSize;
%             
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end
%             
%             optParams.lowerIdx = lowerIdx;
%             optParams.upperIdx = upperIdx;
%             
%             for i=1:params.strikeSize
%                 tvar(i) = params.blackVol(idxNow,i);
%                 lb(i) = 0.01;
%                 ub(i) = 1.0;
%             end
%             
%             for i=1:lowerIdx-1
%                 tvar(i) = tvar(lowerIdx);
%             end
%             
%             for i=upperIdx+1:params.strikeSize
%                 tvar(i) = tvar(upperIdx);
%             end
% 
%             targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
%                 
%             
% %             tol = 0.0001;
% %             maxIter = 1000;
%             
%             tol = params.tol;
%             maxIter = params.maxIter;
%             
%             [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
%             
%             for i=1:params.strikeSize
%                 bootStrapOut.volProxy(i) = x(i);
%                 bootStrapOut.ipos(i) = optParams.ipos(i);
%             end
%             
%             % for later use
%             bootStrapOut.marketStrikes = optParams.marketStrikes;
%             
%             bootStrapOut.volError = volError;
%             bootStrapOut.marketImpVol = marketImpVol;
%             bootStrapOut.modelImpVol = modelImpVol;
%             bootStrapOut.interpBlackPrices = interpBlackPrices;
%             bootStrapOut.dupireOTMPrices = interpOTMPrices;
%             
%             bootStrapOut.nIter = nIter;
%             
%             out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
%             
%             bootStrapOut.lastProb = out.newProb;
%             bootStrapOut.condProb = out.condProb;
%             
%         end
%         
%         function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
%             
%           % Calculate Black OTM Prices as calibration target
%             volSurface = black.volSurface;
%             
%             expiry = volSurface.maturities;
%             strike  = volSurface.strikes;
%             spot =  black.spot.params('rawData');
%             expirySize = length(expiry);
%             strikeSize = length(strike);
%             
%             blackOTMPrices = zeros(expirySize,strikeSize);
%             blackOTMPrices2 = zeros(expirySize,strikeSize);
%             blackVol = zeros(expirySize,strikeSize);
%             blackOTMVega = zeros(expirySize,strikeSize);
%             
%             % we add blackvol as matrix so that in the calibation we don't
%             % need to do 2D interpolation again and again!!
%             % also for the target black price too!!
%             if strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
%                 for k=1:expirySize
%     %                 fwd = black.Fwd(spot,expiry(k));
%     %                 df = black.zeroCurve.DF(expiry(k)/365.0);
%                     for j=1:strikeSize
%                         blackVol(k,j) = volSurface.volSurfaceR(k,j);
%                         fwdMoneyness(k,j) = strike(j);
%                         
%                         if fwdMoneyness(k,j) <= 1
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
%                         else
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
%                         end
%                     end
%                 end
%             else
%                 for k=1:expirySize
%     %                 fwd = black.Fwd(spot,expiry(k));
%     %                 df = black.zeroCurve.DF(expiry(k)/365.0);
%                     for j=1:strikeSize
%                         blackVol(k,j) = volSurface.volSurfaceR(k,j);
%                         if strike(j) <= 0
%                             dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
%                             fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
%                         else
%                             dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
%                             fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
%                         end
% 
%                         if fwdMoneyness(k,j) <= 1
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
%                         else
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
%                         end
%                     end
%                 end
%                 
%             end
%                 
% %             if strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
% %                 for k=1:expirySize
% %                     fwd = black.Fwd(spot,expiry(k));
% %                     df = black.zeroCurve.DF(expiry(k)/365.0);
% %                     fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
% %                     for j=1:strikeSize
% %                         blackVol(k,j) = volSurface.vol(expiry(k),strike(j));
% %                         fwdStrike = strike(j)/fwdFactor;
% %                         if fwdStrike <= 1.0
% % %                             blackOTMPrices(k,j) = black.BlackVanilla(expiry(k),expiry(k),spot,spot*strike(j),volSurface.vol(expiry(k),strike(j)),'P')/fwd/df;
% %                             blackOTMPrices(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,-1);
% %                             blackOTMPrices2(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,-1);
% %                             blackOTMVega(k,j) = BlackFwdVega(1,fwdStrike,blackVol(k,j),expiry(k)/365.0);
% %                         else
% % %                             blackOTMPrices(k,j)  = black.BlackVanilla(expiry(k),expiry(k),spot,spot*strike(j),volSurface.vol(expiry(k),strike(j)),'C')/fwd/df;
% %                             blackOTMPrices(k,j)  = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,1);
% %                             blackOTMPrices2(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,1);
% %                             blackOTMVega(k,j) = BlackFwdVega(1,fwdStrike,blackVol(k,j),expiry(k)/365.0);
% %                             
% %                         end
% %                     end
% %                 end
% %             else
% %                 for k=1:expirySize
% %                     fwd = black.Fwd(spot,expiry(k));
% %                     df = black.zeroCurve.DF(expiry(k)/365.0);
% %                     for j=1:strikeSize
% %                         blackVol(k,j) = volSurface.vol(expiry(k),strike(j));
% %                         if strike(j) <= 1
% %                             blackOTMPrices(k,j) = black.BlackVanilla(expiry(k),expiry(k),spot,fwd*strike(j),volSurface.vol(expiry(k),strike(j)),'P')/fwd/df;
% %                         else
% %                             blackOTMPrices(k,j)  = black.BlackVanilla(expiry(k),expiry(k),spot,fwd*strike(j),volSurface.vol(expiry(k),strike(j)),'C')/fwd/df;
% %                         end
% %                     end
% %                 end
% %             end
% %             
%             
%             params.volSurface = volSurface;
%             params.expiry = expiry;
%             params.strike = strike;
%             
%             params.blackOTMPrices = blackOTMPrices;
%             params.blackVol = blackVol;
%             params.fwdMoneyness = fwdMoneyness;
%             
%             % additional model params
%             eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
%             
%             
%             params.expirySize = expirySize;
%             params.strikeSize =  strikeSize;
%             
%             % optimizer setting
%             params.tol = eqCOMDupireSpotGF.modelParams('tol');
%             params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
%             
%             % vol proxy calibration grid setting
%             
%             params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
%             params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
%             params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
%             params.dK   = eqCOMDupireSpotGF.modelParams('dK');
%             params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
%             
%             % current spot on the grid
%             params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
%             
%             % targetType : 1 = implied vol diff
%             %              2 = price relative error  
%             params.targetType = 1;      
%             
%             % GFType : 1 = matrix exponential moler higham
%             %          2 = one step matrix exponential moler higham & multiplication from
%             %          3 = one step GF & multiplication from right Andreasen & Huge
%             
%             params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
%             
%             params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
%             
%             params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%             
%             params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
%             params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
%             
%             % backGFType : 1 = forward FD(GF)
%             %              2 = implicit FD
%             
%             params.oneStepDt = 1.0/365.0;
%             
%             
%             
%             params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
%             
%             params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
%             params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
%             
%             
%             %output vol proxy
%             out.fwdMoneyness = fwdMoneyness;
%             out.expiry = params.expiry;
%             out.strike = params.strike;
%             
%             out.volProxy = zeros(params.expirySize,params.strikeSize);
%             out.ipos = zeros(params.expirySize,params.strikeSize);
%             out.probMesh = zeros(params.expirySize,params.Ns);
%             
%             out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
%             
%             
%             %initialze the intial probability distribution in the grid
%             initProb = zeros(1,params.Ns);
%             initProb(params.Pricingidx) = 1.0;
%             
%             out.blackOTMPrices = blackOTMPrices;
%             out.blackOTMPrices2= blackOTMPrices2;
%             
%             out.blackVol = blackVol;
%             out.blackOTMVega = blackOTMVega;
%             
%             out.marketImpVol = zeros(expirySize,strikeSize);
%             out.modelImpVol  = zeros(expirySize,strikeSize);
%             
%             out.interpBlackPrices = zeros(expirySize,strikeSize);
%             out.dupireOTMPrices = zeros(expirySize,strikeSize);
%             
%             out.dupireBlackVol = zeros(expirySize,strikeSize);
%             
%             out.volError = zeros(expirySize,strikeSize);
%             out.priceRe = zeros(expirySize,strikeSize);
%             out.priceReOrig = zeros(expirySize,strikeSize);
%             
%             out.marketStrikes = zeros(expirySize,strikeSize);
%             
%             out.Ks =  params.Ks;
%             out.Pricingidx = params.Pricingidx;
%             out.nIter = zeros(expirySize,1);
%             out.localVol = zeros(expirySize,length(out.Ks));
%             out.rmseTotal=0.0;
%             out.fwdVolMseTotal = 0.0;
%             
%             
%             calibrationType = 'bootstrap';
%             if nargin < 3
%                 calibrationType = 'bootstrap'; 
%             else
%                 calibrationType = calibrationFlag;
%             end
%             
%             switch calibrationType
%                 case 'bootstrap'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     tic
%                     % assigning the calibrated volProxy to the local vol
%                     % surface
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
%                     
%                     % generate marginal(cumulative) distribution for mcmc
%                     % simulation
%                     
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
%                     
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     
%                     % calibration gives model schedule, for later use
%                     % general products such as autocallables..
%                     
%                     eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
%                     
%                     params.marketStrikes = out.marketStrikes;
%                     
%                     toc
%                     
% %                     calculate backward scheme for given target
% %                     compareforwardMCFlag = 1 for one step euler scheme
% %                     compareforwardMCFlag = 2 for daily euler scheme
% %                     
% %                     compareforwardMCFlag = 3 for markov chain monte carlo
%                      
%                     
%                     pricingSchemeType = params.pricingSchemeType; 
%                     
%                     tic
%                     
%                     if pricingSchemeType == 1
%                         out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);
% 
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backReOrig = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backPriceOrigRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%   
%                     elseif pricingSchemeType == 3
%                         out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);
% 
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%                     
%                     elseif pricingSchemeType == 4
%                         out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%                         
%                     end
% 
%                     toc
%                 case 'bootstrapForward'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     % update calibration result for eqComDupire
%                     
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
% 
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     % compute implied surface for targetStrikes;
%                     targetExpiry = params.expiry;
%                     
%                     %% at T call & put price
%                     for idx1=1:length(params.expiry)
%                         targetExpiry(idx1) = params.expiry(idx1)-0;
% %                         targetExpiry(idx1) = params.expiry(idx1)+1;
%                     end
%                     
% %                     targetStrikes = out.marketStrikes;
%                     % 0.5% shifted grid
%                     newKs = zeros(length(out.Ks),1);
%                     newKs = zeros(241,1);
%                     
%                     for idx1=1:241
%                         newKs(idx1) = 0.1+ 0.01*(idx1-1);
%                     end
%                     
%                     targetStrikes = zeros(length(targetExpiry),length(newKs));
%                     for idx1=1:size(out.marketStrikes,1)
%                         for idx2= 1:length(newKs)
% %                             targetStrikes(idx1,idx2) = out.Ks(idx2);
%                             targetStrikes(idx1,idx2) = newKs(idx2);
%                         end
%                     end
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                      %% at T-dT call & put price
%                     for idx1=1:length(params.expiry)
%                         targetExpiry(idx1) = params.expiry(idx1)-0.000001;
% %                         targetExpiry(idx1) = params.expiry(idx1)+1;
%                     end
%                     
% %                     targetStrikes = out.marketStrikes;
%                     % 0.5% shifted grid
%                     newKs = zeros(length(out.Ks),1);
%                     newKs = zeros(241,1);
%                     
%                     for idx1=1:241
%                         newKs(idx1) = 0.1+ 0.01*(idx1-1);
%                     end
%                     
%                     targetStrikes = zeros(length(targetExpiry),length(newKs));
%                     for idx1=1:size(out.marketStrikes,1)
%                         for idx2= 1:length(newKs)
% %                             targetStrikes(idx1,idx2) = out.Ks(idx2);
%                             targetStrikes(idx1,idx2) = newKs(idx2);
%                         end
%                     end
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVolPrev = outDummy.impVolSurface;
%                     out.cValueSurfacePrev = outDummy.cValueSurface;
%                     out.pValueSurfacePrev = outDummy.pValueSurface;
%                     out.probMeshIntPrev = outDummy.probMeshInt;
%                     
%                     aaa = 1.0;
%                     
%                 case 'bootstrapForwardOnly'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     % update calibration result for eqComDupire
%                     
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
% 
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     
% %                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
%                     
%                     aaa = 1.0;
%                     
%                     
%                       
% 
%                 otherwise
%                     disp('unImplemented')
%                     
%             end
%         end
             
    end
    
end



In [ ]:
classdef MultiAssetModelIR < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        refModel
        stochasticModelNames
        correlationMatrix
        numOfFactors
    end
    
    methods
        function multiAsset = MultiAssetModelIR(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                multiAsset.refModel = modelParams('refModel');
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix = modelParams('correlationMatrix');
                
                % 1 factor LGM model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                numOfFactors = 0;
                for i=1: length(stochasticModelNames)
                    numOfFactors = numOfFactors + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactors;
                end
                multiAsset.numOfFactors = numOfFactors;
                
            end
        end
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = CommoFwdNMMC(multiAsset,observeTime,modelStates)
            
%             modelNames = multiAsset.modelNameMap.keys;
            numOfFactors = multiAsset.numOfFactors;
            stateSize = size(modelStates,1);
            stochasticFwd = zeros(stateSize,numOfFactors);
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                stochasticFwd(:,i) = multiAsset.modelNameMap(stochasticModelNames{i}).CommoFwdNMMC(observeTime,modelStates(:,i));
            end
             
             out = stochasticFwd;
             
        end
        
        function out = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
%             modelStatesSize = length(comFwd);
            modelStatesSize = size(comFwd,1);
            
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if isKI(idx) > 0
                    out(idx) = 1.0;
                else
                    for ii=1:length(basePrice)
                        if comFwd(idx,ii) <= basePrice(ii)*lowerBarrier
                            out(idx) = 1.0;
                            break;
                        end
                    end
                end
            end
        end
        
        function mcmcOut = inductMCForwardNew(multiAsset,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardNew(fromTime,toTime,lastX(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
            end
            
            mcmcOut.nextX = predictor;
            
        end
        
        function mcmcOut = inductMCMCForwardNew(multiAsset,fromTime,toTime,nextX,nextXIdx,dZ)
            
            predictor = nextX;
            predictorIdx = nextXIdx;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCMCForwardNew(fromTime,toTime,nextX(:,i),nextXIdx(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
                predictorIdx(:,i) = mcOut1.nextXIdx;
            end
            
            mcmcOut.nextX = predictor;
            mcmcOut.nextXIdx = predictorIdx;
            
        end
        
        function columnOut = MCPayoffWorst(multiAsset,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            df_ep = multiAsset.DF(currentTime/365.0);
            
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
        end    
            
        
        function out = computeStepdown2SMC(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            WTIDupire = multiAsset.modelNameMap('WTI');
            expiry = WTIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = 50000;
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);
                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCMC(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            WTIDupire = multiAsset.modelNameMap('WTI');
            expiry = WTIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
%             useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            useQuasiRandomYN = 'false';
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
                
                % we use gaussian copula to generated joint distribution
                % correlated brownian motions
                Z = dZ0';
                UU = zeros(MCTimeStep*numFactors,NMC);
                U = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     W = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*W;
                end
                
                U = H_ncdf(UU);
                
                U = U'; 
                
%                % apply brownian bridge
%                dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
%                U(:,1) = dZ(:,1);
%                for i=2:size(dZ,2)
%                    U(:,i) = dZ(:,i) - dZ(:,i-1); 
%                end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 5000;
                NMC = 50000;
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                UUU = zeros(MCTimeStep*numFactors,NMC);
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                
                UUU = H_ncdf(UU);
                U = UUU';
                
%                 abcd = zeros(MCTimeStep*numFactors,NMC);
%                 for i=1:MCTimeStep*numFactors
%                     for j=1:NMC
%                         abcd(i,j) = H_ncdf(UU(i,j));
%                     end
%                 end

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
           initXIdx = zeros(NMC,numFactors);
           stochasticModelNames = multiAsset.stochasticModelNames;
           for i=1:length(stochasticModelNames)
               Pricingidx = multiAsset.modelNameMap(stochasticModelNames{i}).localVolSurface.Pricingidx;
               initXIdx(:,i) = ones(NMC,1)*Pricingidx;
           end
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);
                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        
%         function out= expmReghai(multiAsset,A,t)
%             % Scale A by power of 2 so that its norm is < 1/2 .
%             A = A*t;
%             [f,e] = log2(norm(A,'inf'));
%             s = max(0,e+1);
%             A = A/2^s;
%             Aexp = eye(size(A,1)) + A;
%             for i=1:s
%                 Aexp = Aexp*Aexp;
%             end
%             out = Aexp;
%         end
%         
%         function path= bbgenerator(eqCOMDupireSpotGF,rn)
%             %generate brownian bridge 
%             
%             [Npaths,Nsteps]=size(rn);
%             path=zeros(Npaths,Nsteps);
%             map= zeros(1,Nsteps);
%             bridgeindex = zeros(1,Nsteps);
%             leftindex = zeros(1,Nsteps);
%             rightindex = zeros(1,Nsteps);
%             sigma = zeros(1,Nsteps);
%             leftweight = zeros(1,Nsteps);
%             rightweight = zeros(1,Nsteps);
%             
%             map(Nsteps)=1;
%             bridgeindex(1)=Nsteps;
%             sigma(1)=sqrt(Nsteps);
%             leftweight(1)=1; 
%             rightweight(1)=1; 
%             j=1;
%             for i=2:Nsteps    
%                 while map(j) 
%                     j=j+1;
%                 end
%                 k=j;
%                 while ~map(k) 
%                     k=k+1;
%                 end
%                 l=j+fix((k-1-j)/2);
%                 map(l)=i;
%                 bridgeindex(i)=l;
%                 leftindex(i)=j;
%                 rightindex(i)=k;
%                 leftweight(i)  = (k-l)/(k+1-j);
%                 rightweight(i) = (l+1-j)/(k+1-j);
%                 sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
%                 j=k+1;
%                 if j>=Nsteps+1
%                     j=1;
%                 end
%             end
% 
%             path(:,Nsteps)=sigma(1)*rn(:,1);
%             for i=2:Nsteps
%                 j=leftindex(i);
%                 k=rightindex(i);
%                 l=bridgeindex(i);
%                 if j~=1
%                     path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
%                 else
%                     path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
%                 end
%             end
%         end
%         
%         function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
%             size = tdma.size;
%             out = vector;
%             
%             for i=2:size
%                 out(i) = out(i) - tdma.ll(i)*out(i-1);
%             end
%             
%             out(size) = out(size)/tdma.dd(size);
%             
%             for i=size-1:-1:1
%                 out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
%             end
%             
%         end
%         
%         function out = LUDecompose(eqCOMDupireSpotGF,tdma)
%             out = tdma;
%             if out.bupdated == 1
%                 for i=1:out.size
%                     out.uu(i) = out.c(i);
%                 end
%                 
%                 out.dd(1) = out.b(1);
%                 for i=2:out.size
%                     out.ll(i) = out.a(i)/out.dd(i-1);
%                     out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
%                 end
%                 
%                 out.bupdated = 0;
%             end
%         
%         end
%         
%         function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
%     
%             x = zeros(n,1);
%             y = zeros(n,1);
%             d = zeros(n,1);
%             %fwd
% 
%             x(1) = 1.0;
%             x(2) = 1.0;
% 
%             for i=3:n
%                 x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
%             end
% 
%             %bwd
%             y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
%             y(n-1) = y(n);
% 
%             for i=n-2:-1:1
%                 y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
%             end
% 
%             %set d
% 
%             d(1) = x(1)*y(1)/b(1);
%             for i=2:n
%                 d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
%             end
% 
% 
%         end
% 
%         
%         function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)
% 
%             localVol = zeros(length(Ks),1);
%             volProxyKs = zeros(1,length(volProxy));
%             try
%                 fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
%             catch ME
%                 aaa = 0;
%             end
%             
%             if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
%                     aaa =1.0;
%                 else
%                     for i=1:length(volProxy)
%                         volProxyKs(i) = Ks(ipos(i));
%                     end
% 
%                     localVol = pchip(volProxyKs,volProxy,Ks);
%                     %flat extrapolation at the ends
%                     for i=1:length(Ks)
%                         if Ks(i) <= Ks(ipos(1))
%                             localVol(i) = localVol(ipos(1));
%                         elseif Ks(i) >= Ks(ipos(length(ipos)))
%                             localVol(i) = localVol(ipos(length(ipos)));
%                         end
%                     end
%                 end
%             elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
%                 for i=1:length(volProxy)
%                     volProxyKs(i) = fwdMoneynessPerExpiry(i);
%                 end
%                 
%                 for i=1:length(Ks)
%                     localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
%                 end   
%                 
%             else
%                 for i=1: length(volProxy)
%     %                 Extrapolation at the left and right
%                     if i == 1
%                         for j=1:ipos(i)-1
%                             localVol(j) = volProxy(i);
%                         end
%                     elseif i == length(volProxy)
%                         for j=ipos(i-1):ipos(i)-1
%                             localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
%                         end
%                         for j=ipos(i):length(Ks)
%                             localVol(j) = volProxy(i);
%                         end
%     %                  linear interpolation in the spot grid in-between    
%                     else
%                         for j=ipos(i-1):ipos(i)-1
%                             localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
%                         end
%                     end
%                 end
%             end
%             
%         end
%                         
%         
%         function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
%             
%             %filling proxy grid vol
%             proxy = zeros(length(lastC),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);
% 
%             %filling end
%             
%             meshC = lastC;
%             meshP = lastP;
%             
%             tdma.size = length(meshC);
%             tdma.bupdated = 0;
%             tdma.a = zeros(tdma.size,1);
%             tdma.b = zeros(tdma.size,1);
%             tdma.c = zeros(tdma.size,1);
%             tdma.uu = zeros(tdma.size,1);
%             tdma.ll = zeros(tdma.size,1);
%             tdma.dd = zeros(tdma.size,1);
%             
%             dK2 = dK*dK;
%             for i=2:tdma.size-1
%                 tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%             end
%             
%             tdma.a(1) = 0;
%             tdma.a(tdma.size) = 0;
%             tdma.c(1) = 0;
%             tdma.c(tdma.size) = 0;
%             tdma.b(1) = 1;
%             tdma.b(tdma.size) = 1;
%             tdma.bupdated = 1;
%             
%             tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%             pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
%             newC = pmeshC;
%             
%             pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
%             newP = pmeshP;
%         end
%         
%         function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
%             
%             %filling proxy grid vol
%             proxy = zeros(length(lastProb),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);
% 
%             %filling end
%             
%             meshProb = lastProb;
%             
%             size = length(meshProb);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             
%             condProb = zeros(size,size);
%             if optParams.params.GFType == -1
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             elseif optParams.params.GFType == 0
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expmdemo1(A_rg*NT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             
%             elseif optParams.params.GFType == 1
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg*NT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             elseif optParams.params.GFType == 2
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg);
%                 
%                 newProb = lastProb;
%                 condProb = eye(size,size);
%                 for t=1:NT
%                     newProb = newProb*A_exp1;
%                     condProb = A_exp1*A_exp1;
%                 end
%                 
%             elseif optParams.params.GFType == 3
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%             
%                 % Test for HugeDecomposition for inverse matrix
%                 [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
%                 A = gallery('tridiag',a1,b,c1);
%                 oneStep = OneStepGF(size,x,y,d,A);
%                 newProb = lastProb;
%                 newProb = newProb*oneStep;
%                 
%                 condProb = oneStep;
%                 
%             else
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 % rescale matrix element to smaller time step
%                 a= a*oneStepDt/dT;
%                 b = (b-1)*oneStepDt/dT + 1.0;
%                 c = c*oneStepDt/dT;
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%             
%                 % Test for HugeDecomposition for inverse matrix
%                 [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
%                 A = gallery('tridiag',a1,b,c1);
%                 oneStep = OneStepGF(size,x,y,d,A);
%                 newProb = lastProb;
%                 %numberOfTimeStep;
%                 %oneStepDt = 1.0/365;
%                 
%                 condProb = eye(size,size);
%                 for t=1:NT
%                     newProb = newProb*oneStep;
%                     condProb = oneStep*oneStep;
%                 end 
%                 
%             end
%             
%             out.newProb = newProb;
%             out.condProb = condProb;
%         end
%         
%         function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = params.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = params.Ks(2) - params.Ks(1);
%             dT = (fromTime - toTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             if params.backGFType == -1
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 newP = A_exp1*lastP;
%                 
%             elseif params.backGFType == 0
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expmdemo1(A_rg*NT);
%                 newP = A_exp1*lastP;
%             elseif params.backGFType == 1
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg*NT);
%                 newP = A_exp1*lastP;
%             elseif params.backGFType == 2
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg);
%                 
%                 newP = lastP;
%                 
%                 for t=1:NT
%                     newP = A_exp1*newP;
%                 end
%                 
%             elseif params.backGFType == -21
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
% %                 A_exp1 = expm(A_rg);
%                 
%                 newP = lastP;
%                 
%                 for t=1:NT
%                     newP = A_exp1*newP;
%                 end
%                 
%             elseif params.backGFType == 3
% 
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%                 
%                 tdma.size = length(lastP);
%                 tdma.a = a;
%                 tdma.b = b;
%                 tdma.c = c;
%                 tdma.uu = zeros(tdma.size,1);
%                 tdma.ll = zeros(tdma.size,1);
%                 tdma.dd = zeros(tdma.size,1);
% 
%                 tdma.bupdated = 1;
% 
%                 tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%                 newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);
% 
%                 
%             elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 % rescale matrix element to smaller time step
%                 a= a*oneStepDt/dT;
%                 b = (b-1)*oneStepDt/dT + 1.0;
%                 c = c*oneStepDt/dT;
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%                 
%                 tdma.size = length(lastP);
%                 tdma.a = a;
%                 tdma.b = b;
%                 tdma.c = c;
%                 tdma.uu = zeros(tdma.size,1);
%                 tdma.ll = zeros(tdma.size,1);
%                 tdma.dd = zeros(tdma.size,1);
% 
%                 tdma.bupdated = 1;
% 
%                 tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%                 
%                 newP = lastP;
%                 for t=1:NT
%                     newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
%                 end
% 
%             else 
%                 disp('unImplemented')
%             end
%         end
%         
%         function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = Ks(2) - Ks(1);
%             dT = (fromTime - toTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             newP = A_exp1*lastP;
%             
%         end
%         
%         function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(Ks),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(proxy);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = Ks(2) - Ks(1);
%             dT = (toTime - fromTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             out = A_exp1;
%             
%         end
%         
%         function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)
% 
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
%         end
%         
%         function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)
% 
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %             Ks = params.Ks;
%             localVol =  zeros(length(x),1);
%             fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             
%             % we use the same linear interpolation scheme as calibration
%             useVolProxyInterpolationYN = 'YES';
%             if strcmp(useVolProxyInterpolationYN,'YES')
%                 
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%             else
%                 volKnotPoints = zeros(length(ipos),1);
%                 for i=1: length(ipos)
%                     volKnotPoints(i) = Ks(ipos(i));
%                 end
% 
% 
%                 for i=1:length(x)
%                     xIdx = find(volKnotPoints <=x(i));
%                     if isempty(xIdx)
%                         localVol(i) = localVolLine(1);
%                     elseif max(xIdx) == ipos(length(ipos))
%                         localVol(i) = localVolLine(length(ipos));
%                     else
%                         localVolIdx = max(xIdx);
%                         weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
%                         localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
%                     end
%                 end
%             end
%             
%         end
%         
%         function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
%             
%             NMC = length(lastX);
%             % we generate conditional probability matrix first
%             condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
%             Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
%             
% %             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
%             uSample = U;
%             
%             nextX = zeros(NMC,1);
%             nextXIdx = zeros(NMC,1);
% %             Ks = params.Ks;
%             nPoints = length(Ks);
%             
%             for i=1:NMC
%                 IndexMc = lastXIdx(i);
%                 j= IndexMc;
%                 cumProb = Qa(IndexMc,:);
%                 idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
%                 nextX(i) = Ks(idxH);
%                 nextXIdx(i) = idxH;
% 
%             end
%             
%             out.nextX = nextX;
%             out.nextXIdx = nextXIdx;
% 
%             
%         end
%         
%         function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
%             
%             timeSchedule = timeScheduleInfo.timeSchedule;
%             dTSchedule365 = timeScheduleInfo.dTSchedule365;
%             volKnotIdx = timeScheduleInfo.volKnowIdx;
%             
%             predictor = lastX;
%             for j=startIdx:endIdx-1
%                 
% %                 localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j),params);
%                 localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
%                 Dt = dTSchedule365(j);
%                 sqrtDt = sqrt(dTSchedule365(j));
%                 for i=1:length(lastX)
%                     predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
% %                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                 end
%             end
%             newX = predictor;
%             
%         end
%         
%         function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
%             
%             predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
%             Dt = (toTime - fromTime)/365.0;
%             sqrtDt = sqrt(Dt);
%             for i=1:length(lastX)
% %                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
%                 predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
%                 
%             end
%             
%             mcmcOut.nextX = predictor;
%             
%         end
%         
%         
%         function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
%             % Calculate Object Function
%             % settings for bisection
%             a = 0.001;
%             b = 10;
%             Tol = 1e-10;
%             MaxIter = 1000;
% 
%             strikes = params.marketStrikes;
%             strikeSize = length(params.marketStrikes);
%             out.vols    = zeros(strikeSize,1);
%             out.interpOTMPrices = zeros(strikeSize,1);
%             out.interpBlackPrices = zeros(strikeSize,1);
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             cValue = meshProb*meshC;
%             pValue = meshProb*meshP;
%             
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 if K <= 1.0
%                       out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
%                      out.interpOTMPrices(i) = pValue(i);
%                      out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
%                      
%                 else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
%                     out.interpOTMPrices(i) = cValue(i);
%                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
%                 end
%             end
%             
%             
%         end
%         
%         function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
%             % Calculate Object Function
%             % settings for bisection
%             a = 0.001;
%             b = 10;
%             Tol = 1e-10;
%             MaxIter = 1000;
% 
%             strikes = params.marketStrikes;
%             strikeSize = length(params.marketStrikes);
%             out.vols    = zeros(strikeSize,1);
%             out.interpOTMPrices = zeros(strikeSize,1);
%             out.interpBlackPrices = zeros(strikeSize,1);
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             cValue = meshProb*meshC;
%             pValue = meshProb*meshP;
%             
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 if K <= 1.0
% %                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
%                      out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
%                      out.interpOTMPrices(i) = pValue(i);
%                      out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
%                      
%                 else
% %                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
%                     out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
%                     out.interpOTMPrices(i) = cValue(i);
%                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
%                 end
%             end
%             
%             
%         end
%         
%         function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
%             expiry = params.expiry;
%             
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             mesh = zeros(modelXSize,1);
%             
%             if strike <= 1.0
%                 for i=1:modelXSize
%                     mesh(i,1) = max(strike- modelX(i),0);
%                 end
% 
%                 volExpiry = expiry(find(expiry <= maturity));
%                 volExpiry = [volExpiry; 0];
%                 timeStep = sort(union(volExpiry,maturity),'ascend');
%                 for i=length(timeStep):-1:2
%                     mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
%                 end
%                 
%                 out = mesh(params.Pricingidx);
%                 
%             else
%                 for i=1:modelXSize
%                     mesh(i,1) = max(modelX(i)-strike,0);
%                 end
% 
%                 volExpiry = expiry(find(expiry <= maturity));
%                 volExpiry = [volExpiry; 0];
%                 timeStep = sort(union(volExpiry,maturity),'ascend');
%                 for i=length(timeStep):-1:2
%                     mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
%                 end
%                 
%                 out = mesh(params.Pricingidx);
%                 
%             end
%             
%         end
% %                              MCPayoff(eqCOMDupireSpotGF,i, scheduleSize, comFwd,isStop,dummyisKI,payoffInfo,hitValue);
%         function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
%             
%             modelStatesSize = length(comFwd);
%             scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
%             strike = payoffInfo.autoCallSchedule.strike(idx);
%             coupon = payoffInfo.autoCallSchedule.coupon(idx);
%             nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
%             lowerBarrier = payoffInfo.lowerBarrier;
%             callStrike1 = payoffInfo.callStrike1;
%             callValue1  = payoffInfo.callValue1;
%             
%             columnOut.isStop = zeros(modelStatesSize,scheduleSize);
%             columnOut.worstP = zeros(modelStatesSize,scheduleSize);
%             
%             columnOut.isStop = columnIn.isStop;
%             columnOut.worstP = columnIn.worstP;
%             
%             columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
%             columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%             columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%             
%             columnOut.payoffColumn = columnIn.payoffColumn;
%             
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%             % early redemption
%             % idx  : early redemption schedule iter
%             % idx11: simulation path iter
%             if idx == 1
%                 for idx11 =1:modelStatesSize % 1st early redemption
%                         % alive path
%                         if comFwd(idx11) >= basePrice*strike % early redemption conditon met
%                            columnOut.isStop(idx11,idx) = 1;
%                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
%                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
%                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                         end
%                         
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                     
%                 end
%                 
%             elseif idx > 1 && idx < scheduleSize
%                 for idx11 =1:modelStatesSize
%                     if columnOut.isStop(idx11,idx-1) == 1 % terminated path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
%                         columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
%                     else
%                         % alive path
%                         if comFwd(idx11) >= basePrice*strike % early redemption conditon met
%                             columnOut.isStop(idx11,idx) = 1;
%                             columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                             columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
%                             columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
%                             columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                         end
%                         
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                     end
%                 end
%                 
%             else % idx == scheduleSize
%                 for idx11 =1:modelStatesSize
%                     if columnOut.isStop(idx11,idx-1) == 1 % terminated path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
%                         columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
% 
%                     else
%                         %alive path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                         % barrier hit path
%                         if isKI(idx11) == 1
%                             if comFwd(idx11) < basePrice*callStrike1 
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
%                             elseif comFwd(idx11) < basePrice*strike
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
%                             else
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                         else % barrier unhit path
%                             if comFwd(idx11) < basePrice*callStrike1 
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
%                             elseif comFwd(idx11) < basePrice*strike
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
%                             else
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                             
%                             if comFwd(idx11) > basePrice*lowerBarrier
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                         end
%                         
%                         % payoff postprocess
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
%                         columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                     end    
%                 end
%             end    
%             
%             
%         end
%         
%         function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%             basePrice = payoffInfo.basePrice;
%             lowerBarrier = payoffInfo.lowerBarrier;
%             modelStatesSize = length(comFwd);
%             out = zeros(modelStatesSize,1);
%             for idx=1:modelStatesSize
%                 if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
%                     out(idx) = 1.0;
%                 end
%             end
%         end
%         
%         % NM : nearest month forward
%         function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%            % NM : nearest month forward
%         function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
%             if observeTime > maturityTime
%                 disp('observeTime is passed maturityTime!')
%                 out = 0;
%                 return;
%             end
%             
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(maturityTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%         function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
% %             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%         function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
% %             out =0;
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% MCMC init 
%             useQuasiRandomYN = 'true';
%             
%             if strcmp(useQuasiRandomYN,'true')
% %                 NMC = 2^14;
% %                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Ps = sobolset(MCTimeStep,'Skip',NMC);
%                 Q = qrandstream(Ps);
% 
%                 reset(Q);
%                 U0 = qrand(Q,NMC);
%                 U = zeros(NMC,MCTimeStep);
% 
%                 dZ0 = zeros(size(U,1),size(U,2));
%     %             dZ = norminv(U);
%                 % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
%                 % generate random number from the end
%                 for i=1:size(U,1)
%                     for j=1:size(U,2)
%                         dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
%                     end
%                 end
%             
%                % apply brownian bridge
%                dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
%                U(:,1) = dZ(:,1);
%                for i=2:size(dZ,2)
%                    U(:,i) = dZ(:,i) - dZ(:,i-1); 
%                end
% %                U = dZ;
%             else
%                 rng('default');
%                 rng(0);
%                 
%                 NMC = 40000;
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
% %                 Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
% 
%             end
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            initX = ones(NMC,1);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
%             
%           %% payoff init
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
%            
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
%            
%            
%            modelStatesSize = NMC;
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
%            
%            % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%            isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
% %           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%            %% Path Generation Forward
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     
%                     nextX = mcmcOut.nextX;
% %                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
% %                 dummyisKI = ones(modelStatesSize,1);
% %                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
% %                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
%                 nMFuture.payoff = comFwd;
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%             
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end
% 
% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
% %             out =0;
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% MCMC init 
%             useQuasiRandomYN = 'true';
%             
%             if strcmp(useQuasiRandomYN,'true')
% %                 NMC = 2^14;
% %                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Ps = sobolset(MCTimeStep,'Skip',NMC);
%                 Q = qrandstream(Ps);
% 
%                 reset(Q);
%                 U0 = qrand(Q,NMC);
%                 U = zeros(NMC,MCTimeStep);
% 
%                 % first random numbers goes to the enddate..
%                 for i=1:size(U,2)
%                     U(:,i) = U0(:,size(U,2)-(i-1));
%                 end
%             else
%                 rng('default');
%                 rng(0);
%                 
%                 NMC = 40000;
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
% %                 Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
% 
%             end
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
%            nextXIdx = initXIdx;
%             
%           %% payoff init
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
%            
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
%            
%            
%            modelStatesSize = NMC;
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
%            
%            % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%            isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
% %           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%            %% Path Generation Forward
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
%                     nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
% %                 dummyisKI = ones(modelStatesSize,1);
% %                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
% %                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
%                 nMFuture.payoff = comFwd;
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%             
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end
% 
% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%              
%         function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            
%            modelStatesSize = length(modelStates);
%            
%            %down barrier touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
%            
%            comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%            strikeMat = autoCallSchedule.strike(scheduleSize);
%            couponMat = autoCallSchedule.coupon(scheduleSize);
%            
% %            strikeOH = strikeMat + overHedgeShift;
%            % samsung overhedge payoff
%            if strcmp(SSpayoffYN,'YES')
%                for i=1:modelStatesSize
%                    if comFwd(i) < basePrice*callStrike1
%                         payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
%                    elseif comFwd(i) < basePrice*strikeMat 
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
%                    else
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
%                    end
%                end
% 
%                payoffStateB.cashflow = payoffStateA.cashflow;
% 
%                for i=1:modelStatesSize
%                     if comFwd(i) > basePrice*lowerBarrier
%                         payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
%                     end
%                end
%                
%            else
%                for i=1:modelStatesSize
%                    if comFwd(i) >= basePrice*strikeMat
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
%                    else
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
%                        payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
%                    end
%                end
% 
%                payoffStateB.cashflow = payoffStateA.cashflow;
% 
%                for i=1:modelStatesSize
%                     if comFwd(i) > basePrice*lowerBarrier
%                         payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
%                     end
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             hitValue.payoff = payoffStateA.cashflow_npv;
%             
%             unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             unhitValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
%                 
%                 endTimeIdx = endTimeIdx + 1;
%             %% induction start
%                 mesh = hitValue.payoff;
%                 meshB = unhitValue.payoff;
%                 for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx -1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%                     for j=1:modelStatesSize
%                         if comFwd(j) <= basePrice*lowerBarrier
%                            meshB(j) = mesh(j); 
%                         end
%                     end
%                 end
%                 
%                 hitValue.payoff = mesh;
%                 unhitValue.payoff = meshB;
%                 nMFuture.payoff = comFwd;
%               %% induction end
%                 
%                 % backward induction in the payoff script
%                 currentNodeIdx = currentNodeIdx -1;
%               %% AtDate operation on Early ExerciseDate
%                 
%                 if i > lastAliveExeriseIdx
%                     currentTime = timeStep(lastTimeIdx);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValue  = hitValue.payoff/onePayoffD;
%                     contiValueB = unhitValue.payoff/onePayoffD;
%                     
%                     %AutoCall applied
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                      % intermediate variables
%                     payoffStateA.cashflow = zeros(modelStatesSize,1);
%                     payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     payoffStateC.cashflow = zeros(modelStatesSize,1);
%                     payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     % samsung overhedge payoff
%                     if strcmp(SSpayoffYN,'YES')
%                         for j=1:modelStatesSize
%                             if comFwd(j)/basePrice >= strikeEarly
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
%                             elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                
%                                payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                             else
% %                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% %                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% % 
% %                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
% %                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
%                                 
%                                 payoffStateA.cashflow(j) = contiValue(j);
%                                 payoffStateB.cashflow(j) = contiValueB(j);
%                            end
%                         end
%                     else
%                         for j=1:modelStatesSize
%                             if comFwd(j) >= basePrice*strikeEarly
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
%                             else
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
%                            end
%                         end
%                         
%                     end
%                     % barrier
%                     for j=1:modelStatesSize
%                         if comFwd(j) <= basePrice*lowerBarrier
%                            payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
%                         end
%                     end
%                     
%                     payoffStateC.cashflow = comFwd;
%                     
%                     for j=1:modelStatesSize
%                         payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
%                         payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
%                         payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
%                     end
%                     
%                     %process payoff at early exercise date
%                     hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%                     hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%                     hitValue.payoff = payoffStateA.cashflow_npv;
%                     
%                     unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%                     unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%                     unhitValue.payoff = payoffStateB.cashflow_npv;
%                     
%                     nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%                     nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%                     nMFuture.payoff = payoffStateC.cashflow_npv;
%                     
%                 end
%                    
%            end
%            
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            
%            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = unhitValue.payoff(Pricingidx);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
%            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
%             % model schedule generation
%             expiryDate = H_Date(vanillaParams.params('maturityDate'));
%             scheduleSize = 1;
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             lastAliveExerciseIdx = 1;
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            modelStatesSize = length(modelStates);
%            
%            %europeanValue
%            europeanValue.npv = 0;
%            europeanValue.payoff = zeros(modelStatesSize,1);
%            europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
%            europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            %americanValue
%            americanValue.npv = 0;
%            americanValue.payoff = zeros(modelStatesSize,1);
%            americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
%            americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
%            
%            comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%            
%            if strcmp(vanillaParams.params('callPutFlag'),'C')
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
%                     payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
%                end
%            else
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
%                     payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             europeanValue.payoff = payoffStateA.cashflow_npv;
%             
%             americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             americanValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            
%            for i=scheduleSize:-1:lastAliveExerciseIdx
%                 % past schedule neglect
%                 startTimeIdx = eventTimeIdx(i);
%                 endTimeIdx = 1;  % valueDate
%                 
%                 endTimeIdx = endTimeIdx + 1;
%             %% induction start
%                 mesh = europeanValue.payoff;
%                 meshB = americanValue.payoff;
%                 for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     % one step backward induction end
%                     lastTimeIdx = lastTimeIdx -1;
%                     currentTime = timeStep(lastTimeIdx);
%                     
%                     %apply american option
%                     comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValueB  = meshB/onePayoffD;
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     if strcmp(vanillaParams.params('callPutFlag'),'C')
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     else
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     end
%                     payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
%                     meshB = payoffStateB.cashflow_npv;
%                     
%                 end
%                 
%                 europeanValue.payoff = mesh;
%                 americanValue.payoff = meshB;
% %                 nMFuture.payoff = comFwd;
%               %% induction end
%                 
%                 % backward induction in the payoff script
%                 currentNodeIdx = currentNodeIdx -1;
%            end
%            
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            
%            europeanValue.npv = europeanValue.payoff(Pricingidx);
%            americanValue.npv = americanValue.payoff(Pricingidx);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
%            out.europeanValue = europeanValue; 
%            out.americanValue = americanValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
%             expiry = params.expiry;
%             
%             initX = ones(NMC,1);
%             
%             payoff = zeros(NMC,length(strikes));
%             volExpiry = expiry(find(expiry <= maturity));
%             volExpiry = [volExpiry; 0];
%             timeStep = sort(union(volExpiry,maturity),'ascend');
%             nextX = initX;
%             %divide time interval into volExpiry
%             for i=2:length(timeStep)
%                 startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
%                 endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
%                 dZStep = dZ(:,startIdx+1:endIdx);
%                 nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
%             end
%             
%             
%             for j=1:length(strikes)
%                 if(strikes(j) <= 1)
%                     payoff(:,j) = max(strikes(j) - nextX,0);
%                 else
%                     payoff(:,j) = max(nextX - strikes(j),0);
%                 end
%             end
%             out = mean(payoff);
%             
%         end
%         
%         function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
%             expiry = params.expiry;
%             Ks = params.Ks;
%             initX = ones(NMC,1);
%             initXIdx = ones(NMC,1)*params.Pricingidx;
%             
%             payoff = zeros(NMC,length(strikes));
%             volExpiry = expiry(find(expiry <= maturity));
%             volExpiry = [volExpiry; 0];
%             timeStep = sort(union(volExpiry,maturity),'ascend');
%             nextX = initX;
%             nextXIdx = initXIdx;
%             for i=2:length(timeStep)
%                 mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
%                 nextX = mcmcOut.nextX;
%                 nextXIdx = mcmcOut.nextXIdx;
%             end
%             
%             
%             for j=1:length(strikes)
%                 if(strikes(j) <= 1)
%                     payoff(:,j) = max(strikes(j) - nextX,0);
%                 else
%                     payoff(:,j) = max(nextX - strikes(j),0);
%                 end
%             end
%             out = mean(payoff);
%             
%         end
%         
%         function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             pdeOTMPrices =  zeros(length(expiry),length(strikes));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
% %                 strikePerExpiry = params.fwdMoneyness(i,:);
%                 for j=1:length(strikePerExpiry)
%                     pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
%                 end
%             end
%             
%             out = pdeOTMPrices;
%             
%         end
%         
%         
%         % compute forward mc(euler scheme) for 8*21 vanilla otm products
%         % generate time steps from the enddate backwards 
%         
%         function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             % we generate the schedule from the enddate
%             timeStepSize = params.mcOneTimeStep;
%             MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
%             dummyTimeSchedule = zeros(MCTimeStep,1);
%             
%             dummyN = 0;
%             for i=1:MCTimeStep
%                 dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
%                 if dummyTimeSchedule(i) < 0
%                     break;
%                 end
%                 dummyN = dummyN +1;
%                 
%             end
%             
%             timeSchedule = zeros(dummyN,1);
%             for i=1:dummyN
%                 timeSchedule(i)= dummyTimeSchedule(i);
%             end
%             
%             volExpiry = [expiry; 0];
%             
%             newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
%             
%             dTSchedule = zeros(length(newTimeSchedule)-1,1);
%             dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
%             for i=length(newTimeSchedule)-1:-1:1
%                 dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
%                 dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
%             end
%             
%             % find the vol expiry points
%             
%             volKnotIdx = zeros(length(expiry),1);
%             for i=1:length(expiry)
%                 volKnotIdx(i) = find(newTimeSchedule==expiry(i));
%                 
%             end
%             
%             timeScheduleInfo.timeSchedule = newTimeSchedule;
%             timeScheduleInfo.dTSchedule = dTSchedule;
%             timeScheduleInfo.dTSchedule365 = dTSchedule365;
%             timeScheduleInfo.volKnowIdx = volKnotIdx;
%             
%             
%             % schedule create end
% %             NMC = params.NMC;
%             NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
% %             Ps = sobolset(length(dTSchedule));
%             
%             Ps = sobolset(length(dTSchedule),'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             Q.reset;
%             U = qrand(Q,NMC);
%             dZ0 = zeros(size(U,1),size(U,2));
% %             dZ = norminv(U);
%             % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
%             % generate random number from the end
%             for i=1:size(U,1)
%                 for j=1:size(U,2)
%                     dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
%                 end
%             end
%             
%            % apply brownian bridge
%            dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
%            
%            % first dZ is dummy , 0
%            dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
%            dZ(:,2) = dZ1(:,1);
%            for i=3:size(dZ1,2)+1
%                 dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
%            end
%            
%             mcOTMPrices =  zeros(length(expiry),size(strikes,2));
% %             mcOTMPrices =  zeros(length(expiry),length(strikes));
%             
%             mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
%                 mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
%                 for j=1:size(strikes,2)
%                     mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
%                 end
%             end
%             
%             out = mcOTMPrices;
%             
%         end
%         
%         
%         % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
%         % generate time steps using only events date 
%         
%         function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
%             mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
%                 mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
%                 for j=1:size(strikes,2)
%                     mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
%                 end
%             end
%             
%             out = mcOTMPrices;
%             
%         end
%         
%         function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
%             
%             expiry = params.expiry;
%             lastP = zeros(1,length(params.Ks));
%             lastP(params.Pricingidx) = 1.0;
%             newP = zeros(1,length(params.Ks));
%             
%             
%             idx = find(maturity <= expiry);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             
%             %relevent volProxy
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             
%             %relevent probMesh and dTDays
%             dTDays = 0;
%             if idxNow~=1
%                 probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
%                 lastP = probMesh;
%                 dTDays = (maturity - expiry(idxNow-1));
%             else % idxNow == 1
%                 dTDays = maturity;
%             end
%             
%             
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
% 
%             %filling end
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
% 
%             dK = Ks(2) - Ks(1);
%             dT = dTDays/365.0;
% 
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
% 
%             a(1) = 0;
%             c(size) = 0;
% 
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
% 
%             b(1) = 1;
%             b(size) = 1;
% 
%             % to create tridiagonal matrix
% 
%             a1 = a(2:size);
%             c1 = c(1:size-1);
% 
%             b_rg(1) = 0;
%             b_rg(size) = 0;
% 
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             newP = lastP*A_exp1;
%             
%             
%             impVol = zeros(1,length(strikes));
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             strikeSize = length(strikes);
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             impVolC = zeros(1,strikeSize);
%             impVolP = zeros(1,strikeSize);
%             impVol  = zeros(1,strikeSize);
%             cValue = newP*meshC;
%             pValue = newP*meshP;
%             
%             out.cValue = cValue;
%             out.pValue = pValue;
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
%                 impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
%                 if K <= 1.0
%                      impVol(i) = impVolP(i);
%                 else
%                      impVol(i) = impVolC(i);
%                 end
%             end
%             
%             out.impVolP = impVolP;
%             out.impVolC = impVolC;
%             out.impVol = impVol;
%             out.probMeshInt = newP;
%             
%         end
%         
%         function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
% %             expiry = params.expiry;
% %             strikes = params.marketStrikes;
% %             
% %             %mc random number initialize
% % %             NMC = 2^14;
% %             NMC = params.NMC;
% %             
% %             MCTimeStep = length(expiry); % -1 because 0
% % 
% %             Ps = sobolset(MCTimeStep,'Skip',NMC);
% % %             Ps = scramble(Ps,'MatousekAffineOwen');
% %             Q = qrandstream(Ps);
% %             
% %             reset(Q);
% %             U0 = qrand(Q,NMC);
% %             U = zeros(NMC,MCTimeStep);
% %             
% %             % first random numbers goes to the enddate..
% %             for i=1:size(U,2)
% %                 U(:,i) = U0(:,size(U,2)-(i-1));
% %             end            
% %             
%             targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
%             targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
%             cValue = zeros(1,size(targetStrikes,2));
%             pValue = zeros(1,size(targetStrikes,2));
%             probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
%             
%             for i=1:length(targetExpiry)
%                 strikePerExpiry = targetStrikes(i,:);
%                 outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
%                 targetImpVolPerExpiry = outDummy.impVol;
%                 cValue = outDummy.cValue;
%                 pValue = outDummy.pValue;
%                 probMeshIntPerExpiry = outDummy.probMeshInt;
%                 
%                 for j=1:size(targetStrikes,2)
%                     targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
%                     cValueSurface(i,j) = cValue(j);
%                     pValueSurface(i,j) = pValue(j);
%                     probMeshInt(i,j) = probMeshIntPerExpiry(j);
%                 end
%             end
%             
%             out.impVolSurface = targetImpVolSurface;
%             out.cValueSurface = cValueSurface;
%             out.pValueSurface = pValueSurface;
%             out.probMeshInt = probMeshInt;
%         end
%         
%         %generate marginal(cumulative distribution) at each event
%         %date(per expiry)
%         %using conditional transition Probability matrix
%         function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
% %             Ks = params.Ks;
% %             nPoints = length(params.Ks);
%             
%             nPoints = length(Ks);
%             condProb = condProbPerExpiry;
%             Qa = zeros(length(Ks),length(Ks));
%             cumM = 0;
%             cumM2 = 0;
%             %first row dealt separately
%             Qa(1,1) = condProb(1,1);
%             cumM2 = 0.0;
%             for j=2:nPoints
%                 cumM2 = cumM2 + condProb(1,j);
%                 Qa(1,j) = Qa(1,1) + cumM2;
%             end
%             
%             for i=2:nPoints
%                 cumM = 0.0;
%                 Qa(i,i) = condProb(i,i);
%                 for j=i-1:-1:1
%                     Qa(i,i) = Qa(i,i) + condProb(i,j);
%                 end
% 
%                 cumM = 0.0;
%                 for j=i-1:-1:1
%                     cumM = cumM + condProb(i,j+1);
%                     Qa(i,j) = Qa(i,i) - cumM;
%                 end
% 
%                 cumM2 = 0.0;
%                 for j=i+1:nPoints
%                     cumM2 = cumM2 + condProb(i,j);
%                     Qa(i,j) = Qa(i,i) + cumM2;
%                 end
%             end
%             
%             out = Qa;
%         end
%         
%         %generate marginal(cumulative distribution) for all eventdates
%         %date(per expiry)
%         %using conditional transition Probability matrix
%         
%         function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
%             Ks = params.Ks;
%             marginal = zeros(params.expirySize,length(Ks),length(Ks));
%             condProbPerExpiry = zeros(length(Ks),length(Ks));
%             marginalPerExpiry = zeros(length(Ks),length(Ks));
%             for i=1:params.expirySize
%                 condProbPerExpiry(:,:) = condProb(i,:,:);
% %                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
%                 marginal(i,:,:) = marginalPerExpiry(:,:);
%             end
%         end
%         
%         
%         % Target function for FixedPointIteration Optimizer
%         
%         function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
%             idxNow = optParams.idxNow;
%             if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
%             mkt_dT=mkt_dT/365.0;
%             
%             
%           %% Solve  Forward PDE(GF) One time
%             outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
%             
%             newProb = outGF.newProb;
%             optParams.meshProb = newProb;
%            %% PDE(GF) Solve End
%            % find the implied vol of dupire model
%            if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
%                target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
%            else
%                target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
%            end
%            marketImpVol = optParams.params.blackVol;
%            expiry = optParams.params.expiry;
%            strike = optParams.params.strike;
%            
%            out.newLocalVol = zeros(length(optParams.strikeSize),1);
%            out.localVolError = zeros(length(optParams.strikeSize),1);
%            out.impVolError =  zeros(length(optParams.strikeSize),1);
%            out.modelImpVol = zeros(length(optParams.strikeSize),1);
%            out.marketImpVol = zeros(length(optParams.strikeSize),1);
%            out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
%            out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
%            out.priceRe = zeros(length(optParams.strikeSize),1);
%            
%            for i=1:optParams.strikeSize
%                 out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
%                 out.localVolError(i) = out.newLocalVol(i) - tvar(i);
%                 out.modelImpVol(i) = target.vols(i);
%                 out.marketImpVol(i) =  marketImpVol(idxNow,i);
%                 out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
%                 out.interpOTMPrices(i) = target.interpOTMPrices(i);
%                 out.interpBlackPrices(i) = target.interpBlackPrices(i);
%                 out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
%            end
%            
%            % for shorter tenor we only fit for the restricted interval
%             if idxNow <= optParams.params.numOfCutoffTenor
%                 for i=1:optParams.lowerIdx-1
%                     out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
%                 end
% 
%                 for i=optParams.upperIdx+1:optParams.strikeSize
%                     out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
%                 end
%             end
%             
%             out.volErrorTotal = 0.0;
%             out.priceReTotal = 0.0;
%             %for one month we only fit betwee lower and upper
%             dummyN = 0;
%             if idxNow <= optParams.params.numOfCutoffTenor
%                 for i=optParams.lowerIdx:optParams.upperIdx
%                     out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
%                     out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
%                     dummyN = dummyN +1;
%                 end
%             else
%                 for i=1:length(out.impVolError)
%                     out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
%                     out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
%                     dummyN = dummyN +1;
%                 end
%             end
%             out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
%             out.priceReTotal = sqrt(out.priceReTotal/dummyN);
%             
%         end
%         
%          % Fixed Point Iteration Optimizer
%          % start from implied vol find the local vol using Reghai's fixed
%          % point iteration optimization
%          % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
%          
%         function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
%             
%             n= 0;
%             errorTry = 1000.0; % Initial ErrorBound make it big enough
%             tvarOld = tvar;
%             if optParams.params.targetType == 1
%                 while (errorTry > tol) && (n < maxIter)
%                     out = targetfunc(tvar);
%                     tvarOld = tvar;
%                     % reduce implied vol error
%                     errorTry = out.volErrorTotal;
%                     tvar = out.newLocalVol;
%                     n = n+1;
%                 end
%                 x = tvar;
%                 finalOut = targetfunc(tvarOld);
%                 fval = finalOut.volErrorTotal;
%                 marketImpVol = finalOut.marketImpVol;
%                 modelImpVol  = finalOut.modelImpVol;
%                 interpBlackPrices = finalOut.interpBlackPrices;
%                 interpOTMPrices = finalOut.interpOTMPrices;
%                 volError     = finalOut.impVolError;
%                 volErrorTotal = finalOut.volErrorTotal;
%                 priceRe      = finalOut.priceRe;
%                 priceReTotal = finalOut.priceReTotal;
%                 nIter = n;
%             else
%                 while (errorTry > tol) && (n < maxIter)
%                     out = targetfunc(tvar);
%                     tvarOld = tvar;
%                     % reduce price relative error
%                     errorTry = out.priceReTotal;
%                     tvar = out.newLocalVol;
%                     n = n+1;
%                 end
%                 x = tvar;
%                 finalOut = targetfunc(tvarOld);
%                 fval = finalOut.volErrorTotal;
%                 marketImpVol = finalOut.marketImpVol;
%                 modelImpVol  = finalOut.modelImpVol;
%                 interpBlackPrices = finalOut.interpBlackPrices;
%                 interpOTMPrices = finalOut.interpOTMPrices;
%                 volError     = finalOut.impVolError;
%                 volErrorTotal = finalOut.volErrorTotal;
%                 priceRe      = finalOut.priceRe;
%                 priceReTotal = finalOut.priceReTotal;
%                 nIter = n;
%             end
%             
%         end
%         
%         % Fixed Local Vol per expiry by bootstrapping
%         
%         function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
%             %initialize calibration parameters
%             
%             optParams.idxNow = idxNow;
%             optParams.dT = dT;
%             optParams.params = params;
%             optParams.ipos = zeros(params.strikeSize,1);
%             optParams.Ks = zeros(params.Ns,1);
%             optParams.Ks = params.Ks;
%             optParams.strikeSize = params.strikeSize;
%             optParams.meshSize = params.Ns;
%             
%             optParams.meshProb = zeros(1,params.Ns);
%             
%             optParams.proxy = zeros(params.Ns,1);
%             optParams.dK = params.Ks(2) - params.Ks(1);
%             
%             optParams.marketStrikes = zeros(params.strikeSize,1);
%             optParams.blackOTMPrices = params.blackOTMPrices;
%             optParams.callCurve = zeros(params.strikeSize,1);
%             optParams.putCurve = zeros(params.strikeSize,1);
%             
%             for i=1:params.strikeSize
%                 optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
%             end
%                 
% %             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
% %                 % market strike points in fwd PDE grid
% %                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
% %                 for i=1:params.strikeSize
% %                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
% %                 end
% %             else
% %                 for i=1:params.strikeSize
% %                     optParams.marketStrikes(i) = params.strike(i);
% %                 end
% %             end
%             
%             % positions to separate vol proxy regimes
%             for i=1:params.strikeSize
%                 optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
%             end
% %             optParams.ipos(params.strikeSize) = optParams.meshSize;
%             
%             optParams.lastProb = lastProb;
%             
%             % exception for one month maturity for restricted interval
%             
%             lowerCutoff = params.lowerCutoffVector(idxNow);
%             upperCutoff = params.upperCutoffVector(idxNow);
%             numOfCutoffTenor = params.numOfCutoffTenor; 
%             
%             lowerIdx = 1;
%             upperIdx = params.strikeSize;
%             
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end
%             
%             optParams.lowerIdx = lowerIdx;
%             optParams.upperIdx = upperIdx;
%             
%             for i=1:params.strikeSize
%                 tvar(i) = params.blackVol(idxNow,i);
%                 lb(i) = 0.01;
%                 ub(i) = 1.0;
%             end
%             
%             for i=1:lowerIdx-1
%                 tvar(i) = tvar(lowerIdx);
%             end
%             
%             for i=upperIdx+1:params.strikeSize
%                 tvar(i) = tvar(upperIdx);
%             end
% 
%             targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
%                 
%             
% %             tol = 0.0001;
% %             maxIter = 1000;
%             
%             tol = params.tol;
%             maxIter = params.maxIter;
%             
%             [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
%             
%             for i=1:params.strikeSize
%                 bootStrapOut.volProxy(i) = x(i);
%                 bootStrapOut.ipos(i) = optParams.ipos(i);
%             end
%             
%             % for later use
%             bootStrapOut.marketStrikes = optParams.marketStrikes;
%             
%             bootStrapOut.volError = volError;
%             bootStrapOut.marketImpVol = marketImpVol;
%             bootStrapOut.modelImpVol = modelImpVol;
%             bootStrapOut.interpBlackPrices = interpBlackPrices;
%             bootStrapOut.dupireOTMPrices = interpOTMPrices;
%             
%             bootStrapOut.nIter = nIter;
%             
%             out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
%             
%             bootStrapOut.lastProb = out.newProb;
%             bootStrapOut.condProb = out.condProb;
%             
%         end
%         
%         function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
%             
%           % Calculate Black OTM Prices as calibration target
%             volSurface = black.volSurface;
%             
%             expiry = volSurface.maturities;
%             strike  = volSurface.strikes;
%             spot =  black.spot.params('rawData');
%             expirySize = length(expiry);
%             strikeSize = length(strike);
%             
%             blackOTMPrices = zeros(expirySize,strikeSize);
%             blackOTMPrices2 = zeros(expirySize,strikeSize);
%             blackVol = zeros(expirySize,strikeSize);
%             blackOTMVega = zeros(expirySize,strikeSize);
%             
%             % we add blackvol as matrix so that in the calibation we don't
%             % need to do 2D interpolation again and again!!
%             % also for the target black price too!!
%             if strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
%                 for k=1:expirySize
%     %                 fwd = black.Fwd(spot,expiry(k));
%     %                 df = black.zeroCurve.DF(expiry(k)/365.0);
%                     for j=1:strikeSize
%                         blackVol(k,j) = volSurface.volSurfaceR(k,j);
%                         fwdMoneyness(k,j) = strike(j);
%                         
%                         if fwdMoneyness(k,j) <= 1
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
%                         else
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
%                         end
%                     end
%                 end
%             else
%                 for k=1:expirySize
%     %                 fwd = black.Fwd(spot,expiry(k));
%     %                 df = black.zeroCurve.DF(expiry(k)/365.0);
%                     for j=1:strikeSize
%                         blackVol(k,j) = volSurface.volSurfaceR(k,j);
%                         if strike(j) <= 0
%                             dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
%                             fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
%                         else
%                             dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
%                             fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
%                         end
% 
%                         if fwdMoneyness(k,j) <= 1
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
%                         else
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
%                         end
%                     end
%                 end
%                 
%             end
%                 
% %             if strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
% %                 for k=1:expirySize
% %                     fwd = black.Fwd(spot,expiry(k));
% %                     df = black.zeroCurve.DF(expiry(k)/365.0);
% %                     fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
% %                     for j=1:strikeSize
% %                         blackVol(k,j) = volSurface.vol(expiry(k),strike(j));
% %                         fwdStrike = strike(j)/fwdFactor;
% %                         if fwdStrike <= 1.0
% % %                             blackOTMPrices(k,j) = black.BlackVanilla(expiry(k),expiry(k),spot,spot*strike(j),volSurface.vol(expiry(k),strike(j)),'P')/fwd/df;
% %                             blackOTMPrices(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,-1);
% %                             blackOTMPrices2(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,-1);
% %                             blackOTMVega(k,j) = BlackFwdVega(1,fwdStrike,blackVol(k,j),expiry(k)/365.0);
% %                         else
% % %                             blackOTMPrices(k,j)  = black.BlackVanilla(expiry(k),expiry(k),spot,spot*strike(j),volSurface.vol(expiry(k),strike(j)),'C')/fwd/df;
% %                             blackOTMPrices(k,j)  = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,1);
% %                             blackOTMPrices2(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,1);
% %                             blackOTMVega(k,j) = BlackFwdVega(1,fwdStrike,blackVol(k,j),expiry(k)/365.0);
% %                             
% %                         end
% %                     end
% %                 end
% %             else
% %                 for k=1:expirySize
% %                     fwd = black.Fwd(spot,expiry(k));
% %                     df = black.zeroCurve.DF(expiry(k)/365.0);
% %                     for j=1:strikeSize
% %                         blackVol(k,j) = volSurface.vol(expiry(k),strike(j));
% %                         if strike(j) <= 1
% %                             blackOTMPrices(k,j) = black.BlackVanilla(expiry(k),expiry(k),spot,fwd*strike(j),volSurface.vol(expiry(k),strike(j)),'P')/fwd/df;
% %                         else
% %                             blackOTMPrices(k,j)  = black.BlackVanilla(expiry(k),expiry(k),spot,fwd*strike(j),volSurface.vol(expiry(k),strike(j)),'C')/fwd/df;
% %                         end
% %                     end
% %                 end
% %             end
% %             
%             
%             params.volSurface = volSurface;
%             params.expiry = expiry;
%             params.strike = strike;
%             
%             params.blackOTMPrices = blackOTMPrices;
%             params.blackVol = blackVol;
%             params.fwdMoneyness = fwdMoneyness;
%             
%             % additional model params
%             eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
%             
%             
%             params.expirySize = expirySize;
%             params.strikeSize =  strikeSize;
%             
%             % optimizer setting
%             params.tol = eqCOMDupireSpotGF.modelParams('tol');
%             params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
%             
%             % vol proxy calibration grid setting
%             
%             params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
%             params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
%             params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
%             params.dK   = eqCOMDupireSpotGF.modelParams('dK');
%             params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
%             
%             % current spot on the grid
%             params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
%             
%             % targetType : 1 = implied vol diff
%             %              2 = price relative error  
%             params.targetType = 1;      
%             
%             % GFType : 1 = matrix exponential moler higham
%             %          2 = one step matrix exponential moler higham & multiplication from
%             %          3 = one step GF & multiplication from right Andreasen & Huge
%             
%             params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
%             
%             params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
%             
%             params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%             
%             params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
%             params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
%             
%             % backGFType : 1 = forward FD(GF)
%             %              2 = implicit FD
%             
%             params.oneStepDt = 1.0/365.0;
%             
%             
%             
%             params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
%             
%             params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
%             params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
%             
%             
%             %output vol proxy
%             out.fwdMoneyness = fwdMoneyness;
%             out.expiry = params.expiry;
%             out.strike = params.strike;
%             
%             out.volProxy = zeros(params.expirySize,params.strikeSize);
%             out.ipos = zeros(params.expirySize,params.strikeSize);
%             out.probMesh = zeros(params.expirySize,params.Ns);
%             
%             out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
%             
%             
%             %initialze the intial probability distribution in the grid
%             initProb = zeros(1,params.Ns);
%             initProb(params.Pricingidx) = 1.0;
%             
%             out.blackOTMPrices = blackOTMPrices;
%             out.blackOTMPrices2= blackOTMPrices2;
%             
%             out.blackVol = blackVol;
%             out.blackOTMVega = blackOTMVega;
%             
%             out.marketImpVol = zeros(expirySize,strikeSize);
%             out.modelImpVol  = zeros(expirySize,strikeSize);
%             
%             out.interpBlackPrices = zeros(expirySize,strikeSize);
%             out.dupireOTMPrices = zeros(expirySize,strikeSize);
%             
%             out.dupireBlackVol = zeros(expirySize,strikeSize);
%             
%             out.volError = zeros(expirySize,strikeSize);
%             out.priceRe = zeros(expirySize,strikeSize);
%             out.priceReOrig = zeros(expirySize,strikeSize);
%             
%             out.marketStrikes = zeros(expirySize,strikeSize);
%             
%             out.Ks =  params.Ks;
%             out.Pricingidx = params.Pricingidx;
%             out.nIter = zeros(expirySize,1);
%             out.localVol = zeros(expirySize,length(out.Ks));
%             out.rmseTotal=0.0;
%             out.fwdVolMseTotal = 0.0;
%             
%             
%             calibrationType = 'bootstrap';
%             if nargin < 3
%                 calibrationType = 'bootstrap'; 
%             else
%                 calibrationType = calibrationFlag;
%             end
%             
%             switch calibrationType
%                 case 'bootstrap'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     tic
%                     % assigning the calibrated volProxy to the local vol
%                     % surface
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
%                     
%                     % generate marginal(cumulative) distribution for mcmc
%                     % simulation
%                     
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
%                     
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     
%                     % calibration gives model schedule, for later use
%                     % general products such as autocallables..
%                     
%                     eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
%                     
%                     params.marketStrikes = out.marketStrikes;
%                     
%                     toc
%                     
% %                     calculate backward scheme for given target
% %                     compareforwardMCFlag = 1 for one step euler scheme
% %                     compareforwardMCFlag = 2 for daily euler scheme
% %                     
% %                     compareforwardMCFlag = 3 for markov chain monte carlo
%                      
%                     
%                     pricingSchemeType = params.pricingSchemeType; 
%                     
%                     tic
%                     
%                     if pricingSchemeType == 1
%                         out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);
% 
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backReOrig = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backPriceOrigRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%   
%                     elseif pricingSchemeType == 3
%                         out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);
% 
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%                     
%                     elseif pricingSchemeType == 4
%                         out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%                         
%                     end
% 
%                     toc
%                 case 'bootstrapForward'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     % update calibration result for eqComDupire
%                     
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
% 
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     % compute implied surface for targetStrikes;
%                     targetExpiry = params.expiry;
%                     
%                     %% at T call & put price
%                     for idx1=1:length(params.expiry)
%                         targetExpiry(idx1) = params.expiry(idx1)-0;
% %                         targetExpiry(idx1) = params.expiry(idx1)+1;
%                     end
%                     
% %                     targetStrikes = out.marketStrikes;
%                     % 0.5% shifted grid
%                     newKs = zeros(length(out.Ks),1);
%                     newKs = zeros(241,1);
%                     
%                     for idx1=1:241
%                         newKs(idx1) = 0.1+ 0.01*(idx1-1);
%                     end
%                     
%                     targetStrikes = zeros(length(targetExpiry),length(newKs));
%                     for idx1=1:size(out.marketStrikes,1)
%                         for idx2= 1:length(newKs)
% %                             targetStrikes(idx1,idx2) = out.Ks(idx2);
%                             targetStrikes(idx1,idx2) = newKs(idx2);
%                         end
%                     end
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                      %% at T-dT call & put price
%                     for idx1=1:length(params.expiry)
%                         targetExpiry(idx1) = params.expiry(idx1)-0.000001;
% %                         targetExpiry(idx1) = params.expiry(idx1)+1;
%                     end
%                     
% %                     targetStrikes = out.marketStrikes;
%                     % 0.5% shifted grid
%                     newKs = zeros(length(out.Ks),1);
%                     newKs = zeros(241,1);
%                     
%                     for idx1=1:241
%                         newKs(idx1) = 0.1+ 0.01*(idx1-1);
%                     end
%                     
%                     targetStrikes = zeros(length(targetExpiry),length(newKs));
%                     for idx1=1:size(out.marketStrikes,1)
%                         for idx2= 1:length(newKs)
% %                             targetStrikes(idx1,idx2) = out.Ks(idx2);
%                             targetStrikes(idx1,idx2) = newKs(idx2);
%                         end
%                     end
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVolPrev = outDummy.impVolSurface;
%                     out.cValueSurfacePrev = outDummy.cValueSurface;
%                     out.pValueSurfacePrev = outDummy.pValueSurface;
%                     out.probMeshIntPrev = outDummy.probMeshInt;
%                     
%                     aaa = 1.0;
%                     
%                 case 'bootstrapForwardOnly'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     % update calibration result for eqComDupire
%                     
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
% 
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     
% %                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
%                     
%                     aaa = 1.0;
%                     
%                     
%                       
% 
%                 otherwise
%                     disp('unImplemented')
%                     
%             end
%         end
             
    end
    
end



In [ ]:
classdef MultiAssetModelEQ3FLocalCorr < Model
    % modelNameMap : dictionary, key : modelName , value : model
    % refModel : discounting model 
    % correlationNameVector : namesVector for correlation matrix
    % correlationMatrix: correlation matrix
    % numOfFactors : number of factors
    
    properties
        modelNameMap
        refModel
        stochasticModelNames
        
        % termCorr
        correlationMatrix_short
        correlationMatrix_long
        lambdaCorr
        
        corrAlpha
        alphaBarrier
        
        termCorr
        
        numOfFactors
        
       %% multiAsset Correlation
        correlationBrigoGamma
        
        localCorrInfo
    end
    
    methods
        function multiAsset = MultiAssetModelEQ3FLocalCorr(mktData,modelParams)
            if nargin > 0
                multiAsset.mktData =  mktData;
                multiAsset.modelParams = modelParams;
                
                multiAsset.modelNameMap = modelParams('modelNameMap');
                multiAsset.refModel = modelParams('refModel');
                multiAsset.stochasticModelNames = modelParams('stochasticModelNames');
                multiAsset.correlationMatrix_short = modelParams('correlationMatrix_short');
                multiAsset.correlationMatrix_long = modelParams('correlationMatrix_long');
                multiAsset.lambdaCorr = modelParams('lambdaCorr');
                
                multiAsset.corrAlpha = modelParams('corrAlpha');
                multiAsset.alphaBarrier = modelParams('alphaBarrier');
                
                % termCorr need to be init
                % maybe generated when init or not
                multiAsset.termCorr = [];
%                 multiAsset.correlationBrigoGamma = modelParams('correlationBrigoGamma');
                
                % 1 factor local vol model
%                 modelNames = multiAsset.modelNameMap.keys;
                stochasticModelNames = multiAsset.stochasticModelNames;
                numOfFactors = 0;
                for i=1: length(stochasticModelNames)
                    numOfFactors = numOfFactors + multiAsset.modelNameMap(stochasticModelNames{i}).numOfFactors;
                end
                multiAsset.numOfFactors = numOfFactors;
                
                localCorrInfo = {};
            end
        end
        
        function out = GenerateTermCorr(multiAsset,valueDate,stepdownParams)
            
             % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
%             autoCallAliveExpiry = autoCallSchedule.days(find(autoCallSchedule.days>=0));
            
            termCorrSize = scheduleSize;
            termCorr = cell(termCorrSize,2);
            
            correlationMatrix_short = multiAsset.correlationMatrix_short;
            correlationMatrix_long = multiAsset.correlationMatrix_long;
            lambdaCorr = multiAsset.lambdaCorr;
            
            tempCorr1 = correlationMatrix_short;
            for i=1:termCorrSize
                if autoCallSchedule.days(i) < 0
                    tempTime = 0;
                else
                    tempTime = autoCallSchedule.days(i);
                end
                
                tempCorr2 = correlationMatrix_long + exp(-1.0*lambdaCorr*tempTime/365.0) * (correlationMatrix_short - correlationMatrix_long);
                termCorr{i,1} = tempTime;
                termCorr{i,2} = 0.5*(tempCorr1 + tempCorr2);
                tempCorr1 = tempCorr2;
            end
            
            multiAsset.termCorr = termCorr; 
            out = termCorr;
                
        end
        
        function out = DF(multiAsset,t)
            out = multiAsset.refModel.DF(t);
        end
        
        function out = CommoFwdNMMC(multiAsset,observeTime,modelStates)
            
%             modelNames = multiAsset.modelNameMap.keys;
            numOfFactors = multiAsset.numOfFactors;
            stateSize = size(modelStates,1);
            stochasticFwd = zeros(stateSize,numOfFactors);
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                stochasticFwd(:,i) = multiAsset.modelNameMap(stochasticModelNames{i}).CommoFwdNMMC(observeTime,modelStates(:,i));
            end
             
             out = stochasticFwd;
             
        end
        
        function out = CommoFwdNMMCReduced(multiAsset,observeTime,futuresStates)
            
            % modelStates  are cell {1}, {2} : 1st asset's model state, {2} 2nd asset's model state    
            numOfFactors = multiAsset.numOfFactors;
            stateSize = size(futuresStates{1},1);
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            stochasticFwd = zeros(stateSize,length(stochasticModelNames));
            for i=1:length(stochasticModelNames)
                stochasticFwd(:,i) = multiAsset.modelNameMap(stochasticModelNames{i}).CommoFwdNMMCReduced(observeTime,futuresStates{i});
            end
             
             out = stochasticFwd;
             
        end
        
        function out = EQFwdMC(multiAsset,observeTime,modelStates)
            
            numOfFactors = multiAsset.numOfFactors;
            stateSize = size(modelStates,1);
            stochasticFwd = zeros(stateSize,numOfFactors);
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                stochasticFwd(:,i) = multiAsset.modelNameMap(stochasticModelNames{i}).EQFwdMC(observeTime,modelStates(:,i));
            end
             
             out = stochasticFwd;
             
        end
        
        function out = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo)
            basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
%             modelStatesSize = length(comFwd);
            modelStatesSize = size(comFwd,1);
            
            out = zeros(modelStatesSize,1);
            for idx=1:modelStatesSize
                if isKI(idx) > 0
                    out(idx) = 1.0;
                else
                    for ii=1:length(basePrice)
                        if comFwd(idx,ii) <= basePrice(ii)*lowerBarrier
                            out(idx) = 1.0;
                            break;
                        end
                    end
                end
            end
        end
        
        
        function mcmcOut = inductMCForwardNewReducedQuantoLocalCorrSimple1S(multiAsset,basePrice,idxNow,termCorr,fromTime,toTime,nextFuturesStates,dZ)
            
            predictor = nextFuturesStates;
            corrector = nextFuturesStates;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            modelStatesSize = size(predictor{1},1);
            
            assetSize = size(predictor,1);
            
            assetPrices = zeros(modelStatesSize,assetSize);
            
            for idxjj=1:assetSize
                if size(predictor{idxjj},2) == 1
                    dummyC = predictor{idxjj};
                    assetPrices(:,idxjj) = dummyC;
                else
                    dummyC = predictor{idxjj};
                    assetPrices(:,idxjj) = dummyC(:,idxNow);
                end
            end
            
            %% 20210203 major change to MultiAssetModel
            % we generalize multAssetModel including singleAssetModel
            % count assetSize and if it is 1 then we skip cholesky
            % decomposition
            
            if assetSize > 1
                % find bestTemp for localCorrelation calculation for each mc path
                bestTemp = zeros(modelStatesSize,1);
                for idxhh=1:modelStatesSize
                    bestTemp(idxhh) = assetPrices(idxhh,1)/basePrice(1);
                    for idxjj=2:assetSize
                        bestTemp(idxhh) = max(bestTemp(idxhh),assetPrices(idxhh,idxjj)/basePrice(idxjj));
                    end
                end

                alphaBarrier = multiAsset.alphaBarrier;
                corrAlpha    = multiAsset.corrAlpha;
                denominator = 1.0/(1.0 - alphaBarrier);

                dummyM = ones(numOfFactors,numOfFactors);

                % have to rewrite later for general cases
    %             dummyM(2,3) = 0.0;
    %             dummyM(2,5) = 0.0;
    %             dummyM(3,2) = 0.0;
    %             dummyM(3,4) = 0.0;
    %             
    %             dummyM(4,3) = 0.0;
    %             dummyM(4,5) = 0.0;
    %             dummyM(5,2) = 0.0;
    %             dummyM(5,4) = 0.0;

    %             cumFactors1 = 0;
    %             cumFactors2 = 0;
    %             
    %             for i=1:length(stochasticModelNames)
    %                 childModel1 = multiAsset.modelNameMap(stochasticModelNames{i});
    %                 numOfFactorsChild1 = childModel.numOfFactors;
    %                 for j=i:length(stochasticModelNames)
    %                     childModel2 = multiAsset.modelNameMap(stochasticModelNames{j});
    %                     numOfFactorsChild2 = childModel.numOfFactors;
    %                     if numOfFactorsChild1 == 1
    %                         continue;
    %                     elseif numOfFactorsChild2 == 1
    %                         continue;
    %                     else
    %                         % more than two factors
    %                         dummyM(cumFactors1+1:
    %                     end
    %                 end
    %                 cumFactors1 = cumFactors1 + numOfFactorsChild1;
    %                 
    %             end

                % bestTemp >= 1.0
                L = chol(termCorr,'lower');
                %  bestTemp <  alphaBarrier
                termCorr3 = termCorr*(1-corrAlpha) + dummyM*corrAlpha;  
                LLL = chol(termCorr3,'lower');

    %             tic
                dWW = zeros(size(dZ));
                for idxhh=1:modelStatesSize
                    dZZ  = dZ(idxhh,:);
                    if bestTemp(idxhh) >= 1.0
                        dW = L*dZZ';
                    elseif bestTemp(idxhh) <= alphaBarrier
                        dW = LLL*dZZ';
                    else
                        termCorr2 = termCorr*(1-corrAlpha*(1.0 - bestTemp(idxhh))*denominator) + ...
                                    dummyM*corrAlpha*(1.0 - bestTemp(idxhh))*denominator;
                        LL = chol(termCorr2,'lower');
                        dW = LL*dZZ';
                    end
                    dWW(idxhh,:) = dW';
                end
                
            else
                % we have only one asset so no need to chol
                dWW = dZ;
            end
%             toc
%             for idxhh=1:modelStatesSize
%                 bestTemp = predictor
%             end
            % termCorr Cholesky Decomposition -> correlated brownian motion
%             L = chol(termCorr,'lower');
%             dW = L*dZ';
%             dWW = dW';
            
            
            
            
            
            
            idxNM = cell(length(stochasticModelNames),1);
            
            cumNumOfFactors = 0;
            for i=1:length(stochasticModelNames)
                childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                numOfFactorsChild = childModel.numOfFactors;
                mcOut1 = childModel.inductMCForwardNewReducedQuanto(fromTime,toTime,predictor{i},dWW(:,cumNumOfFactors+1:cumNumOfFactors+numOfFactorsChild));
                corrector{i} = mcOut1.nextFuturesStates;
                idxNM{i} = mcOut1.idxNM;
                cumNumOfFactors = cumNumOfFactors + numOfFactorsChild;
            end
            
            mcmcOut.nextFuturesStates = corrector;
            mcmcOut.idxNM = idxNM;
            
           %% correlation
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                nextFutures1 = corrector{1};
                nextX1 = nextFutures1(:,idxNM{1});
                
                nextFutures2 = corrector{2};
                nextX2 = nextFutures2(:,idxNM{2});
                
                prevFutures1 = predictor{1};
                lastX1 = prevFutures1(:,idxNM{1});
                
                prevFutures2 = predictor{2};
                lastX2 = prevFutures2(:,idxNM{2});
                
                
                
                deltaX1 = log(nextX1./lastX1);
                deltaX2 = log(nextX2./lastX2);
                
                % in - curve correl
                aa = nextFutures1(:,idxNM{1}:end);
                bb = nextFutures2(:,idxNM{2}:end);
                aa0 = prevFutures1(:,idxNM{1}:end);
                bb0 = prevFutures2(:,idxNM{2}:end);
                deltaA = log(aa./aa0);
                deltaB = log(bb./bb0);
                aa_size = size(aa,2);
                bb_size = size(bb,2);
                
                inCurveCorrA = zeros(aa_size,aa_size); 
                for ii=1:aa_size
                    for jj=1:aa_size
                        inCurveCorrA(ii,jj) = corr(deltaA(:,ii),deltaA(:,jj));
                    end
                end
                
                inCurveCorrB = zeros(bb_size,bb_size); 
                for ii=1:bb_size
                    for jj=1:bb_size
                        inCurveCorrB(ii,jj) = corr(deltaB(:,ii),deltaB(:,jj));
                    end
                end
                
                interCurveCorrAB = zeros(aa_size,bb_size);
                for ii=1:aa_size
                    for jj=1:bb_size
                        interCurveCorrAB(ii,jj) = corr(deltaA(:,ii),deltaB(:,jj));
                    end
                end
                
                
                corrInfo.corrX1X2 = corr(deltaX1,deltaX2);
                
                %curve-curve correl
                corrInfo.inCurveCorrA = inCurveCorrA;
                corrInfo.inCurveCorrB = inCurveCorrB;
                corrInfo.interCurveCorrAB = interCurveCorrAB;
                
             %% correlation between projected co-ordiates(parallel & flattening)
               futuresSize_a = size(aa,2);
               futuresSize_b = size(bb,2);
               
               e1_a = ones(futuresSize_a,1);
               deltaE2_a = 1.0/(futuresSize_a-1);
               e2_a=0.0:deltaE2_a:1.0;
               e2_a = e2_a';
               e2_a = e2_a - mean(e2_a)*ones(futuresSize_a,1);
               e2_a = e2_a*-1.0;
               
               cordPar_a = zeros(size(aa,1),1);
               cordFlat_a =zeros(size(aa,1),1);
               dF_a = aa- aa0;
               normE1_a = e1_a'*e1_a; 
               normE2_a = e2_a'*e2_a; 
               cordPar_a = (dF_a*e1_a)/normE1_a;
               cordFlat_a = (dF_a*e2_a)/normE2_a;
               inCurveCorr_a = corr(cordPar_a,cordFlat_a);
               
               e1_b = ones(futuresSize_b,1);
               deltaE2_b = 1.0/(futuresSize_b-1);
               e2_b=0.0:deltaE2_b:1.0;
               e2_b = e2_b';
               e2_b = e2_b - mean(e2_b)*ones(futuresSize_b,1);
               e2_b = e2_b*-1.0;
               
               cordPar_b = zeros(size(bb,1),1);
               cordFlat_b =zeros(size(bb,1),1);
               dF_b = bb- bb0;
               normE1_b = e1_b'*e1_b; 
               normE2_b = e2_b'*e2_b; 
               cordPar_b = (dF_b*e1_b)/normE1_b;
               cordFlat_b = (dF_b*e2_b)/normE2_b;
               inCurveCorr_b = corr(cordPar_b,cordFlat_b);
               
               projRet = [cordPar_a,cordFlat_a,cordPar_b,cordFlat_b];
               projCorr = corr(projRet);
               aaaa = 1.0;
               
               corrInfo.projCorr = projCorr;
               mcmcOut.corrInfo = corrInfo;
            end
            
            
        end
        
        
        function mcmcOut = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,basePrice,idxNow,termCorr,fromTime,toTime,nextFuturesStates,dZ)
            
            predictor = nextFuturesStates;
            corrector = nextFuturesStates;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            modelStatesSize = size(predictor{1},1);
            
            assetSize = size(predictor,1);
            
            assetPrices = zeros(modelStatesSize,assetSize);
            
            for idxjj=1:assetSize
                if size(predictor{idxjj},2) == 1
                    dummyC = predictor{idxjj};
                    assetPrices(:,idxjj) = dummyC;
                else
                    dummyC = predictor{idxjj};
                    assetPrices(:,idxjj) = dummyC(:,idxNow);
                end
            end
            
            %% 20210203 major change to MultiAssetModel
            % we generalize multAssetModel including singleAssetModel
            % count assetSize and if it is 1 then we skip cholesky
            % decomposition
            
            if assetSize > 1
                % find bestTemp for localCorrelation calculation for each mc path
                bestTemp = zeros(modelStatesSize,1);
                for idxhh=1:modelStatesSize
                    bestTemp(idxhh) = assetPrices(idxhh,1)/basePrice(1);
                    for idxjj=2:assetSize
                        bestTemp(idxhh) = max(bestTemp(idxhh),assetPrices(idxhh,idxjj)/basePrice(idxjj));
                    end
                end

                alphaBarrier = multiAsset.alphaBarrier;
                corrAlpha    = multiAsset.corrAlpha;
                denominator = 1.0/(1.0 - alphaBarrier);

                dummyM = ones(numOfFactors,numOfFactors);

                % have to rewrite later for general cases
    %             dummyM(2,3) = 0.0;
    %             dummyM(2,5) = 0.0;
    %             dummyM(3,2) = 0.0;
    %             dummyM(3,4) = 0.0;
    %             
    %             dummyM(4,3) = 0.0;
    %             dummyM(4,5) = 0.0;
    %             dummyM(5,2) = 0.0;
    %             dummyM(5,4) = 0.0;

    %             cumFactors1 = 0;
    %             cumFactors2 = 0;
    %             
    %             for i=1:length(stochasticModelNames)
    %                 childModel1 = multiAsset.modelNameMap(stochasticModelNames{i});
    %                 numOfFactorsChild1 = childModel.numOfFactors;
    %                 for j=i:length(stochasticModelNames)
    %                     childModel2 = multiAsset.modelNameMap(stochasticModelNames{j});
    %                     numOfFactorsChild2 = childModel.numOfFactors;
    %                     if numOfFactorsChild1 == 1
    %                         continue;
    %                     elseif numOfFactorsChild2 == 1
    %                         continue;
    %                     else
    %                         % more than two factors
    %                         dummyM(cumFactors1+1:
    %                     end
    %                 end
    %                 cumFactors1 = cumFactors1 + numOfFactorsChild1;
    %                 
    %             end

                % bestTemp >= 1.0
                L = chol(termCorr,'lower');
                %  bestTemp <  alphaBarrier
                termCorr3 = termCorr*(1-corrAlpha) + dummyM*corrAlpha;  
                LLL = chol(termCorr3,'lower');

    %             tic
                dWW = zeros(size(dZ));
                for idxhh=1:modelStatesSize
                    dZZ  = dZ(idxhh,:);
                    if bestTemp(idxhh) >= 1.0
                        dW = L*dZZ';
                    elseif bestTemp(idxhh) <= alphaBarrier
                        dW = LLL*dZZ';
                    else
                        termCorr2 = termCorr*(1-corrAlpha*(1.0 - bestTemp(idxhh))*denominator) + ...
                                    dummyM*corrAlpha*(1.0 - bestTemp(idxhh))*denominator;
                        LL = chol(termCorr2,'lower');
                        dW = LL*dZZ';
                    end
                    dWW(idxhh,:) = dW';
                end
                
            else
                % we have only one asset so no need to chol
                dWW = dZ;
            end
%             toc
%             for idxhh=1:modelStatesSize
%                 bestTemp = predictor
%             end
            % termCorr Cholesky Decomposition -> correlated brownian motion
%             L = chol(termCorr,'lower');
%             dW = L*dZ';
%             dWW = dW';
            
            
            
            
            
            
            idxNM = cell(length(stochasticModelNames),1);
            
            cumNumOfFactors = 0;
            for i=1:length(stochasticModelNames)
                childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                numOfFactorsChild = childModel.numOfFactors;
                mcOut1 = childModel.inductMCForwardNewReducedQuanto(fromTime,toTime,predictor{i},dWW(:,cumNumOfFactors+1:cumNumOfFactors+numOfFactorsChild));
                corrector{i} = mcOut1.nextFuturesStates;
                idxNM{i} = mcOut1.idxNM;
                cumNumOfFactors = cumNumOfFactors + numOfFactorsChild;
            end
            
            mcmcOut.nextFuturesStates = corrector;
            mcmcOut.idxNM = idxNM;
            
           %% correlation
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                nextFutures1 = corrector{1};
                nextX1 = nextFutures1(:,idxNM{1});
                
                nextFutures2 = corrector{2};
                nextX2 = nextFutures2(:,idxNM{2});
                
                prevFutures1 = predictor{1};
                lastX1 = prevFutures1(:,idxNM{1});
                
                prevFutures2 = predictor{2};
                lastX2 = prevFutures2(:,idxNM{2});
                
                
                
                deltaX1 = log(nextX1./lastX1);
                deltaX2 = log(nextX2./lastX2);
                
                % in - curve correl
                aa = nextFutures1(:,idxNM{1}:end);
                bb = nextFutures2(:,idxNM{2}:end);
                aa0 = prevFutures1(:,idxNM{1}:end);
                bb0 = prevFutures2(:,idxNM{2}:end);
                deltaA = log(aa./aa0);
                deltaB = log(bb./bb0);
                aa_size = size(aa,2);
                bb_size = size(bb,2);
                
                inCurveCorrA = zeros(aa_size,aa_size); 
                for ii=1:aa_size
                    for jj=1:aa_size
                        inCurveCorrA(ii,jj) = corr(deltaA(:,ii),deltaA(:,jj));
                    end
                end
                
                inCurveCorrB = zeros(bb_size,bb_size); 
                for ii=1:bb_size
                    for jj=1:bb_size
                        inCurveCorrB(ii,jj) = corr(deltaB(:,ii),deltaB(:,jj));
                    end
                end
                
                interCurveCorrAB = zeros(aa_size,bb_size);
                for ii=1:aa_size
                    for jj=1:bb_size
                        interCurveCorrAB(ii,jj) = corr(deltaA(:,ii),deltaB(:,jj));
                    end
                end
                
                
                corrInfo.corrX1X2 = corr(deltaX1,deltaX2);
                
                %curve-curve correl
                corrInfo.inCurveCorrA = inCurveCorrA;
                corrInfo.inCurveCorrB = inCurveCorrB;
                corrInfo.interCurveCorrAB = interCurveCorrAB;
                
             %% correlation between projected co-ordiates(parallel & flattening)
               futuresSize_a = size(aa,2);
               futuresSize_b = size(bb,2);
               
               e1_a = ones(futuresSize_a,1);
               deltaE2_a = 1.0/(futuresSize_a-1);
               e2_a=0.0:deltaE2_a:1.0;
               e2_a = e2_a';
               e2_a = e2_a - mean(e2_a)*ones(futuresSize_a,1);
               e2_a = e2_a*-1.0;
               
               cordPar_a = zeros(size(aa,1),1);
               cordFlat_a =zeros(size(aa,1),1);
               dF_a = aa- aa0;
               normE1_a = e1_a'*e1_a; 
               normE2_a = e2_a'*e2_a; 
               cordPar_a = (dF_a*e1_a)/normE1_a;
               cordFlat_a = (dF_a*e2_a)/normE2_a;
               inCurveCorr_a = corr(cordPar_a,cordFlat_a);
               
               e1_b = ones(futuresSize_b,1);
               deltaE2_b = 1.0/(futuresSize_b-1);
               e2_b=0.0:deltaE2_b:1.0;
               e2_b = e2_b';
               e2_b = e2_b - mean(e2_b)*ones(futuresSize_b,1);
               e2_b = e2_b*-1.0;
               
               cordPar_b = zeros(size(bb,1),1);
               cordFlat_b =zeros(size(bb,1),1);
               dF_b = bb- bb0;
               normE1_b = e1_b'*e1_b; 
               normE2_b = e2_b'*e2_b; 
               cordPar_b = (dF_b*e1_b)/normE1_b;
               cordFlat_b = (dF_b*e2_b)/normE2_b;
               inCurveCorr_b = corr(cordPar_b,cordFlat_b);
               
               projRet = [cordPar_a,cordFlat_a,cordPar_b,cordFlat_b];
               projCorr = corr(projRet);
               aaaa = 1.0;
               
               corrInfo.projCorr = projCorr;
               mcmcOut.corrInfo = corrInfo;
            end
            
            
        end
        
        function mcmcOut = inductMCForwardNewReducedQuantoTermCorr(multiAsset,termCorr,fromTime,toTime,nextFuturesStates,dZ)
            
            predictor = nextFuturesStates;
            corrector = nextFuturesStates;
            
            % termCorr Cholesky Decomposition -> correlated brownian motion
            L = chol(termCorr,'lower');
            dW = L*dZ';
            dWW = dW';
            
            
            numOfFactors = multiAsset.numOfFactors;
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            idxNM = cell(length(stochasticModelNames),1);
            
            cumNumOfFactors = 0;
            for i=1:length(stochasticModelNames)
                childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                numOfFactorsChild = childModel.numOfFactors;
                mcOut1 = childModel.inductMCForwardNewReducedQuanto(fromTime,toTime,predictor{i},dWW(:,cumNumOfFactors+1:cumNumOfFactors+numOfFactorsChild));
                corrector{i} = mcOut1.nextFuturesStates;
                idxNM{i} = mcOut1.idxNM;
                cumNumOfFactors = cumNumOfFactors + numOfFactorsChild;
            end
            
            mcmcOut.nextFuturesStates = corrector;
            mcmcOut.idxNM = idxNM;
            
           %% correlation
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                nextFutures1 = corrector{1};
                nextX1 = nextFutures1(:,idxNM{1});
                
                nextFutures2 = corrector{2};
                nextX2 = nextFutures2(:,idxNM{2});
                
                prevFutures1 = predictor{1};
                lastX1 = prevFutures1(:,idxNM{1});
                
                prevFutures2 = predictor{2};
                lastX2 = prevFutures2(:,idxNM{2});
                
                
                
                deltaX1 = log(nextX1./lastX1);
                deltaX2 = log(nextX2./lastX2);
                
                % in - curve correl
                aa = nextFutures1(:,idxNM{1}:end);
                bb = nextFutures2(:,idxNM{2}:end);
                aa0 = prevFutures1(:,idxNM{1}:end);
                bb0 = prevFutures2(:,idxNM{2}:end);
                deltaA = log(aa./aa0);
                deltaB = log(bb./bb0);
                aa_size = size(aa,2);
                bb_size = size(bb,2);
                
                inCurveCorrA = zeros(aa_size,aa_size); 
                for ii=1:aa_size
                    for jj=1:aa_size
                        inCurveCorrA(ii,jj) = corr(deltaA(:,ii),deltaA(:,jj));
                    end
                end
                
                inCurveCorrB = zeros(bb_size,bb_size); 
                for ii=1:bb_size
                    for jj=1:bb_size
                        inCurveCorrB(ii,jj) = corr(deltaB(:,ii),deltaB(:,jj));
                    end
                end
                
                interCurveCorrAB = zeros(aa_size,bb_size);
                for ii=1:aa_size
                    for jj=1:bb_size
                        interCurveCorrAB(ii,jj) = corr(deltaA(:,ii),deltaB(:,jj));
                    end
                end
                
                
                corrInfo.corrX1X2 = corr(deltaX1,deltaX2);
                
                %curve-curve correl
                corrInfo.inCurveCorrA = inCurveCorrA;
                corrInfo.inCurveCorrB = inCurveCorrB;
                corrInfo.interCurveCorrAB = interCurveCorrAB;
                
             %% correlation between projected co-ordiates(parallel & flattening)
               futuresSize_a = size(aa,2);
               futuresSize_b = size(bb,2);
               
               e1_a = ones(futuresSize_a,1);
               deltaE2_a = 1.0/(futuresSize_a-1);
               e2_a=0.0:deltaE2_a:1.0;
               e2_a = e2_a';
               e2_a = e2_a - mean(e2_a)*ones(futuresSize_a,1);
               e2_a = e2_a*-1.0;
               
               cordPar_a = zeros(size(aa,1),1);
               cordFlat_a =zeros(size(aa,1),1);
               dF_a = aa- aa0;
               normE1_a = e1_a'*e1_a; 
               normE2_a = e2_a'*e2_a; 
               cordPar_a = (dF_a*e1_a)/normE1_a;
               cordFlat_a = (dF_a*e2_a)/normE2_a;
               inCurveCorr_a = corr(cordPar_a,cordFlat_a);
               
               e1_b = ones(futuresSize_b,1);
               deltaE2_b = 1.0/(futuresSize_b-1);
               e2_b=0.0:deltaE2_b:1.0;
               e2_b = e2_b';
               e2_b = e2_b - mean(e2_b)*ones(futuresSize_b,1);
               e2_b = e2_b*-1.0;
               
               cordPar_b = zeros(size(bb,1),1);
               cordFlat_b =zeros(size(bb,1),1);
               dF_b = bb- bb0;
               normE1_b = e1_b'*e1_b; 
               normE2_b = e2_b'*e2_b; 
               cordPar_b = (dF_b*e1_b)/normE1_b;
               cordFlat_b = (dF_b*e2_b)/normE2_b;
               inCurveCorr_b = corr(cordPar_b,cordFlat_b);
               
               projRet = [cordPar_a,cordFlat_a,cordPar_b,cordFlat_b];
               projCorr = corr(projRet);
               aaaa = 1.0;
               
               corrInfo.projCorr = projCorr;
               mcmcOut.corrInfo = corrInfo;
            end
            
            
        end
        
        
        function mcmcOut = inductMCForwardNewReducedQuanto(multiAsset,fromTime,toTime,nextFuturesStates,dZ)
            
            predictor = nextFuturesStates;
            corrector = nextFuturesStates;
            
            numOfFactors = multiAsset.numOfFactors;
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            idxNM = cell(length(stochasticModelNames),1);
            
            cumNumOfFactors = 0;
            for i=1:length(stochasticModelNames)
                childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                numOfFactorsChild = childModel.numOfFactors;
                mcOut1 = childModel.inductMCForwardNewReducedQuanto(fromTime,toTime,predictor{i},dZ(:,cumNumOfFactors+1:cumNumOfFactors+numOfFactorsChild));
                corrector{i} = mcOut1.nextFuturesStates;
                idxNM{i} = mcOut1.idxNM;
                cumNumOfFactors = cumNumOfFactors + numOfFactorsChild;
            end
            
            mcmcOut.nextFuturesStates = corrector;
            mcmcOut.idxNM = idxNM;
            
           %% correlation
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                nextFutures1 = corrector{1};
                nextX1 = nextFutures1(:,idxNM{1});
                
                nextFutures2 = corrector{2};
                nextX2 = nextFutures2(:,idxNM{2});
                
                prevFutures1 = predictor{1};
                lastX1 = prevFutures1(:,idxNM{1});
                
                prevFutures2 = predictor{2};
                lastX2 = prevFutures2(:,idxNM{2});
                
                
                
                deltaX1 = log(nextX1./lastX1);
                deltaX2 = log(nextX2./lastX2);
                
                % in - curve correl
                aa = nextFutures1(:,idxNM{1}:end);
                bb = nextFutures2(:,idxNM{2}:end);
                aa0 = prevFutures1(:,idxNM{1}:end);
                bb0 = prevFutures2(:,idxNM{2}:end);
                deltaA = log(aa./aa0);
                deltaB = log(bb./bb0);
                aa_size = size(aa,2);
                bb_size = size(bb,2);
                
                inCurveCorrA = zeros(aa_size,aa_size); 
                for ii=1:aa_size
                    for jj=1:aa_size
                        inCurveCorrA(ii,jj) = corr(deltaA(:,ii),deltaA(:,jj));
                    end
                end
                
                inCurveCorrB = zeros(bb_size,bb_size); 
                for ii=1:bb_size
                    for jj=1:bb_size
                        inCurveCorrB(ii,jj) = corr(deltaB(:,ii),deltaB(:,jj));
                    end
                end
                
                interCurveCorrAB = zeros(aa_size,bb_size);
                for ii=1:aa_size
                    for jj=1:bb_size
                        interCurveCorrAB(ii,jj) = corr(deltaA(:,ii),deltaB(:,jj));
                    end
                end
                
                
                corrInfo.corrX1X2 = corr(deltaX1,deltaX2);
                
                %curve-curve correl
                corrInfo.inCurveCorrA = inCurveCorrA;
                corrInfo.inCurveCorrB = inCurveCorrB;
                corrInfo.interCurveCorrAB = interCurveCorrAB;
                
             %% correlation between projected co-ordiates(parallel & flattening)
               futuresSize_a = size(aa,2);
               futuresSize_b = size(bb,2);
               
               e1_a = ones(futuresSize_a,1);
               deltaE2_a = 1.0/(futuresSize_a-1);
               e2_a=0.0:deltaE2_a:1.0;
               e2_a = e2_a';
               e2_a = e2_a - mean(e2_a)*ones(futuresSize_a,1);
               e2_a = e2_a*-1.0;
               
               cordPar_a = zeros(size(aa,1),1);
               cordFlat_a =zeros(size(aa,1),1);
               dF_a = aa- aa0;
               normE1_a = e1_a'*e1_a; 
               normE2_a = e2_a'*e2_a; 
               cordPar_a = (dF_a*e1_a)/normE1_a;
               cordFlat_a = (dF_a*e2_a)/normE2_a;
               inCurveCorr_a = corr(cordPar_a,cordFlat_a);
               
               e1_b = ones(futuresSize_b,1);
               deltaE2_b = 1.0/(futuresSize_b-1);
               e2_b=0.0:deltaE2_b:1.0;
               e2_b = e2_b';
               e2_b = e2_b - mean(e2_b)*ones(futuresSize_b,1);
               e2_b = e2_b*-1.0;
               
               cordPar_b = zeros(size(bb,1),1);
               cordFlat_b =zeros(size(bb,1),1);
               dF_b = bb- bb0;
               normE1_b = e1_b'*e1_b; 
               normE2_b = e2_b'*e2_b; 
               cordPar_b = (dF_b*e1_b)/normE1_b;
               cordFlat_b = (dF_b*e2_b)/normE2_b;
               inCurveCorr_b = corr(cordPar_b,cordFlat_b);
               
               projRet = [cordPar_a,cordFlat_a,cordPar_b,cordFlat_b];
               projCorr = corr(projRet);
               aaaa = 1.0;
               
               corrInfo.projCorr = projCorr;
               mcmcOut.corrInfo = corrInfo;
            end
            
            
        end
        
        function mcmcOut = inductMCForwardNewReduced(multiAsset,fromTime,toTime,nextFuturesStates,dZ)
            
            predictor = nextFuturesStates;
            corrector = nextFuturesStates;
            
            numOfFactors = multiAsset.numOfFactors;
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            idxNM = cell(length(stochasticModelNames),1);
            
            cumNumOfFactors = 0;
            for i=1:length(stochasticModelNames)
                childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                numOfFactorsChild = childModel.numOfFactors;
                mcOut1 = childModel.inductMCForwardNewReduced(fromTime,toTime,predictor{i},dZ(:,cumNumOfFactors+1:cumNumOfFactors+numOfFactorsChild));
                corrector{i} = mcOut1.nextFuturesStates;
                idxNM{i} = mcOut1.idxNM;
                cumNumOfFactors = cumNumOfFactors + numOfFactorsChild;
            end
            
            mcmcOut.nextFuturesStates = corrector;
            mcmcOut.idxNM = idxNM;
            
           %% correlation
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                nextFutures1 = corrector{1};
                nextX1 = nextFutures1(:,idxNM{1});
                
                nextFutures2 = corrector{2};
                nextX2 = nextFutures2(:,idxNM{2});
                
                prevFutures1 = predictor{1};
                lastX1 = prevFutures1(:,idxNM{1});
                
                prevFutures2 = predictor{2};
                lastX2 = prevFutures2(:,idxNM{2});
                
                
                
                deltaX1 = log(nextX1./lastX1);
                deltaX2 = log(nextX2./lastX2);
                
                % in - curve correl
                aa = nextFutures1(:,idxNM{1}:end);
                bb = nextFutures2(:,idxNM{2}:end);
                aa0 = prevFutures1(:,idxNM{1}:end);
                bb0 = prevFutures2(:,idxNM{2}:end);
                deltaA = log(aa./aa0);
                deltaB = log(bb./bb0);
                aa_size = size(aa,2);
                bb_size = size(bb,2);
                
                inCurveCorrA = zeros(aa_size,aa_size); 
                for ii=1:aa_size
                    for jj=1:aa_size
                        inCurveCorrA(ii,jj) = corr(deltaA(:,ii),deltaA(:,jj));
                    end
                end
                
                inCurveCorrB = zeros(bb_size,bb_size); 
                for ii=1:bb_size
                    for jj=1:bb_size
                        inCurveCorrB(ii,jj) = corr(deltaB(:,ii),deltaB(:,jj));
                    end
                end
                
                interCurveCorrAB = zeros(aa_size,bb_size);
                for ii=1:aa_size
                    for jj=1:bb_size
                        interCurveCorrAB(ii,jj) = corr(deltaA(:,ii),deltaB(:,jj));
                    end
                end
                
                
                corrInfo.corrX1X2 = corr(deltaX1,deltaX2);
                
                %curve-curve correl
                corrInfo.inCurveCorrA = inCurveCorrA;
                corrInfo.inCurveCorrB = inCurveCorrB;
                corrInfo.interCurveCorrAB = interCurveCorrAB;
                
             %% correlation between projected co-ordiates(parallel & flattening)
               futuresSize_a = size(aa,2);
               futuresSize_b = size(bb,2);
               
               e1_a = ones(futuresSize_a,1);
               deltaE2_a = 1.0/(futuresSize_a-1);
               e2_a=0.0:deltaE2_a:1.0;
               e2_a = e2_a';
               e2_a = e2_a - mean(e2_a)*ones(futuresSize_a,1);
               e2_a = e2_a*-1.0;
               
               cordPar_a = zeros(size(aa,1),1);
               cordFlat_a =zeros(size(aa,1),1);
               dF_a = aa- aa0;
               normE1_a = e1_a'*e1_a; 
               normE2_a = e2_a'*e2_a; 
               cordPar_a = (dF_a*e1_a)/normE1_a;
               cordFlat_a = (dF_a*e2_a)/normE2_a;
               inCurveCorr_a = corr(cordPar_a,cordFlat_a);
               
               e1_b = ones(futuresSize_b,1);
               deltaE2_b = 1.0/(futuresSize_b-1);
               e2_b=0.0:deltaE2_b:1.0;
               e2_b = e2_b';
               e2_b = e2_b - mean(e2_b)*ones(futuresSize_b,1);
               e2_b = e2_b*-1.0;
               
               cordPar_b = zeros(size(bb,1),1);
               cordFlat_b =zeros(size(bb,1),1);
               dF_b = bb- bb0;
               normE1_b = e1_b'*e1_b; 
               normE2_b = e2_b'*e2_b; 
               cordPar_b = (dF_b*e1_b)/normE1_b;
               cordFlat_b = (dF_b*e2_b)/normE2_b;
               inCurveCorr_b = corr(cordPar_b,cordFlat_b);
               
               projRet = [cordPar_a,cordFlat_a,cordPar_b,cordFlat_b];
               projCorr = corr(projRet);
               aaaa = 1.0;
               
               corrInfo.projCorr = projCorr;
               mcmcOut.corrInfo = corrInfo;
            end
            
            
        end
        
        function mcmcOut = inductMCForwardNew(multiAsset,fromTime,toTime,lastX,dZ)
            
            predictor = lastX;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardNew(fromTime,toTime,lastX(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
            end
            
            mcmcOut.nextX = predictor;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                
                % realized correlation
                deltaX = log(predictor./lastX);
                termDeltaX = log(predictor./1.0); 
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.termCorrX1X2 = corr(termDeltaX(:,1),termDeltaX(:,2));
%                 corrInfo.corrX1X3 = corr(deltaX(:,1),deltaX(:,3));
%                 corrInfo.corrX2X3 = corr(deltaX(:,2),deltaX(:,3));

%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
                
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
        end
        
        function mcmcOut = inductMCForwardQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoS1S2 = multiAsset.correlationMatrix(1,2);
 
            
            Lsmall = eye(2,2);
            
            dummyR = rhoS1S2*rhoS1S2/(1-rhoSV(1)^2)/(1-rhoSV(2)^2);
            Lsmall(2,1) = sqrt(dummyR);
            Lsmall(2,2) = sqrt(1.0-dummyR);
            
            L = eye(4,4);
            L(3:4,3:4) = Lsmall;
            % W is (numOfFactors,NMC) matrix
            W = dZ';  
            WCorr = L*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoSiSj = multiAsset.correlationMatrix;
 
            rhoSmall = eye(numOfFactors/2,numOfFactors/2);
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i==j
                        rhoSmall(i,j) = 1.0;
                    else
                        rhoSmall(i,j) = rhoSiSj(i,j)/sqrt(1-rhoSV(i)^2)/sqrt(1-rhoSV(j)^2);
                    end
                end    
            end
            
%             rhoSmall = 0.5*(rhoSmall + rhoSmall');
            
            LSmall = chol(rhoSmall,'lower');
            
            L = eye(numOfFactors,numOfFactors);
            L(numOfFactors/2+1:numOfFactors,numOfFactors/2+1:numOfFactors) = LSmall;
            % W is (numOfFactors,NMC) matrix
            W = dZ';  
            WCorr = L*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaX(:,2));
                corrInfo.covarX1X2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaV(:,1),deltaV(:,2));
                corrInfo.covarV1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaV(:,2));
                corrInfo.covarX1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,2),deltaV(:,1));
                corrInfo.covarX2V1 = dummyCovarX1X2(1,2);
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardPerfectCorrMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoSiSj = multiAsset.correlationMatrix;
 
            rhoSmall = eye(numOfFactors/2,numOfFactors/2);

            rhoTemp = eye(numOfFactors/2 +1,numOfFactors/2 +1);
            
            for i=1:length(stochasticModelNames)
                rhoTemp(1,1+i) = rhoSV(i);
                rhoTemp(1+i,1) = rhoSV(i);
            end
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i==j
                        rhoTemp(1+i,1+j) = rhoSiSj(i,j);
                    else
                        rhoTemp(1+i,1+j) = rhoSiSj(i,j);
                    end
                end
            end
            
            tempEig = eig(rhoTemp);
            if min(tempEig) <= 0
                [xxTemp,iterTemp] = nearcorr_new(rhoTemp,[],1e-8,[],[]);
                rhoTemp = xxTemp;
                
                % we need update the child model's model params rho
                for i=1:length(stochasticModelNames)
                    childModel = multiAsset.modelNameMap(stochasticModelNames{i});
                    childModel.rho = rhoTemp(1,1+i);
                end
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i==j
                        rhoSmall(i,j) = 1.0;
                    else
                        sigma_i = sqrt(1-rhoTemp(1,1+i)^2);
                        sigma_j = sqrt(1-rhoTemp(1,1+j)^2);
                        rhoSmall(i,j) = (rhoTemp(1+i,1+j)-rhoTemp(1,1+i)*rhoTemp(1,1+j)) ...
                                        /(sigma_i*sigma_j);
                         % nearest correlation matrix           
%                         rhoSmall(i,j) = min(0.99,max(-0.99,rhoSmall(i,j))); 
                        if abs(rhoSmall(i,j)) > 1.0
                            error('rhoSmall(i,j) is beyond the limit!');
                        end
                    end
                end    
            end
            
%             for i=1:length(stochasticModelNames)
%                 for j=1:length(stochasticModelNames)
%                     if i==j
%                         rhoSmall(i,j) = 1.0;
%                     else
%                         sigma_i = sqrt(1-rhoSV(i)^2);
%                         sigma_j = sqrt(1-rhoSV(j)^2);
%                         rhoSmall(i,j) = (rhoSiSj(i,j)-rhoSV(i)*rhoSV(j)) ...
%                                         /(sigma_i*sigma_j);
%                          % nearest correlation matrix           
% %                         rhoSmall(i,j) = min(0.99,max(-0.99,rhoSmall(i,j))); 
%                         if abs(rhoSmall(i,j)) > 1.0
%                             error('rhoSmall(i,j) is beyond the limit!');
%                         end
%                     end
%                 end    
%             end

            
%             rhoSmall = 0.5*(rhoSmall + rhoSmall');
            
%             LSmall = chol(rhoSmall,'lower');
            
            rhoBig = eye(numOfFactors/2 +1,numOfFactors/2 +1);
            
            rhoBig(2:numOfFactors/2+1,2:numOfFactors/2+1) = rhoSmall;
            L = chol(rhoBig,'lower');
            % W is (numOfFactors,NMC) matrix
            W = dZ';
            
            % make dW_v_i as dW_v_1 so that variaces are perfectly
            % correlated
            WDummy = zeros(numOfFactors/2 +1,size(W,2));
            
            WDummy(1,:) = W(1,:);
            for i=1:length(stochasticModelNames)
                WDummy(1+i,:) = W(numOfFactors/2+i,:);
            end
            
            WCorrDummy = L*WDummy;
            WCorr = zeros(size(W));
            
            for i=1:length(stochasticModelNames)
                WCorr(i,:) = WCorrDummy(1,:);
                WCorr(numOfFactors/2+i,:) = WCorrDummy(1+i,:);
            end
            
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaX(:,2));
                corrInfo.covarX1X2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaV(:,1),deltaV(:,2));
                corrInfo.covarV1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,1),deltaV(:,2));
                corrInfo.covarX1V2 = dummyCovarX1X2(1,2);
                
                dummyCovarX1X2 = cov(deltaX(:,2),deltaV(:,1));
                corrInfo.covarX2V1 = dummyCovarX1X2(1,2);
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardPerfectCorrMultiQEND(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
           %% all variance processes are independent
           
            corrUL = eye(numOfFactors/2,numOfFactors/2);
            
            corrUR = zeros(numOfFactors/2,numOfFactors/2);
            
            rhoSV = zeros(length(stochasticModelNames),1);
            for i=1:length(stochasticModelNames)
                corrUR(i,i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
                rhoSV(i) = corrUR(i,i);
            end
            
            rhoSiSj = multiAsset.correlationMatrix;
 
            rhoSmall = eye(numOfFactors/2,numOfFactors/2);
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i==j
                        rhoSmall(i,j) = 1.0;
                    else
                        sigma_i = sqrt(1-rhoSV(i)^2);
                        sigma_j = sqrt(1-rhoSV(j)^2);
                        rhoSmall(i,j) = (rhoSiSj(i,j)-rhoSV(i)*rhoSV(j)) ...
                                        /(sigma_i*sigma_j);
                         % nearest correlation matrix           
%                         rhoSmall(i,j) = min(0.99,max(-0.99,rhoSmall(i,j))); 
                    end
                end    
            end
            
%             rhoSmall = 0.5*(rhoSmall + rhoSmall');
            
%             LSmall = chol(rhoSmall,'lower');
            
            rhoBig = eye(numOfFactors/2 +1,numOfFactors/2 +1);
            
            rhoBig(2:numOfFactors/2+1,2:numOfFactors/2+1) = rhoSmall;
            L = chol(rhoBig,'lower');
            % W is (numOfFactors,NMC) matrix
            W = dZ';
            
            % make dW_v_i as dW_v_1 so that variaces are perfectly
            % correlated
            WDummy = zeros(numOfFactors/2 +1,size(W,2));
            
            WDummy(1,:) = W(1,:);
            for i=1:length(stochasticModelNames)
                WDummy(1+i,:) = W(numOfFactors/2+i,:);
            end
            
            WCorrDummy = L*WDummy;
            WCorr = zeros(size(W));
            
            for i=1:length(stochasticModelNames)
                WCorr(i,:) = WCorrDummy(1,:);
                WCorr(numOfFactors/2+i,:) = WCorrDummy(1+i,:);
            end
            
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,i+numOfFactors/2),dZCorr(:,i));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                
                % 1&2
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                % 1&3
                corrInfo.corrX1X3 = corr(deltaX(:,1),deltaX(:,3));
                corrInfo.corrV1V3 = corr(deltaV(:,1),deltaV(:,3));
                
                corrInfo.corrX3V3 = corr(deltaX(:,3),deltaV(:,3));
                
                corrInfo.corrX1V3 = corr(deltaX(:,1),deltaV(:,3));
                corrInfo.corrX3V1 = corr(deltaX(:,3),deltaV(:,1));
                
                % 2&3
                corrInfo.corrX2X3 = corr(deltaX(:,2),deltaX(:,3));
                corrInfo.corrV2V3 = corr(deltaV(:,2),deltaV(:,3));
                
                corrInfo.corrX2V3 = corr(deltaX(:,2),deltaV(:,3));
                corrInfo.corrX3V2 = corr(deltaX(:,3),deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        % input asset Correlation -> output L with parsimonious correlation
        % assumption
        function out = CholeskyDecomposeParsimonious(multiAsset,corrIn)
          %% Cholesky Decomposition Start
          
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
            rhoSiSj = corrIn;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
            
            try
                RPars = chol(corrTotal,'lower');
            catch
                aaa = corrTotal;
                bbb = rhoSiSj(1,2);
            end
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            out = dummyL2;
            
        end
        
        function mcmcOut = inductMCForwardIdyoSyncMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            % idyosyncraticVVCorr for independent variance variance
            % correlation
            idyosyncraticVVCorr = multiAsset.modelParams('idyosyncraticVVCorr');
            
          %% Cholesky Decomposition Start
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        
                        %idyosyncratic correction
                        
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j) + ...
                                                       idyosyncraticVVCorr*sqrt(1.0-rhoSV(i)*rhoSV(i))*sqrt(1.0-rhoSV(j)*rhoSV(j)) ;
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = max(-0.999,min(0.999,corrTotal(2*(i-1)+2,2*(j-1)+2)));
                    end
                    
                end
            end
            
%             %% positiveNess Check
%             [eigV eigD] = eig(corrTotal);
%             numOfPositives = sum(diag(eigD)>=0);
%             %% exception handling : Poor Man's Approach
%             nearCorrTol = 1e-10;
%             if numOfPositives <= size(corrTotal,1)
%                 corrTotalOld = corrTotal;
%                 [nearCorrOut, nearCorrIter] = nearcorr(corrTotalOld,nearCorrTol,[],[],[],[],0);
%                 corrTotal = nearCorrOut;
%             end
        
            RPars = chol(corrTotal,'lower');
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                
                termDeltaX = log(predictorX./1.0);
                corrInfo.termCorrX1X2 = corr(termDeltaX(:,1),termDeltaX(:,2));
                
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                
%                 corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
%                 corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
%                 corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
%                 corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
%                 
%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
%                 
%                 corrInfo.varV1 = var(deltaV(:,1));
%                 corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQE(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
          %% Cholesky Decomposition Start
          
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                
                termDeltaX = log(predictorX./1.0);
                corrInfo.termCorrX1X2 = corr(termDeltaX(:,1),termDeltaX(:,2));
                
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                
%                 corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
%                 corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
%                 corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
%                 corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
%                 
%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
%                 
%                 corrInfo.varV1 = var(deltaV(:,1));
%                 corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQEND(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
          %% Cholesky Decomposition Start
          
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            
          %% Cholesky Decomposition End
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                
                % 1&2
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                % 1&3
                corrInfo.corrX1X3 = corr(deltaX(:,1),deltaX(:,3));
                corrInfo.corrV1V3 = corr(deltaV(:,1),deltaV(:,3));
                
                corrInfo.corrX3V3 = corr(deltaX(:,3),deltaV(:,3));
                
                corrInfo.corrX1V3 = corr(deltaX(:,1),deltaV(:,3));
                corrInfo.corrX3V1 = corr(deltaX(:,3),deltaV(:,1));
                
                % 2&3
                corrInfo.corrX2X3 = corr(deltaX(:,2),deltaX(:,3));
                corrInfo.corrV2V3 = corr(deltaV(:,2),deltaV(:,3));
                
                corrInfo.corrX2V3 = corr(deltaX(:,2),deltaV(:,3));
                corrInfo.corrX3V2 = corr(deltaX(:,3),deltaV(:,2));
                
                
%                 corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
%                 corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
%                 corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
%                 corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
%                 
%                 corrInfo.varX1 = var(deltaX(:,1));
%                 corrInfo.varX2 = var(deltaX(:,2));
%                 
%                 corrInfo.varV1 = var(deltaV(:,1));
%                 corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQE_EffectiveLocalCorr(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                
                corrInfo.covarX1X2 = cov(deltaX(:,1),deltaX(:,2));
                corrInfo.covarV1V2 = cov(deltaV(:,1),deltaV(:,2));
                corrInfo.covarX1V2 = cov(deltaX(:,1),deltaV(:,2));
                corrInfo.covarX2V1 = cov(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQETestVV0(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
%                         
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = 0.0;
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = 0.0;
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = 0.0;
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function mcmcOut = inductMCForwardParsimoniousMultiQETestVV1(multiAsset,fromTime,toTime,lastX,lastV,dZ)
            
            % dZ is (NMC,numOfFactors)
            
            
            stochasticModelNames = multiAsset.stochasticModelNames;
            numOfFactors = multiAsset.numOfFactors;
            corrTotal = zeros(numOfFactors,numOfFactors);
            
          %% all variance processes are independent
            rhoSiSj = multiAsset.correlationMatrix;
            rhoSV = zeros(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                rhoSV(i) = multiAsset.modelNameMap(stochasticModelNames{i}).rho;
            end
            
            Qinv = cell(length(stochasticModelNames),1);
            
            for i=1:length(stochasticModelNames)
                Qinv{i,1} = zeros(2,2);
                dummyVa = sqrt(1-rhoSV(i)*rhoSV(i));
                Qinv{i,1}(1,1) = 1.0/dummyVa;
                Qinv{i,1}(1,2) = -rhoSV(i)/dummyVa;
                Qinv{i,1}(2,1) = 0.0;
                Qinv{i,1}(2,2) = 1.0;
            end
            
            QinvM = zeros(numOfFactors,numOfFactors);
            
            for i=1:length(stochasticModelNames)
                QinvM(2*(i-1)+1:2*(i-1)+2,2*(i-1)+1:2*(i-1)+2) = Qinv{i,1}; 
            end
            
            
            for i=1:length(stochasticModelNames)
                for j=1:length(stochasticModelNames)
                    if i ==  j
                        corrTotal(2*(i-1)+1,2*(i-1)+1) = 1.0;
                        corrTotal(2*(i-1)+2,2*(i-1)+2) = 1.0;
                        corrTotal(2*(i-1)+1,2*(i-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(i-1)+1) = corrTotal(2*(i-1)+1,2*(i-1)+2);
                    else
                        corrTotal(2*(i-1)+1,2*(j-1)+1) = rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(j) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(i) * rhoSiSj(i,j);
%                         corrTotal(2*(i-1)+2,2*(j-1)+2) = rhoSV(i) *rhoSV(j) * rhoSiSj(i,j);
%                         
                        corrTotal(2*(i-1)+1,2*(j-1)+2) = rhoSV(i);
                        corrTotal(2*(i-1)+2,2*(j-1)+1) = rhoSV(j);
                        corrTotal(2*(i-1)+2,2*(j-1)+2) = 0.999;
                    end
                    
                end
            end
        
            RPars = chol(corrTotal,'lower');
            
            
            
            W = dZ';
            % W is (numOfFactors,NMC) matrix
            
            % Tr1 is the transition matrix from  (V1;V2;....,;S1;S2,....)
            % to (S1;V1;S2;V2;....)
            %
            % (S1;V1;S2;V2;....) = Tr1 * (V1;V2;....,;S1;S2,....)
            %
            
            Tr1 = zeros(numOfFactors,numOfFactors);
            for i=1:length(stochasticModelNames)
                Tr1(2*i-1,numOfFactors/2 + i) = 1.0;
                Tr1(2*i,i) = 1.0;
            end
            
            L = QinvM*RPars;
            
            % to make the same cholesky decomposition
            % we compute effective correlation matrix then 
            % cholesky decompose again
            reducedCorr = L*L';
            dummyL = chol(reducedCorr,'lower');
            dummyL2 = dummyL*Tr1;
            WCorr = dummyL2*W;
            dZCorr = WCorr';
            
            predictorX = lastX;
            predictorV = lastV;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCForwardQE(fromTime,toTime,...
                            lastX(:,i),lastV(:,i),dZCorr(:,2 * (i-1) + 1),dZCorr(:,2 * (i-1) + 2));
                predictorX(:,i) = mcOut1.nextX;
                predictorV(:,i) = mcOut1.nextV;
            end
            
            mcmcOut.nextX = predictorX;
            mcmcOut.nextV = predictorV;
            
            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                deltaX = log(predictorX./lastX);
                deltaV = predictorV - lastV;
                corrInfo.corrX1X2 = corr(deltaX(:,1),deltaX(:,2));
                corrInfo.corrV1V2 = corr(deltaV(:,1),deltaV(:,2));
                
                corrInfo.corrX1V1 = corr(deltaX(:,1),deltaV(:,1));
                corrInfo.corrX2V2 = corr(deltaX(:,2),deltaV(:,2));
                
                corrInfo.corrX1V2 = corr(deltaX(:,1),deltaV(:,2));
                corrInfo.corrX2V1 = corr(deltaX(:,2),deltaV(:,1));
                
                corrInfo.varX1 = var(deltaX(:,1));
                corrInfo.varX2 = var(deltaX(:,2));
                
                corrInfo.varV1 = var(deltaV(:,1));
                corrInfo.varV2 = var(deltaV(:,2));
                
                mcmcOut.corrInfo = corrInfo;
                
            end
            
            
        end
        
        function out = GetLocalCorrSLV(multiAsset,fromTime,x,y)
            
            timeStep = multiAsset.localCorrInfo.timeStepLVF;
            
            idx = find(timeStep > fromTime);
            if isempty(idx)
                idxNow = length(timeStep);
            else
                idxNow = min(idx);
            end
            
            localCorrXY = multiAsset.localCorrInfo.localCorrXY{idxNow};
            strikeX = multiAsset.localCorrInfo.strikeX{idxNow};
            strikeY = multiAsset.localCorrInfo.strikeY{idxNow};
            
            strikeXVector = strikeX(:);
            strikeYVector = strikeY(:);
            localCorrXYVector = localCorrXY(:);
            
            F = scatteredInterpolant(strikeXVector,strikeYVector,localCorrXYVector,'linear','nearest');
            out = F(x,y);
            
            out = max(-0.99,min(out,0.99));
            
%             out = griddata(strikeXVector,strikeYVector,localCorrXYVector,x,y);
%             out = interp2FlatExtrap(strikeX,strikeY,localCorrXY',x,y);
            
        end
            
        
        function out = GenerateEffectiveLocalCorr(multiAsset,valueDateH,effectiveLocalCorrParams)
            tic

            stochasticModelNames = multiAsset.stochasticModelNames;
            
            numOfStochasticAssets = length(stochasticModelNames);
            
            childModels = cell(numOfStochasticAssets,1);
            
            for i=1:numOfStochasticAssets
                childModels{i} = multiAsset.modelNameMap(stochasticModelNames{i});
            end
            
            levFuncInfo1 = childModels{1}.levFuncInfo;
            timeStepLVF = levFuncInfo1.timeStepLVF;
            
            timeStep = [0;timeStepLVF];
            timeStep = sort(timeStep,'ascend');
            % timeStepLVF do not have 0 therefore we count 0 additionaly
            totalTimeStepSize = length(timeStepLVF) + 1;
            
            numOfBins = effectiveLocalCorrParams.numOfBins;
            numOfPath = effectiveLocalCorrParams.numOfPath;
            numOfPathPerBins = effectiveLocalCorrParams.numOfPathPerBins;
            numOfPathPerBins1D = numOfPath/numOfBins;
            
          %% initialize EffectiveLocalCorr Info
           localCorrXY = cell(totalTimeStepSize-1,1);
           strikeX = cell(totalTimeStepSize-1,1);
           strikeY = cell(totalTimeStepSize-1,1);
           expectedVarianceX = cell(totalTimeStepSize-1,1);
           

           expectedVarianceY = cell(totalTimeStepSize-1,1);
           
           for idxhh=1:totalTimeStepSize-1
                localCorrXY{idxhh} = zeros(numOfBins,numOfBins);
                strikeX{idxhh} = zeros(numOfBins,numOfBins);
                strikeY{idxhh} = zeros(numOfBins,numOfBins);
                expectedVarianceX{idxhh} = zeros(numOfBins,numOfBins);
                expectedVarianceY{idxhh} = zeros(numOfBins,numOfBins);
           end
                      
           multiAsset.localCorrInfo.localCorrXY = localCorrXY;
           multiAsset.localCorrInfo.timeStepLVF = timeStepLVF;
           multiAsset.localCorrInfo.strikeX = strikeX;
           multiAsset.localCorrInfo.strikeY = strikeY;
           
           multiAsset.localCorrInfo.expectedVarianceX = expectedVarianceX;
           multiAsset.localCorrInfo.expectedVarianceY = expectedVarianceY;
            
          %% local correlation from the local vol model
           rhoSiSj = multiAsset.correlationMatrix;
           corr0 = rhoSiSj(1,2);

           v10 = childModels{1}.v0;
           v20 = childModels{2}.v0;
           smallTime = 0.01;
           lv10 = childModels{1}.GetLocalVol(smallTime,1.0);
           lv20 = childModels{2}.GetLocalVol(smallTime,1.0);
           
          %% localCorrXY initialize
            
            % initial strikes
            dt = timeStep(2) - timeStep(1);
            dx1 = 1.0 * 5.0*lv10*sqrt(dt/365)/numOfBins*2.0;
            dx2 = 1.0 * 5.0*lv20*sqrt(dt/365)/numOfBins*2.0;
            
            for i=1:numOfBins
                for j=1:numOfBins
                    multiAsset.localCorrInfo.strikeX{1}(i,j) = 1.0 + (i-numOfBins/2.0)*dx1;
                    multiAsset.localCorrInfo.strikeY{1}(i,j) = 1.0 + (j-numOfBins/2.0)*dx2;

                    multiAsset.localCorrInfo.expectedVarianceX{1}(i,j) =  childModels{1}.v0;
                    multiAsset.localCorrInfo.expectedVarianceY{1}(i,j) =  childModels{2}.v0;

                    % if we use local Vol & local Corr
                    % this part should be updated to replaced with
                    % local look up part
                    
                    % rho_slv = rho_lv/(sqrt(v1v2)/sqrt(v1*v2));
                    multiAsset.localCorrInfo.localCorrXY{1}(i,j) = 1.0* corr0;
                end
            end
            
           %% MC init 
            useQuasiRandomYN = 'false';
            numOfFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = numOfPath; 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numOfFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = numOfPath; 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numOfFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
            end
            
            currentTime = 0; %NowDate
            currentTimeIdx = 1; % 1st Time Steps
            currentNodeIdx = 1; % no eventDate for nowDate
            lastTimeIdx = currentTimeIdx;
            
            % initial modelState
            %  X  = S(t)/F(0,t)  : spot process
            %  V  = instantaneous variance proces
           
           initX = cell(numOfStochasticAssets,1);
           initV = cell(numOfStochasticAssets,1);
           
           for i=1:numOfStochasticAssets
               initX{i} = ones(NMC,1);
               initV{i} = ones(NMC,1)*childModels{i}.v0;
           end
           
%            initX = ones(NMC,1);
%            initV = ones(NMC,1)*eqHeston.v0;
%            
           nextX = initX;
           nextV = initV;
           
         %% Path Generation Forward
          startTimeIdx = 1;
          endTimeIdx = length(timeStep)-1;
         %% induction Forward start
         %% below for the timeStep(idx) to timeStep(idx+1) 
         %% generated leverageFunction applied to timeStep(idx+1) to timeStep(idx+2)
         %% therefore we only need to generate from startTimeIdx to endTimeIdx-1
          toc

            
          for idx= startTimeIdx:1:endTimeIdx-1
               if idx == endTimeIdx
                 abcd = 1.0;
               end
              
               % Get Effective Local Correlation SLV
               effectivCorrSLV = GetLocalCorrSLV(multiAsset,timeStep(idx) + 0.01,nextX{1},nextX{2});
               Ut = U(:,numOfFactors*(idx-1)+1:numOfFactors*(idx-1)+numOfFactors);
               
               tic
 
               for idxNMC=1:NMC
                    corrIn = eye(2,2);
                    corrIn(1,2) = effectivCorrSLV(idxNMC);
                    corrIn(2,1) = corrIn(1,2);

                    outL = CholeskyDecomposeParsimonious(multiAsset,corrIn);
                    U0 = Ut(idxNMC,1:numOfFactors);
                    dZ = U0';
                    dZCorr = outL*dZ;
                    U0 = dZCorr';
                    Ut(idxNMC,:) = U0;
               end

               toc 
                
                if strcmp(childModels{1}.modelParams('MCScheme'), 'QE')
                    for i=1:numOfStochasticAssets
                        mcmcOut   = childModels{i}.inductMCForwardQE(timeStep(idx),timeStep(idx+1),nextX{i},nextV{i},U(:,2*(i-1)+1),U(:,2*(i-1)+2));
                    
                        nextX{i} = mcmcOut.nextX;
                        nextV{i} = mcmcOut.nextV;

                    end
                    
                    
                end
                
             %% we sort simul path, asceding w.r.t nextX
               sortedX0 = cell(numOfStochasticAssets,1);
               sortIdx0 = cell(numOfStochasticAssets,1);
               
             
               sortedX = cell(numOfStochasticAssets,1);
               sortIdx = cell(numOfStochasticAssets,1);
               sortedV = cell(numOfStochasticAssets,1);
               
               sortIdx2D = cell(numOfStochasticAssets,1);
               interMediatedX = cell(numOfStochasticAssets,1);
               interMediatedV = cell(numOfStochasticAssets,1);
               
               for idxZZ=1:numOfStochasticAssets
                    interMediatedX{idxZZ} = zeros(numOfPathPerBins1D,1);
                    interMediatedV{idxZZ} = zeros(numOfPathPerBins1D,1);
               end
               
               [sortedX0{1}, sortIdx0{1}] = sort(nextX{1},'ascend');
               
               for idxhh1 = 1:numOfBins
                   startIdx = numOfPathPerBins1D*(idxhh1-1)+1;
                   endXIdx = startIdx + numOfPathPerBins1D;
                   interMediatedX{1} = nextX{1}(sortIdx0{1}(startIdx:endXIdx-1));
                   interMediatedX{2} = nextX{2}(sortIdx0{1}(startIdx:endXIdx-1));
                   interMediatedV{1} = nextV{1}(sortIdx0{1}(startIdx:endXIdx-1));
                   interMediatedV{2} = nextV{2}(sortIdx0{1}(startIdx:endXIdx-1));
                   
                   [dummySortedX, dummySortIdx] = sort(interMediatedX{2},'ascend');
                   
                   for idxhh2=1:numOfBins
                       startIdx2 = numOfPathPerBins*(idxhh2-1)+1;
                       endXIdx2 = startIdx2 + numOfPathPerBins;
                       startIdxNew = startIdx + startIdx2 - 1;
                       endXIdxNew  = startIdx  + endXIdx2  - 1; 
                       sortedX{1}(startIdxNew:endXIdxNew-1,1) = interMediatedX{1}(dummySortIdx(startIdx2:endXIdx2-1));
                       sortedX{2}(startIdxNew:endXIdxNew-1,1) = interMediatedX{2}(dummySortIdx(startIdx2:endXIdx2-1));
                       
                       sortedV{1}(startIdxNew:endXIdxNew-1,1) = interMediatedV{1}(dummySortIdx(startIdx2:endXIdx2-1));
                       sortedV{2}(startIdxNew:endXIdxNew-1,1) = interMediatedV{2}(dummySortIdx(startIdx2:endXIdx2-1));
                       
                   end
                   
               end
               
               
%                sortedV{i} = nextV{i}(sortIdx{i});
%                    
%                for i=1:numOfStochasticAssets
%                    [sortedX{i}, sortIdx{i}] = sort(nextX{i},'ascend');
%                    sortedV{i} = nextV{i}(sortIdx{i});
%                end
               
               % X1, X2
               
               startX1Idx = 1;
               endX1Idx = 0;
               
               startX2Idx = 1;
               endX2Idx = 0;
               
%                expectedVariance = cell(numOfStochasticAssets,1);
%                for i=1:numOfStochasticAssets
%                    expectedVariance{i} = zeros(numOfBins,1);
%                end
%                
%                strikesVariance = cell(numOfStochasticAssets,1);
%                for i=1:numOfStochasticAssets
%                    strikesVariance{i} = zeros(numOfBins,1);
%                end
               
               % test version for marginal prob dist
               % Sigma11(S1,S2),Sigma22(S1,S2)
               % Sigma12(S1,S2)
               

               
%                for i=1:numOfStochasticAssets
%                    for idxS1=1:numOfBins
%                        inc = numOfPathPerBins1D;
%                        startX1Idx = 1 + numOfPathPerBins1D*(idxS1-1);
%                        endX1Idx   = startX1Idx + numOfPathPerBins1D;
%                        sum = 0.0;
%                        
%                        for idxW=startX1Idx:endX1Idx-1
%                            sum = sum + sortedV{i}(idxW);
%                        end
% 
%                        %average of variance
%                        sum = sum/inc;
%                        strikesVariance{i}(idxS1) = 0.5*(sortedX{i}(startX1Idx)+sortedX{i}(endX1Idx-1));
%                        expectedVariance{i}(idxS1) = sum;
%                     end
%                end
               
               startX1Idx = 1;
               endX1Idx = 0;
               
               
             %% Effective Joint Marginal Distribution of S1,S2 
             
               for idxS1=1:numOfBins
                   startX1Idx = 1 + numOfBins*numOfPathPerBins*(idxS1-1);
                   endX1Idx   = startX1Idx + numOfPathPerBins;
                   
                   startX2Idx = 1;
                   endX2Idx = 0;
                       
                   for idxS2=1:numOfBins
                       startX2Idx = startX1Idx + numOfPathPerBins*(idxS2-1);
                       endX2Idx = startX2Idx + numOfPathPerBins;
                       
                       sum = 0.0;
                       sumx = 0.0;
                       sumy = 0.0;
                       for idxW=startX2Idx:endX2Idx-1
                           sum  = sum  + sqrt(sortedV{1}(idxW)*sortedV{2}(idxW));
                           sumx = sumx + sqrt(sortedV{1}(idxW)*sortedV{1}(idxW));
                           sumy = sumy + sqrt(sortedV{2}(idxW)*sortedV{2}(idxW));
                       end

                       %average of variance
                       sum = sum/numOfPathPerBins;
                       sumx = sumx/numOfPathPerBins;
                       sumy = sumy/numOfPathPerBins;
                       
                       multiAsset.localCorrInfo.strikeX{idx+1}(idxS1,idxS2) = 0.5*(sortedX{1}(startX2Idx)+sortedX{1}(endX2Idx-1));
                       multiAsset.localCorrInfo.strikeY{idx+1}(idxS1,idxS2) = 0.5*(sortedX{2}(startX2Idx)+sortedX{2}(endX2Idx-1));
 
                       multiAsset.localCorrInfo.expectedVarianceX{idx+1}(idxS1,idxS2) = sumx;
                       multiAsset.localCorrInfo.expectedVarianceY{idx+1}(idxS1,idxS2) = sumy;
                       
                    %% local correlation from the local vol model
                       rhoSiSj_LV = multiAsset.correlationMatrix;
                       corr0_LV = rhoSiSj_LV(1,2); 
                       % rho_slv = rho_lv/(sqrt(v1v2)/sqrt(v1*v2));
                       dummyRal = sum/sqrt(sumx*sumy);
                       
%                        dummyRal2 = min(dummyRal,1.0);
                       corrDummyRal = corr0_LV/dummyRal;
                       
                       corrDummyRal2 = min(corrDummyRal,0.99);
                       
                       if corrDummyRal2 > 1
                           dfr = 1.0;
                       end
                       multiAsset.localCorrInfo.localCorrXY{idx+1}(idxS1,idxS2) = corrDummyRal2; 
                       
                   end
               end
                
               aaa = 1.0;

          end
            
          toc
 
          out = multiAsset.localCorrInfo;
        end
        
        function out = GetEffectiveLocalCorr(multiAsset,fromTime,x,y)
            
            timeStep = multiAsset.localCorrInfo.timeStepLVF;
            
            idx = find(timeStep > fromTime);
            if isempty(idx)
                idxNow = length(timeStep);
            else
                idxNow = min(idx);
            end
            
%             localCorrXY = eqHeston.localCorrInfo.localCorrMap(modelXName,modelYName);
            localCorrXYt = eqHeston.localCorrInfo.localCorrXY{idxNow};
            strikeX = eqHeston.localCorrInfo.StrikeX;
            strikeY = eqHeston.localCorrInfo.StrikeY;
            
            out = interp2(strikeX,strikeY,localCorrXYt,x,y);

        end
        
        function mcmcOut = inductMCMCForwardNew(multiAsset,fromTime,toTime,nextX,nextXIdx,dZ)
            
            predictor = nextX;
            predictorIdx = nextXIdx;
            numOfFactors = multiAsset.numOfFactors;
            stochasticModelNames = multiAsset.stochasticModelNames;
            
            for i=1:length(stochasticModelNames)
                mcOut1 = multiAsset.modelNameMap(stochasticModelNames{i}).inductMCMCForwardNew(fromTime,toTime,nextX(:,i),nextXIdx(:,i),dZ(:,i));
                predictor(:,i) = mcOut1.nextX;
                predictorIdx(:,i) = mcOut1.nextXIdx;
            end
            
            mcmcOut.nextX = predictor;
            mcmcOut.nextXIdx = predictorIdx;
            
        end
        
        function columnOut = MCPayoffWorst(multiAsset,idx,comFwdWorst,currentTime, isStop,isKI,payoffInfo,columnIn)
            
            modelStatesSize = length(comFwdWorst);
            scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
            strike = payoffInfo.autoCallSchedule.strike(idx);
            coupon = payoffInfo.autoCallSchedule.coupon(idx);
            nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
            lowerBarrier = payoffInfo.lowerBarrier;
            callStrike1 = payoffInfo.callStrike1;
            callValue1  = payoffInfo.callValue1;
            
            columnOut.isStop = zeros(modelStatesSize,scheduleSize);
            columnOut.worstP = zeros(modelStatesSize,scheduleSize);
            
            columnOut.isStop = columnIn.isStop;
            columnOut.worstP = columnIn.worstP;
            
            columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
            columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
            columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
            
            columnOut.payoffColumn = columnIn.payoffColumn;
            
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
            df_ep = multiAsset.DF(currentTime/365.0);
            
            % early redemption
            % idx  : early redemption schedule iter
            % idx11: simulation path iter
            if idx == 1
                for idx11 =1:modelStatesSize % 1st early redemption
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                           columnOut.isStop(idx11,idx) = 1;
                           columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                           columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                           columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                           columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    
                end
                
            elseif idx > 1 && idx < scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
                    else
                        % alive path
                        if comFwdWorst(idx11) >= strike % early redemption conditon met
                            columnOut.isStop(idx11,idx) = 1;
                            columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                        end
                        
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                    end
                end
                
            else % idx == scheduleSize
                for idx11 =1:modelStatesSize
                    if columnOut.isStop(idx11,idx-1) == 1 % terminated path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
                        columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv

                    else
                        %alive path
                        columnOut.isStop(idx11,idx) = 1;
                        columnOut.worstP(idx11,idx) = comFwdWorst(idx11);
                        % barrier hit path
                        if isKI(idx11) == 1
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        else % barrier unhit path
                            if comFwdWorst(idx11) < callStrike1 
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwdWorst(idx11);
                            elseif comFwdWorst(idx11) < strike
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwdWorst(idx11) - callStrike1) + nominal*callValue1; 
                            else
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                            
                            if comFwdWorst(idx11) > lowerBarrier
                                columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
                            end
                        end
                        
                        % payoff postprocess
                        columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
                        columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
                    end    
                end
            end    
        end    
            
        
        function out = computeStepdown2SMC(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            childModelSize = length(multiAsset.stochasticModelNames);
            childModels = cell(childModelSize,1);
            
            timeStep = [];
            for idxH=1:childModelSize
                childModels{i,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
                expiry = childModels{i,1}.localVolSurface.expiry;
                volExpiry = expiry(find(expiry <= maturity));
                % we include valueDate in the scheduel
                volExpiry =[volExpiry;0];
                timeStep = sort(union(timeStep,volExpiry),'ascend');
            end
            
            
           
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = 150000;
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdownNSMCReducedSerial(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
          %% add all the options maturities in the timeSteps
            childModelSize = length(multiAsset.stochasticModelNames);
            childModels = cell(childModelSize,1);
            
            timeStep = [];
            for idxH=1:childModelSize
                childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
                expiry = childModels{idxH,1}.localVolSurface.expiry;
                volExpiry = expiry(find(expiry <= maturity));
                % we include valueDate in the scheduel
                volExpiry =[volExpiry;0];
                timeStep = sort(union(timeStep,volExpiry),'ascend');
            end

            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = childModels{1}.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = childModels{1}.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = childModels{1}.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = childModels{1}.modelParams('NMC'); 
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
% 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = dW;
%                 end
%                 
%                 U = UU'; 
                  U = Z';  
            end
            
                      
           
          %% initial model States (reducedFuturesState)
          %% for serial path generation we init single initial state  
           nextFuturesStatesDummy = cell(childModelSize,1);
           
           for idxH=1:childModelSize
%                nextFuturesStates{idxH} = childModels{idxH}.initFuturesStates(NMC);
               nextFuturesStatesDummy{idxH} = childModels{idxH}.initFuturesStates(1);
           end
           
           realizedCorr = zeros(MCTimeStep,10);
           realizedTimeStep = zeros(MCTimeStep,1);
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % worstIdx
           worstIdx.npv = 0;
           worstIdx.payoff = zeros(modelStatesSize,1);
           worstIdx.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           worstIdx.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           isStopSerial = zeros(1,scheduleSize);
           worstPSerial = zeros(1,scheduleSize);
           
           unhitValueSerial.npv = 0;
           unhitValueSerial.payoff = zeros(1,1);
           unhitValueSerial.payoffStates.cashflow = zeros(1,scheduleSize);
           unhitValueSerial.payoffStates.cashflow_npv = zeros(1,scheduleSize);
           
           % Serial path generation ISKI is scalar variable that is reused
           % in the path
           if KIYN == true
%                isKI = ones(modelStatesSize,1);
               isKI = 1.0;
           else
%                isKI = zeros(modelStatesSize,1);
               isKI = 0.0;
           end
           
           % check whether current future price breach the barrier
           nextFuturesStates = nextFuturesStatesDummy;
           currentTime = 0; %NowDate
           comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
          
          %% Serialize Path Generation Forward Start
           
          %% Path Generation Forward
           for idxPath=1:modelStatesSize
               isStopSerial = zeros(1,scheduleSize);
               worstPSerial = zeros(1,scheduleSize);

               unhitValueSerial.npv = 0;
               unhitValueSerial.payoff = zeros(1,1);
               unhitValueSerial.payoffStates.cashflow = zeros(1,scheduleSize);
               unhitValueSerial.payoffStates.cashflow_npv = zeros(1,scheduleSize);
           
               tic
               nextFuturesStates = nextFuturesStatesDummy;
               currentTime = 0; %NowDate
               currentTimeIdx = 1; % 1st Time Steps
               currentNodeIdx = 1; % no eventDate for nowDate
               lastTimeIdx = currentTimeIdx;
               for i=lastAliveExeriseIdx:1:scheduleSize
                    % past schedule neglect
                    if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                    if i > lastAliveExeriseIdx
                        startTimeIdx = eventTimeIdx(i-1);
                        endTimeIdx = eventTimeIdx(i);
                    else
                        startTimeIdx = 1; % valueDate
                        endTimeIdx = eventTimeIdx(i);  
                    end

                    endTimeIdx = endTimeIdx - 1;

                %% induction Forward start
                    origCorr = multiAsset.termCorr{i,2};
                    idxNow = i;
                    for idx= startTimeIdx:1:endTimeIdx
    %                     mcmcOut   = inductMCForwardNewReduced(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
    %                     mcmcOut   = inductMCForwardNewReducedQuanto(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
    %                     mcmcOut   = inductMCForwardNewReducedQuantoTermCorr(multiAsset,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                        mcmcOut   = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,basePrice,idxNow,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(idxPath,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));

                        nextFuturesStates = mcmcOut.nextFuturesStates;


                        if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                            realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;

                            realizedTimeStep(idx) = timeStep(idx+1);
                        end

    %                     nextXIdx = mcmcOut.nextXIdx;
                        % one step backward induction end
                        % barrier check
                        lastTimeIdx = lastTimeIdx + 1;
                        currentTime = timeStep(lastTimeIdx);
                        comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
                        % per each path check whether Barrier was breached 
                        isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                    end


                    %forward payoff handling
                    columnIn.isStop = isStopSerial;
                    columnIn.worstP = worstPSerial;
                    columnIn.payoffColumn = unhitValueSerial;

                    comFwdWorst = comFwd(1)/basePrice(1);
%                     comFwdWorstIdx = ones(modelStatesSize,1);
                    comFwdWorstIdx = 1;
                    
                    for jj=2:childModelSize
                        if comFwdWorst >= comFwd(jj)/basePrice(jj)
                             comFwdWorst = comFwd(jj)/basePrice(jj);
                            comFwdWorstIdx = jj;
                        end
                    end
                    
                    columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                    isStopSerial = columnOut.isStop;
                    worstPSerial = columnOut.worstP;
                    unhitValueSerial = columnOut.payoffColumn;

                    %serialize
                    worstP(idxPath,:) = worstPSerial;
                    isStop(idxPath,:) = isStopSerial;
                    
                    unhitValue.payoff(idxPath) = unhitValueSerial.payoff;
                    
                    unhitValue.payoffStates.cashflow(idxPath,:) = unhitValueSerial.payoffStates.cashflow;
                    unhitValue.payoffStates.cashflow_npv(idxPath,:) = unhitValueSerial.payoffStates.cashflow_npv;
                    
                    nMFuture.payoffStates.cashflow(idxPath,currentNodeIdx) = comFwd(1);
                    nMFuture.payoffStates.cashflow_npv(idxPath,currentNodeIdx) = comFwd(1);
                    nMFuture.payoff = comFwd(1);

                    worstIdx.payoffStates.cashflow(idxPath,currentNodeIdx) = comFwdWorstIdx;
                    worstIdx.payoffStates.cashflow_npv(idxPath,currentNodeIdx) = comFwdWorstIdx;
                    worstIdx.payoff = comFwdWorstIdx;


                    %induct forward for eventDate
                    currentNodeIdx = currentNodeIdx + 1;
               end
               toc
           end
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
           
          %% Serialize Path Generation Forward End
          
          
          
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           worstIdx.npv = mean(worstIdx.payoff);
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
           out.worstIdx = worstIdx;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
              out.realizedCorr = realizedCorr;
              out.realizedTimeStep = realizedTimeStep;
           end 
           
        end
        
        function out = computeVanillaNSMC(multiAsset,valueDate,vanillaParams)
             % model schedule generation
            expiryDate = H_Date(vanillaParams.params('maturityDate'));
            scheduleSize = 1;
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
%             Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
            
            expiredChance = 0;
            aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
            
          %% add all the options maturities in the timeSteps
            childModelSize = length(multiAsset.stochasticModelNames);
            childModels = cell(childModelSize,1);
            
            timeStep = [];
            for idxH=1:childModelSize
                childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
                expiry = childModels{idxH,1}.localVolSurface.expiry;
                volExpiry = expiry(find(expiry <= maturity));
                % we include valueDate in the scheduel
                volExpiry =[volExpiry;0];
                timeStep = sort(union(timeStep,volExpiry),'ascend');
            end
            
            
            if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = childModels{1}.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');    
            end
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            lastAliveExerciseIdx = 1;
            for i= scheduleSize:-1:lastAliveExerciseIdx
                idx = find(timeStep == maturity);
                eventTimeIdx(i) = min(idx);
            end
            
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             
%           %% add all the options maturities in the timeSteps
%             childModelSize = length(multiAsset.stochasticModelNames);
%             childModels = cell(childModelSize,1);
%             
%             timeStep = [];
%             for idxH=1:childModelSize
%                 childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
%                 expiry = childModels{idxH,1}.localVolSurface.expiry;
%                 volExpiry = expiry(find(expiry <= maturity));
%                 % we include valueDate in the scheduel
%                 volExpiry =[volExpiry;0];
%                 timeStep = sort(union(timeStep,volExpiry),'ascend');
%             end
%             
% %             WTIDupire = multiAsset.modelNameMap('WTI');
% %             expiry = WTIDupire.localVolSurface.expiry;
% %             volExpiry = expiry(find(expiry <= maturity));
% %             % we include valueDate in the scheduel
% %             volExpiry =[volExpiry;0];
% %             
% %             BrentDupire = multiAsset.modelNameMap('Brent');
% %             expiry2 = BrentDupire.localVolSurface.expiry;
% %             volExpiry2 = expiry2(find(expiry2 <= maturity));
% %             % we include valueDate in the scheduel
% %             volExpiry2 =[volExpiry2;0];
% %             
% %             volExpiry = sort(union(volExpiry,volExpiry2),'ascend');
% %             timeStep = volExpiry;
% 
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = childModels{1}.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = childModels{1}.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = childModels{1}.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = childModels{1}.modelParams('NMC'); 
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                
%                 factorCorrel = eye(numFactors,numFactors);
%                 factorCorrel = multiAsset.modelParams('correlationMatrix');
% %                 for idxH=1:numFactors/2
% %                     factorCorrel(idxH,numFactors/2 + idxH) = multiAsset.correlationBrigoGamma;
% %                     factorCorrel(numFactors/2 + idxH,idxH) = multiAsset.correlationBrigoGamma;
% %                 end
%                     
%                 L = chol(factorCorrel,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = dW;
                end
                
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
          %% initial model States (reducedFuturesState)
           nextFuturesStates = cell(childModelSize,1);
           
           for idxH=1:childModelSize
               nextFuturesStates{idxH} = childModels{idxH}.initFuturesStates(NMC);
           end
           
           realizedCorr = zeros(MCTimeStep,10);
           realizedTimeStep = zeros(MCTimeStep,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            initX = ones(NMC,numFactors);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
            
          %% payoff init
            %% payoff at maturity
           strike =  vanillaParams.params('strike');
           
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            % added for oveHedgeVector
%            overHedgeSpreadVector = stepdownParams.params('overHedgeSpreadVector');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
           
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.overHedgeSpreadVector = overHedgeSpreadVector;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           
            %europeanValue
           europeanValue.npv = 0;
           europeanValue.payoff = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           %americanValue
           americanValue.npv = 0;
           americanValue.payoff = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
           americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
           
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % worstIdx
%            worstIdx.npv = 0;
%            worstIdx.payoff = zeros(modelStatesSize,1);
%            worstIdx.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            worstIdx.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
%            isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExerciseIdx:1:scheduleSize
                % past schedule neglect
                startTimeIdx = 1; % valueDate
                endTimeIdx = eventTimeIdx(i);  
                                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
%                 comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(timeStep(1),maturity,nextX);
%                 worstP(:,1) = comFwd(:);
                    
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
                origCorr = multiAsset.termCorr{i,2};
                idxNow = i;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     nextX = mcmcOut.nextX;
                    
%                     mcmcOut   = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,basePrice,idxNow,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    mcmcOut   = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,strike,idxNow,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextFuturesStates = mcmcOut.nextFuturesStates;

                    % one step forward induction end
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdContractMMC(currentTime,maturity,nextX);
%                     worstP(:,idx+1) = comFwd(:);
                end
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
           lastTimeIdx = currentTimeIdx;
           
           comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
           
           if strcmp(vanillaParams.params('callPutFlag'),'C')
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(comFwd(i,1) - strike,0);
                    payoffStateB.cashflow(i) = max(comFwd(i,1) - strike,0);
               end
           else
               for i=1:modelStatesSize
                    payoffStateA.cashflow(i) = max(-comFwd(i,1) + strike,0);
                    payoffStateB.cashflow(i) = max(-comFwd(i,1) + strike,0);
               end
           end
           
           payoffStateC.cashflow = comFwd;
           
           df_ep = multiAsset.DF(currentTime/365.0);
           
           % we don't discount forward price
           for i=1:modelStatesSize
                payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
                payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
                payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
           end
           
          %% Forward Induction End
          
           %process payoff at maturity
            
            europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
            europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
            europeanValue.payoff = payoffStateA.cashflow_npv;
            europeanValue.payoffUnd = payoffStateA.cashflow;
            
            americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
            americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
            americanValue.payoff = payoffStateB.cashflow_npv;
            
            nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
            nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
            nMFuture.payoff = payoffStateC.cashflow_npv;
            
           lastTimeIdx = currentTimeIdx;
           
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
           
           europeanValue.npv = mean(europeanValue.payoff);
           europeanValue.npvUnd = mean(europeanValue.payoffUnd);
           
           americanValue.npv = mean(americanValue.payoff);
           nMFuture.npv = mean(nMFuture.payoff);
            
           out.europeanValue = europeanValue; 
           out.americanValue = americanValue;
           out.nMFuture = nMFuture;
          
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 origCorr = multiAsset.termCorr{i,2};
%                 idxNow = i;
%                 for idx= startTimeIdx:1:endTimeIdx
% %                     mcmcOut   = inductMCForwardNewReduced(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
% %                     mcmcOut   = inductMCForwardNewReducedQuanto(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
% %                     mcmcOut   = inductMCForwardNewReducedQuantoTermCorr(multiAsset,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
%                     mcmcOut   = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,basePrice,idxNow,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
%                     
%                     nextFuturesStates = mcmcOut.nextFuturesStates;
%                     
%                     
%                     if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
%                         realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
%                         
%                         realizedTimeStep(idx) = timeStep(idx+1);
%                     end
%                     
% %                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
%                 
%                 comFwdWorst = comFwd(:,1)/basePrice(1);
%                 comFwdWorstIdx = ones(modelStatesSize,1);
%                 for ii=1:modelStatesSize
%                     for jj=2:childModelSize
%                         if comFwdWorst(ii) >= comFwd(ii,jj)/basePrice(jj)
%                              comFwdWorst(ii) = comFwd(ii,jj)/basePrice(jj);
%                             comFwdWorstIdx(ii) = jj;
%                         end
% %                         comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
%                     end
%                 end
%                 
%                 columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
%                 nMFuture.payoff = comFwd(:,1);
%                 
%                 worstIdx.payoffStates.cashflow(:,currentNodeIdx) = comFwdWorstIdx(:,1);
%                 worstIdx.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwdWorstIdx(:,1);
%                 worstIdx.payoff = comFwdWorstIdx(:,1);
%                 
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
% %                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = multiAsset.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
% %                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
% %                                 exerciseValue(idx) = contiValue(idx);
% %                             elseif worstP(idx,i-1) < strikeEarly
% %                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
% %                             end
%                             
%                              if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpreadVector(i-1)
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpreadVector(i-1));
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end

% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            worstIdx.npv = mean(worstIdx.payoff);
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%            out.worstIdx = worstIdx;
%            if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
%               out.realizedCorr = realizedCorr;
%               out.realizedTimeStep = realizedTimeStep;
%            end 
           
        end
        
        function out = computeStepdownNSMCReduced_ohVec(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
          %% add all the options maturities in the timeSteps
            childModelSize = length(multiAsset.stochasticModelNames);
            childModels = cell(childModelSize,1);
            
            timeStep = [];
            %% 20220518 no vol expiry for time step for SABR model
            for idxH=1:childModelSize
                childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
%                 expiry = childModels{idxH,1}.localVolSurface.expiry;
%                 volExpiry = expiry(find(expiry <= maturity));
%                 % we include valueDate in the scheduel
%                 volExpiry =[volExpiry;0];
%                 timeStep = sort(union(timeStep,volExpiry),'ascend');
            end

%             for idxH=1:childModelSize
%                 childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
%                 expiry = childModels{idxH,1}.localVolSurface.expiry;
%                 volExpiry = expiry(find(expiry <= maturity));
%                 % we include valueDate in the scheduel
%                 volExpiry =[volExpiry;0];
%                 timeStep = sort(union(timeStep,volExpiry),'ascend');
%             end
            
%             WTIDupire = multiAsset.modelNameMap('WTI');
%             expiry = WTIDupire.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             BrentDupire = multiAsset.modelNameMap('Brent');
%             expiry2 = BrentDupire.localVolSurface.expiry;
%             volExpiry2 = expiry2(find(expiry2 <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry2 =[volExpiry2;0];
%             
%             volExpiry = sort(union(volExpiry,volExpiry2),'ascend');
%             timeStep = volExpiry;

            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = childModels{1}.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = childModels{1}.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = childModels{1}.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = childModels{1}.modelParams('NMC'); 
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                
%                 factorCorrel = eye(numFactors,numFactors);
%                 factorCorrel = multiAsset.modelParams('correlationMatrix');
% %                 for idxH=1:numFactors/2
% %                     factorCorrel(idxH,numFactors/2 + idxH) = multiAsset.correlationBrigoGamma;
% %                     factorCorrel(numFactors/2 + idxH,idxH) = multiAsset.correlationBrigoGamma;
% %                 end
%                     
%                 L = chol(factorCorrel,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = dW;
                end
                
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
          %% initial model States (reducedFuturesState)
           nextFuturesStates = cell(childModelSize,1);
           
           for idxH=1:childModelSize
               nextFuturesStates{idxH} = childModels{idxH}.initFuturesStates(NMC);
           end
           
           realizedCorr = zeros(MCTimeStep,10);
           realizedTimeStep = zeros(MCTimeStep,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            initX = ones(NMC,numFactors);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           % added for oveHedgeVector
           overHedgeSpreadVector = stepdownParams.params('overHedgeSpreadVector');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.overHedgeSpreadVector = overHedgeSpreadVector;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % worstIdx
           worstIdx.npv = 0;
           worstIdx.payoff = zeros(modelStatesSize,1);
           worstIdx.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           worstIdx.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                origCorr = multiAsset.termCorr{i,2};
                idxNow = i;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNewReduced(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
%                     mcmcOut   = inductMCForwardNewReducedQuanto(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
%                     mcmcOut   = inductMCForwardNewReducedQuantoTermCorr(multiAsset,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    mcmcOut   = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,basePrice,idxNow,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextFuturesStates = mcmcOut.nextFuturesStates;
                    
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
                    
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);
                comFwdWorstIdx = ones(modelStatesSize,1);
                for ii=1:modelStatesSize
                    for jj=2:childModelSize
                        if comFwdWorst(ii) >= comFwd(ii,jj)/basePrice(jj)
                             comFwdWorst(ii) = comFwd(ii,jj)/basePrice(jj);
                            comFwdWorstIdx(ii) = jj;
                        end
%                         comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                worstIdx.payoffStates.cashflow(:,currentNodeIdx) = comFwdWorstIdx(:,1);
                worstIdx.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwdWorstIdx(:,1);
                worstIdx.payoff = comFwdWorstIdx(:,1);
                
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
                            
                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpreadVector(i-1)
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpreadVector(i-1));
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           worstIdx.npv = mean(worstIdx.payoff);
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
           out.worstIdx = worstIdx;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
              out.realizedCorr = realizedCorr;
              out.realizedTimeStep = realizedTimeStep;
           end 
           
        end
        
        function out = computeStepdownNSMCReduced(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
          %% add all the options maturities in the timeSteps
            childModelSize = length(multiAsset.stochasticModelNames);
            childModels = cell(childModelSize,1);
            
            timeStep = [];
            for idxH=1:childModelSize
                childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
                expiry = childModels{idxH,1}.localVolSurface.expiry;
                volExpiry = expiry(find(expiry <= maturity));
                % we include valueDate in the scheduel
                volExpiry =[volExpiry;0];
                timeStep = sort(union(timeStep,volExpiry),'ascend');
            end
            
%             WTIDupire = multiAsset.modelNameMap('WTI');
%             expiry = WTIDupire.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             BrentDupire = multiAsset.modelNameMap('Brent');
%             expiry2 = BrentDupire.localVolSurface.expiry;
%             volExpiry2 = expiry2(find(expiry2 <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry2 =[volExpiry2;0];
%             
%             volExpiry = sort(union(volExpiry,volExpiry2),'ascend');
%             timeStep = volExpiry;

            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = childModels{1}.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = childModels{1}.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = childModels{1}.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = childModels{1}.modelParams('NMC'); 
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                
%                 factorCorrel = eye(numFactors,numFactors);
%                 factorCorrel = multiAsset.modelParams('correlationMatrix');
% %                 for idxH=1:numFactors/2
% %                     factorCorrel(idxH,numFactors/2 + idxH) = multiAsset.correlationBrigoGamma;
% %                     factorCorrel(numFactors/2 + idxH,idxH) = multiAsset.correlationBrigoGamma;
% %                 end
%                     
%                 L = chol(factorCorrel,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = dW;
                end
                
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
          %% initial model States (reducedFuturesState)
           nextFuturesStates = cell(childModelSize,1);
           
           for idxH=1:childModelSize
               nextFuturesStates{idxH} = childModels{idxH}.initFuturesStates(NMC);
           end
           
           realizedCorr = zeros(MCTimeStep,10);
           realizedTimeStep = zeros(MCTimeStep,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            initX = ones(NMC,numFactors);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % worstIdx
           worstIdx.npv = 0;
           worstIdx.payoff = zeros(modelStatesSize,1);
           worstIdx.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           worstIdx.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                origCorr = multiAsset.termCorr{i,2};
                idxNow = i;
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNewReduced(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
%                     mcmcOut   = inductMCForwardNewReducedQuanto(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
%                     mcmcOut   = inductMCForwardNewReducedQuantoTermCorr(multiAsset,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    mcmcOut   = inductMCForwardNewReducedQuantoLocalCorr(multiAsset,basePrice,idxNow,origCorr,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextFuturesStates = mcmcOut.nextFuturesStates;
                    
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
                    
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);
                comFwdWorstIdx = ones(modelStatesSize,1);
                for ii=1:modelStatesSize
                    for jj=2:childModelSize
                        if comFwdWorst(ii) >= comFwd(ii,jj)/basePrice(jj)
                             comFwdWorst(ii) = comFwd(ii,jj)/basePrice(jj);
                            comFwdWorstIdx(ii) = jj;
                        end
%                         comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                worstIdx.payoffStates.cashflow(:,currentNodeIdx) = comFwdWorstIdx(:,1);
                worstIdx.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwdWorstIdx(:,1);
                worstIdx.payoff = comFwdWorstIdx(:,1);
                
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                            
%                             % overHedgeSpreadVector
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpreadVector(i-1)
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpreadVector(i-1));
%                             end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           worstIdx.npv = mean(worstIdx.payoff);
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
           out.worstIdx = worstIdx;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
              out.realizedCorr = realizedCorr;
              out.realizedTimeStep = realizedTimeStep;
           end 
           
        end
        
        function out = computeStepdown2SMCReduced(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
          %% add all the options maturities in the timeSteps
            childModelSize = length(multiAsset.stochasticModelNames);
            childModels = cell(childModelSize,1);
            
            timeStep = [];
            for idxH=1:childModelSize
                childModels{idxH,1} = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxH});
                expiry = childModels{idxH,1}.localVolSurface.expiry;
                volExpiry = expiry(find(expiry <= maturity));
                % we include valueDate in the scheduel
                volExpiry =[volExpiry;0];
                timeStep = sort(union(timeStep,volExpiry),'ascend');
            end
            
%             WTIDupire = multiAsset.modelNameMap('WTI');
%             expiry = WTIDupire.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             BrentDupire = multiAsset.modelNameMap('Brent');
%             expiry2 = BrentDupire.localVolSurface.expiry;
%             volExpiry2 = expiry2(find(expiry2 <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry2 =[volExpiry2;0];
%             
%             volExpiry = sort(union(volExpiry,volExpiry2),'ascend');
%             timeStep = volExpiry;

            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = childModels{1}.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = childModels{1}.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep*numOfFactors);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = childModels{1}.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
                NMC = childModels{1}.modelParams('NMC'); 
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                
                factorCorrel = eye(numFactors,numFactors);
                correlMatrix = multiAsset.modelParams('correlationMatrix');
%                 for idxH=1:numFactors/2
%                     factorCorrel(idxH,numFactors/2 + idxH) = multiAsset.correlationBrigoGamma;
%                     factorCorrel(numFactors/2 + idxH,idxH) = multiAsset.correlationBrigoGamma;
%                 end
                    
                L = chol(factorCorrel,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
          %% initial model States (reducedFuturesState)
           nextFuturesStates = cell(childModelSize,1);
           
           for idxH=1:childModelSize
               nextFuturesStates{idxH} = childModels{idxH}.initFuturesStates(NMC);
           end
           
           realizedCorr = zeros(MCTimeStep,10);
           realizedTimeStep = zeros(MCTimeStep,1);
%            reducedForwardCurve = eqCOMDupireSpotGF.modelParams('reducedFutures');
%            reducedForwardCurveSize = size(reducedForwardCurve,1);
%            reducedForwardMaturity = reducedForwardCurve(:,1); 
%            
%            initFuturesStatesDefault = zeros(reducedForwardCurveSize,NMC);
%            
%            for idx=1:reducedForwardCurveSize
%                 initFuturesStatesDefault(idx,:) = reducedForwardCurve(idx,2)*ones(1,NMC);
%            end
%            
%            % we choose transpose for convenience
%            initFuturesStates = initFuturesStatesDefault';
%            
%            nextFuturesStates = initFuturesStates;
%            
%            initX = ones(NMC,numFactors);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNewReduced(multiAsset,timeStep(idx),timeStep(idx+1),nextFuturesStates,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextFuturesStates = mcmcOut.nextFuturesStates;
                    
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
                    
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMCReduced(currentTime,nextFuturesStates);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:childModelSize
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
           
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
              out.realizedCorr = realizedCorr;
              out.realizedTimeStep = realizedTimeStep;
           end 
           
        end
        
        function out = computeStepdown2SMCEQLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIDupire = multiAsset.modelNameMap('HSCEI');
            expiry = HSCEIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = HSCEIDupire.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIDupire.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIDupire.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
         
           % corr , var1, var2 
           realizedCorr = zeros(MCTimeStep,3);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.termCorrX1X2;
%                         realizedCorr(idx,2) = mcmcOut.corrInfo.varX1;
%                         realizedCorr(idx,3) = mcmcOut.corrInfo.varX2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                out.realizedCorr = realizedCorr;
                out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown3SMCEQLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIDupire = multiAsset.modelNameMap('HSCEI');
            expiry = HSCEIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = HSCEIDupire.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIDupire.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIDupire.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                U = UU'; 

            end
         
           % corr , var1, var2 
           realizedCorr = zeros(MCTimeStep,3);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
           
           nextX = initX;
%            nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
%                         realizedCorr(idx,2) = mcmcOut.corrInfo.corrX1X3;
%                         realizedCorr(idx,3) = mcmcOut.corrInfo.corrX2X3;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end
%                     nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                out.realizedCorr = realizedCorr;
                out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCEQHeston(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIHeston = multiAsset.modelNameMap('HSCEI');
            expiry = HSCEIHeston.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = HSCEIHeston.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
           
            % corr X1X2,V1V2,X1V2,X2V1,X1V1,X2V2,
            % var X1,X2,V1,V2
            % COVAR x1x2,v1v2,x1v2,x2v1
            
           realizedCorr = zeros(MCTimeStep,6 + 4 + 4);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           HSCEIHeston = multiAsset.modelNameMap('HSCEI');
           SX5EHeston  = multiAsset.modelNameMap('SX5E');
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*HSCEIHeston.v0;
           
           initX2 = ones(NMC,1);
           initV2 = ones(NMC,1)*SX5EHeston.v0;

           initX = [initX1,initX2];
           initV = [initV1,initV2];
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE')
                        mcmcOut   = inductMCForwardMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE')
                        mcmcOut   = inductMCForwardPerfectCorrMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'IdyoSyncMultiQE')
                        mcmcOut   = inductMCForwardIdyoSyncMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV0')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV0(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV1')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV1(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
%                         realizedCorr(idx,7) = mcmcOut.corrInfo.varX1;
%                         realizedCorr(idx,8) = mcmcOut.corrInfo.varX2;
%                         
%                         
%                         realizedCorr(idx,9) = mcmcOut.corrInfo.varV1;
%                         realizedCorr(idx,10) = mcmcOut.corrInfo.varV2;
                        
%                         realizedCorr(idx,7) = mcmcOut.corrInfo.termCorrX1X2;
                        
                        if strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE') || ...
                           strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE') 
                            realizedCorr(idx,11) = mcmcOut.corrInfo.covarX1X2;
                            realizedCorr(idx,12) = mcmcOut.corrInfo.covarV1V2;
                            realizedCorr(idx,13) = mcmcOut.corrInfo.covarX1V2;
                            realizedCorr(idx,14) = mcmcOut.corrInfo.covarX2V1;
                            
                        end
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown3SMCEQHeston(multiAsset,valueDate,stepdownParams)
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            % we use stochasticModelNames{1} for generic purpose
            DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
            
            expiry = DummyHeston.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
            else
                timeStepP = DummyHeston.modelParams('mcOneTimeStep');
                dailyTimeSteps = [maturity:-timeStepP:0]';
                dailyTimeSteps = [dailyTimeSteps;0];
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = DummyHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = DummyHeston.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = DummyHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
           
            % corr X1X2,V1V2,X1V2,X2V1,X1V1,X2V2,
            % var X1,X2,V1,V2
            % COVAR x1x2,v1v2,x1v2,x2v1
            
%            realizedCorr = zeros(MCTimeStep,6 + 4 + 4);
           realizedCorr = zeros(MCTimeStep,100);
           
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           numOfAsset = length(multiAsset.stochasticModelNames);
           
           DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*DummyHeston.v0;
           
           initX = initX1;
           initV = initV1;
           
           for idxHong=2:numOfAsset
               DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxHong});
               initXDummy = ones(NMC,1);
               initVDummy = ones(NMC,1)*DummyHeston.v0;
               
               initX = [initX,initXDummy];
               initV = [initV,initVDummy];
               
           end
           
           nextX = initX;
           nextV = initV;
           
%            HSCEIHeston = multiAsset.modelNameMap('HSCEI');
%            SX5EHeston  = multiAsset.modelNameMap('SX5E');
%            
%            initX1 = ones(NMC,1);
%            initV1 = ones(NMC,1)*HSCEIHeston.v0;
%            
%            initX2 = ones(NMC,1);
%            initV2 = ones(NMC,1)*SX5EHeston.v0;
% 
%            initX = [initX1,initX2];
%            initV = [initV1,initV2];
%            
%            nextX = initX;
%            nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE')
                        mcmcOut   = inductMCForwardPerfectCorrMultiQEND(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQEND(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        % 1& 2
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
                        % 1& 3
                        realizedCorr(idx,7) = mcmcOut.corrInfo.corrX1X3;
                        realizedCorr(idx,8) = mcmcOut.corrInfo.corrV1V3;
                        realizedCorr(idx,9) = mcmcOut.corrInfo.corrX1V3;
                        realizedCorr(idx,10) = mcmcOut.corrInfo.corrX3V1;
                        
                        realizedCorr(idx,11) = mcmcOut.corrInfo.corrX3V3;
                        
                        % 2&3
                        realizedCorr(idx,12) = mcmcOut.corrInfo.corrX2X3;
                        realizedCorr(idx,13) = mcmcOut.corrInfo.corrV2V3;
                        realizedCorr(idx,14) = mcmcOut.corrInfo.corrX2V3;
                        realizedCorr(idx,15) = mcmcOut.corrInfo.corrX3V2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 
           
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCEQHestonSLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            HSCEIHeston = multiAsset.modelNameMap('HSCEI');
%             expiry = HSCEIHeston.localVolSurface.expiry;
            expiry = HSCEIHeston.levFuncInfo.timeStepLVF;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = HSCEIHeston.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = HSCEIHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = HSCEIDupire.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = HSCEIHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
            
           realizedCorr = zeros(MCTimeStep,6+4);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           HSCEIHeston = multiAsset.modelNameMap('HSCEI');
           SX5EHeston  = multiAsset.modelNameMap('SX5E');
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*HSCEIHeston.v0;
           
           initX2 = ones(NMC,1);
           initV2 = ones(NMC,1)*SX5EHeston.v0;

           initX = [initX1,initX2];
           initV = [initV1,initV2];
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'Euler')
                        mcmcOut   = inductMCForwardEuler(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'QE')
                        mcmcOut   = inductMCForwardQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE')
                        mcmcOut   = inductMCForwardMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                     elseif strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE')
                        mcmcOut   = inductMCForwardPerfectCorrMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'IdyoSyncMultiQE')
                        mcmcOut   = inductMCForwardIdyoSyncMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQE(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE_EffectiveLocalCorr')
                        mcmcOut   = inductMCForwardParsimoniousMultiQE_EffectiveLocalCorr(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV0')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV0(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQETestVV1')
                        mcmcOut   = inductMCForwardParsimoniousMultiQETestVV1(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
%                         realizedCorr(idx,7) = mcmcOut.corrInfo.varX1;
%                         realizedCorr(idx,8) = mcmcOut.corrInfo.varX2;
%                         
%                         
%                         realizedCorr(idx,9) = mcmcOut.corrInfo.varV1;
%                         realizedCorr(idx,10) = mcmcOut.corrInfo.varV2;
                        
%                         realizedCorr(idx,7) = mcmcOut.corrInfo.termCorrX1X2;
                        
                        if strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE') || ...
                           strcmp(multiAsset.modelParams('MCScheme'), 'MultiQE') 
                            realizedCorr(idx,11) = mcmcOut.corrInfo.covarX1X2;
                            realizedCorr(idx,12) = mcmcOut.corrInfo.covarV1V2;
                            realizedCorr(idx,13) = mcmcOut.corrInfo.covarX1V2;
                            realizedCorr(idx,14) = mcmcOut.corrInfo.covarX2V1;
                        
                        end
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 

           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown3SMCEQHestonSLV(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
%             expiry = HSCEIHeston.localVolSurface.expiry;
            expiry = DummyHeston.levFuncInfo.timeStepLVF;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%             else
%                 timeStepP = HSCEIHeston.modelParams('mcOneTimeStep');
%                 dailyTimeSteps = [maturity:-timeStepP:0]';
%                 dailyTimeSteps = [dailyTimeSteps;0];
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
            useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
                NMC = 2^14;
%                 NMC = 2^16;
                NMC = DummyHeston.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
            
               % apply brownian bridge
               dZ = DummyHeston.bbgenerator(dZ0);
               U(:,1) = dZ(:,1);
               for i=2:size(dZ,2)
                   U(:,i) = dZ(:,i) - dZ(:,i-1); 
               end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 50000;
                NMC = DummyHeston.modelParams('NMC'); 
                
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
                U = Z';
%                 % correlated brownian motions
%                 UU = zeros(MCTimeStep*numFactors,NMC);
%                 L = chol(multiAsset.correlationMatrix,'lower');
%                 
%                 for i=1:MCTimeStep
%                      dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
%                      UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
%                 end
%                 U = UU'; 

            end
            
           realizedCorr = zeros(MCTimeStep,100);
           realizedTimeStep = zeros(MCTimeStep,1);
           
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           numOfAsset = length(multiAsset.stochasticModelNames);
           
           DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{1});
           
           initX1 = ones(NMC,1);
           initV1 = ones(NMC,1)*DummyHeston.v0;
           
           initX = initX1;
           initV = initV1;
           
           for idxHong=2:numOfAsset
               DummyHeston = multiAsset.modelNameMap(multiAsset.stochasticModelNames{idxHong});
               initXDummy = ones(NMC,1);
               initVDummy = ones(NMC,1)*DummyHeston.v0;
               
               initX = [initX,initXDummy];
               initV = [initV,initVDummy];
               
           end
           
           nextX = initX;
           nextV = initV;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.EQFwdMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    if strcmp(multiAsset.modelParams('MCScheme'), 'PerfectCorrMultiQE')
                        mcmcOut   = inductMCForwardPerfectCorrMultiQEND(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    elseif strcmp(multiAsset.modelParams('MCScheme'), 'ParsimoniousMultiQE')
                        mcmcOut   = inductMCForwardParsimoniousMultiQEND(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextV,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    else
                        error('unImplemented MC Scheme');
                    end
                    
                    nextX = mcmcOut.nextX;
                    nextV = mcmcOut.nextV;
                    
                    if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
                        % 1& 2
                        realizedCorr(idx,1) = mcmcOut.corrInfo.corrX1X2;
                        realizedCorr(idx,2) = mcmcOut.corrInfo.corrV1V2;
                        realizedCorr(idx,3) = mcmcOut.corrInfo.corrX1V2;
                        realizedCorr(idx,4) = mcmcOut.corrInfo.corrX2V1;
                        
                        realizedCorr(idx,5) = mcmcOut.corrInfo.corrX1V1;
                        realizedCorr(idx,6) = mcmcOut.corrInfo.corrX2V2;
                        
                        % 1& 3
                        realizedCorr(idx,7) = mcmcOut.corrInfo.corrX1X3;
                        realizedCorr(idx,8) = mcmcOut.corrInfo.corrV1V3;
                        realizedCorr(idx,9) = mcmcOut.corrInfo.corrX1V3;
                        realizedCorr(idx,10) = mcmcOut.corrInfo.corrX3V1;
                        
                        realizedCorr(idx,11) = mcmcOut.corrInfo.corrX3V3;
                        
                        % 2&3
                        realizedCorr(idx,12) = mcmcOut.corrInfo.corrX2X3;
                        realizedCorr(idx,13) = mcmcOut.corrInfo.corrV2V3;
                        realizedCorr(idx,14) = mcmcOut.corrInfo.corrX2V3;
                        realizedCorr(idx,15) = mcmcOut.corrInfo.corrX3V2;
                        
                        realizedTimeStep(idx) = timeStep(idx+1);
                    end

                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.EQFwdMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);

                % Heston model numFactors=4 , numOfFactors/2 =2
                for ii=1:modelStatesSize
                    for jj=2:numFactors/2
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue;
           if strcmp(multiAsset.modelParams('computeRealizedCorrelationYN'),'true')
               out.realizedCorr = realizedCorr;
               out.realizedTimeStep = realizedTimeStep;
           end 

           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        function out = computeStepdown2SMCMC(multiAsset,valueDate,stepdownParams)
%             out =0;
            % model schedule generation
            scheduleInt = stepdownParams.params('autoCallScheduleInt');
            scheduleSize = size(scheduleInt,1);
            autoCallSchedule=[];
            for i=1:scheduleSize
                autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
                autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
                autoCallSchedule.strike(i) = scheduleInt(i,2);
                autoCallSchedule.coupon(i) = scheduleInt(i,3);
            end
            autoCallSchedule.scheduleSize = scheduleSize;
            
            expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
            
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            expiredChance = 0;
            aliveChance = 0;
            
            WTIDupire = multiAsset.modelNameMap('WTI');
            expiry = WTIDupire.localVolSurface.expiry;
            volExpiry = expiry(find(expiry <= maturity));
            % we include valueDate in the scheduel
            volExpiry =[volExpiry;0];
            
            timeStep = volExpiry;
            if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
                dailyTimeSteps = [maturity:-1:0]';
                timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
                
            end
            % we include exerciseDates
            for i= 1:scheduleSize
                daysToExercise = autoCallSchedule.days(i);
                if daysToExercise <= 0
                    expiredChance = expiredChance +1;
                    continue;
                else
                    aliveChance = aliveChance + 1;
                    timeStep = [timeStep; daysToExercise];
                end 
            end
            lastAliveExeriseIdx = 1 + expiredChance;
            
            
            timeStep = sort(union(timeStep,maturity),'ascend');
            totalTimeStepSize = length(timeStep);
            eventTimeIdx = zeros(scheduleSize,1);
            
            for i= scheduleSize:-1:lastAliveExeriseIdx
                idx = find(timeStep == autoCallSchedule.days(i));
                eventTimeIdx(i) = min(idx);
            end
           
          %% MC init 
%             useQuasiRandomYN = multiAsset.modelParams('useQuasiRandomYN');
            useQuasiRandomYN = 'false';
            useQuasiRandomYN = 'true';
            
            numFactors = multiAsset.numOfFactors;
            
            if strcmp(useQuasiRandomYN,'true')
%                 NMC = 2^14;
                NMC = 2^18;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
                Ps = sobolset(MCTimeStep*numFactors,'Skip',NMC);
                Q = qrandstream(Ps);

                reset(Q);
                U0 = qrand(Q,NMC);
                U = zeros(NMC,MCTimeStep);

                dZ0 = zeros(size(U,1),size(U,2));
    %             dZ = norminv(U);
                % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
                % generate random number from the end
                for i=1:size(U,1)
                    for j=1:size(U,2)
                        dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
                    end
                end
                
                % we use gaussian copula to generated joint distribution
                % correlated brownian motions
                Z = dZ0';
                UU = zeros(MCTimeStep*numFactors,NMC);
                U = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                
                for i=1:MCTimeStep
                     W = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*W;
                end
                
                U = H_ncdf(UU);
                
                U = U'; 
                
%                % apply brownian bridge
%                dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
%                U(:,1) = dZ(:,1);
%                for i=2:size(dZ,2)
%                    U(:,i) = dZ(:,i) - dZ(:,i-1); 
%                end
%                U = dZ;
            else
                rng('default');
                rng(0);
                
%                 NMC = 5000;
%                 NMC = 50000;
                NMC = 150000;
                
                MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Z= randn(NMC/2,MCTimeStep);
                Z= randn(NMC/2,MCTimeStep*numFactors);
                Z=[Z;-Z];
                Z=Z';
                
                %TimeInversion
                sizeZ = size(Z,1);
                A = zeros(sizeZ,sizeZ);
                for i=1:sizeZ
                    A(i,sizeZ + 1 -i) = 1;
                end
                Z = A*Z;
                
%                 U = Z';
                % correlated brownian motions
                UU = zeros(MCTimeStep*numFactors,NMC);
                L = chol(multiAsset.correlationMatrix,'lower');
                UUU = zeros(MCTimeStep*numFactors,NMC);
                for i=1:MCTimeStep
                     dW = Z(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:);
                     UU(numFactors*(i-1) + 1:numFactors*(i-1) + numFactors,:) = L*dW;
                end
                
                UUU = H_ncdf(UU);
                U = UUU';
                
%                 abcd = zeros(MCTimeStep*numFactors,NMC);
%                 for i=1:MCTimeStep*numFactors
%                     for j=1:NMC
%                         abcd(i,j) = H_ncdf(UU(i,j));
%                     end
%                 end

            end
            
            
           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = ones(NMC,numFactors);
           initXIdx = zeros(NMC,numFactors);
           stochasticModelNames = multiAsset.stochasticModelNames;
           for i=1:length(stochasticModelNames)
               Pricingidx = multiAsset.modelNameMap(stochasticModelNames{i}).localVolSurface.Pricingidx;
               initXIdx(:,i) = ones(NMC,1)*Pricingidx;
           end
           nextX = initX;
           nextXIdx = initXIdx;
            
          %% payoff init
           nominal = stepdownParams.params('nominal');
           basePrice = stepdownParams.params('basePrice');
           lowerBarrier = stepdownParams.params('lowerBarrier');
           overHedgeShift = stepdownParams.params('overHedgeShift');
           overHedgeSpread = stepdownParams.params('overHedgeSpread');
           
           callStrike1 = stepdownParams.params('callStrike1');
           callValue1 = stepdownParams.params('callValue1');
           SSpayoffYN = stepdownParams.params('SSpayoffYN');
           KIYN = stepdownParams.params('KIYN');
           
           payoffInfo.nominal = nominal;
           payoffInfo.basePrice = basePrice;
           payoffInfo.lowerBarrier = lowerBarrier;
           payoffInfo.overHedgeShift = overHedgeShift;
           payoffInfo.overHedgeSpread = overHedgeSpread;
           
           payoffInfo.callStrike1 = callStrike1;
           payoffInfo.callValue1 = callValue1;
           payoffInfo.SSpayoffYN = SSpayoffYN;
           
           payoffInfo.autoCallSchedule = autoCallSchedule;
           
           
           modelStatesSize = NMC;
           %down barrier already touch
           hitValue.npv = 0;
           hitValue.payoff = zeros(modelStatesSize,1);
           hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % down barrier no touch
           unhitValue.npv = 0;
           unhitValue.payoff = zeros(modelStatesSize,1);
           unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % dummy nearest month future price
           nMFuture.npv = 0;
           nMFuture.payoff = zeros(modelStatesSize,1);
           nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
           nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(modelStatesSize,1);
           payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateB.cashflow = zeros(modelStatesSize,1);
           payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
           
           payoffStateC.cashflow = zeros(modelStatesSize,1);
           payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
           
           % variable to use for early redemption check and barrier check 
           isStop = zeros(modelStatesSize,scheduleSize);
           worstP = zeros(modelStatesSize,scheduleSize);
           
           if KIYN == true
               isKI = ones(modelStatesSize,1);
           else
               isKI = zeros(modelStatesSize,1);
           end
           
           % check whether current future price breach the barrier
           comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
           isKI = multiAsset.BarrierCheck(comFwd,isKI,payoffInfo);
%           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
           %% Path Generation Forward
           for i=lastAliveExeriseIdx:1:scheduleSize
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i-1);
                    endTimeIdx = eventTimeIdx(i);
                else
                    startTimeIdx = 1; % valueDate
                    endTimeIdx = eventTimeIdx(i);  
                end
                
                endTimeIdx = endTimeIdx - 1;
                
            %% induction Forward start
                for idx= startTimeIdx:1:endTimeIdx
                    mcmcOut   = inductMCMCForwardNew(multiAsset,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,numFactors*(idx-1) + 1:numFactors*(idx-1) + numFactors));
                    
                    nextX = mcmcOut.nextX;
                    nextXIdx = mcmcOut.nextXIdx;
                    % one step backward induction end
                    % barrier check
                    lastTimeIdx = lastTimeIdx + 1;
                    currentTime = timeStep(lastTimeIdx);
                    comFwd = multiAsset.CommoFwdNMMC(currentTime,nextX);
                    % per each path check whether Barrier was breached 
                    isKI = BarrierCheck(multiAsset,comFwd,isKI,payoffInfo);
                end
                
                
                %forward payoff handling
                columnIn.isStop = isStop;
                columnIn.worstP = worstP;
                columnIn.payoffColumn = unhitValue;
                
                comFwdWorst = comFwd(:,1)/basePrice(1);
                for ii=1:modelStatesSize
                    for jj=2:numFactors
                        comFwdWorst(ii) = min(comFwdWorst(ii),comFwd(ii,jj)/basePrice(jj));
                    end
                end
                
                columnOut = multiAsset.MCPayoffWorst(i,comFwdWorst,currentTime,isStop,isKI,payoffInfo,columnIn);
                isStop = columnOut.isStop;
                worstP = columnOut.worstP;
                unhitValue = columnOut.payoffColumn;
                
                
                nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd(:,1);
                nMFuture.payoff = comFwd(:,1);
                
                %induct forward for eventDate
                currentNodeIdx = currentNodeIdx + 1;
           end
          
%            lastTimeIdx = currentTimeIdx;
          %% Forward Induction End
            
          %% Backward Induction Start (Regression)
           for i=scheduleSize:-1:lastAliveExeriseIdx
                % past schedule neglect
                if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
                if i > lastAliveExeriseIdx
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = eventTimeIdx(i-1);
                else
                    startTimeIdx = eventTimeIdx(i);
                    endTimeIdx = 1;  % valueDate
                end
%                 endTimeIdx = endTimeIdx + 1;
                currentTime = timeStep(endTimeIdx);
                
                %induct backward for eventDate
                currentNodeIdx = currentNodeIdx - 1;
                nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
              %% AtDate operation on Early ExerciseDate
                if i > lastAliveExeriseIdx
                    strikeEarly = autoCallSchedule.strike(i-1);
                    couponEarly = autoCallSchedule.coupon(i-1);
                    
                    exerciseValue = zeros(modelStatesSize,1);
                    contiValue = zeros(modelStatesSize,1);
                    
%                     onePayoffD = ones(modelStatesSize,1);
                    onePayoffD = multiAsset.DF(currentTime/365.0);
                    
                    % we only use paths that is still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 
                            contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
                        else
                            contiValue(idx) = unhitValue.payoff(idx);
                        end
                    end
                    
%                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
                    contiValue = contiValue/10000;
                    
                    decisionVar1 = worstP(:,i-1);
                    % we use upto 2nd order polynomial as regression
                    % equatioin
                    dimensionLS = 4;
                    regMatrix = ones(modelStatesSize,dimensionLS);
                    for idx_r=1:modelStatesSize
                        regMatrix(idx_r,2) = decisionVar1(idx_r);
                        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
                        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
                    end
                    % we use only alive path at the execise date as
                    % regression input
                    beta =zeros(dimensionLS,1);
                    
                    numSim = 0;
                    regMatrix2 = zeros(dimensionLS,dimensionLS);
                    contiValue2 = zeros(dimensionLS,1);
%                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
                    aliveVector = zeros(modelStatesSize,1);
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
                            numSim = numSim +1;
%                             aliveMatrix(idx,idx) = 1;
                            aliveVector(idx) = 1;
                            
                            if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
                                exerciseValue(idx) = contiValue(idx);
                            elseif worstP(idx,i-1) < strikeEarly
                                exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
                            end
                        else % for terminated path we do nothing
                            exerciseValue(idx) = contiValue(idx);
%                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
                        end
                        
                    end
                    
                    % if we have live paths, then we do the regression
                    if numSim > 0
%                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
%                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
                        regMatrix2 = zeros(dimensionLS,dimensionLS);
                        contiValue2 = zeros(dimensionLS,1);
                        for ii=1:dimensionLS
                            regVi = regMatrix(:,ii);
                            for jj=1:dimensionLS
                                regVj = regMatrix(:,jj);
                                for idx3=1:modelStatesSize
                                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                                end
                            end
                            for idx3=1:modelStatesSize
                                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
                            end
                        end
                        
                        beta = regress(contiValue2,regMatrix2);
                        regContiValue = regMatrix*beta;
                        
                        for idx=1:modelStatesSize
                            if isStop(idx,i-1) == 0 %live path
                                % regContiValue is only used for decision making
                                if regContiValue(idx) <= exerciseValue(idx) 
                                    contiValue(idx) = exerciseValue(idx);
                                end
                            else  % terminated path we do nothing
                                contiValue(idx) = exerciseValue(idx);
                            end
                        end
                    else % there is no alive path then we do nothing
                        for idx=1:modelStatesSize
                            contiValue(idx) = exerciseValue(idx);
                        end
                    end
                    
                    % we only apply regression to those paths that are still alive for regression
                    for idx=1:modelStatesSize
                        if isStop(idx,i-1) == 0  % live path
                            unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
                        else
                            unhitValue.payoff(idx) = contiValue(idx);
                        end
                    end
                    
                    unhitValue.payoff = unhitValue.payoff * 10000;
                    
                    
                end
                % we are at the start of the period
                
            end

%            hitValue.npv = hitValue.payoff(Pricingidx);
           unhitValue.npv = mean(unhitValue.payoff);
           nMFuture.npv =   mean(nMFuture.payoff);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
           
%            out.hitValue = hitValue; 
           out.unhitValue = unhitValue;
           out.nMFuture = nMFuture; 
        end
        
        
%         function out= expmReghai(multiAsset,A,t)
%             % Scale A by power of 2 so that its norm is < 1/2 .
%             A = A*t;
%             [f,e] = log2(norm(A,'inf'));
%             s = max(0,e+1);
%             A = A/2^s;
%             Aexp = eye(size(A,1)) + A;
%             for i=1:s
%                 Aexp = Aexp*Aexp;
%             end
%             out = Aexp;
%         end
%         
%         function path= bbgenerator(eqCOMDupireSpotGF,rn)
%             %generate brownian bridge 
%             
%             [Npaths,Nsteps]=size(rn);
%             path=zeros(Npaths,Nsteps);
%             map= zeros(1,Nsteps);
%             bridgeindex = zeros(1,Nsteps);
%             leftindex = zeros(1,Nsteps);
%             rightindex = zeros(1,Nsteps);
%             sigma = zeros(1,Nsteps);
%             leftweight = zeros(1,Nsteps);
%             rightweight = zeros(1,Nsteps);
%             
%             map(Nsteps)=1;
%             bridgeindex(1)=Nsteps;
%             sigma(1)=sqrt(Nsteps);
%             leftweight(1)=1; 
%             rightweight(1)=1; 
%             j=1;
%             for i=2:Nsteps    
%                 while map(j) 
%                     j=j+1;
%                 end
%                 k=j;
%                 while ~map(k) 
%                     k=k+1;
%                 end
%                 l=j+fix((k-1-j)/2);
%                 map(l)=i;
%                 bridgeindex(i)=l;
%                 leftindex(i)=j;
%                 rightindex(i)=k;
%                 leftweight(i)  = (k-l)/(k+1-j);
%                 rightweight(i) = (l+1-j)/(k+1-j);
%                 sigma(i) = sqrt(((l+1-j)*(k-l))/(k+1-j));
%                 j=k+1;
%                 if j>=Nsteps+1
%                     j=1;
%                 end
%             end
% 
%             path(:,Nsteps)=sigma(1)*rn(:,1);
%             for i=2:Nsteps
%                 j=leftindex(i);
%                 k=rightindex(i);
%                 l=bridgeindex(i);
%                 if j~=1
%                     path(:,l)=leftweight(i)*path(:,j-1) + rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
%                 else
%                     path(:,l) = rightweight(i)*path(:,k) + sigma(i)*rn(:,i);
%                 end
%             end
%         end
%         
%         function out = TDMASolve(eqCOMDupireSpotGF,tdma,vector)
%             size = tdma.size;
%             out = vector;
%             
%             for i=2:size
%                 out(i) = out(i) - tdma.ll(i)*out(i-1);
%             end
%             
%             out(size) = out(size)/tdma.dd(size);
%             
%             for i=size-1:-1:1
%                 out(i) = (out(i) - out(i+1)*tdma.uu(i))/tdma.dd(i);
%             end
%             
%         end
%         
%         function out = LUDecompose(eqCOMDupireSpotGF,tdma)
%             out = tdma;
%             if out.bupdated == 1
%                 for i=1:out.size
%                     out.uu(i) = out.c(i);
%                 end
%                 
%                 out.dd(1) = out.b(1);
%                 for i=2:out.size
%                     out.ll(i) = out.a(i)/out.dd(i-1);
%                     out.dd(i) = out.b(i) - out.c(i-1)*out.ll(i);
%                 end
%                 
%                 out.bupdated = 0;
%             end
%         
%         end
%         
%         function [ x,y,d] = HugeDecomp(eqCOMDupireSpotGF,n,a,b,c)
%     
%             x = zeros(n,1);
%             y = zeros(n,1);
%             d = zeros(n,1);
%             %fwd
% 
%             x(1) = 1.0;
%             x(2) = 1.0;
% 
%             for i=3:n
%                 x(i) = x(i-1) - (a(i-1)/b(i-1))*(c(i-2)/b(i-2))*x(i-2);
%             end
% 
%             %bwd
%             y(n) = 1.0/(x(n)-(a(n)/b(n))*(c(n-1)/b(n-1))*x(n-1));
%             y(n-1) = y(n);
% 
%             for i=n-2:-1:1
%                 y(i) = y(i+1) - (a(i+2)/b(i+2))*(c(i+1)/b(i+1))*y(i+2);
%             end
% 
%             %set d
% 
%             d(1) = x(1)*y(1)/b(1);
%             for i=2:n
%                 d(i) = -(a(i)/b(i))*(y(i)/y(i-1))*d(i-1) + x(i)*y(i)/b(i);
%             end
% 
% 
%         end
% 
%         
%         function localVol = GetLocalVolFromProxy(eqCOMDupireSpotGF,volProxy,Ks,ipos,idxNow)
% 
%             localVol = zeros(length(Ks),1);
%             volProxyKs = zeros(1,length(volProxy));
%             try
%                 fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
%             catch ME
%                 aaa = 0;
%             end
%             
%             if strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'monotonicCubicSpline')
%                 if strcmp(eqCOMDupireSpotGF.modelParams('VolProxyOnMarketStrikeYN') ,'Yes')
%                     aaa =1.0;
%                 else
%                     for i=1:length(volProxy)
%                         volProxyKs(i) = Ks(ipos(i));
%                     end
% 
%                     localVol = pchip(volProxyKs,volProxy,Ks);
%                     %flat extrapolation at the ends
%                     for i=1:length(Ks)
%                         if Ks(i) <= Ks(ipos(1))
%                             localVol(i) = localVol(ipos(1));
%                         elseif Ks(i) >= Ks(ipos(length(ipos)))
%                             localVol(i) = localVol(ipos(length(ipos)));
%                         end
%                     end
%                 end
%             elseif strcmp(eqCOMDupireSpotGF.modelParams('LocalVolInterpol') ,'matlabLinearMarketFwdMoneyness')
%                 for i=1:length(volProxy)
%                     volProxyKs(i) = fwdMoneynessPerExpiry(i);
%                 end
%                 
%                 for i=1:length(Ks)
%                     localVol(i) = H_interpolation(volProxyKs,volProxy,Ks(i),1);
%                 end   
%                 
%             else
%                 for i=1: length(volProxy)
%     %                 Extrapolation at the left and right
%                     if i == 1
%                         for j=1:ipos(i)-1
%                             localVol(j) = volProxy(i);
%                         end
%                     elseif i == length(volProxy)
%                         for j=ipos(i-1):ipos(i)-1
%                             localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
%                         end
%                         for j=ipos(i):length(Ks)
%                             localVol(j) = volProxy(i);
%                         end
%     %                  linear interpolation in the spot grid in-between    
%                     else
%                         for j=ipos(i-1):ipos(i)-1
%                             localVol(j) = (Ks(j)-Ks(ipos(i-1)))*(volProxy(i)-volProxy(i-1))/(Ks(ipos(i))-Ks(ipos(i-1))) + volProxy(i-1);
%                         end
%                     end
%                 end
%             end
%             
%         end
%                         
%         
%         function [newC newP] = Solve1DPDE(eqCOMDupireSpotGF,tvar,dT,dK,lastC,lastP,optParams)
%             
%             %filling proxy grid vol
%             proxy = zeros(length(lastC),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);
% 
%             %filling end
%             
%             meshC = lastC;
%             meshP = lastP;
%             
%             tdma.size = length(meshC);
%             tdma.bupdated = 0;
%             tdma.a = zeros(tdma.size,1);
%             tdma.b = zeros(tdma.size,1);
%             tdma.c = zeros(tdma.size,1);
%             tdma.uu = zeros(tdma.size,1);
%             tdma.ll = zeros(tdma.size,1);
%             tdma.dd = zeros(tdma.size,1);
%             
%             dK2 = dK*dK;
%             for i=2:tdma.size-1
%                 tdma.a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 tdma.b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 tdma.c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%             end
%             
%             tdma.a(1) = 0;
%             tdma.a(tdma.size) = 0;
%             tdma.c(1) = 0;
%             tdma.c(tdma.size) = 0;
%             tdma.b(1) = 1;
%             tdma.b(tdma.size) = 1;
%             tdma.bupdated = 1;
%             
%             tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%             pmeshC = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshC);
%             newC = pmeshC;
%             
%             pmeshP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,meshP);
%             newP = pmeshP;
%         end
%         
%         function out = Solve1DGF(eqCOMDupireSpotGF,tvar,dT,dK,lastProb,optParams)
%             
%             %filling proxy grid vol
%             proxy = zeros(length(lastProb),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(tvar,optParams.Ks,optParams.ipos,optParams.idxNow);
% 
%             %filling end
%             
%             meshProb = lastProb;
%             
%             size = length(meshProb);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*optParams.Ks(i)*optParams.Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             
%             condProb = zeros(size,size);
%             if optParams.params.GFType == -1
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             elseif optParams.params.GFType == 0
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expmdemo1(A_rg*NT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             
%             elseif optParams.params.GFType == 1
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg*NT);
%                 newProb = lastProb*A_exp1;
%                 condProb =A_exp1;
%             elseif optParams.params.GFType == 2
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg);
%                 
%                 newProb = lastProb;
%                 condProb = eye(size,size);
%                 for t=1:NT
%                     newProb = newProb*A_exp1;
%                     condProb = A_exp1*A_exp1;
%                 end
%                 
%             elseif optParams.params.GFType == 3
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%             
%                 % Test for HugeDecomposition for inverse matrix
%                 [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
%                 A = gallery('tridiag',a1,b,c1);
%                 oneStep = OneStepGF(size,x,y,d,A);
%                 newProb = lastProb;
%                 newProb = newProb*oneStep;
%                 
%                 condProb = oneStep;
%                 
%             else
%                 oneStepDt = optParams.params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 % rescale matrix element to smaller time step
%                 a= a*oneStepDt/dT;
%                 b = (b-1)*oneStepDt/dT + 1.0;
%                 c = c*oneStepDt/dT;
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%             
%                 % Test for HugeDecomposition for inverse matrix
%                 [ x,y,d] = eqCOMDupireSpotGF.HugeDecomp(size,a,b,c);
%                 A = gallery('tridiag',a1,b,c1);
%                 oneStep = OneStepGF(size,x,y,d,A);
%                 newProb = lastProb;
%                 %numberOfTimeStep;
%                 %oneStepDt = 1.0/365;
%                 
%                 condProb = eye(size,size);
%                 for t=1:NT
%                     newProb = newProb*oneStep;
%                     condProb = oneStep*oneStep;
%                 end 
%                 
%             end
%             
%             out.newProb = newProb;
%             out.condProb = condProb;
%         end
%         
%         function newP = inductPDEBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP,params)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = params.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = params.Ks(2) - params.Ks(1);
%             dT = (fromTime - toTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             if params.backGFType == -1
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%                 newP = A_exp1*lastP;
%                 
%             elseif params.backGFType == 0
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expmdemo1(A_rg*NT);
%                 newP = A_exp1*lastP;
%             elseif params.backGFType == 1
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg*NT);
%                 newP = A_exp1*lastP;
%             elseif params.backGFType == 2
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = expm(A_rg);
%                 
%                 newP = lastP;
%                 
%                 for t=1:NT
%                     newP = A_exp1*newP;
%                 end
%                 
%             elseif params.backGFType == -21
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 A_rg = full(gallery('tridiag',-a1*oneStepDt/dT,-b_rg*oneStepDt/dT,-c1*oneStepDt/dT));
%                 A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,1.0);
% %                 A_exp1 = expm(A_rg);
%                 
%                 newP = lastP;
%                 
%                 for t=1:NT
%                     newP = A_exp1*newP;
%                 end
%                 
%             elseif params.backGFType == 3
% 
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%                 
%                 tdma.size = length(lastP);
%                 tdma.a = a;
%                 tdma.b = b;
%                 tdma.c = c;
%                 tdma.uu = zeros(tdma.size,1);
%                 tdma.ll = zeros(tdma.size,1);
%                 tdma.dd = zeros(tdma.size,1);
% 
%                 tdma.bupdated = 1;
% 
%                 tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%                 newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,lastP);
% 
%                 
%             elseif (params.backGFType == 4) || (params.backGFType == 14) || (params.backGFType == 24)
%                 oneStepDt = params.oneStepDt;
%                 NT = round(dT/oneStepDt);
%                 
%                 % rescale matrix element to smaller time step
%                 a= a*oneStepDt/dT;
%                 b = (b-1)*oneStepDt/dT + 1.0;
%                 c = c*oneStepDt/dT;
%                 
%                 a1 = a(2:size);
%                 c1 = c(1:size-1);
%                 
%                 tdma.size = length(lastP);
%                 tdma.a = a;
%                 tdma.b = b;
%                 tdma.c = c;
%                 tdma.uu = zeros(tdma.size,1);
%                 tdma.ll = zeros(tdma.size,1);
%                 tdma.dd = zeros(tdma.size,1);
% 
%                 tdma.bupdated = 1;
% 
%                 tdmaLU =  eqCOMDupireSpotGF.LUDecompose(tdma);
%                 
%                 newP = lastP;
%                 for t=1:NT
%                     newP = eqCOMDupireSpotGF.TDMASolve(tdmaLU,newP);
%                 end
% 
%             else 
%                 disp('unImplemented')
%             end
%         end
%         
%         function newP = inductGFBackward(eqCOMDupireSpotGF,fromTime,toTime,lastP)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = Ks(2) - Ks(1);
%             dT = (fromTime - toTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             newP = A_exp1*lastP;
%             
%         end
%         
%         function out = generateCondProb(eqCOMDupireSpotGF,fromTime,toTime)
%             
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(Ks),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
%             %filling end
%             
%             size = length(proxy);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
%             
%             dK = Ks(2) - Ks(1);
%             dT = (toTime - fromTime)/365.0;
%             
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
%             
%             a(1) = 0;
%             c(size) = 0;
%             
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
%             
%             b(1) = 1;
%             b(size) = 1;
%             
%             % to create tridiagonal matrix
%             
%             a1 = a(2:size);
%             c1 = c(1:size-1);
%             
%             b_rg(1) = 0;
%             b_rg(size) = 0;
%             
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             out = A_exp1;
%             
%         end
%         
%         function marginal = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime)
% 
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry >= toTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             marginal(:,:) = eqCOMDupireSpotGF.localVolSurface.marginal(idxNow,:,:);
%         end
%         
%         function localVol = InterpolateLocalVol(eqCOMDupireSpotGF,x,fromTime)
% 
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             idx = find(eqCOMDupireSpotGF.localVolSurface.expiry > fromTime);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             localVolLine = eqCOMDupireSpotGF.localVolSurface.localVol(idxNow,:);
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %             Ks = params.Ks;
%             localVol =  zeros(length(x),1);
%             fwdMoneynessPerExpiry = eqCOMDupireSpotGF.fwdMoneyness(idxNow,:);
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             
%             % we use the same linear interpolation scheme as calibration
%             useVolProxyInterpolationYN = 'YES';
%             if strcmp(useVolProxyInterpolationYN,'YES')
%                 
%                 for ii =1:length(x)
%                     localVol(ii) = H_interpolation(fwdMoneynessPerExpiry,volProxy,x(ii),1);
%                 end
%             else
%                 volKnotPoints = zeros(length(ipos),1);
%                 for i=1: length(ipos)
%                     volKnotPoints(i) = Ks(ipos(i));
%                 end
% 
% 
%                 for i=1:length(x)
%                     xIdx = find(volKnotPoints <=x(i));
%                     if isempty(xIdx)
%                         localVol(i) = localVolLine(1);
%                     elseif max(xIdx) == ipos(length(ipos))
%                         localVol(i) = localVolLine(length(ipos));
%                     else
%                         localVolIdx = max(xIdx);
%                         weight = (x(i)-Ks(ipos(localVolIdx)))/(Ks(ipos(localVolIdx)+1)-Ks(ipos(localVolIdx)));
%                         localVol(i) = weight*localVolLine(ipos(localVolIdx)+1)+(1-weight)*localVolLine(ipos(localVolIdx));
%                     end
%                 end
%             end
%             
%         end
%         
%         function out = inductMCMCForward(eqCOMDupireSpotGF,fromTime,toTime,lastX,lastXIdx,U,Ks)
%             
%             NMC = length(lastX);
%             % we generate conditional probability matrix first
%             condProb = eqCOMDupireSpotGF.generateCondProb(fromTime,toTime);
%             Qa = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProb,Ks);
%             
% %             Qa = InterpolateMarginal(eqCOMDupireSpotGF,fromTime,toTime);
%             uSample = U;
%             
%             nextX = zeros(NMC,1);
%             nextXIdx = zeros(NMC,1);
% %             Ks = params.Ks;
%             nPoints = length(Ks);
%             
%             for i=1:NMC
%                 IndexMc = lastXIdx(i);
%                 j= IndexMc;
%                 cumProb = Qa(IndexMc,:);
%                 idxH = min(find(uSample(i)*Qa(IndexMc,end) < cumProb));
%                 nextX(i) = Ks(idxH);
%                 nextXIdx(i) = idxH;
% 
%             end
%             
%             out.nextX = nextX;
%             out.nextXIdx = nextXIdx;
% 
%             
%         end
%         
%         function newX = inductMCForward(eqCOMDupireSpotGF,fromTime,toTime,startIdx,endIdx,timeScheduleInfo,lastX,dZ,params)
%             
%             timeSchedule = timeScheduleInfo.timeSchedule;
%             dTSchedule365 = timeScheduleInfo.dTSchedule365;
%             volKnotIdx = timeScheduleInfo.volKnowIdx;
%             
%             predictor = lastX;
%             for j=startIdx:endIdx-1
%                 
% %                 localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j),params);
%                 localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,timeSchedule(j));
%                 Dt = dTSchedule365(j);
%                 sqrtDt = sqrt(dTSchedule365(j));
%                 for i=1:length(lastX)
%                     predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
% %                     predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i,j-startIdx+1)); 
%                 end
%             end
%             newX = predictor;
%             
%         end
%         
%         function mcmcOut = inductMCForwardNew(eqCOMDupireSpotGF,fromTime,toTime,lastX,dZ)
%             
%             predictor = lastX;
%             localVol = eqCOMDupireSpotGF.InterpolateLocalVol(predictor,fromTime);
%             Dt = (toTime - fromTime)/365.0;
%             sqrtDt = sqrt(Dt);
%             for i=1:length(lastX)
% %                 predictor(i) = predictor(i)*(1 + localVol(i)*sqrtDt*dZ(i)); 
%                 predictor(i) = predictor(i)*exp(-0.5*localVol(i)*localVol(i)*Dt + localVol(i)*sqrtDt*dZ(i)); 
%                 
%             end
%             
%             mcmcOut.nextX = predictor;
%             
%         end
%         
%         
%         function out = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,meshProb,params)
%             % Calculate Object Function
%             % settings for bisection
%             a = 0.001;
%             b = 10;
%             Tol = 1e-10;
%             MaxIter = 1000;
% 
%             strikes = params.marketStrikes;
%             strikeSize = length(params.marketStrikes);
%             out.vols    = zeros(strikeSize,1);
%             out.interpOTMPrices = zeros(strikeSize,1);
%             out.interpBlackPrices = zeros(strikeSize,1);
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             cValue = meshProb*meshC;
%             pValue = meshProb*meshP;
%             
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 if K <= 1.0
%                       out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
%                      out.interpOTMPrices(i) = pValue(i);
%                      out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
%                      
%                 else
%                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
%                     out.interpOTMPrices(i) = cValue(i);
%                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
%                 end
%             end
%             
%             
%         end
%         
%         function out = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,meshProb,params)
%             % Calculate Object Function
%             % settings for bisection
%             a = 0.001;
%             b = 10;
%             Tol = 1e-10;
%             MaxIter = 1000;
% 
%             strikes = params.marketStrikes;
%             strikeSize = length(params.marketStrikes);
%             out.vols    = zeros(strikeSize,1);
%             out.interpOTMPrices = zeros(strikeSize,1);
%             out.interpBlackPrices = zeros(strikeSize,1);
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             cValue = meshProb*meshC;
%             pValue = meshProb*meshP;
%             
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 if K <= 1.0
% %                      out.vols(i) = blackVolLBR(pValue(i),1.0,K, params.params.expiry(idxNow)/365.0,-1);
%                      out.vols(i) = BisecBlackFwdIV('P',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,pValue(i),Tol,MaxIter); 
%                      out.interpOTMPrices(i) = pValue(i);
%                      out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,-1);
%                      
%                 else
% %                     out.vols(i) = blackVolLBR(cValue(i),1.0,K,params.params.expiry(idxNow)/365.0,1);
%                     out.vols(i) = BisecBlackFwdIV('C',1.0,K,0.0,params.params.expiry(idxNow)/365.0,a,b,cValue(i),Tol,MaxIter); 
%                     out.interpOTMPrices(i) = cValue(i);
%                     out.interpBlackPrices(i) = blackPrice(1.0,K,out.vols(i),params.params.expiry(idxNow)/365.0,1);
%                 end
%             end
%             
%             
%         end
%         
%         function out = computeBackwardPDEOne(eqCOMDupireSpotGF,maturity,strike,params)
%             expiry = params.expiry;
%             
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             
%             mesh = zeros(modelXSize,1);
%             
%             if strike <= 1.0
%                 for i=1:modelXSize
%                     mesh(i,1) = max(strike- modelX(i),0);
%                 end
% 
%                 volExpiry = expiry(find(expiry <= maturity));
%                 volExpiry = [volExpiry; 0];
%                 timeStep = sort(union(volExpiry,maturity),'ascend');
%                 for i=length(timeStep):-1:2
%                     mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
%                 end
%                 
%                 out = mesh(params.Pricingidx);
%                 
%             else
%                 for i=1:modelXSize
%                     mesh(i,1) = max(modelX(i)-strike,0);
%                 end
% 
%                 volExpiry = expiry(find(expiry <= maturity));
%                 volExpiry = [volExpiry; 0];
%                 timeStep = sort(union(volExpiry,maturity),'ascend');
%                 for i=length(timeStep):-1:2
%                     mesh = eqCOMDupireSpotGF.inductPDEBackward(timeStep(i),timeStep(i-1),mesh,params);
%                 end
%                 
%                 out = mesh(params.Pricingidx);
%                 
%             end
%             
%         end
% %                              MCPayoff(eqCOMDupireSpotGF,i, scheduleSize, comFwd,isStop,dummyisKI,payoffInfo,hitValue);
%         function columnOut = MCPayoff(eqCOMDupireSpotGF,idx,comFwd,currentTime, isStop,isKI,payoffInfo,columnIn)
%             
%             modelStatesSize = length(comFwd);
%             scheduleSize = payoffInfo.autoCallSchedule.scheduleSize;
%             strike = payoffInfo.autoCallSchedule.strike(idx);
%             coupon = payoffInfo.autoCallSchedule.coupon(idx);
%             nominal = payoffInfo.nominal;
%             basePrice = payoffInfo.basePrice;
%             lowerBarrier = payoffInfo.lowerBarrier;
%             callStrike1 = payoffInfo.callStrike1;
%             callValue1  = payoffInfo.callValue1;
%             
%             columnOut.isStop = zeros(modelStatesSize,scheduleSize);
%             columnOut.worstP = zeros(modelStatesSize,scheduleSize);
%             
%             columnOut.isStop = columnIn.isStop;
%             columnOut.worstP = columnIn.worstP;
%             
%             columnOut.payoffColumn.payoff = zeros(modelStatesSize,1);
%             columnOut.payoffColumn.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%             columnOut.payoffColumn.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%             
%             columnOut.payoffColumn = columnIn.payoffColumn;
%             
%             df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%             % early redemption
%             % idx  : early redemption schedule iter
%             % idx11: simulation path iter
%             if idx == 1
%                 for idx11 =1:modelStatesSize % 1st early redemption
%                         % alive path
%                         if comFwd(idx11) >= basePrice*strike % early redemption conditon met
%                            columnOut.isStop(idx11,idx) = 1;
%                            columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                            columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
%                            columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
%                            columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                         end
%                         
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                     
%                 end
%                 
%             elseif idx > 1 && idx < scheduleSize
%                 for idx11 =1:modelStatesSize
%                     if columnOut.isStop(idx11,idx-1) == 1 % terminated path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
%                         columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
%                     else
%                         % alive path
%                         if comFwd(idx11) >= basePrice*strike % early redemption conditon met
%                             columnOut.isStop(idx11,idx) = 1;
%                             columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                             columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal*(1.0 + coupon);
%                             columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = nominal*(1.0 + coupon)*df_ep;
%                             columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                         end
%                         
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                     end
%                 end
%                 
%             else % idx == scheduleSize
%                 for idx11 =1:modelStatesSize
%                     if columnOut.isStop(idx11,idx-1) == 1 % terminated path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = columnOut.worstP(idx11,idx-1);
%                         columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx-1); %cached cashflow
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx-1); %cached cashflow_npv
% 
%                     else
%                         %alive path
%                         columnOut.isStop(idx11,idx) = 1;
%                         columnOut.worstP(idx11,idx) = comFwd(idx11)/basePrice;
%                         % barrier hit path
%                         if isKI(idx11) == 1
%                             if comFwd(idx11) < basePrice*callStrike1 
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
%                             elseif comFwd(idx11) < basePrice*strike
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
%                             else
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                         else % barrier unhit path
%                             if comFwd(idx11) < basePrice*callStrike1 
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * callValue1/callStrike1* comFwd(idx11)/basePrice;
%                             elseif comFwd(idx11) < basePrice*strike
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon - callValue1)/(strike-callStrike1)*(comFwd(idx11)/basePrice - callStrike1) + nominal*callValue1; 
%                             else
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                             
%                             if comFwd(idx11) > basePrice*lowerBarrier
%                                 columnOut.payoffColumn.payoffStates.cashflow(idx11,idx) = nominal * (1.0 + coupon);
%                             end
%                         end
%                         
%                         % payoff postprocess
%                         columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx) = columnOut.payoffColumn.payoffStates.cashflow(idx11,idx)*df_ep;
%                         columnOut.payoffColumn.payoff(idx11) = columnOut.payoffColumn.payoffStates.cashflow_npv(idx11,idx);
%                     end    
%                 end
%             end    
%             
%             
%         end
%         
%         function out = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%             basePrice = payoffInfo.basePrice;
%             lowerBarrier = payoffInfo.lowerBarrier;
%             modelStatesSize = length(comFwd);
%             out = zeros(modelStatesSize,1);
%             for idx=1:modelStatesSize
%                 if (isKI(idx) > 0) || (comFwd(idx) <= basePrice*lowerBarrier)
%                     out(idx) = 1.0;
%                 end
%             end
%         end
%         
%         % NM : nearest month forward
%         function out = CommoFwdNM(eqCOMDupireSpotGF,observeTime)
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%            % NM : nearest month forward
%         function out = CommoFwdContractM(eqCOMDupireSpotGF,observeTime,maturityTime)
%             if observeTime > maturityTime
%                 disp('observeTime is passed maturityTime!')
%                 out = 0;
%                 return;
%             end
%             
%             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(maturityTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%         function out = CommoFwdNMMC(eqCOMDupireSpotGF,observeTime,modelStates)
% %             modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%             forwardCurve = eqCOMDupireSpotGF.forwardCurve.params('rawData');
%             forwardCurveSize = size(forwardCurve,1);
%             forwardMaturity = forwardCurve(:,1); 
%             idx = find(observeTime <= forwardMaturity);
%              if isempty(idx)
%                 idxNM = forwardCurveSize;
%              else
%                 idxNM = min(idx);
%              end
%              
%              %fwd(0,T_NM)
%              fwd_NM = forwardCurve(idxNM,2);
%              stateSize = length(modelStates);
%              stochasticFwd = zeros(stateSize,1);
%              
%              for i=1:stateSize
%                 stochasticFwd(i) = fwd_NM * modelStates(i); 
%              end
%              
%              out = stochasticFwd;
%         end
%         
%         function out = computeStepdown1SMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
% %             out =0;
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% MCMC init 
%             useQuasiRandomYN = 'true';
%             
%             if strcmp(useQuasiRandomYN,'true')
% %                 NMC = 2^14;
% %                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Ps = sobolset(MCTimeStep,'Skip',NMC);
%                 Q = qrandstream(Ps);
% 
%                 reset(Q);
%                 U0 = qrand(Q,NMC);
%                 U = zeros(NMC,MCTimeStep);
% 
%                 dZ0 = zeros(size(U,1),size(U,2));
%     %             dZ = norminv(U);
%                 % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
%                 % generate random number from the end
%                 for i=1:size(U,1)
%                     for j=1:size(U,2)
%                         dZ0(i,j) = r8_normal_01_cdf_inverse(U0(i,j));
%                     end
%                 end
%             
%                % apply brownian bridge
%                dZ = eqCOMDupireSpotGF.bbgenerator(dZ0);
%                U(:,1) = dZ(:,1);
%                for i=2:size(dZ,2)
%                    U(:,i) = dZ(:,i) - dZ(:,i-1); 
%                end
% %                U = dZ;
%             else
%                 rng('default');
%                 rng(0);
%                 
%                 NMC = 40000;
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
% %                 Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
% 
%             end
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            initX = ones(NMC,1);
% %            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
% %            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
% %            nextXIdx = initXIdx;
%             
%           %% payoff init
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
%            
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
%            
%            
%            modelStatesSize = NMC;
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
%            
%            % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%            isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
% %           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%            %% Path Generation Forward
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCForwardNew(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,U(:,idx));
%                     
%                     nextX = mcmcOut.nextX;
% %                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
% %                 dummyisKI = ones(modelStatesSize,1);
% %                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
% %                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
%                 nMFuture.payoff = comFwd;
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%             
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end
% 
% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeStepdown1SMCMC(eqCOMDupireSpotGF,valueDate,stepdownParams)
% %             out =0;
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             autoCallSchedule.scheduleSize = scheduleSize;
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% MCMC init 
%             useQuasiRandomYN = 'true';
%             
%             if strcmp(useQuasiRandomYN,'true')
% %                 NMC = 2^14;
% %                 NMC = 2^16;
%                 NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
%                 Ps = sobolset(MCTimeStep,'Skip',NMC);
%                 Q = qrandstream(Ps);
% 
%                 reset(Q);
%                 U0 = qrand(Q,NMC);
%                 U = zeros(NMC,MCTimeStep);
% 
%                 % first random numbers goes to the enddate..
%                 for i=1:size(U,2)
%                     U(:,i) = U0(:,size(U,2)-(i-1));
%                 end
%             else
%                 rng('default');
%                 rng(0);
%                 
%                 NMC = 40000;
%                 MCTimeStep = totalTimeStepSize -1; % nowDate (0) should be skipped
% %                 Z= randn(NMC/2,MCTimeStep);
%                 Z= rand(NMC/2,MCTimeStep);
%                 Z=[Z;-Z];
%                 Z=Z';
%                 
%                 %TimeInversion
%                 sizeZ = size(Z,1);
%                 A = zeros(sizeZ,sizeZ);
%                 for i=1:sizeZ
%                     A(i,sizeZ + 1 -i) = 1;
%                 end
%                 Z = A*Z;
%                 
%                 U = Z';
% 
%             end
%            currentTime = 0; %NowDate
%            currentTimeIdx = 1; % 1st Time Steps
%            currentNodeIdx = 1; % no eventDate for nowDate
%            lastTimeIdx = currentTimeIdx;
%            
%            % initial modelState
%            initX = ones(NMC,1);
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            initXIdx = ones(NMC,1)*Pricingidx;
%            
%            nextX = initX;
%            nextXIdx = initXIdx;
%             
%           %% payoff init
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            KIYN = stepdownParams.params('KIYN');
%            
%            payoffInfo.nominal = nominal;
%            payoffInfo.basePrice = basePrice;
%            payoffInfo.lowerBarrier = lowerBarrier;
%            payoffInfo.overHedgeShift = overHedgeShift;
%            payoffInfo.overHedgeSpread = overHedgeSpread;
%            
%            payoffInfo.callStrike1 = callStrike1;
%            payoffInfo.callValue1 = callValue1;
%            payoffInfo.SSpayoffYN = SSpayoffYN;
%            
%            payoffInfo.autoCallSchedule = autoCallSchedule;
%            
%            
%            modelStatesSize = NMC;
%            %down barrier already touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % variable to use for early redemption check and barrier check 
%            isStop = zeros(modelStatesSize,scheduleSize);
%            worstP = zeros(modelStatesSize,scheduleSize);
%            
%            if KIYN == true
%                isKI = ones(modelStatesSize,1);
%            else
%                isKI = zeros(modelStatesSize,1);
%            end
%            
%            % check whether current future price breach the barrier
%            comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%            isKI = eqCOMDupireSpotGF.BarrierCheck(comFwd,isKI,payoffInfo);
% %           BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo)
%            %% Path Generation Forward
%            for i=lastAliveExeriseIdx:1:scheduleSize
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i-1);
%                     endTimeIdx = eventTimeIdx(i);
%                 else
%                     startTimeIdx = 1; % valueDate
%                     endTimeIdx = eventTimeIdx(i);  
%                 end
%                 
%                 endTimeIdx = endTimeIdx - 1;
%                 
%             %% induction Forward start
%                 Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%                 for idx= startTimeIdx:1:endTimeIdx
%                     mcmcOut   = inductMCMCForward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx+1),nextX,nextXIdx,U(:,idx),Ks);
%                     nextX = mcmcOut.nextX;
%                     nextXIdx = mcmcOut.nextXIdx;
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx + 1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNMMC(currentTime,nextX);
%                     % per each path check whether Barrier was breached 
%                     isKI = BarrierCheck(eqCOMDupireSpotGF,comFwd,isKI,payoffInfo);
%                 end
%                 
%                 
%                 %forward payoff handling
%                 columnIn.isStop = isStop;
%                 columnIn.worstP = worstP;
%                 columnIn.payoffColumn = unhitValue;
% %                 dummyisKI = ones(modelStatesSize,1);
% %                 columnout   = MCPayoff(eqCOMDupireSpotGF,i,comFwd,currentTime,isStop,dummyisKI,payoffInfo,columnIn);
% %                 
%                 columnOut = eqCOMDupireSpotGF.MCPayoff(i,comFwd,currentTime,isStop,isKI,payoffInfo,columnIn);
%                 isStop = columnOut.isStop;
%                 worstP = columnOut.worstP;
%                 unhitValue = columnOut.payoffColumn;
%                 
%                 
%                 nMFuture.payoffStates.cashflow(:,currentNodeIdx) = comFwd;
%                 nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = comFwd ;
%                 nMFuture.payoff = comFwd;
%                 
%                 %induct forward for eventDate
%                 currentNodeIdx = currentNodeIdx + 1;
%            end
%           
% %            lastTimeIdx = currentTimeIdx;
%           %% Forward Induction End
%             
%           %% Backward Induction Start (Regression)
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
% %                 endTimeIdx = endTimeIdx + 1;
%                 currentTime = timeStep(endTimeIdx);
%                 
%                 %induct backward for eventDate
%                 currentNodeIdx = currentNodeIdx - 1;
%                 nMFuture.payoff = nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx);
%               %% AtDate operation on Early ExerciseDate
%                 if i > lastAliveExeriseIdx
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                     exerciseValue = zeros(modelStatesSize,1);
%                     contiValue = zeros(modelStatesSize,1);
%                     
% %                     onePayoffD = ones(modelStatesSize,1);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     
%                     % we only use paths that is still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 
%                             contiValue(idx) = unhitValue.payoff(idx)/onePayoffD;
%                         else
%                             contiValue(idx) = unhitValue.payoff(idx);
%                         end
%                     end
%                     
% %                     contiValue = unhitValue.payoffStates.cashflow(:,i-1);
%                     contiValue = contiValue/10000;
%                     
%                     decisionVar1 = worstP(:,i-1);
%                     % we use upto 2nd order polynomial as regression
%                     % equatioin
%                     dimensionLS = 4;
%                     regMatrix = ones(modelStatesSize,dimensionLS);
%                     for idx_r=1:modelStatesSize
%                         regMatrix(idx_r,2) = decisionVar1(idx_r);
%                         regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
%                         regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
%                     end
%                     % we use only alive path at the execise date as
%                     % regression input
%                     beta =zeros(dimensionLS,1);
%                     
%                     numSim = 0;
%                     regMatrix2 = zeros(dimensionLS,dimensionLS);
%                     contiValue2 = zeros(dimensionLS,1);
% %                     aliveMatrix = zeros(modelStatesSize,modelStatesSize);
%                     aliveVector = zeros(modelStatesSize,1);
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0 % alive path at the start of the period (early exercise date
%                             numSim = numSim +1;
% %                             aliveMatrix(idx,idx) = 1;
%                             aliveVector(idx) = 1;
%                             
%                             if worstP(idx,i-1) < strikeEarly - couponEarly/overHedgeSpread
%                                 exerciseValue(idx) = contiValue(idx);
%                             elseif worstP(idx,i-1) < strikeEarly
%                                 exerciseValue(idx) = (1.0 + couponEarly + (worstP(idx,i-1) - strikeEarly)*overHedgeSpread);
%                             end
%                         else % for terminated path we do nothing
%                             exerciseValue(idx) = contiValue(idx);
% %                             exerciseValue(idx) = unhitValue.payoffStates.cashflow(idx,i-1)/10000;
%                         end
%                         
%                     end
%                     
%                     % if we have live paths, then we do the regression
%                     if numSim > 0
% %                         regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
% %                         contiValue2 = regMatrix' * aliveMatrix * contiValue;
%                         regMatrix2 = zeros(dimensionLS,dimensionLS);
%                         contiValue2 = zeros(dimensionLS,1);
%                         for ii=1:dimensionLS
%                             regVi = regMatrix(:,ii);
%                             for jj=1:dimensionLS
%                                 regVj = regMatrix(:,jj);
%                                 for idx3=1:modelStatesSize
%                                     regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
%                                 end
%                             end
%                             for idx3=1:modelStatesSize
%                                 contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*contiValue(idx3);
%                             end
%                         end
%                         
%                         beta = regress(contiValue2,regMatrix2);
%                         regContiValue = regMatrix*beta;
%                         
%                         for idx=1:modelStatesSize
%                             if isStop(idx,i-1) == 0 %live path
%                                 % regContiValue is only used for decision making
%                                 if regContiValue(idx) <= exerciseValue(idx) 
%                                     contiValue(idx) = exerciseValue(idx);
%                                 end
%                             else  % terminated path we do nothing
%                                 contiValue(idx) = exerciseValue(idx);
%                             end
%                         end
%                     else % there is no alive path then we do nothing
%                         for idx=1:modelStatesSize
%                             contiValue(idx) = exerciseValue(idx);
%                         end
%                     end
%                     
%                     % we only apply regression to those paths that are still alive for regression
%                     for idx=1:modelStatesSize
%                         if isStop(idx,i-1) == 0  % live path
%                             unhitValue.payoff(idx) = contiValue(idx)*onePayoffD;
%                         else
%                             unhitValue.payoff(idx) = contiValue(idx);
%                         end
%                     end
%                     
%                     unhitValue.payoff = unhitValue.payoff * 10000;
%                     
%                     
%                 end
%                 % we are at the start of the period
%                 
%             end
% 
% %            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = mean(unhitValue.payoff);
%            nMFuture.npv =   mean(nMFuture.payoff);
% %            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
% %            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%              
%         function out = computeStepdown1SGF(eqCOMDupireSpotGF,valueDate,stepdownParams)
%             % model schedule generation
%             scheduleInt = stepdownParams.params('autoCallScheduleInt');
%             scheduleSize = size(scheduleInt,1);
%             autoCallSchedule=[];
%             for i=1:scheduleSize
%                 autoCallSchedule.exerciseDate(i) = H_Date(scheduleInt(i,1));
%                 autoCallSchedule.days(i) = DateDiff(autoCallSchedule.exerciseDate(i),valueDate);
%                 autoCallSchedule.strike(i) = scheduleInt(i,2);
%                 autoCallSchedule.coupon(i) = scheduleInt(i,3);
%             end
%             
%             expiryDate = autoCallSchedule.exerciseDate(scheduleSize);
%             
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(stepdownParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             % we include exerciseDates
%             for i= 1:scheduleSize
%                 daysToExercise = autoCallSchedule.days(i);
%                 if daysToExercise <= 0
%                     expiredChance = expiredChance +1;
%                     continue;
%                 else
%                     aliveChance = aliveChance + 1;
%                     timeStep = [timeStep; daysToExercise];
%                 end 
%             end
%             lastAliveExeriseIdx = 1 + expiredChance;
%             
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             for i= scheduleSize:-1:lastAliveExeriseIdx
%                 idx = find(timeStep == autoCallSchedule.days(i));
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            nominal = stepdownParams.params('nominal');
%            basePrice = stepdownParams.params('basePrice');
%            lowerBarrier = stepdownParams.params('lowerBarrier');
%            overHedgeShift = stepdownParams.params('overHedgeShift');
%            overHedgeSpread = stepdownParams.params('overHedgeSpread');
%            
%            callStrike1 = stepdownParams.params('callStrike1');
%            callValue1 = stepdownParams.params('callValue1');
%            SSpayoffYN = stepdownParams.params('SSpayoffYN');
%            
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            
%            modelStatesSize = length(modelStates);
%            
%            %down barrier touch
%            hitValue.npv = 0;
%            hitValue.payoff = zeros(modelStatesSize,1);
%            hitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            hitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % down barrier no touch
%            unhitValue.npv = 0;
%            unhitValue.payoff = zeros(modelStatesSize,1);
%            unhitValue.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            unhitValue.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,scheduleSize);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,scheduleSize);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
%            
%            comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%            strikeMat = autoCallSchedule.strike(scheduleSize);
%            couponMat = autoCallSchedule.coupon(scheduleSize);
%            
% %            strikeOH = strikeMat + overHedgeShift;
%            % samsung overhedge payoff
%            if strcmp(SSpayoffYN,'YES')
%                for i=1:modelStatesSize
%                    if comFwd(i) < basePrice*callStrike1
%                         payoffStateA.cashflow(i) = nominal * callValue1/callStrike1* comFwd(i)/basePrice; 
%                    elseif comFwd(i) < basePrice*strikeMat 
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat - callValue1)/(strikeMat-callStrike1)*(comFwd(i)/basePrice - callStrike1) + nominal*callValue1; 
%                    else
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat);
%                    end
%                end
% 
%                payoffStateB.cashflow = payoffStateA.cashflow;
% 
%                for i=1:modelStatesSize
%                     if comFwd(i) > basePrice*lowerBarrier
%                         payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
%                     end
%                end
%                
%            else
%                for i=1:modelStatesSize
%                    if comFwd(i) >= basePrice*strikeMat
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat); 
%                    else
%                        payoffStateA.cashflow(i) = nominal * (1.0 + couponMat + (comFwd(i)/basePrice-strikeMat)*overHedgeSpread);
%                        payoffStateA.cashflow(i) = max(payoffStateA.cashflow(i),nominal * comFwd(i)/basePrice);
%                    end
%                end
% 
%                payoffStateB.cashflow = payoffStateA.cashflow;
% 
%                for i=1:modelStatesSize
%                     if comFwd(i) > basePrice*lowerBarrier
%                         payoffStateB.cashflow(i) = nominal * (1.0 + couponMat);
%                     end
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             hitValue.payoff = payoffStateA.cashflow_npv;
%             
%             unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             unhitValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            
%            for i=scheduleSize:-1:lastAliveExeriseIdx
%                 % past schedule neglect
%                 if DateDiff(valueDate,autoCallSchedule.exerciseDate(i)) >= 0, continue;end
%                 if i > lastAliveExeriseIdx
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = eventTimeIdx(i-1);
%                 else
%                     startTimeIdx = eventTimeIdx(i);
%                     endTimeIdx = 1;  % valueDate
%                 end
%                 
%                 endTimeIdx = endTimeIdx + 1;
%             %% induction start
%                 mesh = hitValue.payoff;
%                 meshB = unhitValue.payoff;
%                 for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     % one step backward induction end
%                     % barrier check
%                     lastTimeIdx = lastTimeIdx -1;
%                     currentTime = timeStep(lastTimeIdx);
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%                     for j=1:modelStatesSize
%                         if comFwd(j) <= basePrice*lowerBarrier
%                            meshB(j) = mesh(j); 
%                         end
%                     end
%                 end
%                 
%                 hitValue.payoff = mesh;
%                 unhitValue.payoff = meshB;
%                 nMFuture.payoff = comFwd;
%               %% induction end
%                 
%                 % backward induction in the payoff script
%                 currentNodeIdx = currentNodeIdx -1;
%               %% AtDate operation on Early ExerciseDate
%                 
%                 if i > lastAliveExeriseIdx
%                     currentTime = timeStep(lastTimeIdx);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValue  = hitValue.payoff/onePayoffD;
%                     contiValueB = unhitValue.payoff/onePayoffD;
%                     
%                     %AutoCall applied
%                     comFwd = eqCOMDupireSpotGF.CommoFwdNM(currentTime);
%                     strikeEarly = autoCallSchedule.strike(i-1);
%                     couponEarly = autoCallSchedule.coupon(i-1);
%                     
%                      % intermediate variables
%                     payoffStateA.cashflow = zeros(modelStatesSize,1);
%                     payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     payoffStateC.cashflow = zeros(modelStatesSize,1);
%                     payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%                     
%                     % samsung overhedge payoff
%                     if strcmp(SSpayoffYN,'YES')
%                         for j=1:modelStatesSize
%                             if comFwd(j)/basePrice >= strikeEarly
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
%                             elseif comFwd(j)/basePrice >= strikeEarly - couponEarly/overHedgeSpread
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                
%                                payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                             else
% %                                 payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% %                                 payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
% % 
% %                                 payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
% %                                 payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
%                                 
%                                 payoffStateA.cashflow(j) = contiValue(j);
%                                 payoffStateB.cashflow(j) = contiValueB(j);
%                            end
%                         end
%                     else
%                         for j=1:modelStatesSize
%                             if comFwd(j) >= basePrice*strikeEarly
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly);
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly);
%                             else
%                                payoffStateA.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateA.cashflow(j) = max(payoffStateA.cashflow(j),contiValue(j));
%                                payoffStateB.cashflow(j) = nominal * (1.0 + couponEarly + (comFwd(j)/basePrice-strikeEarly)*overHedgeSpread);
%                                payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j));
%                            end
%                         end
%                         
%                     end
%                     % barrier
%                     for j=1:modelStatesSize
%                         if comFwd(j) <= basePrice*lowerBarrier
%                            payoffStateB.cashflow(j) = payoffStateA.cashflow(j);
%                         end
%                     end
%                     
%                     payoffStateC.cashflow = comFwd;
%                     
%                     for j=1:modelStatesSize
%                         payoffStateA.cashflow_npv(j) = payoffStateA.cashflow(j)*onePayoffD;
%                         payoffStateB.cashflow_npv(j) = payoffStateB.cashflow(j)*onePayoffD;
%                         payoffStateC.cashflow_npv(j) = payoffStateC.cashflow(j)*1.0;
%                     end
%                     
%                     %process payoff at early exercise date
%                     hitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%                     hitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%                     hitValue.payoff = payoffStateA.cashflow_npv;
%                     
%                     unhitValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%                     unhitValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%                     unhitValue.payoff = payoffStateB.cashflow_npv;
%                     
%                     nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%                     nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%                     nMFuture.payoff = payoffStateC.cashflow_npv;
%                     
%                 end
%                    
%            end
%            
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            
%            hitValue.npv = hitValue.payoff(Pricingidx);
%            unhitValue.npv = unhitValue.payoff(Pricingidx);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
%            out.hitValue = hitValue; 
%            out.unhitValue = unhitValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeVanillaGF(eqCOMDupireSpotGF,valueDate,vanillaParams)
%             % model schedule generation
%             expiryDate = H_Date(vanillaParams.params('maturityDate'));
%             scheduleSize = 1;
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
%             
%             expiredChance = 0;
%             aliveChance = 0;
%             expiry = eqCOMDupireSpotGF.localVolSurface.expiry;
%             volExpiry = expiry(find(expiry <= maturity));
%             % we include valueDate in the scheduel
%             volExpiry =[volExpiry;0];
%             
%             timeStep = volExpiry;
%             if strcmp(vanillaParams.params('dailyTimeStepsYN'),'YES')
%                 dailyTimeSteps = [maturity:-1:0]';
%                 timeStep = sort(union(timeStep,dailyTimeSteps),'ascend');
%                 
%             end
%             
%             timeStep = sort(union(timeStep,maturity),'ascend');
%             totalTimeStepSize = length(timeStep);
%             eventTimeIdx = zeros(scheduleSize,1);
%             
%             lastAliveExerciseIdx = 1;
%             for i= scheduleSize:-1:lastAliveExerciseIdx
%                 idx = find(timeStep == maturity);
%                 eventTimeIdx(i) = min(idx);
%             end
%            
%           %% grid generation based on basePrice
%             
%            
%           %% payoff at maturity
%            strike =  vanillaParams.params('strike');
%            modelStates = eqCOMDupireSpotGF.localVolSurface.Ks;
%            modelStatesSize = length(modelStates);
%            
%            %europeanValue
%            europeanValue.npv = 0;
%            europeanValue.payoff = zeros(modelStatesSize,1);
%            europeanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
%            europeanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            %americanValue
%            americanValue.npv = 0;
%            americanValue.payoff = zeros(modelStatesSize,1);
%            americanValue.payoffStates.cashflow = zeros(modelStatesSize,1);
%            americanValue.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            
%            % dummy nearest month future price
%            nMFuture.npv = 0;
%            nMFuture.payoff = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow = zeros(modelStatesSize,1);
%            nMFuture.payoffStates.cashflow_npv = zeros(modelStatesSize,1);
%            
%            % intermediate variables
%            payoffStateA.cashflow = zeros(modelStatesSize,1);
%            payoffStateA.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateB.cashflow = zeros(modelStatesSize,1);
%            payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%            
%            payoffStateC.cashflow = zeros(modelStatesSize,1);
%            payoffStateC.cashflow_npv = zeros(modelStatesSize,1);
%            
%            currentTime = timeStep(totalTimeStepSize);
%            currentTimeIdx = totalTimeStepSize;
%            currentNodeIdx = scheduleSize;
%            
%            comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%            
%            if strcmp(vanillaParams.params('callPutFlag'),'C')
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(comFwd(i) - strike,0);
%                     payoffStateB.cashflow(i) = max(comFwd(i) - strike,0);
%                end
%            else
%                for i=1:modelStatesSize
%                     payoffStateA.cashflow(i) = max(-comFwd(i) + strike,0);
%                     payoffStateB.cashflow(i) = max(-comFwd(i) + strike,0);
%                end
%            end
%            
%            payoffStateC.cashflow = comFwd;
%            
%            df_ep = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%            
%            % we don't discount forward price
%            for i=1:modelStatesSize
%                 payoffStateA.cashflow_npv(i) = payoffStateA.cashflow(i)*df_ep;
%                 payoffStateB.cashflow_npv(i) = payoffStateB.cashflow(i)*df_ep;
%                 payoffStateC.cashflow_npv(i) =  payoffStateC.cashflow(i)*1.0;
%            end
%            
%            
%            %process payoff at maturity
%             europeanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateA.cashflow;
%             europeanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateA.cashflow_npv ;
%             europeanValue.payoff = payoffStateA.cashflow_npv;
%             
%             americanValue.payoffStates.cashflow(:,currentNodeIdx) = payoffStateB.cashflow;
%             americanValue.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateB.cashflow_npv ;
%             americanValue.payoff = payoffStateB.cashflow_npv;
%             
%             nMFuture.payoffStates.cashflow(:,currentNodeIdx) = payoffStateC.cashflow;
%             nMFuture.payoffStates.cashflow_npv(:,currentNodeIdx) = payoffStateC.cashflow_npv ;
%             nMFuture.payoff = payoffStateC.cashflow_npv;
%             
%            lastTimeIdx = currentTimeIdx;
%            
%            
%            for i=scheduleSize:-1:lastAliveExerciseIdx
%                 % past schedule neglect
%                 startTimeIdx = eventTimeIdx(i);
%                 endTimeIdx = 1;  % valueDate
%                 
%                 endTimeIdx = endTimeIdx + 1;
%             %% induction start
%                 mesh = europeanValue.payoff;
%                 meshB = americanValue.payoff;
%                 for idx= startTimeIdx:-1:endTimeIdx
%                     mesh  = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),mesh);
%                     meshB = inductGFBackward(eqCOMDupireSpotGF,timeStep(idx),timeStep(idx-1),meshB);
%                     % one step backward induction end
%                     lastTimeIdx = lastTimeIdx -1;
%                     currentTime = timeStep(lastTimeIdx);
%                     
%                     %apply american option
%                     comFwd = eqCOMDupireSpotGF.CommoFwdContractM(currentTime,maturity);
%                     onePayoffD = eqCOMDupireSpotGF.zeroCurve.DF(currentTime/365.0);
%                     contiValueB  = meshB/onePayoffD;
%                     
%                     payoffStateB.cashflow = zeros(modelStatesSize,1);
%                     payoffStateB.cashflow_npv = zeros(modelStatesSize,1);
%                     if strcmp(vanillaParams.params('callPutFlag'),'C')
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( comFwd(j) - strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     else
%                         for j=1:modelStatesSize
%                             payoffStateB.cashflow(j) = max( -comFwd(j) + strike,0);
%                             payoffStateB.cashflow(j) = max(payoffStateB.cashflow(j),contiValueB(j)); 
%                         end
%                     end
%                     payoffStateB.cashflow_npv = payoffStateB.cashflow*onePayoffD;
%                     meshB = payoffStateB.cashflow_npv;
%                     
%                 end
%                 
%                 europeanValue.payoff = mesh;
%                 americanValue.payoff = meshB;
% %                 nMFuture.payoff = comFwd;
%               %% induction end
%                 
%                 % backward induction in the payoff script
%                 currentNodeIdx = currentNodeIdx -1;
%            end
%            
%            Pricingidx = eqCOMDupireSpotGF.localVolSurface.Pricingidx;
%            
%            europeanValue.npv = europeanValue.payoff(Pricingidx);
%            americanValue.npv = americanValue.payoff(Pricingidx);
%            nMFuture.npv = nMFuture.payoff(Pricingidx);
%            
%            out.europeanValue = europeanValue; 
%            out.americanValue = americanValue;
%            out.nMFuture = nMFuture; 
%         end
%         
%         function out = computeForwardMCPerExpiry(eqCOMDupireSpotGF,idxNow,maturity,timeScheduleInfo,strikes,dZ,NMC,params)
%             expiry = params.expiry;
%             
%             initX = ones(NMC,1);
%             
%             payoff = zeros(NMC,length(strikes));
%             volExpiry = expiry(find(expiry <= maturity));
%             volExpiry = [volExpiry; 0];
%             timeStep = sort(union(volExpiry,maturity),'ascend');
%             nextX = initX;
%             %divide time interval into volExpiry
%             for i=2:length(timeStep)
%                 startIdx = find(timeScheduleInfo.timeSchedule == timeStep(i-1));
%                 endIdx = find(timeScheduleInfo.timeSchedule == timeStep(i));
%                 dZStep = dZ(:,startIdx+1:endIdx);
%                 nextX = eqCOMDupireSpotGF.inductMCForward(timeStep(i-1),timeStep(i),startIdx,endIdx,timeScheduleInfo,nextX,dZStep,params);
%             end
%             
%             
%             for j=1:length(strikes)
%                 if(strikes(j) <= 1)
%                     payoff(:,j) = max(strikes(j) - nextX,0);
%                 else
%                     payoff(:,j) = max(nextX - strikes(j),0);
%                 end
%             end
%             out = mean(payoff);
%             
%         end
%         
%         function out = computeForwardMCMCPerExpiry(eqCOMDupireSpotGF,maturity,strikes,U,NMC,params)
%             expiry = params.expiry;
%             Ks = params.Ks;
%             initX = ones(NMC,1);
%             initXIdx = ones(NMC,1)*params.Pricingidx;
%             
%             payoff = zeros(NMC,length(strikes));
%             volExpiry = expiry(find(expiry <= maturity));
%             volExpiry = [volExpiry; 0];
%             timeStep = sort(union(volExpiry,maturity),'ascend');
%             nextX = initX;
%             nextXIdx = initXIdx;
%             for i=2:length(timeStep)
%                 mcmcOut = eqCOMDupireSpotGF.inductMCMCForward(timeStep(i-1),timeStep(i),nextX,nextXIdx,U(:,i-1),Ks);
%                 nextX = mcmcOut.nextX;
%                 nextXIdx = mcmcOut.nextXIdx;
%             end
%             
%             
%             for j=1:length(strikes)
%                 if(strikes(j) <= 1)
%                     payoff(:,j) = max(strikes(j) - nextX,0);
%                 else
%                     payoff(:,j) = max(nextX - strikes(j),0);
%                 end
%             end
%             out = mean(payoff);
%             
%         end
%         
%         function out = computeBackwardPDEOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             pdeOTMPrices =  zeros(length(expiry),length(strikes));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
% %                 strikePerExpiry = params.fwdMoneyness(i,:);
%                 for j=1:length(strikePerExpiry)
%                     pdeOTMPrices(i,j) = eqCOMDupireSpotGF.computeBackwardPDEOne(expiry(i),strikePerExpiry(j),params);
%                 end
%             end
%             
%             out = pdeOTMPrices;
%             
%         end
%         
%         
%         % compute forward mc(euler scheme) for 8*21 vanilla otm products
%         % generate time steps from the enddate backwards 
%         
%         function out = computeForwardMCOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             % we generate the schedule from the enddate
%             timeStepSize = params.mcOneTimeStep;
%             MCTimeStep = round(expiry(length(expiry))/timeStepSize) +1; 
%             dummyTimeSchedule = zeros(MCTimeStep,1);
%             
%             dummyN = 0;
%             for i=1:MCTimeStep
%                 dummyTimeSchedule(i)= expiry(length(expiry)) -timeStepSize*(i-1);
%                 if dummyTimeSchedule(i) < 0
%                     break;
%                 end
%                 dummyN = dummyN +1;
%                 
%             end
%             
%             timeSchedule = zeros(dummyN,1);
%             for i=1:dummyN
%                 timeSchedule(i)= dummyTimeSchedule(i);
%             end
%             
%             volExpiry = [expiry; 0];
%             
%             newTimeSchedule = sort(union(volExpiry,timeSchedule),'ascend');
%             
%             dTSchedule = zeros(length(newTimeSchedule)-1,1);
%             dTSchedule365 = zeros(length(newTimeSchedule)-1,1);
%             for i=length(newTimeSchedule)-1:-1:1
%                 dTSchedule(i) = newTimeSchedule(i+1) - newTimeSchedule(i);
%                 dTSchedule365(i) = (newTimeSchedule(i+1) - newTimeSchedule(i))/365.0;
%             end
%             
%             % find the vol expiry points
%             
%             volKnotIdx = zeros(length(expiry),1);
%             for i=1:length(expiry)
%                 volKnotIdx(i) = find(newTimeSchedule==expiry(i));
%                 
%             end
%             
%             timeScheduleInfo.timeSchedule = newTimeSchedule;
%             timeScheduleInfo.dTSchedule = dTSchedule;
%             timeScheduleInfo.dTSchedule365 = dTSchedule365;
%             timeScheduleInfo.volKnowIdx = volKnotIdx;
%             
%             
%             % schedule create end
% %             NMC = params.NMC;
%             NMC = eqCOMDupireSpotGF.modelParams('NMC'); 
% %             Ps = sobolset(length(dTSchedule));
%             
%             Ps = sobolset(length(dTSchedule),'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             Q.reset;
%             U = qrand(Q,NMC);
%             dZ0 = zeros(size(U,1),size(U,2));
% %             dZ = norminv(U);
%             % Use Accurate Normal Inverse Method , ASA241 by Michael Wichura
%             % generate random number from the end
%             for i=1:size(U,1)
%                 for j=1:size(U,2)
%                     dZ0(i,j) = r8_normal_01_cdf_inverse(U(i,j));
%                 end
%             end
%             
%            % apply brownian bridge
%            dZ1 = eqCOMDupireSpotGF.bbgenerator(dZ0);
%            
%            % first dZ is dummy , 0
%            dZ = zeros(size(dZ1,1),size(dZ1,2) + 1);
%            dZ(:,2) = dZ1(:,1);
%            for i=3:size(dZ1,2)+1
%                 dZ(:,i) = dZ1(:,i-1) -  dZ1(:,i-2);
%            end
%            
%             mcOTMPrices =  zeros(length(expiry),size(strikes,2));
% %             mcOTMPrices =  zeros(length(expiry),length(strikes));
%             
%             mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
%                 mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCPerExpiry(i,expiry(i),timeScheduleInfo,strikePerExpiry,dZ,NMC,params);
%                 for j=1:size(strikes,2)
%                     mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
%                 end
%             end
%             
%             out = mcOTMPrices;
%             
%         end
%         
%         
%         % compute forward mcmc(Markov Chain Monte Carlo scheme) for 8*21 vanilla otm products
%         % generate time steps using only events date 
%         
%         function out = computeForwardMCMCOTMTotal(eqCOMDupireSpotGF,params)
%             expiry = params.expiry;
%             strikes = params.marketStrikes;
%             
%             %mc random number initialize
% %             NMC = 2^14;
%             NMC = params.NMC;
%             
%             MCTimeStep = length(expiry); % -1 because 0
% 
%             Ps = sobolset(MCTimeStep,'Skip',NMC);
% %             Ps = scramble(Ps,'MatousekAffineOwen');
%             Q = qrandstream(Ps);
%             
%             reset(Q);
%             U0 = qrand(Q,NMC);
%             U = zeros(NMC,MCTimeStep);
%             
%             % first random numbers goes to the enddate..
%             for i=1:size(U,2)
%                 U(:,i) = U0(:,size(U,2)-(i-1));
%             end            
%             
%             mcOTMPrices =  zeros(length(expiry),size(strikes,2));
%             mcOTMPricesPerExpiry = zeros(1,size(strikes,2));
%             for i=1:length(expiry)
%                 strikePerExpiry = strikes(i,:);
%                 mcOTMPricesPerExpiry = eqCOMDupireSpotGF.computeForwardMCMCPerExpiry(expiry(i),strikePerExpiry,U,NMC,params);
%                 for j=1:size(strikes,2)
%                     mcOTMPrices(i,j) = mcOTMPricesPerExpiry(j);
%                 end
%             end
%             
%             out = mcOTMPrices;
%             
%         end
%         
%         function out = computeForwardImpVolPerExpiry(eqCOMDupireSpotGF,maturity,strikes,params)
%             
%             expiry = params.expiry;
%             lastP = zeros(1,length(params.Ks));
%             lastP(params.Pricingidx) = 1.0;
%             newP = zeros(1,length(params.Ks));
%             
%             
%             idx = find(maturity <= expiry);
%             if isempty(idx)
%                 idxNow = length(expiry);
%             else
%                 idxNow = min(idx);
%             end
%             
%             
%             %relevent volProxy
%             volProxy = eqCOMDupireSpotGF.localVolSurface.volProxy(idxNow,:);
%             
%             %relevent probMesh and dTDays
%             dTDays = 0;
%             if idxNow~=1
%                 probMesh = eqCOMDupireSpotGF.localVolSurface.probMesh(idxNow-1,:);
%                 lastP = probMesh;
%                 dTDays = (maturity - expiry(idxNow-1));
%             else % idxNow == 1
%                 dTDays = maturity;
%             end
%             
%             
%             ipos = eqCOMDupireSpotGF.localVolSurface.ipos(idxNow,:);
%             Ks = eqCOMDupireSpotGF.localVolSurface.Ks;
%             %filling proxy grid vol
%             proxy = zeros(length(lastP),1);
%             proxy = eqCOMDupireSpotGF.GetLocalVolFromProxy(volProxy,Ks,ipos,idxNow);
% 
%             %filling end
%             size = length(lastP);
%             a = zeros(size,1);
%             b = zeros(size,1);
%             c = zeros(size,1);
%             b_rg = zeros(size,1);
% 
%             dK = Ks(2) - Ks(1);
%             dT = dTDays/365.0;
% 
%             dK2 = dK*dK;
%             for i=1:size
%                 a(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b(i) = 1 + dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 b_rg(i) = dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%                 c(i) = -0.5*dT*proxy(i)*proxy(i)*Ks(i)*Ks(i)/dK2;
%             end
% 
%             a(1) = 0;
%             c(size) = 0;
% 
%             % absorbing boundary condition
%             c(1) = 0;
%             a(size) = 0;
% 
%             b(1) = 1;
%             b(size) = 1;
% 
%             % to create tridiagonal matrix
% 
%             a1 = a(2:size);
%             c1 = c(1:size-1);
% 
%             b_rg(1) = 0;
%             b_rg(size) = 0;
% 
%             A_rg = full(gallery('tridiag',-a1*1.0/dT,-b_rg*1.0/dT,-c1*1.0/dT));
%             A_exp1 = eqCOMDupireSpotGF.expmReghai(A_rg,dT);
%             newP = lastP*A_exp1;
%             
%             
%             impVol = zeros(1,length(strikes));
%             
%             %meshmatrixC(i,j) : max(x_i - k_j,0)
%             %meshmatrixP(i,j) : max(-x_i + k_j,0)
%             % payoff at maturity
%             modelX = params.Ks;
%             modelXSize = length(params.Ks);
%             strikeSize = length(strikes);
%             meshC = zeros(modelXSize,strikeSize);
%             meshP = zeros(modelXSize,strikeSize);
%             
%             for i=1:modelXSize
%                 for j=1:strikeSize
%                     meshC(i,j) = max(modelX(i) - strikes(j),0);
%                     meshP(i,j) = max(strikes(j) - modelX(i),0);
%                 end
%             end
%             
%             cValue = zeros(1,strikeSize);
%             pValue = zeros(1,strikeSize);
%             
%             impVolC = zeros(1,strikeSize);
%             impVolP = zeros(1,strikeSize);
%             impVol  = zeros(1,strikeSize);
%             cValue = newP*meshC;
%             pValue = newP*meshP;
%             
%             out.cValue = cValue;
%             out.pValue = pValue;
%             % use jackel's Lets Be Rational compiled code(mex) for price and implied vol calculation
%             % blackVolLBR
%             % blackPrice
%             
%             for i=1:strikeSize
%                 K = strikes(i);
%                 impVolP(i) = blackVolLBR(pValue(i),1.0,K, maturity/365.0,-1);
%                 impVolC(i) = blackVolLBR(cValue(i),1.0,K, maturity/365.0,1);
%                 if K <= 1.0
%                      impVol(i) = impVolP(i);
%                 else
%                      impVol(i) = impVolC(i);
%                 end
%             end
%             
%             out.impVolP = impVolP;
%             out.impVolC = impVolC;
%             out.impVol = impVol;
%             out.probMeshInt = newP;
%             
%         end
%         
%         function out = computeForwardImpVol(eqCOMDupireSpotGF,params,targetExpiry,targetStrikes)
% %             expiry = params.expiry;
% %             strikes = params.marketStrikes;
% %             
% %             %mc random number initialize
% % %             NMC = 2^14;
% %             NMC = params.NMC;
% %             
% %             MCTimeStep = length(expiry); % -1 because 0
% % 
% %             Ps = sobolset(MCTimeStep,'Skip',NMC);
% % %             Ps = scramble(Ps,'MatousekAffineOwen');
% %             Q = qrandstream(Ps);
% %             
% %             reset(Q);
% %             U0 = qrand(Q,NMC);
% %             U = zeros(NMC,MCTimeStep);
% %             
% %             % first random numbers goes to the enddate..
% %             for i=1:size(U,2)
% %                 U(:,i) = U0(:,size(U,2)-(i-1));
% %             end            
% %             
%             targetImpVolSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             cValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             pValueSurface = zeros(length(targetExpiry),size(targetStrikes,2));
%             probMeshInt = zeros(length(targetExpiry),size(targetStrikes,2));
%             targetImpVolPerExpiry = zeros(1,size(targetStrikes,2));
%             cValue = zeros(1,size(targetStrikes,2));
%             pValue = zeros(1,size(targetStrikes,2));
%             probMeshIntPerExpiry = zeros(1,size(targetStrikes,2));
%             
%             for i=1:length(targetExpiry)
%                 strikePerExpiry = targetStrikes(i,:);
%                 outDummy = eqCOMDupireSpotGF.computeForwardImpVolPerExpiry(targetExpiry(i),strikePerExpiry,params);
%                 targetImpVolPerExpiry = outDummy.impVol;
%                 cValue = outDummy.cValue;
%                 pValue = outDummy.pValue;
%                 probMeshIntPerExpiry = outDummy.probMeshInt;
%                 
%                 for j=1:size(targetStrikes,2)
%                     targetImpVolSurface(i,j) = targetImpVolPerExpiry(j);
%                     cValueSurface(i,j) = cValue(j);
%                     pValueSurface(i,j) = pValue(j);
%                     probMeshInt(i,j) = probMeshIntPerExpiry(j);
%                 end
%             end
%             
%             out.impVolSurface = targetImpVolSurface;
%             out.cValueSurface = cValueSurface;
%             out.pValueSurface = pValueSurface;
%             out.probMeshInt = probMeshInt;
%         end
%         
%         %generate marginal(cumulative distribution) at each event
%         %date(per expiry)
%         %using conditional transition Probability matrix
%         function out = generateMarginalPerExpiry(eqCOMDupireSpotGF,condProbPerExpiry,Ks)
% %             Ks = params.Ks;
% %             nPoints = length(params.Ks);
%             
%             nPoints = length(Ks);
%             condProb = condProbPerExpiry;
%             Qa = zeros(length(Ks),length(Ks));
%             cumM = 0;
%             cumM2 = 0;
%             %first row dealt separately
%             Qa(1,1) = condProb(1,1);
%             cumM2 = 0.0;
%             for j=2:nPoints
%                 cumM2 = cumM2 + condProb(1,j);
%                 Qa(1,j) = Qa(1,1) + cumM2;
%             end
%             
%             for i=2:nPoints
%                 cumM = 0.0;
%                 Qa(i,i) = condProb(i,i);
%                 for j=i-1:-1:1
%                     Qa(i,i) = Qa(i,i) + condProb(i,j);
%                 end
% 
%                 cumM = 0.0;
%                 for j=i-1:-1:1
%                     cumM = cumM + condProb(i,j+1);
%                     Qa(i,j) = Qa(i,i) - cumM;
%                 end
% 
%                 cumM2 = 0.0;
%                 for j=i+1:nPoints
%                     cumM2 = cumM2 + condProb(i,j);
%                     Qa(i,j) = Qa(i,i) + cumM2;
%                 end
%             end
%             
%             out = Qa;
%         end
%         
%         %generate marginal(cumulative distribution) for all eventdates
%         %date(per expiry)
%         %using conditional transition Probability matrix
%         
%         function marginal = generateMarginal(eqCOMDupireSpotGF,condProb,params)
%             Ks = params.Ks;
%             marginal = zeros(params.expirySize,length(Ks),length(Ks));
%             condProbPerExpiry = zeros(length(Ks),length(Ks));
%             marginalPerExpiry = zeros(length(Ks),length(Ks));
%             for i=1:params.expirySize
%                 condProbPerExpiry(:,:) = condProb(i,:,:);
% %                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,params);
%                 marginalPerExpiry = eqCOMDupireSpotGF.generateMarginalPerExpiry(condProbPerExpiry,Ks);
%                 marginal(i,:,:) = marginalPerExpiry(:,:);
%             end
%         end
%         
%         
%         % Target function for FixedPointIteration Optimizer
%         
%         function  out  = TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams)
%             idxNow = optParams.idxNow;
%             if  idxNow ~= 1 mkt_dT = optParams.params.expiry(idxNow) - optParams.params.expiry(idxNow-1);else mkt_dT =  optParams.params.expiry(1);end;
%             mkt_dT=mkt_dT/365.0;
%             
%             
%           %% Solve  Forward PDE(GF) One time
%             outGF = eqCOMDupireSpotGF.Solve1DGF(tvar,mkt_dT,optParams.dK,optParams.lastProb,optParams);
%             
%             newProb = outGF.newProb;
%             optParams.meshProb = newProb;
%            %% PDE(GF) Solve End
%            % find the implied vol of dupire model
%            if strcmp(eqCOMDupireSpotGF.modelParams('UseLetsBeRational'),'YES')
%                target = InterpolateTargetStrikeVolFPI(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
%            else
%                target = InterpolateTargetStrikeVolFPIBisec(eqCOMDupireSpotGF,idxNow,newProb,optParams); 
%            end
%            marketImpVol = optParams.params.blackVol;
%            expiry = optParams.params.expiry;
%            strike = optParams.params.strike;
%            
%            out.newLocalVol = zeros(length(optParams.strikeSize),1);
%            out.localVolError = zeros(length(optParams.strikeSize),1);
%            out.impVolError =  zeros(length(optParams.strikeSize),1);
%            out.modelImpVol = zeros(length(optParams.strikeSize),1);
%            out.marketImpVol = zeros(length(optParams.strikeSize),1);
%            out.interpOTMPrices = zeros(length(optParams.strikeSize),1);
%            out.interpBlackPrices = zeros(length(optParams.strikeSize),1);
%            out.priceRe = zeros(length(optParams.strikeSize),1);
%            
%            for i=1:optParams.strikeSize
%                 out.newLocalVol(i) = tvar(i)*(marketImpVol(idxNow,i))/target.vols(i);
%                 out.localVolError(i) = out.newLocalVol(i) - tvar(i);
%                 out.modelImpVol(i) = target.vols(i);
%                 out.marketImpVol(i) =  marketImpVol(idxNow,i);
%                 out.impVolError(i) = out.modelImpVol(i) - out.marketImpVol(i);
%                 out.interpOTMPrices(i) = target.interpOTMPrices(i);
%                 out.interpBlackPrices(i) = target.interpBlackPrices(i);
%                 out.priceRe(i) = out.interpOTMPrices(i)/optParams.params.blackOTMPrices(idxNow,i)-1.0;
%            end
%            
%            % for shorter tenor we only fit for the restricted interval
%             if idxNow <= optParams.params.numOfCutoffTenor
%                 for i=1:optParams.lowerIdx-1
%                     out.newLocalVol(i) = out.newLocalVol(optParams.lowerIdx);
%                 end
% 
%                 for i=optParams.upperIdx+1:optParams.strikeSize
%                     out.newLocalVol(i) = out.newLocalVol(optParams.upperIdx);
%                 end
%             end
%             
%             out.volErrorTotal = 0.0;
%             out.priceReTotal = 0.0;
%             %for one month we only fit betwee lower and upper
%             dummyN = 0;
%             if idxNow <= optParams.params.numOfCutoffTenor
%                 for i=optParams.lowerIdx:optParams.upperIdx
%                     out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
%                     out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
%                     dummyN = dummyN +1;
%                 end
%             else
%                 for i=1:length(out.impVolError)
%                     out.volErrorTotal = out.volErrorTotal + out.impVolError(i)*out.impVolError(i);
%                     out.priceReTotal = out.priceReTotal + out.priceRe(i)*out.priceRe(i);
%                     dummyN = dummyN +1;
%                 end
%             end
%             out.volErrorTotal = sqrt(out.volErrorTotal/dummyN);
%             out.priceReTotal = sqrt(out.priceReTotal/dummyN);
%             
%         end
%         
%          % Fixed Point Iteration Optimizer
%          % start from implied vol find the local vol using Reghai's fixed
%          % point iteration optimization
%          % LocalVol(n+1) = LocalVol(n)*(marketImpVol)/ModelImplVol(n);
%          
%         function [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter] = FixedPointIter(eqCOMDupireSpotGF,targetfunc,tvar,tol,maxIter,optParams)
%             
%             n= 0;
%             errorTry = 1000.0; % Initial ErrorBound make it big enough
%             tvarOld = tvar;
%             if optParams.params.targetType == 1
%                 while (errorTry > tol) && (n < maxIter)
%                     out = targetfunc(tvar);
%                     tvarOld = tvar;
%                     % reduce implied vol error
%                     errorTry = out.volErrorTotal;
%                     tvar = out.newLocalVol;
%                     n = n+1;
%                 end
%                 x = tvar;
%                 finalOut = targetfunc(tvarOld);
%                 fval = finalOut.volErrorTotal;
%                 marketImpVol = finalOut.marketImpVol;
%                 modelImpVol  = finalOut.modelImpVol;
%                 interpBlackPrices = finalOut.interpBlackPrices;
%                 interpOTMPrices = finalOut.interpOTMPrices;
%                 volError     = finalOut.impVolError;
%                 volErrorTotal = finalOut.volErrorTotal;
%                 priceRe      = finalOut.priceRe;
%                 priceReTotal = finalOut.priceReTotal;
%                 nIter = n;
%             else
%                 while (errorTry > tol) && (n < maxIter)
%                     out = targetfunc(tvar);
%                     tvarOld = tvar;
%                     % reduce price relative error
%                     errorTry = out.priceReTotal;
%                     tvar = out.newLocalVol;
%                     n = n+1;
%                 end
%                 x = tvar;
%                 finalOut = targetfunc(tvarOld);
%                 fval = finalOut.volErrorTotal;
%                 marketImpVol = finalOut.marketImpVol;
%                 modelImpVol  = finalOut.modelImpVol;
%                 interpBlackPrices = finalOut.interpBlackPrices;
%                 interpOTMPrices = finalOut.interpOTMPrices;
%                 volError     = finalOut.impVolError;
%                 volErrorTotal = finalOut.volErrorTotal;
%                 priceRe      = finalOut.priceRe;
%                 priceReTotal = finalOut.priceReTotal;
%                 nIter = n;
%             end
%             
%         end
%         
%         % Fixed Local Vol per expiry by bootstrapping
%         
%         function bootStrapOut = FindVolProxy(eqCOMDupireSpotGF,idxNow,dT,lastProb,params)
%             %initialize calibration parameters
%             
%             optParams.idxNow = idxNow;
%             optParams.dT = dT;
%             optParams.params = params;
%             optParams.ipos = zeros(params.strikeSize,1);
%             optParams.Ks = zeros(params.Ns,1);
%             optParams.Ks = params.Ks;
%             optParams.strikeSize = params.strikeSize;
%             optParams.meshSize = params.Ns;
%             
%             optParams.meshProb = zeros(1,params.Ns);
%             
%             optParams.proxy = zeros(params.Ns,1);
%             optParams.dK = params.Ks(2) - params.Ks(1);
%             
%             optParams.marketStrikes = zeros(params.strikeSize,1);
%             optParams.blackOTMPrices = params.blackOTMPrices;
%             optParams.callCurve = zeros(params.strikeSize,1);
%             optParams.putCurve = zeros(params.strikeSize,1);
%             
%             for i=1:params.strikeSize
%                 optParams.marketStrikes(i) = params.fwdMoneyness(idxNow,i);
%             end
%                 
% %             if strcmp(params.volSurface.params('moneyNessType'),'spotMoneyNess')
% %                 % market strike points in fwd PDE grid
% %                 fwdFactor = eqCOMDupireSpotGF.FwdFactor(params.expiry(idxNow));
% %                 for i=1:params.strikeSize
% %                     optParams.marketStrikes(i) = params.strike(i)/fwdFactor;
% %                 end
% %             else
% %                 for i=1:params.strikeSize
% %                     optParams.marketStrikes(i) = params.strike(i);
% %                 end
% %             end
%             
%             % positions to separate vol proxy regimes
%             for i=1:params.strikeSize
%                 optParams.ipos(i) = floor( (optParams.marketStrikes(i) - optParams.Ks(1))/optParams.dK) + 1;  % add default 1
%             end
% %             optParams.ipos(params.strikeSize) = optParams.meshSize;
%             
%             optParams.lastProb = lastProb;
%             
%             % exception for one month maturity for restricted interval
%             
%             lowerCutoff = params.lowerCutoffVector(idxNow);
%             upperCutoff = params.upperCutoffVector(idxNow);
%             numOfCutoffTenor = params.numOfCutoffTenor; 
%             
%             lowerIdx = 1;
%             upperIdx = params.strikeSize;
%             
%             if idxNow <= numOfCutoffTenor
%                 lowerIdx = min(find(optParams.marketStrikes >= lowerCutoff));
%                 upperIdx = max(find(optParams.marketStrikes <= upperCutoff));
%             end
%             
%             optParams.lowerIdx = lowerIdx;
%             optParams.upperIdx = upperIdx;
%             
%             for i=1:params.strikeSize
%                 tvar(i) = params.blackVol(idxNow,i);
%                 lb(i) = 0.01;
%                 ub(i) = 1.0;
%             end
%             
%             for i=1:lowerIdx-1
%                 tvar(i) = tvar(lowerIdx);
%             end
%             
%             for i=upperIdx+1:params.strikeSize
%                 tvar(i) = tvar(upperIdx);
%             end
% 
%             targetfunc = @(tvar) TargetFunctionVolProxyFPI(eqCOMDupireSpotGF,tvar,optParams);
%                 
%             
% %             tol = 0.0001;
% %             maxIter = 1000;
%             
%             tol = params.tol;
%             maxIter = params.maxIter;
%             
%             [x, fval,marketImpVol,modelImpVol,interpBlackPrices, interpOTMPrices,volError,volErrorTotal, priceRe, priceReTotal,nIter]  = eqCOMDupireSpotGF.FixedPointIter(targetfunc,tvar,tol,maxIter,optParams);
%             
%             for i=1:params.strikeSize
%                 bootStrapOut.volProxy(i) = x(i);
%                 bootStrapOut.ipos(i) = optParams.ipos(i);
%             end
%             
%             % for later use
%             bootStrapOut.marketStrikes = optParams.marketStrikes;
%             
%             bootStrapOut.volError = volError;
%             bootStrapOut.marketImpVol = marketImpVol;
%             bootStrapOut.modelImpVol = modelImpVol;
%             bootStrapOut.interpBlackPrices = interpBlackPrices;
%             bootStrapOut.dupireOTMPrices = interpOTMPrices;
%             
%             bootStrapOut.nIter = nIter;
%             
%             out = eqCOMDupireSpotGF.Solve1DGF(bootStrapOut.volProxy,dT,optParams.dK,lastProb,optParams);
%             
%             bootStrapOut.lastProb = out.newProb;
%             bootStrapOut.condProb = out.condProb;
%             
%         end
%         
%         function out = CalibrateToVolSurface(eqCOMDupireSpotGF,black,calibrationFlag)
%             
%           % Calculate Black OTM Prices as calibration target
%             volSurface = black.volSurface;
%             
%             expiry = volSurface.maturities;
%             strike  = volSurface.strikes;
%             spot =  black.spot.params('rawData');
%             expirySize = length(expiry);
%             strikeSize = length(strike);
%             
%             blackOTMPrices = zeros(expirySize,strikeSize);
%             blackOTMPrices2 = zeros(expirySize,strikeSize);
%             blackVol = zeros(expirySize,strikeSize);
%             blackOTMVega = zeros(expirySize,strikeSize);
%             
%             % we add blackvol as matrix so that in the calibation we don't
%             % need to do 2D interpolation again and again!!
%             % also for the target black price too!!
%             if strcmp(volSurface.params('moneyNessType'),'fwdMoneyNess')
%                 for k=1:expirySize
%     %                 fwd = black.Fwd(spot,expiry(k));
%     %                 df = black.zeroCurve.DF(expiry(k)/365.0);
%                     for j=1:strikeSize
%                         blackVol(k,j) = volSurface.volSurfaceR(k,j);
%                         fwdMoneyness(k,j) = strike(j);
%                         
%                         if fwdMoneyness(k,j) <= 1
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
%                         else
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
%                         end
%                     end
%                 end
%             else
%                 for k=1:expirySize
%     %                 fwd = black.Fwd(spot,expiry(k));
%     %                 df = black.zeroCurve.DF(expiry(k)/365.0);
%                     for j=1:strikeSize
%                         blackVol(k,j) = volSurface.volSurfaceR(k,j);
%                         if strike(j) <= 0
%                             dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
%                             fwdMoneyness(k,j) = exp(norminv(-1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
%                         else
%                             dummyStd = blackVol(k,j)*sqrt(expiry(k)/365.0);
%                             fwdMoneyness(k,j) = exp(-1.0*norminv(1.0*strike(j),0,1)*dummyStd+0.5*dummyStd*dummyStd);
%                         end
% 
%                         if fwdMoneyness(k,j) <= 1
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'P');
%                         else
%                             blackOTMPrices(k,j) = black.BlackVanillaFwd(expiry(k),1.0,fwdMoneyness(k,j),blackVol(k,j),'C');
%                         end
%                     end
%                 end
%                 
%             end
%                 
% %             if strcmp(volSurface.params('moneyNessType'),'spotMoneyNess')
% %                 for k=1:expirySize
% %                     fwd = black.Fwd(spot,expiry(k));
% %                     df = black.zeroCurve.DF(expiry(k)/365.0);
% %                     fwdFactor = eqCOMDupireSpotGF.FwdFactor(expiry(k));
% %                     for j=1:strikeSize
% %                         blackVol(k,j) = volSurface.vol(expiry(k),strike(j));
% %                         fwdStrike = strike(j)/fwdFactor;
% %                         if fwdStrike <= 1.0
% % %                             blackOTMPrices(k,j) = black.BlackVanilla(expiry(k),expiry(k),spot,spot*strike(j),volSurface.vol(expiry(k),strike(j)),'P')/fwd/df;
% %                             blackOTMPrices(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,-1);
% %                             blackOTMPrices2(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,-1);
% %                             blackOTMVega(k,j) = BlackFwdVega(1,fwdStrike,blackVol(k,j),expiry(k)/365.0);
% %                         else
% % %                             blackOTMPrices(k,j)  = black.BlackVanilla(expiry(k),expiry(k),spot,spot*strike(j),volSurface.vol(expiry(k),strike(j)),'C')/fwd/df;
% %                             blackOTMPrices(k,j)  = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,1);
% %                             blackOTMPrices2(k,j) = blackPrice(1.0,spot*strike(j)/fwd,blackVol(k,j),expiry(k)/365.0,1);
% %                             blackOTMVega(k,j) = BlackFwdVega(1,fwdStrike,blackVol(k,j),expiry(k)/365.0);
% %                             
% %                         end
% %                     end
% %                 end
% %             else
% %                 for k=1:expirySize
% %                     fwd = black.Fwd(spot,expiry(k));
% %                     df = black.zeroCurve.DF(expiry(k)/365.0);
% %                     for j=1:strikeSize
% %                         blackVol(k,j) = volSurface.vol(expiry(k),strike(j));
% %                         if strike(j) <= 1
% %                             blackOTMPrices(k,j) = black.BlackVanilla(expiry(k),expiry(k),spot,fwd*strike(j),volSurface.vol(expiry(k),strike(j)),'P')/fwd/df;
% %                         else
% %                             blackOTMPrices(k,j)  = black.BlackVanilla(expiry(k),expiry(k),spot,fwd*strike(j),volSurface.vol(expiry(k),strike(j)),'C')/fwd/df;
% %                         end
% %                     end
% %                 end
% %             end
% %             
%             
%             params.volSurface = volSurface;
%             params.expiry = expiry;
%             params.strike = strike;
%             
%             params.blackOTMPrices = blackOTMPrices;
%             params.blackVol = blackVol;
%             params.fwdMoneyness = fwdMoneyness;
%             
%             % additional model params
%             eqCOMDupireSpotGF.fwdMoneyness = fwdMoneyness;
%             
%             
%             params.expirySize = expirySize;
%             params.strikeSize =  strikeSize;
%             
%             % optimizer setting
%             params.tol = eqCOMDupireSpotGF.modelParams('tol');
%             params.maxIter = eqCOMDupireSpotGF.modelParams('maxIter');
%             
%             % vol proxy calibration grid setting
%             
%             params.Kmin = eqCOMDupireSpotGF.modelParams('Kmin');
%             params.Kmax = eqCOMDupireSpotGF.modelParams('Kmax');
%             params.Ns   = eqCOMDupireSpotGF.modelParams('Ns');
%             params.dK   = eqCOMDupireSpotGF.modelParams('dK');
%             params.Ks   = eqCOMDupireSpotGF.modelParams('Ks');
%             
%             % current spot on the grid
%             params.Pricingidx = floor((1.0-params.Ks(1))/params.dK) + 1;
%             
%             % targetType : 1 = implied vol diff
%             %              2 = price relative error  
%             params.targetType = 1;      
%             
%             % GFType : 1 = matrix exponential moler higham
%             %          2 = one step matrix exponential moler higham & multiplication from
%             %          3 = one step GF & multiplication from right Andreasen & Huge
%             
%             params.GFType = eqCOMDupireSpotGF.modelParams('GFType');
%             
%             params.backGFType = eqCOMDupireSpotGF.modelParams('backGFType');
%             
%             params.mcOneTimeStep = eqCOMDupireSpotGF.modelParams('mcOneTimeStep');
%             
%             params.pricingSchemeType = eqCOMDupireSpotGF.modelParams('pricingSchemeType');
%             params.NMC = eqCOMDupireSpotGF.modelParams('NMC');
%             
%             % backGFType : 1 = forward FD(GF)
%             %              2 = implicit FD
%             
%             params.oneStepDt = 1.0/365.0;
%             
%             
%             
%             params.numOfCutoffTenor = eqCOMDupireSpotGF.modelParams('numOfCutoffTenor');
%             
%             params.lowerCutoffVector = eqCOMDupireSpotGF.modelParams('lowerCutoffVector');
%             params.upperCutoffVector = eqCOMDupireSpotGF.modelParams('upperCutoffVector');
%             
%             
%             %output vol proxy
%             out.fwdMoneyness = fwdMoneyness;
%             out.expiry = params.expiry;
%             out.strike = params.strike;
%             
%             out.volProxy = zeros(params.expirySize,params.strikeSize);
%             out.ipos = zeros(params.expirySize,params.strikeSize);
%             out.probMesh = zeros(params.expirySize,params.Ns);
%             
%             out.condProb = zeros(params.expirySize,params.Ns,params.Ns);
%             
%             
%             %initialze the intial probability distribution in the grid
%             initProb = zeros(1,params.Ns);
%             initProb(params.Pricingidx) = 1.0;
%             
%             out.blackOTMPrices = blackOTMPrices;
%             out.blackOTMPrices2= blackOTMPrices2;
%             
%             out.blackVol = blackVol;
%             out.blackOTMVega = blackOTMVega;
%             
%             out.marketImpVol = zeros(expirySize,strikeSize);
%             out.modelImpVol  = zeros(expirySize,strikeSize);
%             
%             out.interpBlackPrices = zeros(expirySize,strikeSize);
%             out.dupireOTMPrices = zeros(expirySize,strikeSize);
%             
%             out.dupireBlackVol = zeros(expirySize,strikeSize);
%             
%             out.volError = zeros(expirySize,strikeSize);
%             out.priceRe = zeros(expirySize,strikeSize);
%             out.priceReOrig = zeros(expirySize,strikeSize);
%             
%             out.marketStrikes = zeros(expirySize,strikeSize);
%             
%             out.Ks =  params.Ks;
%             out.Pricingidx = params.Pricingidx;
%             out.nIter = zeros(expirySize,1);
%             out.localVol = zeros(expirySize,length(out.Ks));
%             out.rmseTotal=0.0;
%             out.fwdVolMseTotal = 0.0;
%             
%             
%             calibrationType = 'bootstrap';
%             if nargin < 3
%                 calibrationType = 'bootstrap'; 
%             else
%                 calibrationType = calibrationFlag;
%             end
%             
%             switch calibrationType
%                 case 'bootstrap'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     tic
%                     % assigning the calibrated volProxy to the local vol
%                     % surface
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
%                     
%                     % generate marginal(cumulative) distribution for mcmc
%                     % simulation
%                     
%                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
%                     
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = params.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = params.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     
%                     % calibration gives model schedule, for later use
%                     % general products such as autocallables..
%                     
%                     eqCOMDupireSpotGF.modelSchedule = eqCOMDupireSpotGF.localVolSurface.expiry;
%                     
%                     params.marketStrikes = out.marketStrikes;
%                     
%                     toc
%                     
% %                     calculate backward scheme for given target
% %                     compareforwardMCFlag = 1 for one step euler scheme
% %                     compareforwardMCFlag = 2 for daily euler scheme
% %                     
% %                     compareforwardMCFlag = 3 for markov chain monte carlo
%                      
%                     
%                     pricingSchemeType = params.pricingSchemeType; 
%                     
%                     tic
%                     
%                     if pricingSchemeType == 1
%                         out.pdeOTMPrices = eqCOMDupireSpotGF.computeBackwardPDEOTMTotal(params);
% 
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backReOrig = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backPriceOrigRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.pdeOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 out.backReOrig(i,j) = out.pdeOTMPrices(i,j)/out.blackOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.pdeOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backPriceOrigRmseTotal = out.backPriceOrigRmseTotal + out.backReOrig(j,k)*out.backReOrig(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backPriceOrigRmseTotal= sqrt(out.backPriceOrigRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%   
%                     elseif pricingSchemeType == 3
%                         out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCOTMTotal(params);
% 
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%                     
%                     elseif pricingSchemeType == 4
%                         out.mcOTMPrices = eqCOMDupireSpotGF.computeForwardMCMCOTMTotal(params);
%                         out.backRe = zeros(expirySize,strikeSize);
%                         out.backwardVol = zeros(expirySize,strikeSize);
%                         out.backVolError = zeros(expirySize,strikeSize);
% 
%                         out.backPriceRmseTotal = 0.0;
%                         out.backVolMseTotal = 0.0;
%                         for i=1:expirySize
%                             for j=1:strikeSize
%                                 out.backRe(i,j) = out.mcOTMPrices(i,j)/out.dupireOTMPrices(i,j)-1;
%                                 if out.marketStrikes(i,j) <=1.0
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,-1);
%                                 else
%                                     out.backwardVol(i,j) = blackVolLBR(out.mcOTMPrices(i,j),1,out.marketStrikes(i,j),params.expiry(i)/365.0,1);
%                                 end
%                                 out.backVolError(i,j) = out.backwardVol(i,j) - out.marketImpVol(i,j);
% 
%                             end
%                         end
% 
%                         N=0;
%                         for j=1:expirySize
%                             for k=1:strikeSize
%                                 out.backPriceRmseTotal = out.backPriceRmseTotal + out.backRe(j,k)*out.backRe(j,k);
%                                 out.backVolMseTotal = out.backVolMseTotal +out.backVolError(j,k)*out.backVolError(j,k);
%                                 N=N+1;
%                             end
%                         end
% 
%                         out.backPriceRmseTotal = sqrt(out.backPriceRmseTotal/N);
%                         out.backVolMseTotal=sqrt(out.backVolMseTotal/N);
%                         
%                     end
% 
%                     toc
%                 case 'bootstrapForward'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     % update calibration result for eqComDupire
%                     
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
% 
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     % compute implied surface for targetStrikes;
%                     targetExpiry = params.expiry;
%                     
%                     %% at T call & put price
%                     for idx1=1:length(params.expiry)
%                         targetExpiry(idx1) = params.expiry(idx1)-0;
% %                         targetExpiry(idx1) = params.expiry(idx1)+1;
%                     end
%                     
% %                     targetStrikes = out.marketStrikes;
%                     % 0.5% shifted grid
%                     newKs = zeros(length(out.Ks),1);
%                     newKs = zeros(241,1);
%                     
%                     for idx1=1:241
%                         newKs(idx1) = 0.1+ 0.01*(idx1-1);
%                     end
%                     
%                     targetStrikes = zeros(length(targetExpiry),length(newKs));
%                     for idx1=1:size(out.marketStrikes,1)
%                         for idx2= 1:length(newKs)
% %                             targetStrikes(idx1,idx2) = out.Ks(idx2);
%                             targetStrikes(idx1,idx2) = newKs(idx2);
%                         end
%                     end
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVol = outDummy.impVolSurface;
%                     out.cValueSurface = outDummy.cValueSurface;
%                     out.pValueSurface = outDummy.pValueSurface;
%                     out.probMeshInt = outDummy.probMeshInt;
%                     
%                      %% at T-dT call & put price
%                     for idx1=1:length(params.expiry)
%                         targetExpiry(idx1) = params.expiry(idx1)-0.000001;
% %                         targetExpiry(idx1) = params.expiry(idx1)+1;
%                     end
%                     
% %                     targetStrikes = out.marketStrikes;
%                     % 0.5% shifted grid
%                     newKs = zeros(length(out.Ks),1);
%                     newKs = zeros(241,1);
%                     
%                     for idx1=1:241
%                         newKs(idx1) = 0.1+ 0.01*(idx1-1);
%                     end
%                     
%                     targetStrikes = zeros(length(targetExpiry),length(newKs));
%                     for idx1=1:size(out.marketStrikes,1)
%                         for idx2= 1:length(newKs)
% %                             targetStrikes(idx1,idx2) = out.Ks(idx2);
%                             targetStrikes(idx1,idx2) = newKs(idx2);
%                         end
%                     end
%                     
%                     outDummy = eqCOMDupireSpotGF.computeForwardImpVol(params,targetExpiry,targetStrikes);
%                     out.forwardImpVolPrev = outDummy.impVolSurface;
%                     out.cValueSurfacePrev = outDummy.cValueSurface;
%                     out.pValueSurfacePrev = outDummy.pValueSurface;
%                     out.probMeshIntPrev = outDummy.probMeshInt;
%                     
%                     aaa = 1.0;
%                     
%                 case 'bootstrapForwardOnly'
%                     
%                     tic
%                     %vol proxy calibration
%                     lastProb = initProb;
%                     
%                     for i=1:expirySize
%                         if  i ~= 1 dT = params.expiry(i) - params.expiry(i-1);else dT =  params.expiry(i);end;
%                         dT = dT/365.0;
%                         bootStrapOut = eqCOMDupireSpotGF.FindVolProxy(i,dT,lastProb,params);
%                         for j=1:strikeSize
%                             out.volProxy(i,j) = bootStrapOut.volProxy(j);
%                             out.ipos(i,j)     = bootStrapOut.ipos(j);
%                             out.volError(i,j) = bootStrapOut.volError(j);
%                             out.marketImpVol(i,j) =  bootStrapOut.marketImpVol(j);
%                             out.modelImpVol(i,j) =  bootStrapOut.modelImpVol(j);
%                             out.interpBlackPrices(i,j) = bootStrapOut.interpBlackPrices(j);
%                             out.dupireOTMPrices(i,j) = bootStrapOut.dupireOTMPrices(j);
%                             out.priceRe(i,j) = out.dupireOTMPrices(i,j)/out.interpBlackPrices(i,j)-1.0;
%                             out.priceReOrig(i,j) = out.dupireOTMPrices(i,j)/out.blackOTMPrices(i,j)-1.0;
%                             out.marketStrikes(i,j) = bootStrapOut.marketStrikes(j);
%                         end
%                         
%                         out.nIter(i) = bootStrapOut.nIter;
%                         tempLocalVol = eqCOMDupireSpotGF.GetLocalVolFromProxy(bootStrapOut.volProxy,out.Ks,bootStrapOut.ipos,i);
%                         for j=1:params.Ns
%                             out.probMesh(i,j) = bootStrapOut.lastProb(j);
%                             out.localVol(i,j) = tempLocalVol(j);
%                         end
%                         
%                         out.condProb(i,:,:) = bootStrapOut.condProb;
%                         % Initial Payoff for the next period
%                         lastProb = bootStrapOut.lastProb;
%                         
%                     end
%                     
%                     N=0;
%                     for j=1:expirySize
%                         for k=1:strikeSize
%                             out.rmseTotal = out.rmseTotal + out.priceReOrig(j,k)*out.priceReOrig(j,k);
%                             out.fwdVolMseTotal = out.fwdVolMseTotal + out.volError(j,k)*out.volError(j,k);
%                             N=N+1;
%                         end
%                     end
%                    
%                     out.rmseTotal = sqrt(out.rmseTotal/N);
%                     out.fwdVolMseTotal = sqrt(out.fwdVolMseTotal/N);
%                     toc
%                     
%                     % update calibration result for eqComDupire
%                     
%                     eqCOMDupireSpotGF.localVolSurface.volProxy = out.volProxy;
%                     eqCOMDupireSpotGF.localVolSurface.ipos = out.ipos;
%                     eqCOMDupireSpotGF.localVolSurface.localVol = out.localVol;
%                     eqCOMDupireSpotGF.localVolSurface.probMesh = out.probMesh;
%                     eqCOMDupireSpotGF.localVolSurface.condProb = out.condProb;
% 
%                     eqCOMDupireSpotGF.localVolSurface.Ks = out.Ks;
%                     eqCOMDupireSpotGF.localVolSurface.Pricingidx = out.Pricingidx;
%                     eqCOMDupireSpotGF.localVolSurface.expiry = out.expiry;
%                     eqCOMDupireSpotGF.localVolSurface.strike = out.strike;
%                     eqCOMDupireSpotGF.localVolSurface.marketStrikes = out.marketStrikes;
%                     
% %                     eqCOMDupireSpotGF.localVolSurface.marginal = eqCOMDupireSpotGF.generateMarginal(out.condProb,params);
%                     
%                     aaa = 1.0;
%                     
%                     
%                       
% 
%                 otherwise
%                     disp('unImplemented')
%                     
%             end
%         end
             
    end
    
end



In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
